In [3]:
import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [4]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [5]:
gc.enable()

buro_bal = pd.read_csv('../input/bureau_balance.csv')
print('Buro bal shape : ', buro_bal.shape)

print('transform to dummies')
buro_bal = pd.concat([buro_bal, pd.get_dummies(buro_bal.STATUS, prefix='buro_bal_status')], axis=1).drop('STATUS', axis=1)

print('Counting buros')
buro_counts = buro_bal[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()
buro_bal['buro_count'] = buro_bal['SK_ID_BUREAU'].map(buro_counts['MONTHS_BALANCE'])

print('averaging buro bal')
avg_buro_bal = buro_bal.groupby('SK_ID_BUREAU').mean()

avg_buro_bal.columns = ['avg_buro_' + f_ for f_ in avg_buro_bal.columns]
del buro_bal
gc.collect()

print('Read Bureau')
buro = pd.read_csv('../input/bureau.csv')

print('Go to dummies')
buro_credit_active_dum = pd.get_dummies(buro.CREDIT_ACTIVE, prefix='ca_')
buro_credit_currency_dum = pd.get_dummies(buro.CREDIT_CURRENCY, prefix='cu_')
buro_credit_type_dum = pd.get_dummies(buro.CREDIT_TYPE, prefix='ty_')

buro_full = pd.concat([buro, buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum], axis=1)
# buro_full.columns = ['buro_' + f_ for f_ in buro_full.columns]

del buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum
gc.collect()

print('Merge with buro avg')
buro_full = buro_full.merge(right=avg_buro_bal.reset_index(), how='left', on='SK_ID_BUREAU', suffixes=('', '_bur_bal'))

print('Counting buro per SK_ID_CURR')
nb_bureau_per_curr = buro_full[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
buro_full['SK_ID_BUREAU'] = buro_full['SK_ID_CURR'].map(nb_bureau_per_curr['SK_ID_BUREAU'])

print('Averaging bureau')
avg_buro = buro_full.groupby('SK_ID_CURR').mean()
print(avg_buro.head())

del buro, buro_full
gc.collect()

print('Read prev')
prev = pd.read_csv('../input/previous_application.csv')

prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]

print('Go to dummies')
prev_dum = pd.DataFrame()
for f_ in prev_cat_features:
    prev_dum = pd.concat([prev_dum, pd.get_dummies(prev[f_], prefix=f_).astype(np.uint8)], axis=1)

prev = pd.concat([prev, prev_dum], axis=1)

del prev_dum
gc.collect()

print('Counting number of Prevs')
nb_prev_per_curr = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
prev['SK_ID_PREV'] = prev['SK_ID_CURR'].map(nb_prev_per_curr['SK_ID_PREV'])

print('Averaging prev')
avg_prev = prev.groupby('SK_ID_CURR').mean()
print(avg_prev.head())
del prev
gc.collect()

print('Reading POS_CASH')
pos = pd.read_csv('../input/POS_CASH_balance.csv')

print('Go to dummies')
pos = pd.concat([pos, pd.get_dummies(pos['NAME_CONTRACT_STATUS'])], axis=1)

print('Compute nb of prevs per curr')
nb_prevs = pos[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
pos['SK_ID_PREV'] = pos['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

print('Go to averages')
avg_pos = pos.groupby('SK_ID_CURR').mean()

del pos, nb_prevs
gc.collect()

print('Reading CC balance')
cc_bal = pd.read_csv('../input/credit_card_balance.csv')

print('Go to dummies')
cc_bal = pd.concat([cc_bal, pd.get_dummies(cc_bal['NAME_CONTRACT_STATUS'], prefix='cc_bal_status_')], axis=1)

nb_prevs = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
cc_bal['SK_ID_PREV'] = cc_bal['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

print('Compute average')
avg_cc_bal = cc_bal.groupby('SK_ID_CURR').mean()
avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]

del cc_bal, nb_prevs
gc.collect()

print('Reading Installments')
inst = pd.read_csv('../input/installments_payments.csv')
nb_prevs = inst[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
inst['SK_ID_PREV'] = inst['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

avg_inst = inst.groupby('SK_ID_CURR').mean()
avg_inst.columns = ['inst_' + f_ for f_ in avg_inst.columns]

print('Read data and test')
data = pd.read_csv('../input/application_train.csv')
test = pd.read_csv('../input/application_test.csv')
print('Shapes : ', data.shape, test.shape)


print('Read data and test')
train = pd.read_csv('../input/application_train.csv')
test = pd.read_csv('../input/application_test.csv')
print('Shapes : ', train.shape, train.shape)

categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]
categorical_feats
for f_ in categorical_feats:
    train[f_], indexer = pd.factorize(train[f_])
    test[f_] = indexer.get_indexer(test[f_])
    
train = train.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')

del avg_buro, avg_prev
gc.collect()

ID = test.SK_ID_CURR

train.columns = train.columns.str.replace('[^A-Za-z0-9_]', '_')
test.columns = test.columns.str.replace('[^A-Za-z0-9_]', '_')

In [6]:
floattypes = []
inttypes = []
stringtypes = []
for c in test.columns:
    if(train[c].dtype=='object'):
        train[c] = train[c].astype('str')
        stringtypes.append(c)
    elif(train[c].dtype=='int64'):
        train[c] = train[c].astype('int32')
        inttypes.append(c)
    else:
        train[c] = train[c].astype('float32')
        floattypes.append(c)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for col in stringtypes:
    train['te_'+col] = 0.
    test['te_'+col] = 0.
    SMOOTHING = test[~test[col].isin(train[col])].shape[0]/test.shape[0]
    _, test['te_'+col] = target_encode(train[col], 
                                      test[col], 
                                      target=train['TARGET'], 
                                      min_samples_leaf=100,
                                      smoothing=SMOOTHING,
                                      noise_level=0.0)
    for f, (vis_index, blind_index) in enumerate(kf.split(train)):
        _, train.loc[blind_index, 'te_'+col] = target_encode(train.loc[vis_index, col], 
                                                            train.loc[blind_index, col], 
                                                            target=train.loc[vis_index,'TARGET'], 
                                                            min_samples_leaf=100,
                                                            smoothing=SMOOTHING,
                                                            noise_level=0.0)

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for col in inttypes:
    train['te_'+col] = 0.
    test['te_'+col] = 0.
    SMOOTHING = test[~test[col].isin(train[col])].shape[0]/test.shape[0]
    _, test['te_'+col] = target_encode(train[col], 
                                      test[col], 
                                      target=train['TARGET'], 
                                      min_samples_leaf=100,
                                      smoothing=SMOOTHING,
                                      noise_level=0.0)
    for f, (vis_index, blind_index) in enumerate(kf.split(train.index)):
        _, train.loc[blind_index, 'te_'+col] = target_encode(train.loc[vis_index, col], 
                                                            train.loc[blind_index, col], 
                                                            target=train.loc[vis_index,'TARGET'], 
                                                            min_samples_leaf=100,
                                                            smoothing=SMOOTHING,
                                                            noise_level=0.0)


In [8]:
test.insert(1,'TARGET',-1)
alldata = pd.concat([train,test])
del train ,test
gc.collect()

In [9]:
features = list(set(alldata.columns).difference(['SK_ID_CURR','TARGET']))

In [10]:
alldata[features] = alldata[features].astype('float32')

In [11]:
alldata['nans'] = alldata.isnull().sum(axis=1)

In [12]:
for col in inttypes[1:]:
    x = alldata[col].value_counts().reset_index(drop=False)
    x.columns = [col,'cnt_'+col]
    x['cnt_'+col]/=alldata.shape[0]
    alldata = alldata.merge(x,on=col,how='left')

In [13]:
for c in features:
    ss = StandardScaler()
    alldata.loc[~alldata[c].isnull(),c] = ss.fit_transform(alldata.loc[~alldata[c].isnull(),c].values.reshape(-1,1))
    alldata[c].fillna(alldata[c].mean(),inplace=True)

In [14]:
train = alldata[alldata.TARGET>-1]
test = alldata[alldata.TARGET==-1]

In [15]:
def Output(p):
    return 1./(1.+np.exp(-p))


In [16]:
def GPI(data):
    return Output(  0.100000*np.tanh(((((((((data["EXT_SOURCE_3"]) * 2.0)) - (((data["te_OCCUPATION_TYPE"]) - ((((((data["EXT_SOURCE_1"]) > (data["DAYS_CREDIT"]))*1.)) + (data["EXT_SOURCE_2"]))))))) * (-2.0))) - (data["EXT_SOURCE_2"]))) +
                    0.100000*np.tanh(((((((((data["te_OCCUPATION_TYPE"]) - (data["EXT_SOURCE_3"]))) + ((-1.0*((((data["EXT_SOURCE_1"]) * 2.0))))))) - (data["EXT_SOURCE_3"]))) - (((data["EXT_SOURCE_3"]) + (((data["EXT_SOURCE_2"]) * 2.0)))))) +
                    0.100000*np.tanh(((((((2.0) * ((((((((data["EXT_SOURCE_1"]) + (data["EXT_SOURCE_2"]))/2.0)) * (-2.0))) - (data["EXT_SOURCE_3"]))))) * 2.0)) * 2.0)) +
                    0.100000*np.tanh(((data["te_OCCUPATION_TYPE"]) + ((((-1.0*((2.0)))) * (((((data["EXT_SOURCE_2"]) + (data["ca__Closed"]))) + (((data["CODE_REJECT_REASON_XAP"]) + (data["EXT_SOURCE_1"]))))))))) +
                    0.100000*np.tanh(((((((((((((data["te_ORGANIZATION_TYPE"]) + ((-1.0*((((data["EXT_SOURCE_3"]) * 2.0))))))) * 2.0)) + (data["NAME_CONTRACT_STATUS_Refused"]))) - (((data["EXT_SOURCE_1"]) * 2.0)))) - (data["EXT_SOURCE_1"]))) * 2.0)) +
                    0.100000*np.tanh(((((np.where(data["EXT_SOURCE_3"]>0, -2.0, ((((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) - (data["EXT_SOURCE_2"]))) - (data["EXT_SOURCE_3"]))) * 2.0) )) - (data["EXT_SOURCE_2"]))) * 2.0)) +
                    0.100000*np.tanh(((((data["NAME_CONTRACT_STATUS_Refused"]) + (((-1.0) - (np.minimum(((data["EXT_SOURCE_2"])), ((data["EXT_SOURCE_2"])))))))) - (((((data["EXT_SOURCE_3"]) - (np.tanh((data["te_OCCUPATION_TYPE"]))))) - (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))))) +
                    0.100000*np.tanh(((data["NAME_PRODUCT_TYPE_walk_in"]) + (((data["te_NAME_EDUCATION_TYPE"]) + ((-1.0*((((data["EXT_SOURCE_1"]) + (((data["EXT_SOURCE_3"]) - (((np.minimum(((data["te_ORGANIZATION_TYPE"])), ((data["te_NAME_EDUCATION_TYPE"])))) - (data["EXT_SOURCE_2"])))))))))))))) +
                    0.100000*np.tanh(((((-1.0) - (data["EXT_SOURCE_2"]))) + (((data["cc_bal_AMT_BALANCE"]) - ((((-1.0*((data["cc_bal_AMT_BALANCE"])))) - (np.minimum(((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) - (data["EXT_SOURCE_3"])))), ((3.0)))))))))) +
                    0.100000*np.tanh(((((((data["te_REGION_RATING_CLIENT_W_CITY"]) + (((data["te_CODE_GENDER"]) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))))) * 2.0)) - (((data["EXT_SOURCE_2"]) + (data["EXT_SOURCE_3"]))))) +
                    0.100000*np.tanh(((data["cnt_FLAG_DOCUMENT_3"]) - (((np.where(data["EXT_SOURCE_1"]>0, 3.0, data["EXT_SOURCE_2"] )) + (((np.maximum(((data["DAYS_FIRST_DRAWING"])), ((data["DAYS_FIRST_DRAWING"])))) - (data["cnt_FLAG_DOCUMENT_3"]))))))) +
                    0.100000*np.tanh(((((data["te_NAME_EDUCATION_TYPE"]) - (data["EXT_SOURCE_3"]))) + (((((np.minimum(((3.0)), ((data["NAME_PRODUCT_TYPE_walk_in"])))) + (((data["ty__Microloan"]) * 2.0)))) + (data["CNT_INSTALMENT_FUTURE"]))))) +
                    0.100000*np.tanh(((((((data["DEF_30_CNT_SOCIAL_CIRCLE"]) - ((((data["EXT_SOURCE_2"]) > (data["cnt_FLAG_OWN_CAR"]))*1.)))) + (((data["cc_bal_CNT_DRAWINGS_CURRENT"]) + (((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) * 2.0)) - (data["SK_ID_PREV_y"]))))))) - (data["SK_ID_PREV_y"]))) +
                    0.100000*np.tanh(((((((data["te_ORGANIZATION_TYPE"]) + (((data["NAME_YIELD_GROUP_high"]) + (((((data["te_REG_CITY_NOT_LIVE_CITY"]) + (data["CNT_PAYMENT"]))) * 2.0)))))) - (data["EXT_SOURCE_2"]))) + (((data["CNT_PAYMENT"]) + (data["NAME_YIELD_GROUP_high"]))))) +
                    0.100000*np.tanh(((data["cnt_FLAG_OWN_CAR"]) - (((np.where(data["cnt_NAME_CONTRACT_TYPE"]>0, (-1.0*((((data["ty__Microloan"]) - (((data["DAYS_FIRST_DRAWING"]) + (data["AMT_ANNUITY"]))))))), np.tanh((1.0)) )) * 2.0)))) +
                    0.100000*np.tanh(((((((((((data["ty__Microloan"]) - (data["EXT_SOURCE_1"]))) - (data["EXT_SOURCE_1"]))) - (data["cnt_CODE_GENDER"]))) + ((((((data["CODE_REJECT_REASON_SCOFR"]) + (data["ty__Microloan"]))/2.0)) + (data["DAYS_ID_PUBLISH"]))))) * 2.0)) +
                    0.100000*np.tanh(((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) - (data["cnt_NAME_FAMILY_STATUS"]))) + (((np.where(0.0>0, -1.0, ((((data["cnt_NAME_CONTRACT_TYPE"]) + (data["CNT_INSTALMENT_FUTURE"]))) + (data["avg_buro_buro_bal_status_1"])) )) + (data["ty__Microloan"]))))) +
                    0.100000*np.tanh(((((((np.maximum(((data["NAME_CLIENT_TYPE_New"])), ((data["DEF_30_CNT_SOCIAL_CIRCLE"])))) + ((-1.0*((data["RATE_DOWN_PAYMENT"])))))) - (((data["SK_ID_PREV_y"]) - (((((data["DEF_60_CNT_SOCIAL_CIRCLE"]) * 2.0)) - (data["NAME_YIELD_GROUP_low_action"]))))))) * 2.0)) +
                    0.100000*np.tanh(((data["te_WALLSMATERIAL_MODE"]) + ((((data["NAME_CONTRACT_STATUS_Refused"]) + (((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) + ((((2.0) + (np.minimum(((1.0)), ((((data["AMT_ANNUITY_x"]) * ((11.28408813476562500))))))))/2.0)))))/2.0)))) +
                    0.100000*np.tanh((((((((((((data["CNT_PAYMENT"]) - (data["NAME_SELLER_INDUSTRY_Furniture"]))) - (data["AMT_ANNUITY"]))) + (((data["CNT_PAYMENT"]) - (data["AMT_ANNUITY"]))))) + (data["cnt_FLAG_OWN_CAR"]))/2.0)) - (data["PRODUCT_COMBINATION_POS_industry_with_interest"]))) +
                    0.100000*np.tanh(((((((((((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]) + (np.tanh((((data["ty__Microloan"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) + ((((1.0)) - ((((-1.0*((data["ty__Microloan"])))) * 2.0)))))) +
                    0.100000*np.tanh((-1.0*((((((data["CODE_GENDER"]) + (data["PRODUCT_COMBINATION_Cash_X_Sell__low"]))) - (((data["te_ORGANIZATION_TYPE"]) + (((data["SK_DPD"]) - (((((data["OWN_CAR_AGE"]) * 2.0)) * ((-1.0*((3.0))))))))))))))) +
                    0.100000*np.tanh((((((data["CNT_INSTALMENT_FUTURE"]) * (((((np.minimum(((data["CNT_INSTALMENT_FUTURE"])), ((data["CNT_INSTALMENT_FUTURE"])))) * 2.0)) / 2.0)))) + (np.where(data["ty__Mortgage"]>0, -3.0, data["cnt_NAME_EDUCATION_TYPE"] )))/2.0)) +
                    0.100000*np.tanh((((data["DAYS_LAST_PHONE_CHANGE"]) + (((((data["avg_buro_buro_bal_status_1"]) + (np.tanh((((-3.0) - (((data["EXT_SOURCE_3"]) * 2.0)))))))) - (data["CHANNEL_TYPE_Channel_of_corporate_sales"]))))/2.0)) +
                    0.100000*np.tanh((((((-1.0*((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) + (data["NAME_YIELD_GROUP_low_action"])))))) - (((data["inst_NUM_INSTALMENT_VERSION"]) - (((data["AMT_ANNUITY_x"]) - (data["CHANNEL_TYPE_Channel_of_corporate_sales"]))))))) + ((-1.0*((data["NAME_YIELD_GROUP_low_action"])))))) +
                    0.100000*np.tanh(((((data["CNT_PAYMENT"]) - ((((data["RATE_DOWN_PAYMENT"]) + (((((data["RATE_DOWN_PAYMENT"]) / 2.0)) * 2.0)))/2.0)))) - (np.maximum(((data["NAME_YIELD_GROUP_low_normal"])), ((np.maximum(((data["ty__Mortgage"])), ((data["PRODUCT_COMBINATION_Cash_Street__low"]))))))))) +
                    0.100000*np.tanh(((((((data["ty__Microloan"]) - (data["inst_AMT_PAYMENT"]))) + (((((((((data["cnt_DAYS_BIRTH"]) + (data["cnt_FLAG_DOCUMENT_18"]))/2.0)) + (data["CODE_REJECT_REASON_SCOFR"]))) + (data["ty__Microloan"]))/2.0)))) * 2.0)) +
                    0.100000*np.tanh(((((((((((data["cnt_ORGANIZATION_TYPE"]) + (data["SK_DPD"]))/2.0)) + (((data["SK_DPD"]) + ((-1.0*((data["cnt_FLAG_DOCUMENT_6"])))))))) + (data["SK_DPD"]))) + (np.maximum(((data["FLAG_DOCUMENT_6"])), ((data["CNT_INSTALMENT_FUTURE"])))))/2.0)) +
                    0.100000*np.tanh(np.where(data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]>0, np.minimum(((-2.0)), ((data["Active"]))), (((data["CODE_REJECT_REASON_HC"]) + (np.maximum(((((((data["NAME_GOODS_CATEGORY_Mobile"]) + (data["DAYS_REGISTRATION"]))) - (data["inst_SK_ID_PREV"])))), ((data["NAME_SELLER_INDUSTRY_Connectivity"])))))/2.0) )) +
                    0.100000*np.tanh((((data["cnt_WEEKDAY_APPR_PROCESS_START"]) + (((-3.0) * (np.maximum(((data["AMT_CREDIT_SUM_LIMIT"])), ((np.maximum(((((data["FLAG_DOCUMENT_16"]) - (data["AMT_CREDIT_SUM_LIMIT"])))), ((data["CHANNEL_TYPE_Channel_of_corporate_sales"]))))))))))/2.0)) +
                    0.099961*np.tanh(np.where(data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]>0, ((np.tanh((data["NAME_PAYMENT_TYPE_XNA"]))) + (data["CHANNEL_TYPE_AP___Cash_loan_"])), ((((((((data["SK_DPD"]) + (data["NAME_PAYMENT_TYPE_XNA"]))/2.0)) - (data["Active"]))) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))/2.0) )) +
                    0.100000*np.tanh((((data["avg_buro_buro_bal_status_1"]) + (((data["ca__Sold"]) + ((((((((data["avg_buro_buro_bal_status_1"]) + (data["REGION_RATING_CLIENT_W_CITY"]))) - (data["ty__Mortgage"]))) + (((data["NFLAG_INSURED_ON_APPROVAL"]) + (data["DEF_60_CNT_SOCIAL_CIRCLE"]))))/2.0)))))/2.0)) +
                    0.094568*np.tanh((((((data["NAME_CLIENT_TYPE_New"]) - (np.maximum(((((-3.0) - (data["NAME_CLIENT_TYPE_New"])))), (((((data["FLAG_DOCUMENT_18"]) + (data["PRODUCT_COMBINATION_Cash_X_Sell__low"]))/2.0))))))) + ((-1.0*((((data["inst_NUM_INSTALMENT_VERSION"]) / 2.0))))))/2.0)) +
                    0.099922*np.tanh(((np.maximum(((np.minimum(((2.0)), ((((np.minimum((((1.86563777923583984))), ((data["CNT_INSTALMENT_FUTURE"])))) - (data["MONTHS_BALANCE"]))))))), ((data["FLAG_DOCUMENT_2"])))) + (np.minimum(((data["NAME_GOODS_CATEGORY_XNA"])), ((0.0)))))) +
                    0.100000*np.tanh((((np.where(np.maximum(((data["AMT_ANNUITY_x"])), ((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((data["AMT_ANNUITY_x"]))))))>0, 1.0, data["AMT_ANNUITY_x"] )) + (np.where(data["te_NAME_FAMILY_STATUS"]>0, ((2.0) / 2.0), data["te_NAME_FAMILY_STATUS"] )))/2.0)) +
                    0.100000*np.tanh((((-1.0*(((-1.0*(((((np.minimum(((-1.0)), ((data["AMT_GOODS_PRICE_x"])))) > (((data["inst_AMT_PAYMENT"]) * 2.0)))*1.)))))))) - ((((data["AMT_GOODS_PRICE_x"]) > (data["PRODUCT_COMBINATION_Cash_Street__low"]))*1.)))) +
                    0.099961*np.tanh(((np.where(data["FLAG_DOCUMENT_16"]>0, data["cnt_FLAG_DOCUMENT_16"], ((np.minimum((((5.91657543182373047))), ((data["cnt_FLAG_DOCUMENT_16"])))) * 2.0) )) - ((((((data["CODE_REJECT_REASON_SCO"]) + (data["PRODUCT_COMBINATION_Cash_Street__low"]))/2.0)) / 2.0)))) +
                    0.100000*np.tanh((((((data["cnt_FLAG_DOCUMENT_13"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) + (((data["FLAG_DOCUMENT_2"]) + (((data["NAME_GOODS_CATEGORY_Insurance"]) + (np.tanh(((-1.0*((data["te_FLAG_DOCUMENT_20"])))))))))))/2.0)) +
                    0.093396*np.tanh((-1.0*((((data["CHANNEL_TYPE_Channel_of_corporate_sales"]) - (((((data["FLAG_WORK_PHONE"]) / 2.0)) - (np.maximum(((data["AMT_REQ_CREDIT_BUREAU_QRT"])), ((((((data["AMT_REQ_CREDIT_BUREAU_QRT"]) * 2.0)) * 2.0)))))))))))) +
                    0.094373*np.tanh(np.where(data["SK_DPD"]>0, ((((-1.0) * (data["te_OCCUPATION_TYPE"]))) - (((((data["AMT_ANNUITY_x"]) + (data["AMT_ANNUITY_x"]))) / 2.0))), data["AMT_ANNUITY_x"] )) +
                    0.100000*np.tanh(((data["AMT_GOODS_PRICE_x"]) * ((((-1.0*((data["AMT_GOODS_PRICE_x"])))) - ((((data["FLAG_DOCUMENT_2"]) < (((data["AMT_GOODS_PRICE_x"]) * (((((-1.0*((data["AMT_GOODS_PRICE_x"])))) + (-1.0))/2.0)))))*1.)))))) +
                    0.100000*np.tanh(((((((-1.0*((data["te_FLAG_EMP_PHONE"])))) + (data["cnt_FLAG_PHONE"]))) + (np.where(((data["cnt_DAYS_EMPLOYED"]) + (data["FLAG_EMP_PHONE"]))>0, 1.0, -1.0 )))/2.0)) +
                    0.094920*np.tanh(((((-1.0*((data["avg_buro_buro_bal_status_0"])))) + ((((data["cnt_NAME_INCOME_TYPE"]) + (((((((data["NAME_GOODS_CATEGORY_Insurance"]) > (data["cc_bal_SK_DPD_DEF"]))*1.)) < ((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["cnt_NAME_INCOME_TYPE"]))/2.0)) / 2.0)))*1.)))/2.0)))/2.0)) +
                    0.099961*np.tanh(((data["CNT_INSTALMENT_FUTURE"]) * ((((3.0) < (((data["CNT_PAYMENT"]) * ((((5.0)) / 2.0)))))*1.)))) +
                    0.100000*np.tanh(((((data["SK_DPD"]) * 2.0)) + ((((((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) / 2.0)) + (data["cc_bal_CNT_DRAWINGS_CURRENT"]))/2.0)) - (((((data["te_OCCUPATION_TYPE"]) / 2.0)) / 2.0)))))) +
                    0.097968*np.tanh(((np.maximum((((((data["ca__Closed"]) + (data["te_CNT_CHILDREN"]))/2.0))), ((((np.minimum(((data["ca__Closed"])), ((data["cnt_FLAG_DOCUMENT_2"])))) / 2.0))))) - (np.tanh((((data["ca__Closed"]) * 2.0)))))) +
                    0.099922*np.tanh(np.where(data["cc_bal_SK_DPD_DEF"]>0, (3.0), ((np.maximum(((-3.0)), ((data["cc_bal_SK_DPD_DEF"])))) - (((np.maximum(((data["YEARS_BUILD_MODE"])), ((((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * 2.0))))) - ((-1.0*((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))))))) )) +
                    0.099805*np.tanh(((((((data["cc_bal_AMT_RECIVABLE"]) + (data["HOUSETYPE_MODE"]))/2.0)) + (((np.where(data["HOUSETYPE_MODE"]>0, 0.0, data["cc_bal_AMT_BALANCE"] )) - ((((data["cc_bal_AMT_BALANCE"]) > (data["cnt_FLAG_DOCUMENT_2"]))*1.)))))/2.0)) +
                    0.099961*np.tanh(np.minimum(((((1.0) - (data["ty__Mortgage"])))), ((((((((2.0) + (data["DAYS_ID_PUBLISH"]))/2.0)) > ((1.74468076229095459)))*1.))))) +
                    0.100000*np.tanh((-1.0*((((((data["FLAG_DOCUMENT_13"]) * 2.0)) + (np.maximum(((np.maximum(((data["AMT_CREDIT_SUM_LIMIT"])), (((-1.0*((data["AMT_CREDIT_SUM_LIMIT"])))))))), ((0.0))))))))) +
                    0.099961*np.tanh(((np.minimum(((data["cnt_FLAG_DOCUMENT_18"])), ((np.tanh(((((data["AMT_CREDIT_x"]) + (np.minimum(((1.0)), ((((((data["AMT_CREDIT_x"]) / 2.0)) - (-1.0)))))))/2.0))))))) * 2.0)) +
                    0.099961*np.tanh((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (((((np.maximum(((np.where(data["ty__Car_loan"]>0, 1.0, data["ty__Microloan"] ))), ((data["NAME_GOODS_CATEGORY_Insurance"])))) - (data["ty__Car_loan"]))) / 2.0)))/2.0)) +
                    0.099883*np.tanh((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (((((np.maximum(((np.maximum(((data["SK_ID_PREV_x"])), ((((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)))))), ((((-2.0) + (data["FLAG_DOCUMENT_2"])))))) + (np.tanh((data["cnt_HOUR_APPR_PROCESS_START_x"]))))) / 2.0)))/2.0)) +
                    0.099922*np.tanh(np.where(((data["cnt_FLAG_WORK_PHONE"]) + (np.tanh((data["inst_AMT_PAYMENT"]))))>0, data["FLAG_DOCUMENT_2"], np.where(data["cc_bal_SK_DPD_DEF"]>0, data["cc_bal_SK_DPD_DEF"], (-1.0*((data["inst_AMT_PAYMENT"]))) ) )) +
                    0.099805*np.tanh(((np.tanh(((((data["NAME_GOODS_CATEGORY_Construction_Materials"]) + ((-1.0*((np.maximum(((data["NAME_CONTRACT_STATUS_Unused_offer"])), ((0.0))))))))/2.0)))) + (((np.tanh((np.where(data["CODE_REJECT_REASON_CLIENT"]>0, -3.0, data["CHANNEL_TYPE_AP___Cash_loan_"] )))) / 2.0)))) +
                    0.098554*np.tanh(((((np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) - (((((((data["cnt_NAME_HOUSING_TYPE"]) + (data["cnt_FLAG_DOCUMENT_2"]))/2.0)) + (data["NAME_TYPE_SUITE_Unaccompanied"]))/2.0)))) / 2.0)) +
                    0.099844*np.tanh((((np.tanh((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]))) + (((((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) * (((np.maximum(((data["cc_bal_SK_DPD_DEF"])), ((data["PRODUCT_COMBINATION_Cash_Street__low"])))) / 2.0)))) * (((np.maximum(((data["cc_bal_SK_DPD_DEF"])), ((data["PRODUCT_COMBINATION_Cash_Street__low"])))) / 2.0)))))/2.0)) +
                    0.100000*np.tanh((((((np.where(data["NONLIVINGAPARTMENTS_MODE"]>0, (14.59643173217773438), np.minimum(((data["NONLIVINGAPARTMENTS_MODE"])), ((data["WEEKDAY_APPR_PROCESS_START_SATURDAY"]))) )) + (data["te_REG_REGION_NOT_LIVE_REGION"]))/2.0)) * (data["WEEKDAY_APPR_PROCESS_START_SATURDAY"]))) +
                    0.100000*np.tanh(((np.where((((data["FLAG_DOCUMENT_2"]) > ((-1.0*((((((data["SK_DPD"]) * (data["SK_DPD"]))) * 2.0))))))*1.)>0, data["SK_DPD"], 3.0 )) * 2.0)) +
                    0.099961*np.tanh((((data["cnt_FLAG_DOCUMENT_14"]) + (np.where(data["NAME_GOODS_CATEGORY_House_Construction"]>0, -2.0, (((((data["AMT_CREDIT_y"]) / 2.0)) + (np.maximum(((((data["AMT_CREDIT_y"]) / 2.0))), ((data["NAME_GOODS_CATEGORY_Insurance"])))))/2.0) )))/2.0)) +
                    0.099961*np.tanh((((((data["NONLIVINGAPARTMENTS_MODE"]) * 2.0)) > ((((data["NAME_PRODUCT_TYPE_walk_in"]) + ((((((((data["NONLIVINGAPARTMENTS_MODE"]) + (data["cnt_FLAG_DOCUMENT_2"]))) * 2.0)) < (data["te_NAME_CONTRACT_TYPE"]))*1.)))/2.0)))*1.)) +
                    0.099140*np.tanh(np.where(data["NAME_PRODUCT_TYPE_walk_in"]>0, (-1.0*(((((2.0) < (data["NAME_PRODUCT_TYPE_walk_in"]))*1.)))), (((((data["Returned_to_the_store"]) + (data["te_REG_CITY_NOT_LIVE_CITY"]))/2.0)) / 2.0) )) +
                    0.100000*np.tanh(((np.where(data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]>0, (-1.0*((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))), ((((((((((data["te_REG_REGION_NOT_LIVE_REGION"]) + (((data["HOUR_APPR_PROCESS_START_y"]) / 2.0)))/2.0)) / 2.0)) * (-3.0))) + (data["FLAG_DOCUMENT_2"]))/2.0) )) / 2.0)) +
                    0.100000*np.tanh(np.where((((((data["inst_NUM_INSTALMENT_VERSION"]) + (-2.0))) > (data["cc_bal_SK_ID_PREV"]))*1.)>0, -2.0, (((data["inst_AMT_PAYMENT"]) < ((((-2.0) + (((data["cc_bal_SK_ID_PREV"]) + (data["inst_NUM_INSTALMENT_VERSION"]))))/2.0)))*1.) )) +
                    0.099883*np.tanh((((np.where(data["cnt_CODE_GENDER"]>0, np.tanh((data["REGION_POPULATION_RELATIVE"])), data["cc_bal_SK_ID_PREV"] )) + ((((((((data["REGION_POPULATION_RELATIVE"]) / 2.0)) < (data["cnt_CODE_GENDER"]))*1.)) * (data["REGION_POPULATION_RELATIVE"]))))/2.0)) +
                    0.099961*np.tanh(np.where(np.minimum(((np.where(data["cc_bal_SK_DPD_DEF"]>0, 2.0, data["DAYS_FIRST_DUE"] ))), ((data["cc_bal_cc_bal_status__Sent_proposal"])))>0, 3.0, ((data["cc_bal_SK_DPD_DEF"]) - (((data["cc_bal_cc_bal_status__Sent_proposal"]) / 2.0))) )) +
                    0.075342*np.tanh((((((((data["REGION_RATING_CLIENT_W_CITY"]) - (((data["AMT_REQ_CREDIT_BUREAU_MON"]) * (np.where(data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"]>0, data["AMT_REQ_CREDIT_BUREAU_MON"], 3.0 )))))) + ((((data["REGION_RATING_CLIENT_W_CITY"]) < (data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"]))*1.)))/2.0)) / 2.0)) +
                    0.088980*np.tanh(((np.tanh(((((((-1.0*((np.tanh((data["DAYS_ENDDATE_FACT"])))))) / 2.0)) - (data["NAME_GOODS_CATEGORY_Gardening"]))))) - (np.maximum(((data["cc_bal_cc_bal_status__Sent_proposal"])), ((data["NAME_GOODS_CATEGORY_Gardening"])))))) +
                    0.100000*np.tanh((((data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"]) + (((((((data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"]) * ((-1.0*((data["SK_ID_PREV_y"])))))) * 2.0)) + (((((((0.0) + (((-3.0) / 2.0)))/2.0)) > (data["SK_ID_PREV_y"]))*1.)))))/2.0)) +
                    0.099805*np.tanh(((np.tanh((((((-1.0*((np.tanh((3.0)))))) + (((data["CNT_INSTALMENT_FUTURE"]) - (np.maximum(((-1.0)), ((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"])))))))/2.0)))) / 2.0)) +
                    0.100000*np.tanh(((np.where(np.minimum(((((3.0) * 2.0))), ((((-1.0) + (data["CNT_INSTALMENT"])))))>0, 3.0, data["inst_AMT_PAYMENT"] )) * (np.minimum(((np.tanh((data["inst_AMT_PAYMENT"])))), ((data["cc_bal_SK_DPD_DEF"])))))) +
                    0.099961*np.tanh(np.where(data["cc_bal_SK_DPD_DEF"]>0, data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"], (((((((data["inst_AMT_PAYMENT"]) < (np.tanh((((data["FLAG_DOCUMENT_2"]) - (np.tanh(((-1.0*((np.tanh((-2.0)))))))))))))*1.)) * 2.0)) * 2.0) )) +
                    0.099883*np.tanh((((-1.0*(((((((((data["AMT_CREDIT_x"]) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))) * 2.0)) < (((np.where((11.91235828399658203)>0, -3.0, data["cc_bal_SK_DPD_DEF"] )) / 2.0)))*1.))))) / 2.0)) +
                    0.088667*np.tanh(((0.0) - (np.minimum((((((-1.0) < (np.minimum(((data["ENTRANCES_AVG"])), ((((((data["inst_AMT_PAYMENT"]) * 2.0)) * (data["ENTRANCES_AVG"])))))))*1.))), ((data["ENTRANCES_MEDI"])))))) +
                    0.099648*np.tanh((((((data["FLAG_DOCUMENT_13"]) * ((-1.0*(((((data["FLAG_DOCUMENT_13"]) + (1.0))/2.0))))))) + ((((((((-1.0*(((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"]))))))) + (data["FLAG_WORK_PHONE"]))/2.0)) + (data["cc_bal_SK_DPD_DEF"]))/2.0)))/2.0)) +
                    0.099218*np.tanh(np.tanh((((np.maximum(((((((-1.0*((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((data["cc_bal_SK_DPD_DEF"]))))))) > (data["cnt_FLAG_DOCUMENT_2"]))*1.))), ((-3.0)))) * ((((data["CNT_INSTALMENT_FUTURE"]) + (((data["CNT_INSTALMENT_FUTURE"]) * 2.0)))/2.0)))))) +
                    0.099961*np.tanh(((np.where(data["PRODUCT_COMBINATION_Cash_X_Sell__low"]>0, ((((-1.0*((data["PRODUCT_COMBINATION_Cash_X_Sell__low"])))) + ((((2.0) + (data["cc_bal_SK_DPD_DEF"]))/2.0)))/2.0), (-1.0*(((((data["CNT_INSTALMENT"]) > ((-1.0*((data["PRODUCT_COMBINATION_Cash_X_Sell__low"])))))*1.)))) )) / 2.0)) +
                    0.099297*np.tanh(np.where(data["inst_AMT_PAYMENT"]>0, data["cc_bal_SK_DPD_DEF"], np.maximum(((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) / 2.0))), ((np.tanh((np.maximum(((data["cc_bal_CNT_DRAWINGS_CURRENT"])), ((((((data["inst_AMT_PAYMENT"]) * 2.0)) * (np.tanh((data["PRODUCT_COMBINATION_POS_mobile_with_interest"])))))))))))) )) +
                    0.099531*np.tanh((((((((data["NONLIVINGAPARTMENTS_MODE"]) + ((((12.74562358856201172)) * (data["cc_bal_SK_DPD_DEF"]))))/2.0)) + ((((2.0) < (data["PRODUCT_COMBINATION_Card_X_Sell"]))*1.)))) + (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0)))) +
                    0.096991*np.tanh(((((np.minimum(((data["cnt_FLAG_DOCUMENT_11"])), ((np.minimum(((np.tanh((((np.tanh((data["CODE_REJECT_REASON_LIMIT"]))) - (data["cnt_FLAG_DOCUMENT_2"])))))), (((6.59259939193725586)))))))) + ((((data["cc_bal_SK_DPD_DEF"]) < (data["CODE_REJECT_REASON_LIMIT"]))*1.)))) / 2.0)) +
                    0.099844*np.tanh(np.where((((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) + ((1.57356655597686768)))) + (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))/2.0)>0, data["cc_bal_SK_DPD_DEF"], ((2.0) * 2.0) )) +
                    0.099023*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["cnt_FLAG_DOCUMENT_2"], np.minimum(((np.minimum(((data["cnt_FLAG_DOCUMENT_16"])), ((data["cnt_FLAG_EMAIL"]))))), ((np.minimum(((data["cc_bal_SK_DPD_DEF"])), (((((-1.0*((data["cnt_FLAG_DOCUMENT_16"])))) * (data["CODE_REJECT_REASON_SCO"])))))))) )) +
                    0.099726*np.tanh(((((((-1.0*((((((data["DAYS_LAST_DUE_1ST_VERSION"]) + (data["NAME_TYPE_SUITE_Children"]))) / 2.0))))) + ((((data["CODE_GENDER"]) < (((data["DAYS_LAST_DUE_1ST_VERSION"]) * (data["FLAG_DOCUMENT_2"]))))*1.)))/2.0)) / 2.0)) +
                    0.099531*np.tanh(np.minimum((((((((((((((data["FLAG_OWN_CAR"]) / 2.0)) / 2.0)) + (data["FLAG_OWN_CAR"]))) + (((data["FLAG_OWN_CAR"]) - (data["ty__Car_loan"]))))/2.0)) * (data["ty__Car_loan"])))), ((0.0)))) +
                    0.099805*np.tanh(((((0.0) + (((np.where(data["cu__currency_3"]>0, ((data["cu__currency_3"]) / 2.0), ((data["cnt_FLAG_DOCUMENT_15"]) - ((-1.0*((np.tanh((data["cc_bal_SK_DPD_DEF"]))))))) )) * 2.0)))) - (data["cnt_FLAG_DOCUMENT_2"]))) +
                    0.081829*np.tanh((((np.where(data["CNT_INSTALMENT_FUTURE"]>0, data["AMT_CREDIT_y"], data["NAME_GOODS_CATEGORY_Insurance"] )) > (np.minimum((((((data["CNT_INSTALMENT_FUTURE"]) < (2.0))*1.))), ((np.where(3.0>0, (12.38264274597167969), ((data["CNT_INSTALMENT_FUTURE"]) * 2.0) ))))))*1.)) +
                    0.100000*np.tanh(((-1.0) * ((((((((0.0) < (data["CNT_INSTALMENT"]))*1.)) * (data["Active"]))) * (data["Active"]))))) +
                    0.099805*np.tanh(np.where((((data["CNT_PAYMENT"]) + (-3.0))/2.0)>0, data["DAYS_LAST_DUE"], ((np.maximum((((((data["DAYS_LAST_DUE"]) > (2.0))*1.))), (((((1.0) < (data["CNT_PAYMENT"]))*1.))))) / 2.0) )) +
                    0.099883*np.tanh(np.minimum(((((2.0) - (data["inst_AMT_PAYMENT"])))), (((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) < ((-1.0*(((((data["inst_AMT_PAYMENT"]) < (1.0))*1.))))))*1.))))) +
                    0.099570*np.tanh((((-1.0*(((((((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) > (((((((data["FLAG_DOCUMENT_2"]) * 2.0)) * 2.0)) - (((data["PRODUCT_COMBINATION_Cash_Street__low"]) + (-3.0))))))*1.)) / 2.0)) * 2.0))))) * 2.0)) +
                    0.100000*np.tanh((((((-1.0*(((((3.0) < (data["NAME_GOODS_CATEGORY_Furniture"]))*1.))))) + ((((data["cnt_FLAG_DOCUMENT_14"]) + ((((data["cnt_FLAG_DOCUMENT_14"]) < (np.maximum(((((data["cc_bal_SK_DPD_DEF"]) * 2.0))), ((data["cnt_FLAG_DOCUMENT_14"])))))*1.)))/2.0)))) * 2.0)) +
                    0.099687*np.tanh(((((np.tanh((((data["WEEKDAY_APPR_PROCESS_START_MONDAY"]) - (data["Active"]))))) / 2.0)) / 2.0)) +
                    0.099687*np.tanh(((((((((np.where((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"])))>0, np.where(data["CNT_INSTALMENT_FUTURE"]>0, (2.0), ((data["ca__Sold"]) + (-1.0)) ), 0.0 )) / 2.0)) / 2.0)) / 2.0)) / 2.0)) +
                    0.099766*np.tanh(((((((2.0) * (data["te_FLAG_OWN_REALTY"]))) + ((((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + (((data["cnt_FLAG_DOCUMENT_2"]) - (data["NAME_GOODS_CATEGORY_Insurance"]))))/2.0)) * 2.0)))) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) +
                    0.099609*np.tanh(np.tanh((((data["DAYS_BIRTH"]) * (((1.0) + (np.minimum(((np.tanh(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > (((data["DAYS_BIRTH"]) * (((data["cnt_NAME_FAMILY_STATUS"]) / 2.0)))))*1.))))), ((data["cnt_NAME_FAMILY_STATUS"])))))))))) +
                    0.098007*np.tanh(np.where(((data["CNT_FAM_MEMBERS"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))>0, data["DAYS_EMPLOYED"], ((((-1.0*((data["cnt_CNT_CHILDREN"])))) > (data["CNT_CHILDREN"]))*1.) )) +
                    0.086284*np.tanh(((((((((-1.0) / 2.0)) + (data["cc_bal_SK_DPD_DEF"]))/2.0)) + ((((data["AMT_CREDIT_y"]) > (((data["ty__Mortgage"]) - (((data["cnt_FLAG_DOCUMENT_2"]) - (data["ty__Mortgage"]))))))*1.)))/2.0)) +
                    0.093044*np.tanh(((np.minimum(((((((data["SK_ID_PREV_y"]) * (data["inst_AMT_PAYMENT"]))) * (2.0)))), ((((np.minimum(((((data["SK_ID_PREV_y"]) * (data["inst_AMT_PAYMENT"])))), ((3.0)))) / 2.0))))) * 2.0)) +
                    0.100000*np.tanh(((((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) < ((((data["SK_DPD"]) + (np.tanh((np.tanh(((((-1.0*((data["SK_DPD"])))) / 2.0)))))))/2.0)))*1.)) * 2.0)) +
                    0.100000*np.tanh((-1.0*((np.where((((-2.0) + ((-1.0*((data["EXT_SOURCE_2"])))))/2.0)>0, data["EXT_SOURCE_2"], ((((((data["EXT_SOURCE_2"]) < (((-3.0) * 2.0)))*1.)) > (data["EXT_SOURCE_2"]))*1.) ))))) +
                    0.098632*np.tanh(np.where(np.tanh((((-3.0) + (data["AMT_REQ_CREDIT_BUREAU_YEAR"]))))>0, -3.0, ((np.where(data["AMT_REQ_CREDIT_BUREAU_YEAR"]>0, data["CODE_REJECT_REASON_XAP"], data["FLAG_DOCUMENT_2"] )) / 2.0) )) +
                    0.082923*np.tanh(((np.minimum(((((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) < (data["SK_ID_PREV_x"]))*1.)) / 2.0)) + (data["NAME_CASH_LOAN_PURPOSE_Car_repairs"]))/2.0))), (((((((6.0)) - (((data["te_LIVE_REGION_NOT_WORK_REGION"]) * 2.0)))) * 2.0))))) * 2.0)) +
                    0.098867*np.tanh(np.maximum(((data["NAME_SELLER_INDUSTRY_Auto_technology"])), (((-1.0*((np.where(data["inst_SK_ID_PREV"]>0, np.maximum(((2.0)), ((2.0))), ((data["SK_ID_PREV_y"]) / 2.0) )))))))) +
                    0.079640*np.tanh(np.tanh((((np.tanh((((np.minimum(((data["EXT_SOURCE_2"])), ((((-2.0) * (np.tanh((((data["EXT_SOURCE_2"]) - (1.0)))))))))) * 2.0)))) + (((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)))))) +
                    0.099883*np.tanh(np.maximum((((((1.0) < (np.where(data["EXT_SOURCE_2"]>0, -2.0, data["ty__Consumer_credit"] )))*1.))), ((((-2.0) - (np.minimum(((-1.0)), ((data["EXT_SOURCE_2"]))))))))) +
                    0.099766*np.tanh(np.where(data["AMT_CREDIT_SUM_DEBT"]>0, (-1.0*((((((-3.0) + (data["AMT_CREDIT_SUM_DEBT"]))) * 2.0)))), ((((data["AMT_CREDIT_SUM_DEBT"]) * 2.0)) * 2.0) )) +
                    0.100000*np.tanh((((((data["ty__Microloan"]) > (((3.0) + (((np.minimum(((data["ca__Active"])), ((((-1.0) * ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > (data["DAYS_EMPLOYED"]))*1.))))))) + (-2.0))))))*1.)) * 2.0)) +
                    0.087651*np.tanh(np.where(np.tanh((np.where((2.0)>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], (3.98223018646240234) )))>0, data["SK_ID_BUREAU"], ((((data["SK_ID_BUREAU"]) * 2.0)) * (((data["SK_ID_PREV_x"]) / 2.0))) )) +
                    0.100000*np.tanh((-1.0*(((((((((data["FLAG_EMP_PHONE"]) < ((-1.0*((data["DAYS_EMPLOYED"])))))*1.)) * (3.0))) * ((((data["FLAG_EMP_PHONE"]) < ((-1.0*((data["DAYS_EMPLOYED"])))))*1.))))))) +
                    0.090778*np.tanh(((((data["NAME_CLIENT_TYPE_Repeater"]) / 2.0)) * ((((data["FLAG_DOCUMENT_2"]) + (np.where(data["SK_ID_PREV_x"]>0, 2.0, data["HOUR_APPR_PROCESS_START_y"] )))/2.0)))) +
                    0.091403*np.tanh((((data["PRODUCT_COMBINATION_POS_household_without_interest"]) + (((data["SK_ID_BUREAU"]) * (((np.minimum((((((data["DAYS_LAST_DUE_1ST_VERSION"]) + (np.maximum(((data["SK_ID_BUREAU"])), ((data["SK_ID_BUREAU"])))))/2.0))), ((1.0)))) / 2.0)))))/2.0)) +
                    0.099844*np.tanh((((-1.0*(((((np.where(data["cc_bal_SK_DPD"]>0, data["NAME_CONTRACT_STATUS_Canceled"], np.tanh(((((data["cc_bal_SK_DPD"]) < ((-1.0*((((data["NAME_CONTRACT_STATUS_Canceled"]) / 2.0))))))*1.))) )) > (2.0))*1.))))) * 2.0)) +
                    0.100000*np.tanh((((np.maximum((((((data["SK_ID_PREV_y"]) < (((((data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"]) + (data["cc_bal_SK_DPD"]))) - ((-1.0*((-1.0)))))))*1.))), ((data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"])))) > ((-1.0*((data["FLAG_DOCUMENT_5"])))))*1.)) +
                    0.100000*np.tanh(((np.maximum(((data["EXT_SOURCE_3"])), ((np.maximum(((data["EXT_SOURCE_3"])), (((((-3.0) + (((data["EXT_SOURCE_3"]) * (data["EXT_SOURCE_3"]))))/2.0)))))))) * 2.0)) +
                    0.100000*np.tanh(((np.tanh((((((data["DAYS_CREDIT"]) + (np.minimum(((data["ty__Credit_card"])), ((np.minimum(((data["ty__Credit_card"])), ((1.0))))))))) + (np.minimum(((data["ty__Credit_card"])), ((((data["DAYS_CREDIT"]) / 2.0))))))))) / 2.0)) +
                    0.100000*np.tanh(((np.where(np.minimum((((6.0))), (((((data["EXT_SOURCE_3"]) < (-2.0))*1.))))>0, 1.0, ((data["AMT_REQ_CREDIT_BUREAU_DAY"]) * 2.0) )) - (data["YEARS_BEGINEXPLUATATION_MODE"]))) +
                    0.079601*np.tanh(((np.where(data["LIVINGAPARTMENTS_MEDI"]>0, -3.0, (((data["NAME_YIELD_GROUP_low_action"]) < (np.where(data["avg_buro_MONTHS_BALANCE"]>0, (((0.90152645111083984)) / 2.0), data["LIVINGAPARTMENTS_AVG"] )))*1.) )) * (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))) +
                    0.083665*np.tanh(np.where(data["avg_buro_MONTHS_BALANCE"]>0, data["SK_ID_BUREAU"], ((((data["SK_ID_BUREAU"]) * (-1.0))) * (data["ty__Consumer_credit"])) )) +
                    0.100000*np.tanh((((((((-2.0) / 2.0)) - (((data["te_NAME_FAMILY_STATUS"]) * (data["DAYS_CREDIT_ENDDATE"]))))) > (np.minimum(((data["DAYS_CREDIT_ENDDATE"])), ((data["NAME_GOODS_CATEGORY_Insurance"])))))*1.)) +
                    0.089723*np.tanh(np.where(np.maximum(((-1.0)), (((-1.0*((np.minimum(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])), ((((((((-1.0*((-2.0)))) < (data["SK_ID_BUREAU"]))*1.)) / 2.0))))))))))>0, data["SK_ID_BUREAU"], (-1.0*((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) )) +
                    0.099922*np.tanh(np.minimum((((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) > (np.minimum(((np.where(data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]>0, 3.0, data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"] ))), ((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"])))))*1.))), (((((((2.0) + (2.0))/2.0)) - (data["NAME_YIELD_GROUP_low_action"])))))) +
                    0.099805*np.tanh(((((((((data["cnt_FLAG_DOCUMENT_15"]) + (data["cnt_FLAG_DOCUMENT_15"]))) + (((data["cc_bal_cc_bal_status__Sent_proposal"]) * (data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]))))) * 2.0)) + (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0)))) +
                    0.099531*np.tanh((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) + ((-1.0*((((((((((data["LIVINGAPARTMENTS_MEDI"]) + (data["AMT_CREDIT_x"]))) + (data["AMT_CREDIT_x"]))) + (data["LIVINGAPARTMENTS_MEDI"]))) * (data["cc_bal_CNT_DRAWINGS_CURRENT"])))))))/2.0)) +
                    0.100000*np.tanh((((data["NAME_GOODS_CATEGORY_Insurance"]) < (np.minimum(((((data["SK_DPD"]) - (((np.where(0.0>0, data["cnt_FLAG_DOCUMENT_13"], data["NAME_CASH_LOAN_PURPOSE_Furniture"] )) * 2.0))))), ((data["cnt_FLAG_DOCUMENT_13"])))))*1.)) +
                    0.099883*np.tanh(((((((-1.0*((((data["NAME_YIELD_GROUP_high"]) / 2.0))))) - ((((np.minimum(((data["inst_SK_ID_PREV"])), (((((-2.0) > ((7.0)))*1.))))) > (((data["NAME_YIELD_GROUP_high"]) / 2.0)))*1.)))) + (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))/2.0)) +
                    0.099922*np.tanh(np.minimum((((-1.0*((np.maximum(((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * 2.0))), ((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"])))))))), ((np.maximum(((np.maximum(((((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]) * 2.0))), ((((data["PRODUCT_COMBINATION_Card_X_Sell"]) / 2.0)))))), ((data["CNT_INSTALMENT_FUTURE"]))))))) +
                    0.068308*np.tanh(((np.where(((1.0) + (np.minimum(((data["cnt_DAYS_BIRTH"])), ((3.0)))))>0, (-1.0*((((np.minimum(((data["SK_ID_PREV_y"])), ((data["SK_ID_PREV_y"])))) / 2.0)))), -2.0 )) / 2.0)) +
                    0.093630*np.tanh((((((data["DAYS_CREDIT_ENDDATE"]) > (((((np.minimum(((np.maximum(((data["DAYS_LAST_DUE_1ST_VERSION"])), ((data["FLAG_DOCUMENT_13"]))))), (((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + ((2.0)))/2.0))))) - (0.0))) * 2.0)))*1.)) / 2.0)) +
                    0.090660*np.tanh(((data["DAYS_FIRST_DUE"]) * ((((data["NAME_CLIENT_TYPE_Repeater"]) < ((((-1.0) + (data["CODE_REJECT_REASON_VERIF"]))/2.0)))*1.)))) +
                    0.099101*np.tanh(((((((data["NAME_CONTRACT_STATUS_Approved"]) < (data["NAME_CONTRACT_STATUS_Approved"]))*1.)) + ((((-1.0*((data["inst_AMT_PAYMENT"])))) * (np.maximum(((data["NAME_CONTRACT_STATUS_Approved"])), (((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))))))))/2.0)) +
                    0.099922*np.tanh(np.maximum(((data["cu__currency_3"])), (((((((data["CNT_INSTALMENT_FUTURE"]) + (((data["cu__currency_3"]) - (np.minimum((((((5.11337184906005859)) - (data["PRODUCT_COMBINATION_Cash_Street__high"])))), ((data["cu__currency_3"])))))))) > (3.0))*1.))))) +
                    0.099922*np.tanh(np.minimum(((((data["ORGANIZATION_TYPE"]) * (data["NAME_GOODS_CATEGORY_Gardening"])))), ((np.maximum(((data["NAME_CONTRACT_STATUS_Approved"])), (((((data["ORGANIZATION_TYPE"]) + (0.0))/2.0)))))))) +
                    0.099844*np.tanh((-1.0*(((((((data["FLAG_DOCUMENT_13"]) + ((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + ((((-1.0*((((data["SK_ID_PREV_x"]) - (((np.tanh((0.0))) * 2.0))))))) + (data["ORGANIZATION_TYPE"]))))/2.0)))/2.0)) / 2.0))))) +
                    0.099922*np.tanh(((np.minimum(((np.maximum(((((np.maximum(((data["NONLIVINGAPARTMENTS_MODE"])), ((((data["NONLIVINGAPARTMENTS_MODE"]) * (((data["FLAG_CONT_MOBILE"]) * 2.0))))))) + (-2.0)))), ((data["cnt_FLAG_DOCUMENT_17"]))))), ((2.0)))) * 2.0)) +
                    0.099609*np.tanh(((np.maximum(((((-3.0) / 2.0))), (((4.0))))) * (((np.tanh(((10.0)))) * (((((((data["PRODUCT_COMBINATION_Card_Street"]) * (((data["cc_bal_SK_DPD_DEF"]) * 2.0)))) * 2.0)) * 2.0)))))) +
                    0.098867*np.tanh(((np.where(-1.0>0, data["DAYS_ID_PUBLISH"], ((data["SK_ID_PREV_y"]) * ((-1.0*((np.tanh(((((((data["DAYS_ID_PUBLISH"]) > ((((data["CODE_REJECT_REASON_VERIF"]) + (0.0))/2.0)))*1.)) * 2.0)))))))) )) / 2.0)) +
                    0.099609*np.tanh(np.minimum(((((((3.0) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * (data["NAME_CLIENT_TYPE_New"])))), (((((((((np.maximum(((data["FLAG_DOCUMENT_2"])), ((-3.0)))) / 2.0)) < (data["SK_DPD"]))*1.)) * (data["NAME_CLIENT_TYPE_New"])))))) +
                    0.099844*np.tanh(((data["MONTHS_BALANCE"]) * ((((np.where(data["SK_DPD"]>0, 3.0, ((data["SK_DPD"]) * 2.0) )) + (np.where(data["MONTHS_BALANCE"]>0, data["SK_DPD"], data["inst_NUM_INSTALMENT_VERSION"] )))/2.0)))) +
                    0.097655*np.tanh(((((-1.0*(((((1.0) < (data["RATE_DOWN_PAYMENT"]))*1.))))) + (((data["Demand"]) - (data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))))/2.0)) +
                    0.100000*np.tanh(np.where(data["SK_DPD"]>0, data["FLAG_DOCUMENT_16"], (((data["cnt_FLAG_DOCUMENT_16"]) + (np.where(data["te_FLAG_EMAIL"]>0, data["cnt_FLAG_DOCUMENT_16"], np.tanh((np.tanh((data["cc_bal_SK_ID_PREV"])))) )))/2.0) )) +
                    0.099648*np.tanh((((((data["SK_DPD"]) + (((np.where(data["SK_DPD"]>0, data["FLAG_DOCUMENT_16"], (-1.0*(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["FLAG_DOCUMENT_16"]))/2.0)))) )) * 2.0)))/2.0)) / 2.0)) +
                    0.099140*np.tanh(((np.where(data["te_WALLSMATERIAL_MODE"]>0, np.minimum(((data["cc_bal_SK_DPD"])), ((np.minimum(((data["cnt_FLAG_DOCUMENT_16"])), ((data["SK_DPD"])))))), data["cnt_FLAG_DOCUMENT_16"] )) - ((((0.13569834828376770)) * (data["te_FLAG_EMAIL"]))))) +
                    0.099961*np.tanh(((((-1.0*((data["SK_DPD"])))) > ((((data["SK_DPD"]) < (((((((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)) * 2.0)) * 2.0)))*1.)))*1.)) +
                    0.099726*np.tanh(((np.minimum(((((np.tanh((data["NAME_CONTRACT_STATUS_Approved"]))) / 2.0))), ((((data["AMT_CREDIT_x"]) / 2.0))))) / 2.0)) +
                    0.100000*np.tanh(np.where((((data["AMT_ANNUITY_x"]) + ((((data["AMT_ANNUITY_x"]) > (((data["AMT_GOODS_PRICE_x"]) * 2.0)))*1.)))/2.0)>0, np.tanh((data["te_NAME_CONTRACT_TYPE"])), data["NAME_CONTRACT_TYPE"] )) +
                    0.084760*np.tanh((-1.0*((((data["AMT_GOODS_PRICE_x"]) * (np.maximum(((np.maximum(((np.tanh((np.tanh((data["AMT_GOODS_PRICE_x"])))))), ((np.where(data["AMT_GOODS_PRICE_x"]>0, data["NAME_GOODS_CATEGORY_Insurance"], -3.0 )))))), ((((data["CNT_INSTALMENT_FUTURE"]) * 2.0)))))))))) +
                    0.099805*np.tanh(((((((np.maximum(((data["FLAG_DOCUMENT_2"])), ((data["Active"])))) * ((-1.0*((data["CNT_INSTALMENT"])))))) + (np.where(data["Active"]>0, (-1.0*((data["FLAG_DOCUMENT_2"]))), data["AMT_CREDIT_x"] )))) / 2.0)) +
                    0.099531*np.tanh(np.where(data["SK_DPD"]>0, data["CNT_INSTALMENT_FUTURE"], np.maximum(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) - (((data["AMT_ANNUITY_x"]) / 2.0)))) / 2.0))), (((((data["SK_DPD"]) > (((3.0) - (data["CNT_INSTALMENT_FUTURE"]))))*1.)))) )) +
                    0.099805*np.tanh(((((((data["AMT_CREDIT_x"]) * ((((1.0) + (data["LIVINGAPARTMENTS_MEDI"]))/2.0)))) * ((((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["AMT_CREDIT_x"], np.tanh((2.0)) )) + (data["LIVINGAPARTMENTS_MEDI"]))/2.0)))) / 2.0)) +
                    0.100000*np.tanh(((((((data["CHANNEL_TYPE_AP___Cash_loan_"]) > (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["AMT_GOODS_PRICE_x"]))))*1.)) + ((((-1.0) + ((((2.0) < (((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0)))*1.)))/2.0)))/2.0)) +
                    0.082141*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, np.minimum(((3.0)), ((np.tanh((data["SK_DPD"]))))), np.minimum(((np.maximum(((((data["WEEKDAY_APPR_PROCESS_START_WEDNESDAY"]) * 2.0))), ((data["FLAG_DOCUMENT_2"]))))), ((((data["AMT_CREDIT_x"]) / 2.0)))) )) +
                    0.099844*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["FLAG_DOCUMENT_18"], (-1.0*((np.maximum(((data["FLAG_DOCUMENT_18"])), ((np.tanh((((data["AMT_ANNUITY_x"]) - ((((2.0) > (1.0))*1.))))))))))) )) +
                    0.099766*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["FLAG_DOCUMENT_18"], ((((data["te_FLAG_DOCUMENT_18"]) + (np.minimum(((((3.0) * 2.0))), ((((data["LIVINGAPARTMENTS_MEDI"]) / 2.0))))))) / 2.0) )) +
                    0.096952*np.tanh(np.minimum((((-1.0*((data["NONLIVINGAREA_MODE"]))))), (((-1.0*((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["FLOORSMAX_MEDI"], data["FLAG_DOCUMENT_18"] )))))))) +
                    0.099844*np.tanh(((((((4.0)) < (((((data["SK_DPD"]) * 2.0)) - (((((data["Active"]) * (((data["SK_DPD"]) / 2.0)))) + (np.minimum(((data["Active"])), ((data["Active"])))))))))*1.)) * 2.0)) +
                    0.099922*np.tanh((-1.0*((((np.minimum(((data["cnt_FLAG_DOCUMENT_2"])), (((((8.0)) - (np.maximum(((np.maximum(((data["SK_DPD"])), ((data["cc_bal_SK_DPD"]))))), (((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["SK_DPD"]))/2.0)))))))))) / 2.0))))) +
                    0.099766*np.tanh((-1.0*((np.maximum(((data["NAME_SELLER_INDUSTRY_MLM_partners"])), (((((data["NAME_GOODS_CATEGORY_Photo___Cinema_Equipment"]) > ((((1.0) > (((data["AMT_CREDIT_x"]) * (data["NAME_GOODS_CATEGORY_Photo___Cinema_Equipment"]))))*1.)))*1.)))))))) +
                    0.100000*np.tanh((((((np.maximum((((1.12741851806640625))), ((data["AMT_CREDIT_x"])))) / 2.0)) < (((data["AMT_CREDIT_x"]) * (data["OWN_CAR_AGE"]))))*1.)) +
                    0.099922*np.tanh(np.minimum((((-1.0*((((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]) - ((-1.0*((((data["CODE_REJECT_REASON_VERIF"]) - ((8.51573276519775391))))))))))))), ((np.minimum(((data["cnt_FLAG_DOCUMENT_17"])), ((data["FLAG_DOCUMENT_17"]))))))) +
                    0.099492*np.tanh(((((data["avg_buro_buro_bal_status_5"]) + (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Car_repairs"])), ((((np.where(data["cnt_NAME_EDUCATION_TYPE"]>0, (-1.0*((np.tanh((0.0))))), data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"] )) / 2.0))))))) - (data["FLAG_DOCUMENT_15"]))) +
                    0.082376*np.tanh(np.where(data["avg_buro_buro_bal_status_2"]>0, data["NAME_GOODS_CATEGORY_Insurance"], ((data["AMT_CREDIT_x"]) - ((((data["avg_buro_buro_count"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0))) )) +
                    0.099922*np.tanh(((((data["AMT_ANNUITY_x"]) * (np.minimum(((((((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) * 2.0)) * (-2.0)))), (((-1.0*(((((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) < (((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) * 2.0)))*1.)))))))))) / 2.0)) +
                    0.077804*np.tanh(np.tanh((np.tanh((np.where(data["AMT_GOODS_PRICE_x"]>0, ((-1.0) + (data["AMT_ANNUITY_x"])), ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["AMT_GOODS_PRICE_x"]))) * ((((((1.0)) * 2.0)) * 2.0))) )))))) +
                    0.099726*np.tanh((-1.0*((np.tanh((((data["inst_AMT_PAYMENT"]) * ((((np.maximum(((data["AMT_GOODS_PRICE_x"])), ((-1.0)))) < (data["AMT_ANNUITY_x"]))*1.))))))))) +
                    0.100000*np.tanh(((np.where(data["CNT_INSTALMENT_FUTURE"]>0, (((data["AMT_ANNUITY_x"]) < (((data["inst_AMT_INSTALMENT"]) - ((-1.0*((np.where(0.0>0, (2.0), ((-2.0) / 2.0) ))))))))*1.), data["inst_AMT_INSTALMENT"] )) / 2.0)) +
                    0.100000*np.tanh(((((((data["avg_buro_buro_bal_status_C"]) + ((((data["inst_NUM_INSTALMENT_VERSION"]) < (((((((((((data["inst_AMT_PAYMENT"]) / 2.0)) + (data["AMT_ANNUITY_x"]))/2.0)) + (data["NAME_SELLER_INDUSTRY_Connectivity"]))/2.0)) * (3.0))))*1.)))/2.0)) > (1.0))*1.)) +
                    0.099648*np.tanh(((((data["inst_AMT_PAYMENT"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * (np.maximum(((((data["AMT_ANNUITY_x"]) * 2.0))), ((data["inst_AMT_PAYMENT"])))))) +
                    0.099844*np.tanh((((((((np.where(0.0>0, 0.0, np.minimum((((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + (-2.0))/2.0))), ((data["AMT_ANNUITY_x"]))) )) - (1.0))) - (data["AMT_ANNUITY_x"]))) > (-1.0))*1.)) +
                    0.099844*np.tanh(((np.minimum((((-1.0*((data["AMT_CREDIT_x"]))))), ((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((data["AMT_CREDIT_x"]) * (data["inst_DAYS_INSTALMENT"]))))))))) * (data["inst_DAYS_INSTALMENT"]))) +
                    0.099336*np.tanh(np.where((-1.0*(((((-2.0) + (data["cc_bal_AMT_BALANCE"]))/2.0))))>0, ((np.minimum(((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"])), (((-1.0*((data["cc_bal_AMT_BALANCE"]))))))) / 2.0), (11.17006969451904297) )) +
                    0.098046*np.tanh((((data["NAME_YIELD_GROUP_XNA"]) > (((((((data["AMT_GOODS_PRICE_x"]) + ((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) + ((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + (data["DAYS_FIRST_DRAWING"]))/2.0)))/2.0)))/2.0)) + ((2.82667350769042969)))/2.0)))*1.)) +
                    0.099922*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Medicine"]>0, 3.0, ((((np.where(data["NAME_GOODS_CATEGORY_Medicine"]>0, (3.0), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (data["NAME_CONTRACT_STATUS_Canceled"])) )) * (data["NAME_CONTRACT_STATUS_Canceled"]))) * (data["NAME_CONTRACT_STATUS_Canceled"])) )) +
                    0.099844*np.tanh(((np.tanh((data["avg_buro_buro_bal_status_2"]))) + (((((-2.0) - (data["NAME_CONTRACT_STATUS_Refused"]))) * ((((np.tanh((data["cnt_FLAG_DOCUMENT_2"]))) < (data["avg_buro_buro_bal_status_2"]))*1.)))))) +
                    0.086166*np.tanh(((np.where((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["AMT_ANNUITY_x"]))) > (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))*1.)>0, -1.0, data["FLAG_DOCUMENT_2"] )) * ((((((data["AMT_ANNUITY_x"]) + ((-1.0*((2.0)))))/2.0)) / 2.0)))) +
                    0.097265*np.tanh(np.where(-2.0>0, data["NAME_GOODS_CATEGORY_Insurance"], ((np.minimum(((np.tanh((data["cnt_CNT_CHILDREN"])))), ((np.minimum(((np.tanh((data["cnt_FLAG_OWN_CAR"])))), ((3.0))))))) * (np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["AMT_GOODS_PRICE_x"]))))) )) +
                    0.098632*np.tanh(np.where(data["DAYS_LAST_DUE_1ST_VERSION"]>0, ((((data["cnt_FLAG_DOCUMENT_2"]) - (data["NAME_CONTRACT_STATUS_Refused"]))) / 2.0), data["NAME_GOODS_CATEGORY_Insurance"] )) +
                    0.099961*np.tanh((((((((-3.0) > ((((((np.minimum(((data["NAME_CONTRACT_TYPE_Revolving_loans"])), (((((data["AMT_CREDIT_y"]) > (3.0))*1.))))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) - (data["AMT_CREDIT_y"]))))*1.)) * 2.0)) * 2.0)) +
                    0.099726*np.tanh((((-1.0*(((((data["DAYS_LAST_DUE_1ST_VERSION"]) > (3.0))*1.))))) - ((-1.0*(((-1.0*((((np.minimum(((((data["DAYS_LAST_DUE_1ST_VERSION"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"])))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) * (data["NAME_SELLER_INDUSTRY_Connectivity"]))))))))))) +
                    0.099726*np.tanh(((data["NAME_GOODS_CATEGORY_Insurance"]) - (np.maximum(((np.tanh((((((data["AMT_ANNUITY_x"]) - (data["cc_bal_CNT_DRAWINGS_CURRENT"]))) + (-2.0)))))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"])))))) +
                    0.093748*np.tanh(((((((data["AMT_REQ_CREDIT_BUREAU_QRT"]) / 2.0)) - (((data["AMT_CREDIT_x"]) * 2.0)))) * (((data["AMT_REQ_CREDIT_BUREAU_QRT"]) / 2.0)))) +
                    0.099492*np.tanh((((((np.tanh((((-2.0) * ((((((data["AMT_GOODS_PRICE_x"]) + (((data["AMT_GOODS_PRICE_x"]) * 2.0)))/2.0)) * (((data["AMT_ANNUITY_x"]) / 2.0)))))))) + (data["cnt_FLAG_WORK_PHONE"]))/2.0)) * (data["AMT_GOODS_PRICE_x"]))) +
                    0.099922*np.tanh(((((((-1.0*((np.tanh(((-1.0*((((((((np.tanh((data["cnt_FLAG_DOCUMENT_2"]))) > ((10.0)))*1.)) + ((-1.0*((data["AMT_CREDIT_x"])))))/2.0)))))))))) > (data["te_NAME_EDUCATION_TYPE"]))*1.)) * (data["AMT_CREDIT_x"]))) +
                    0.099531*np.tanh(((((((-1.0*((data["FLAG_DOCUMENT_18"])))) + (np.minimum(((data["NAME_EDUCATION_TYPE"])), (((-1.0*(((((np.where(-1.0>0, -3.0, data["FLAG_DOCUMENT_14"] )) + (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))/2.0)))))))))/2.0)) / 2.0)) +
                    0.087143*np.tanh(((((np.where(data["CODE_GENDER"]>0, data["FLAG_DOCUMENT_11"], 2.0 )) - (data["CODE_GENDER"]))) * (data["cnt_FLAG_DOCUMENT_11"]))) +
                    0.099844*np.tanh(np.where(data["cnt_FLAG_DOCUMENT_17"]>0, (((data["SK_DPD"]) > (((((np.tanh((data["NAME_GOODS_CATEGORY_Insurance"]))) - ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)))) + ((13.44596672058105469)))))*1.), data["cnt_FLAG_DOCUMENT_17"] )) +
                    0.099922*np.tanh((((((data["cu__currency_3"]) + ((((0.0) > (np.maximum(((np.tanh((data["AMT_DOWN_PAYMENT"])))), (((-1.0*((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]))))))))*1.)))/2.0)) - (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (-3.0))))) +
                    0.099609*np.tanh(np.tanh((((np.where((((data["AMT_REQ_CREDIT_BUREAU_YEAR"]) + (-2.0))/2.0)>0, data["CNT_INSTALMENT"], np.minimum(((data["cc_bal_CNT_DRAWINGS_CURRENT"])), ((data["NAME_GOODS_CATEGORY_Insurance"]))) )) / 2.0)))) +
                    0.099883*np.tanh(((np.maximum((((-1.0*((((data["inst_AMT_PAYMENT"]) / 2.0)))))), ((np.where(np.where(0.0>0, -3.0, data["PRODUCT_COMBINATION_Cash_X_Sell__low"] )>0, -3.0, ((data["inst_AMT_PAYMENT"]) / 2.0) ))))) / 2.0)) +
                    0.077648*np.tanh((-1.0*(((((((2.0) < (((np.maximum(((np.maximum((((((((-1.0) > (data["cnt_FLAG_DOCUMENT_2"]))*1.)) / 2.0))), ((data["AMT_REQ_CREDIT_BUREAU_YEAR"]))))), ((data["inst_AMT_PAYMENT"])))) * 2.0)))*1.)) / 2.0))))) +
                    0.089879*np.tanh((-1.0*(((((((data["AMT_DOWN_PAYMENT"]) + (np.tanh(((((((((data["AMT_DOWN_PAYMENT"]) * (data["OCCUPATION_TYPE"]))) + (data["NAME_CLIENT_TYPE_Refreshed"]))/2.0)) * 2.0)))))/2.0)) * (data["OCCUPATION_TYPE"])))))) +
                    0.099922*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["FLAG_DOCUMENT_6"], (((data["FLAG_DOCUMENT_2"]) + (np.where(data["NAME_CLIENT_TYPE_Refreshed"]>0, (-1.0*((data["te_FLAG_DOCUMENT_6"]))), 0.0 )))/2.0) )) +
                    0.099648*np.tanh((((((data["AMT_DOWN_PAYMENT"]) + (((data["AMT_DOWN_PAYMENT"]) * (((data["SK_ID_PREV_x"]) / 2.0)))))) > ((((((data["AMT_DOWN_PAYMENT"]) + (data["FLAG_DOCUMENT_2"]))) > ((((0.0)) * 2.0)))*1.)))*1.)) +
                    0.100000*np.tanh((((-1.0*((np.where(((((data["cnt_FLAG_DOCUMENT_2"]) / 2.0)) + (data["ca__Bad_debt"]))>0, data["te_OCCUPATION_TYPE"], (((data["te_OCCUPATION_TYPE"]) < ((-1.0*(((-1.0*((data["te_OCCUPATION_TYPE"]))))))))*1.) ))))) / 2.0)) +
                    0.093005*np.tanh((((data["cnt_FLAG_DOCUMENT_2"]) < (((data["NAME_GOODS_CATEGORY_Insurance"]) * (((data["cnt_FLAG_DOCUMENT_2"]) + (np.where(((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) - (-1.0))>0, data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"], np.tanh((data["cnt_FLAG_DOCUMENT_2"])) )))))))*1.)) +
                    0.099375*np.tanh(((np.where(data["cc_bal_CNT_DRAWINGS_CURRENT"]>0, (((-1.0*(((((data["NAME_GOODS_CATEGORY_Fitness"]) < (data["cc_bal_CNT_DRAWINGS_CURRENT"]))*1.))))) / 2.0), data["cc_bal_CNT_DRAWINGS_CURRENT"] )) / 2.0)) +
                    0.099844*np.tanh(((((((data["cc_bal_AMT_BALANCE"]) > (data["cc_bal_AMT_TOTAL_RECEIVABLE"]))*1.)) + (np.minimum(((((((data["NAME_GOODS_CATEGORY_Insurance"]) - (((data["NAME_GOODS_CATEGORY_Gardening"]) / 2.0)))) / 2.0))), ((data["cc_bal_AMT_TOTAL_RECEIVABLE"])))))/2.0)) +
                    0.098515*np.tanh((((((((np.maximum(((3.0)), ((((data["DAYS_FIRST_DUE"]) * (data["NAME_GOODS_CATEGORY_Fitness"])))))) < (((data["AMT_CREDIT_x"]) * 2.0)))*1.)) - ((((data["NAME_GOODS_CATEGORY_Weapon"]) + (data["NAME_GOODS_CATEGORY_Fitness"]))/2.0)))) / 2.0)) +
                    0.099922*np.tanh((-1.0*(((((data["NAME_SELLER_INDUSTRY_MLM_partners"]) + ((((1.0) < (((data["AMT_GOODS_PRICE_x"]) - ((((((-1.0*((np.tanh((data["AMT_GOODS_PRICE_x"])))))) / 2.0)) * ((-1.0*((-3.0)))))))))*1.)))/2.0))))) +
                    0.099140*np.tanh(((((((((data["AMT_CREDIT_x"]) + (((((data["AMT_CREDIT_x"]) / 2.0)) / 2.0)))/2.0)) > ((((data["ca__Bad_debt"]) + (np.tanh(((8.0)))))/2.0)))*1.)) / 2.0)) +
                    0.099218*np.tanh(((((np.where(np.where(data["SK_ID_PREV_x"]>0, data["cnt_FLAG_CONT_MOBILE"], ((data["DAYS_FIRST_DUE"]) * (data["cnt_FLAG_CONT_MOBILE"])) )>0, data["cnt_FLAG_CONT_MOBILE"], (-1.0*((data["FLAG_CONT_MOBILE"]))) )) / 2.0)) * 2.0)) +
                    0.099883*np.tanh((-1.0*((np.where(data["cc_bal_cc_bal_status__Sent_proposal"]>0, np.maximum(((np.maximum((((((2.0) < (data["AMT_GOODS_PRICE_x"]))*1.))), (((-1.0*((data["FLAG_DOCUMENT_2"])))))))), ((data["cc_bal_cc_bal_status__Sent_proposal"]))), ((data["FLAG_DOCUMENT_2"]) * 2.0) ))))) +
                    0.099922*np.tanh((((6.36812734603881836)) * (((((np.maximum(((data["AMT_CREDIT_MAX_OVERDUE"])), ((((((((data["AMT_CREDIT_MAX_OVERDUE"]) * 2.0)) * ((14.96181964874267578)))) * 2.0))))) * 2.0)) * 2.0)))) +
                    0.060571*np.tanh(np.where(data["WALLSMATERIAL_MODE"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], (((((1.0) + (np.tanh((np.tanh((data["cnt_NAME_INCOME_TYPE"]))))))) + (data["WALLSMATERIAL_MODE"]))/2.0) )) +
                    0.099844*np.tanh(np.tanh((((((((data["NAME_INCOME_TYPE"]) < (1.0))*1.)) < (np.where(data["NAME_INCOME_TYPE"]>0, data["AMT_CREDIT_x"], np.minimum(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0))), (((((-1.0*((0.0)))) / 2.0)))) )))*1.)))) +
                    0.099922*np.tanh((((-1.0) + (((((((data["AMT_GOODS_PRICE_x"]) > (np.tanh((((((-3.0) * (-3.0))) * (data["cnt_FLAG_DOCUMENT_2"]))))))*1.)) > (data["AMT_GOODS_PRICE_x"]))*1.)))/2.0)) +
                    0.099844*np.tanh((((data["AMT_CREDIT_x"]) > (np.where(np.maximum(((data["FLAG_DOCUMENT_2"])), (((((data["AMT_CREDIT_x"]) + (data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]))/2.0))))>0, 3.0, data["cnt_FLAG_DOCUMENT_2"] )))*1.)) +
                    0.099883*np.tanh(np.tanh((np.where((((-3.0) > (((data["SK_ID_BUREAU"]) * (data["AMT_CREDIT_x"]))))*1.)>0, data["AMT_CREDIT_x"], ((((data["SK_ID_BUREAU"]) * (data["AMT_CREDIT_x"]))) / 2.0) )))) +
                    0.099766*np.tanh(((np.minimum(((((((np.tanh((1.0))) - (data["avg_buro_buro_bal_status_1"]))) - (data["avg_buro_buro_bal_status_1"])))), ((data["avg_buro_buro_bal_status_1"])))) * (((data["SK_ID_BUREAU"]) / 2.0)))) +
                    0.099844*np.tanh(((((((np.where((((2.0) < (data["SK_ID_BUREAU"]))*1.)>0, (10.58298397064208984), data["NAME_GOODS_CATEGORY_Insurance"] )) * 2.0)) * 2.0)) * 2.0)) +
                    0.099922*np.tanh(((((((np.minimum(((data["cc_bal_cc_bal_status__Refused"])), (((-1.0*((data["avg_buro_buro_bal_status_1"]))))))) - (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, -1.0, data["avg_buro_buro_bal_status_1"] )))) * (data["avg_buro_buro_bal_status_1"]))) + (data["cc_bal_cc_bal_status__Refused"]))) +
                    0.099492*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["te_FLAG_DOCUMENT_3"], ((((data["te_FLAG_DOCUMENT_3"]) + (data["te_FLAG_DOCUMENT_3"]))) * ((-1.0*((np.where((((2.0) > (data["AMT_CREDIT_x"]))*1.)>0, 0.0, (13.56991291046142578) )))))) )) +
                    0.098828*np.tanh((((1.0) < ((((((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, np.where(data["AMT_CREDIT_x"]>0, data["cnt_FLAG_DOCUMENT_2"], data["NAME_GOODS_CATEGORY_Direct_Sales"] ), ((data["AMT_CREDIT_x"]) / 2.0) )) / 2.0)) + (data["AMT_CREDIT_x"]))/2.0)))*1.)) +
                    0.099805*np.tanh((-1.0*((np.where(((((data["AMT_REQ_CREDIT_BUREAU_MON"]) * 2.0)) * (data["CHANNEL_TYPE_Car_dealer"]))>0, (((((data["AMT_GOODS_PRICE_x"]) / 2.0)) + (data["CHANNEL_TYPE_Car_dealer"]))/2.0), ((1.0) * (0.0)) ))))) +
                    0.081594*np.tanh(np.where(np.where(data["cnt_ORGANIZATION_TYPE"]>0, (4.0), data["FLAG_DOCUMENT_3"] )>0, np.tanh(((((np.tanh((((data["AMT_CREDIT_x"]) / 2.0)))) + ((-1.0*((data["ty__Another_type_of_loan"])))))/2.0))), data["FLAG_DOCUMENT_3"] )) +
                    0.100000*np.tanh(np.where(data["AMT_GOODS_PRICE_x"]>0, ((data["te_NAME_CONTRACT_TYPE"]) / 2.0), (-1.0*(((((((data["te_NAME_CONTRACT_TYPE"]) < (-1.0))*1.)) * (data["AMT_GOODS_PRICE_x"]))))) )) +
                    0.092380*np.tanh(((data["cnt_FLAG_DOCUMENT_8"]) - (((((1.0) * ((((data["AMT_GOODS_PRICE_x"]) > (2.0))*1.)))) + (data["te_FLAG_DOCUMENT_8"]))))) +
                    0.094920*np.tanh(((((((((data["te_FLAG_DOCUMENT_2"]) < (0.0))*1.)) > (data["cnt_ORGANIZATION_TYPE"]))*1.)) - ((((data["cnt_ORGANIZATION_TYPE"]) < (np.maximum(((-2.0)), (((-1.0*((data["te_FLAG_DOCUMENT_17"]))))))))*1.)))) +
                    0.100000*np.tanh(np.where((-1.0*((data["BASEMENTAREA_MEDI"])))>0, data["FLAG_DOCUMENT_8"], np.tanh(((((((data["BASEMENTAREA_MEDI"]) + (data["FLAG_DOCUMENT_8"]))/2.0)) * (data["cnt_REG_REGION_NOT_WORK_REGION"])))) )) +
                    0.099648*np.tanh((((((3.0) < (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, ((data["te_REG_REGION_NOT_LIVE_REGION"]) * 2.0), 2.0 )))*1.)) - ((((((data["te_WALLSMATERIAL_MODE"]) > ((((3.0) > (data["cnt_EMERGENCYSTATE_MODE"]))*1.)))*1.)) * 2.0)))) +
                    0.099844*np.tanh((((-1.0*(((((np.minimum(((data["cnt_FLAG_DOCUMENT_2"])), ((data["FLOORSMAX_AVG"])))) + ((-1.0*((data["FLOORSMAX_AVG"])))))/2.0))))) - ((((data["FLOORSMAX_MEDI"]) > (((2.0) / 2.0)))*1.)))) +
                    0.099453*np.tanh(((((((((((-1.0*((((data["FLOORSMAX_AVG"]) / 2.0))))) / 2.0)) > ((((((data["FLOORSMAX_MEDI"]) / 2.0)) < (((data["FLOORSMAX_MODE"]) / 2.0)))*1.)))*1.)) - (((data["FLOORSMAX_MODE"]) / 2.0)))) / 2.0)) +
                    0.099492*np.tanh((-1.0*(((((data["te_REG_REGION_NOT_LIVE_REGION"]) > (((((np.maximum(((data["te_REG_REGION_NOT_LIVE_REGION"])), ((1.0)))) + ((-1.0*((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((1.0) / 2.0))))))))) + (data["AMT_CREDIT_x"]))))*1.))))) +
                    0.079406*np.tanh(np.where(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * ((((data["AMT_REQ_CREDIT_BUREAU_MON"]) + (data["cnt_FLAG_DOCUMENT_5"]))/2.0)))) * 2.0)>0, (((((data["cnt_FLAG_DOCUMENT_5"]) < (data["AMT_REQ_CREDIT_BUREAU_MON"]))*1.)) * ((1.60125052928924561))), ((data["cnt_NAME_TYPE_SUITE"]) / 2.0) )) +
                    0.099726*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["AMT_REQ_CREDIT_BUREAU_MON"], ((np.minimum((((9.0))), (((-1.0*((((((((14.71726322174072266)) - (data["AMT_REQ_CREDIT_BUREAU_MON"]))) < (data["AMT_REQ_CREDIT_BUREAU_MON"]))*1.)))))))) * 2.0) )) +
                    0.099766*np.tanh(np.maximum(((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) > ((((((-1.0) / 2.0)) < (data["AMT_GOODS_PRICE_x"]))*1.)))*1.))), ((((data["cnt_FLAG_DOCUMENT_2"]) * 2.0))))) +
                    0.099531*np.tanh(np.where(data["cnt_FLAG_DOCUMENT_17"]>0, ((((((((data["cnt_FLAG_DOCUMENT_17"]) / 2.0)) - (data["NAME_GOODS_CATEGORY_Weapon"]))) * (data["FLAG_DOCUMENT_3"]))) * 2.0), data["cnt_FLAG_DOCUMENT_17"] )) +
                    0.096991*np.tanh(np.minimum(((((((-1.0*((((((((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) / 2.0)) / 2.0)) * 2.0))))) + (((data["AMT_GOODS_PRICE_x"]) * (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))))/2.0))), (((((4.0)) * 2.0))))) +
                    0.099414*np.tanh(np.maximum(((data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"])), (((((((((8.98015975952148438)) * (((((((3.0)) + (np.minimum(((((data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]) / 2.0))), ((3.0)))))) < (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))*1.)))) * 2.0)) * 2.0))))) +
                    0.099179*np.tanh(((((np.maximum(((data["FLAG_DOCUMENT_2"])), (((((data["NAME_TYPE_SUITE"]) > (3.0))*1.))))) * (3.0))) - (((data["CHANNEL_TYPE_Car_dealer"]) / 2.0)))) +
                    0.092927*np.tanh(((np.where(data["cc_bal_cc_bal_status__Refused"]>0, data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"], ((data["NONLIVINGAPARTMENTS_MODE"]) - (data["cnt_FLAG_DOCUMENT_2"])) )) * 2.0)) +
                    0.099570*np.tanh(((((np.where(data["cc_bal_cc_bal_status__Refused"]>0, (((-1.0*((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) / 2.0), ((np.tanh((np.tanh((((data["PRODUCT_COMBINATION_Cash"]) * 2.0)))))) / 2.0) )) - (data["NAME_GOODS_CATEGORY_Weapon"]))) - ((0.0)))) +
                    0.099609*np.tanh((((data["NAME_CONTRACT_STATUS_Approved"]) > (((((-1.0*((data["FLAG_DOCUMENT_10"])))) > (np.where(data["PRODUCT_COMBINATION_Cash_Street__high"]>0, (((data["NAME_CONTRACT_STATUS_Approved"]) + (data["NAME_CONTRACT_STATUS_Canceled"]))/2.0), -1.0 )))*1.)))*1.)) +
                    0.099218*np.tanh(np.tanh((((data["cc_bal_cc_bal_status__Refused"]) - ((((np.where(data["cc_bal_cc_bal_status__Refused"]>0, ((data["AMT_DOWN_PAYMENT"]) - (-2.0)), data["NAME_CONTRACT_STATUS_Canceled"] )) < (data["NAME_CONTRACT_STATUS_Canceled"]))*1.)))))) +
                    0.083548*np.tanh(np.maximum(((((((np.where(data["NAME_CASH_LOAN_PURPOSE_Furniture"]>0, data["Returned_to_the_store"], np.tanh((-3.0)) )) * 2.0)) + (data["avg_buro_buro_count"])))), ((((((((1.0) + (data["avg_buro_buro_count"]))/2.0)) < (0.0))*1.))))) +
                    0.099805*np.tanh(((((np.where((-1.0*((data["EXT_SOURCE_3"])))>0, (((((3.0) + (data["cnt_NAME_EDUCATION_TYPE"]))/2.0)) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"])), (-1.0*((((np.tanh((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))) * 2.0)))) )) * 2.0)) * 2.0)) +
                    0.099922*np.tanh((((-1.0*(((((data["NAME_GOODS_CATEGORY_Insurance"]) > ((((data["EXT_SOURCE_3"]) < (data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]))*1.)))*1.))))) + ((((data["EXT_SOURCE_3"]) < (-2.0))*1.)))) +
                    0.089136*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Furniture"]>0, data["AMT_REQ_CREDIT_BUREAU_MON"], ((data["AMT_REQ_CREDIT_BUREAU_WEEK"]) * (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"])), ((((((np.tanh((-3.0))) + (data["AMT_REQ_CREDIT_BUREAU_MON"]))) * 2.0)))))) )) +
                    0.099766*np.tanh(np.tanh((((np.where(data["avg_buro_MONTHS_BALANCE"]>0, (((np.minimum(((-2.0)), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) + (1.0))/2.0), ((data["NAME_GOODS_CATEGORY_Insurance"]) + ((((0.0) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.))) )) / 2.0)))) +
                    0.099453*np.tanh(((data["Returned_to_the_store"]) * ((((((data["Returned_to_the_store"]) > (np.minimum(((data["ty__Credit_card"])), ((((data["NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer"]) * (data["DAYS_REGISTRATION"])))))))*1.)) - (data["DAYS_REGISTRATION"]))))) +
                    0.100000*np.tanh((((-1.0*(((((data["cc_bal_AMT_DRAWINGS_POS_CURRENT"]) + (((((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (-1.0))))/2.0)) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)))/2.0))))) / 2.0)) +
                    0.097968*np.tanh(np.minimum((((-1.0*((data["NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer"]))))), (((((((((np.minimum(((data["NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer"])), ((2.0)))) + (data["PRODUCT_COMBINATION_Cash"]))/2.0)) / 2.0)) / 2.0))))) +
                    0.099336*np.tanh((((((-1.0*((np.minimum(((((data["NAME_GOODS_CATEGORY_Education"]) * 2.0))), ((data["NAME_GOODS_CATEGORY_Education"]))))))) + ((((((((data["CNT_INSTALMENT"]) + (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))/2.0)) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * 2.0)))) * 2.0)) +
                    0.097694*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Fitness"]>0, (((data["te_REG_CITY_NOT_WORK_CITY"]) > (1.0))*1.), ((((2.80854535102844238)) < (((np.maximum(((1.0)), ((data["NAME_GOODS_CATEGORY_Insurance"])))) * (np.maximum(((data["AMT_CREDIT_y"])), ((data["FLAG_DOCUMENT_10"])))))))*1.) )) +
                    0.099570*np.tanh(((np.minimum(((((((data["NAME_GOODS_CATEGORY_Fitness"]) * 2.0)) * 2.0))), ((np.tanh((np.minimum(((data["NAME_GOODS_CATEGORY_Fitness"])), ((((2.0) - (data["te_LIVE_CITY_NOT_WORK_CITY"]))))))))))) * (data["REG_CITY_NOT_WORK_CITY"]))) +
                    0.071082*np.tanh(((np.tanh((np.tanh(((((data["cnt_FLAG_DOCUMENT_10"]) + ((((data["cnt_REG_REGION_NOT_WORK_REGION"]) < (data["REG_REGION_NOT_WORK_REGION"]))*1.)))/2.0)))))) - (data["NAME_GOODS_CATEGORY_Fitness"]))) +
                    0.099922*np.tanh(((((np.where((((data["FLAG_DOCUMENT_10"]) > (data["CODE_REJECT_REASON_VERIF"]))*1.)>0, data["PRODUCT_COMBINATION_Cash_Street__low"], (((3.0) + (2.0))/2.0) )) * (data["te_REG_REGION_NOT_LIVE_REGION"]))) / 2.0)) +
                    0.030520*np.tanh((((((data["cnt_FLAG_DOCUMENT_10"]) + (((((((-1.0*((((data["NAME_GOODS_CATEGORY_Weapon"]) - (data["FLAG_DOCUMENT_2"])))))) < ((-1.0*((data["cnt_FLAG_DOCUMENT_10"])))))*1.)) - (data["NAME_GOODS_CATEGORY_Weapon"]))))) + (data["FLAG_DOCUMENT_2"]))/2.0)) +
                    0.096561*np.tanh(((np.minimum((((((((data["cnt_FLAG_DOCUMENT_13"]) + (1.0))/2.0)) + (data["FLAG_WORK_PHONE"])))), (((((((data["cnt_FLAG_DOCUMENT_10"]) > (((((-1.0*((data["AMT_GOODS_PRICE_x"])))) < (0.0))*1.)))*1.)) / 2.0))))) / 2.0)) +
                    0.099883*np.tanh((((-1.0) > (np.where(data["AMT_CREDIT_x"]>0, (((((3.0) + (2.0))/2.0)) * (data["AMT_INCOME_TOTAL"])), (-1.0*((data["AMT_INCOME_TOTAL"]))) )))*1.)) +
                    0.099414*np.tanh(np.minimum(((data["cc_bal_SK_DPD"])), ((np.minimum(((((data["AMT_CREDIT_x"]) * ((((data["cnt_FLAG_DOCUMENT_10"]) < (data["cc_bal_SK_DPD"]))*1.))))), ((((data["AMT_CREDIT_x"]) + (np.tanh((np.maximum(((2.0)), ((data["cc_bal_cc_bal_status__Refused"])))))))))))))) +
                    0.078703*np.tanh((((data["ca__Bad_debt"]) + ((-1.0*(((((data["AMT_GOODS_PRICE_x"]) > (((np.maximum(((-2.0)), ((np.minimum(((2.0)), ((((data["NAME_SELLER_INDUSTRY_Construction"]) / 2.0)))))))) * 2.0)))*1.))))))/2.0)) +
                    0.087691*np.tanh(((((((((((data["FLAG_DOCUMENT_10"]) > (np.tanh((data["AMT_CREDIT_x"]))))*1.)) * ((((3.0)) * (((data["FLAG_DOCUMENT_10"]) * ((8.92413711547851562)))))))) < (data["AMT_CREDIT_x"]))*1.)) / 2.0)) +
                    0.099883*np.tanh(np.minimum(((((data["AMT_GOODS_PRICE_x"]) * (np.tanh(((((data["DAYS_FIRST_DUE"]) < (data["AMT_ANNUITY_x"]))*1.))))))), (((-1.0*((data["FLAG_DOCUMENT_15"]))))))) +
                    0.099922*np.tanh(np.maximum(((((((2.0)) < (np.maximum(((((np.where(2.0>0, data["NAME_YIELD_GROUP_XNA"], data["cnt_FLAG_DOCUMENT_10"] )) * (data["AMT_ANNUITY_x"])))), ((2.0)))))*1.))), ((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) * (data["AMT_GOODS_PRICE_x"])))))) +
                    0.095193*np.tanh(np.tanh((((((((1.21274137496948242)) + (data["AMT_ANNUITY_x"]))) < ((((data["AMT_ANNUITY_x"]) > (data["AMT_GOODS_PRICE_x"]))*1.)))*1.)))) +
                    0.082337*np.tanh(np.minimum(((((data["AMT_CREDIT_x"]) / 2.0))), ((np.tanh((np.tanh(((-1.0*((np.maximum(((((data["FLAG_DOCUMENT_10"]) * 2.0))), ((data["AMT_INCOME_TOTAL"])))))))))))))) +
                    0.067370*np.tanh(np.where(data["AMT_GOODS_PRICE_x"]>0, data["AMT_CREDIT_SUM_DEBT"], ((((((data["AMT_GOODS_PRICE_x"]) < (data["AMT_CREDIT_SUM_DEBT"]))*1.)) + (np.tanh((data["AMT_CREDIT_SUM_DEBT"]))))/2.0) )) +
                    0.098085*np.tanh(np.minimum(((data["ty__Mortgage"])), (((((-1.0*(((((data["ty__Mortgage"]) > (((((((data["NAME_CONTRACT_STATUS_Refused"]) > (0.0))*1.)) + (((data["AMT_INCOME_TOTAL"]) / 2.0)))/2.0)))*1.))))) / 2.0))))) +
                    0.099961*np.tanh(np.maximum(((((np.maximum(((data["cnt_FLAG_DOCUMENT_10"])), ((data["AMT_INCOME_TOTAL"])))) * (data["DAYS_FIRST_DUE"])))), (((((((-2.0) < (((-3.0) + (data["AMT_INCOME_TOTAL"]))))*1.)) - ((0.17902497947216034))))))) +
                    0.099844*np.tanh(np.where(((((((data["NAME_SELLER_INDUSTRY_Construction"]) + (((1.0) + (data["AMT_CREDIT_SUM"]))))/2.0)) > (data["AMT_CREDIT_SUM"]))*1.)>0, np.where(data["NAME_CONTRACT_STATUS_Refused"]>0, data["AMT_CREDIT_SUM"], 0.0 ), data["AMT_CREDIT_x"] )) +
                    0.088785*np.tanh(np.maximum((((((np.tanh((np.where(data["OWN_CAR_AGE"]>0, ((data["ty__Mortgage"]) * (-1.0)), -1.0 )))) > (data["OWN_CAR_AGE"]))*1.))), ((((data["OWN_CAR_AGE"]) / 2.0))))) +
                    0.085033*np.tanh(((np.minimum(((((data["te_FLAG_OWN_CAR"]) * ((((((data["CODE_REJECT_REASON_VERIF"]) + (data["NAME_SELLER_INDUSTRY_Construction"]))) > (data["CODE_REJECT_REASON_VERIF"]))*1.))))), ((data["NAME_GOODS_CATEGORY_Insurance"])))) - (((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)))) +
                    0.099453*np.tanh(np.tanh(((((((data["AMT_CREDIT_x"]) < (data["PRODUCT_COMBINATION_POS_mobile_without_interest"]))*1.)) * (data["CODE_REJECT_REASON_VERIF"]))))) +
                    0.099883*np.tanh(((((((((data["AMT_CREDIT_SUM"]) / 2.0)) + (data["AMT_CREDIT_SUM"]))/2.0)) > ((((((((((data["AMT_CREDIT_SUM"]) * 2.0)) * 2.0)) + (-3.0))/2.0)) - (np.minimum(((-2.0)), ((3.0)))))))*1.)) +
                    0.096717*np.tanh(np.tanh((((((data["AMT_CREDIT_SUM"]) * (data["SK_ID_BUREAU"]))) + (((np.minimum(((data["SK_ID_BUREAU"])), ((np.tanh(((3.93321013450622559))))))) / 2.0)))))) +
                    0.076202*np.tanh(np.maximum(((-2.0)), ((((data["AMT_CREDIT_SUM"]) * (np.where((-1.0*((data["te_FLAG_OWN_CAR"])))>0, np.tanh((np.tanh((data["cnt_FLAG_OWN_CAR"])))), np.minimum(((data["AMT_CREDIT_SUM"])), (((-1.0*((-3.0)))))) ))))))) +
                    0.099844*np.tanh(((((((((((data["AMT_CREDIT_MAX_OVERDUE"]) - (data["FLAG_DOCUMENT_2"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
                    0.083118*np.tanh(np.maximum((((((data["PRODUCT_COMBINATION_POS_household_with_interest"]) + (-2.0))/2.0))), ((np.maximum((((((data["PRODUCT_COMBINATION_POS_household_with_interest"]) + (-2.0))/2.0))), ((np.where(data["DAYS_CREDIT"]>0, (-1.0*((data["CODE_REJECT_REASON_VERIF"]))), (-1.0*((data["AMT_CREDIT_SUM"]))) )))))))) +
                    0.057093*np.tanh(np.where(((data["DAYS_LAST_DUE_1ST_VERSION"]) / 2.0)>0, np.tanh((((((data["AMT_CREDIT_x"]) / 2.0)) / 2.0))), np.minimum((((((data["DAYS_CREDIT"]) + (data["AMT_CREDIT_x"]))/2.0))), ((0.0))) )) +
                    0.099726*np.tanh(np.minimum(((((data["AMT_ANNUITY_x"]) * (data["AMT_GOODS_PRICE_x"])))), ((np.tanh((np.tanh((np.tanh(((((((data["cnt_FLAG_DOCUMENT_10"]) - (data["AMT_GOODS_PRICE_x"]))) + (((data["AMT_ANNUITY_x"]) / 2.0)))/2.0))))))))))) +
                    0.099453*np.tanh(((np.minimum((((((data["FLAG_DOCUMENT_2"]) > (((data["cc_bal_cc_bal_status__Refused"]) + (data["FLAG_DOCUMENT_2"]))))*1.))), ((np.tanh((((((-1.0*((-2.0)))) < (data["AMT_GOODS_PRICE_x"]))*1.))))))) * 2.0)) +
                    0.097186*np.tanh(((np.where(data["cc_bal_cc_bal_status__Refused"]>0, data["NAME_GOODS_CATEGORY_Insurance"], ((np.maximum(((data["cc_bal_cc_bal_status__Refused"])), ((((((data["EXT_SOURCE_3"]) + (data["EXT_SOURCE_3"]))) / 2.0))))) * (((((data["DAYS_LAST_DUE_1ST_VERSION"]) / 2.0)) * 2.0))) )) * 2.0)) +
                    0.099805*np.tanh(((np.where(data["DAYS_LAST_DUE_1ST_VERSION"]>0, data["NAME_PAYMENT_TYPE_Cash_through_the_bank"], (((data["NAME_PORTFOLIO_Cards"]) > ((((data["NAME_PAYMENT_TYPE_XNA"]) < ((((0.0) + (((data["PRODUCT_COMBINATION_Card_X_Sell"]) + (0.0))))/2.0)))*1.)))*1.) )) / 2.0)) +
                    0.079445*np.tanh((((np.where(data["NAME_CONTRACT_STATUS_Refused"]>0, data["FLAG_DOCUMENT_2"], ((data["cc_bal_CNT_DRAWINGS_CURRENT"]) / 2.0) )) + (np.tanh(((((data["AMT_CREDIT_x"]) + (np.minimum(((data["cc_bal_CNT_DRAWINGS_CURRENT"])), ((data["cc_bal_CNT_DRAWINGS_CURRENT"])))))/2.0)))))/2.0)) +
                    0.099922*np.tanh(np.maximum(((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((np.tanh((data["inst_AMT_PAYMENT"]))))))), ((((np.minimum((((((data["AMT_GOODS_PRICE_x"]) < (data["AMT_ANNUITY_x"]))*1.))), ((((data["AMT_GOODS_PRICE_x"]) - (((data["inst_AMT_PAYMENT"]) * 2.0))))))) / 2.0))))) +
                    0.099648*np.tanh(np.tanh((np.where(((((-1.0*((np.minimum(((data["CNT_INSTALMENT_FUTURE"])), ((((((13.17097568511962891)) < (-3.0))*1.)))))))) < (data["PRODUCT_COMBINATION_Cash"]))*1.)>0, (-1.0*((data["AMT_GOODS_PRICE_x"]))), 0.0 )))) +
                    0.098437*np.tanh(np.minimum(((0.0)), (((-1.0*(((((((((data["CODE_REJECT_REASON_XAP"]) / 2.0)) + (data["CODE_REJECT_REASON_HC"]))) > (2.0))*1.)))))))) +
                    0.084564*np.tanh(np.minimum(((np.tanh((data["cc_bal_cc_bal_status__Refused"])))), (((((data["CNT_PAYMENT"]) + (((((0.0)) < (np.minimum(((2.0)), ((data["cc_bal_cc_bal_status__Refused"])))))*1.)))/2.0))))) +
                    0.099883*np.tanh(((((((data["AMT_CREDIT_x"]) * (np.maximum(((data["cc_bal_SK_DPD"])), (((((0.0) < (((np.tanh((data["NAME_GOODS_CATEGORY_Insurance"]))) / 2.0)))*1.))))))) / 2.0)) / 2.0)) +
                    0.099297*np.tanh((((data["AMT_CREDIT_x"]) > (((data["AMT_DOWN_PAYMENT"]) - (((((data["AMT_DOWN_PAYMENT"]) * (data["CODE_REJECT_REASON_VERIF"]))) + (-2.0))))))*1.)) +
                    0.098984*np.tanh(((np.minimum(((((np.minimum((((((7.0)) / 2.0))), ((((data["AMT_GOODS_PRICE_x"]) * 2.0))))) * (data["CODE_REJECT_REASON_VERIF"])))), ((np.tanh((0.0)))))) * (((np.tanh((data["AMT_ANNUITY_x"]))) * 2.0)))) +
                    0.090543*np.tanh((((data["AMT_DOWN_PAYMENT"]) > (np.tanh(((((2.0) > (data["AMT_CREDIT_x"]))*1.)))))*1.)) +
                    0.099844*np.tanh(((data["CODE_REJECT_REASON_VERIF"]) * (((((data["AMT_CREDIT_x"]) + ((((-2.0) + (np.tanh((data["cnt_FLAG_DOCUMENT_10"]))))/2.0)))) * (data["CODE_REJECT_REASON_VERIF"]))))) +
                    0.098671*np.tanh((-1.0*(((((-3.0) > (((((np.minimum(((data["cnt_FLAG_DOCUMENT_2"])), ((data["RATE_DOWN_PAYMENT"])))) * (data["RATE_DOWN_PAYMENT"]))) - (data["RATE_DOWN_PAYMENT"]))))*1.))))) +
                    0.099453*np.tanh(((((data["DAYS_FIRST_DRAWING"]) + (((np.where(data["CHANNEL_TYPE_Car_dealer"]>0, data["CODE_REJECT_REASON_VERIF"], (-1.0*((data["CODE_REJECT_REASON_VERIF"]))) )) * ((-1.0*((data["CHANNEL_TYPE_Car_dealer"])))))))) * (data["CODE_REJECT_REASON_VERIF"]))) +
                    0.099375*np.tanh((((((((-1.0*((np.tanh((((((4.0)) < (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))*1.))))))) * (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) +
                    0.095584*np.tanh((((((0.0) - (((((data["CODE_REJECT_REASON_VERIF"]) * (data["AMT_CREDIT_x"]))) * 2.0)))) + (((((((data["CODE_REJECT_REASON_VERIF"]) * (data["AMT_CREDIT_x"]))) * 2.0)) * (data["AMT_CREDIT_x"]))))/2.0)) +
                    0.099492*np.tanh(((((data["cnt_FLAG_DOCUMENT_14"]) + (((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["FLAG_DOCUMENT_2"]))) * (data["AMT_ANNUITY_x"]))))) + (((data["AMT_ANNUITY_x"]) * ((-1.0*((data["cnt_FLAG_DOCUMENT_14"])))))))) +
                    0.074326*np.tanh(((((((((np.tanh(((((data["AMT_CREDIT_x"]) > ((((-1.0) < (-2.0))*1.)))*1.)))) + (data["AMT_CREDIT_x"]))/2.0)) + ((((-1.0*((data["CODE_REJECT_REASON_VERIF"])))) + (data["cc_bal_cc_bal_status__Refused"]))))/2.0)) / 2.0)) +
                    0.099883*np.tanh(((data["AMT_ANNUITY_x"]) * (((data["AMT_GOODS_PRICE_x"]) * (((data["AMT_ANNUITY_x"]) * (((data["cnt_FLAG_DOCUMENT_2"]) * (np.minimum(((0.0)), ((np.minimum(((data["cnt_FLAG_DOCUMENT_2"])), (((-1.0*(((4.51322889328002930))))))))))))))))))) +
                    0.099805*np.tanh(((((np.where(data["AMT_CREDIT_MAX_OVERDUE"]>0, data["AMT_CREDIT_MAX_OVERDUE"], (-1.0*((((data["NAME_GOODS_CATEGORY_Fitness"]) + (((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) / 2.0)))))) )) * (2.0))) * 2.0)) +
                    0.086362*np.tanh(((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"])), ((np.minimum(((data["CHANNEL_TYPE_Stone"])), ((np.tanh((((np.tanh((((-1.0) + (((data["CHANNEL_TYPE_Stone"]) / 2.0)))))) / 2.0)))))))))) * 2.0)) +
                    0.099531*np.tanh(np.minimum(((np.where(data["NAME_GOODS_CATEGORY_Weapon"]>0, -3.0, np.tanh((((data["FLAG_DOCUMENT_17"]) * 2.0))) ))), ((((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["FLAG_DOCUMENT_17"], data["cnt_FLAG_DOCUMENT_17"] )) * 2.0))))) +
                    0.087182*np.tanh(((np.tanh((np.minimum(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["WEEKDAY_APPR_PROCESS_START_SATURDAY"]))) + (data["cnt_FLAG_DOCUMENT_17"])))), ((data["cnt_FLAG_DOCUMENT_10"])))))) + (((((data["WEEKDAY_APPR_PROCESS_START_SATURDAY"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * (data["WEEKDAY_APPR_PROCESS_START_SATURDAY"]))))) +
                    0.099687*np.tanh(((((np.tanh(((-1.0*(((((3.0) < (((((data["NAME_SELLER_INDUSTRY_Construction"]) / 2.0)) + (data["Amortized_debt"]))))*1.))))))) - (((data["NAME_SELLER_INDUSTRY_Construction"]) * (data["FLAG_DOCUMENT_4"]))))) * (data["NAME_SELLER_INDUSTRY_Construction"]))) +
                    0.097538*np.tanh((((((((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) / 2.0)) - (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]))) + ((((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) + ((((((0.0) + (data["NAME_GOODS_CATEGORY_Construction_Materials"]))) > ((((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) < (data["NAME_CASH_LOAN_PURPOSE_Medicine"]))*1.)))*1.)))/2.0)))/2.0)) +
                    0.096717*np.tanh(np.minimum(((((data["ty__Another_type_of_loan"]) * (((data["NAME_GOODS_CATEGORY_Clothing_and_Accessories"]) / 2.0))))), (((((10.93089485168457031)) - (((data["NAME_GOODS_CATEGORY_Clothing_and_Accessories"]) * 2.0))))))) +
                    0.099805*np.tanh(np.where(data["cc_bal_cc_bal_status__Refused"]>0, (-1.0*(((-1.0*((((data["YEARS_BEGINEXPLUATATION_MEDI"]) * (np.minimum(((np.minimum(((data["WEEKDAY_APPR_PROCESS_START_TUESDAY"])), (((-1.0*((data["CODE_REJECT_REASON_VERIF"])))))))), (((-1.0*((data["CODE_REJECT_REASON_VERIF"])))))))))))))), data["CODE_REJECT_REASON_VERIF"] )) +
                    0.099687*np.tanh((-1.0*((((((data["NAME_GOODS_CATEGORY_Education"]) * (((data["HOUSETYPE_MODE"]) * (((data["NAME_GOODS_CATEGORY_Insurance"]) + ((4.77548122406005859)))))))) + (((((14.97221946716308594)) < (data["CODE_REJECT_REASON_VERIF"]))*1.))))))) +
                    0.099726*np.tanh(((data["FONDKAPREMONT_MODE"]) * (((np.tanh((((data["NAME_GOODS_CATEGORY_Fitness"]) * (((-2.0) * (np.maximum(((-2.0)), ((((data["FONDKAPREMONT_MODE"]) * (data["WEEKDAY_APPR_PROCESS_START_TUESDAY"])))))))))))) * 2.0)))) +
                    0.070379*np.tanh(np.where(((((((data["FONDKAPREMONT_MODE"]) + (data["WEEKDAY_APPR_PROCESS_START_TUESDAY"]))) * (data["NAME_GOODS_CATEGORY_Fitness"]))) * 2.0)>0, (((((data["WEEKDAY_APPR_PROCESS_START_TUESDAY"]) * (2.0))) > (data["NAME_GOODS_CATEGORY_Fitness"]))*1.), ((-1.0) / 2.0) )) +
                    0.097890*np.tanh(np.where(data["NAME_TYPE_SUITE_Unaccompanied"]>0, data["cnt_FLAG_DOCUMENT_10"], (((((data["YEARS_BUILD_AVG"]) * 2.0)) > (np.maximum(((data["YEARS_BUILD_AVG"])), (((-1.0*((data["CODE_REJECT_REASON_VERIF"]))))))))*1.) )) +
                    0.070457*np.tanh((-1.0*(((((data["NAME_GOODS_CATEGORY_Fitness"]) > (((data["EMERGENCYSTATE_MODE"]) * (((data["FLAG_DOCUMENT_10"]) - (np.tanh((((data["NAME_GOODS_CATEGORY_Fitness"]) * (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))))))))))*1.))))) +
                    0.088355*np.tanh(np.maximum(((data["cnt_FLAG_DOCUMENT_10"])), ((((np.maximum(((-2.0)), ((np.maximum(((1.0)), (((11.60319614410400391)))))))) * ((((((((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) - (data["FLAG_DOCUMENT_10"]))) * 2.0)) > (3.0))*1.))))))) +
                    0.099726*np.tanh(((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) * ((-1.0*(((((-1.0*((((data["cnt_FLAG_DOCUMENT_10"]) - (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])))))) + ((((((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) / 2.0)) < (((data["cc_bal_SK_ID_PREV"]) / 2.0)))*1.))))))))) +
                    0.099961*np.tanh((((((data["cc_bal_AMT_RECIVABLE"]) > (data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]))*1.)) * ((((((3.0) + (-2.0))/2.0)) + (data["DAYS_LAST_DUE"]))))) +
                    0.100000*np.tanh(np.where(data["inst_SK_ID_PREV"]>0, data["FLAG_DOCUMENT_10"], np.maximum((((((((data["Completed"]) > ((6.0)))*1.)) / 2.0))), ((np.maximum(((data["inst_NUM_INSTALMENT_NUMBER"])), ((data["cc_bal_AMT_PAYMENT_CURRENT"])))))) )) +
                    0.068699*np.tanh(np.minimum((((((-1.0*((data["FLAG_DOCUMENT_10"])))) * 2.0))), ((((data["inst_SK_ID_PREV"]) * (data["CNT_INSTALMENT"])))))) +
                    0.099414*np.tanh((((data["inst_AMT_INSTALMENT"]) > (np.tanh((((((((data["REG_CITY_NOT_LIVE_CITY"]) + (data["CNT_INSTALMENT_FUTURE"]))/2.0)) > (((-1.0) / 2.0)))*1.)))))*1.)) +
                    0.099726*np.tanh((-1.0*(((((np.maximum(((np.minimum(((((data["AMT_APPLICATION"]) * 2.0))), (((2.26555514335632324)))))), ((1.0)))) < (data["AMT_ANNUITY"]))*1.))))) +
                    0.079406*np.tanh((((((data["NAME_YIELD_GROUP_middle"]) < ((((data["te_REG_CITY_NOT_LIVE_CITY"]) + (-3.0))/2.0)))*1.)) + ((((((((-1.0*((data["inst_AMT_PAYMENT"])))) / 2.0)) / 2.0)) / 2.0)))) +
                    0.085463*np.tanh((-1.0*(((((np.tanh((((data["te_REG_CITY_NOT_LIVE_CITY"]) * ((-1.0*((((data["te_REG_CITY_NOT_LIVE_CITY"]) + ((-1.0*((data["REG_CITY_NOT_LIVE_CITY"]))))))))))))) < (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.))))) +
                    0.061313*np.tanh((((((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), (((((0.0)) * 2.0))))) > (data["FLAG_DOCUMENT_10"]))*1.)) + (((data["AMT_CREDIT_x"]) * (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])), ((0.0)))))))) +
                    0.099883*np.tanh((((((data["NAME_CASH_LOAN_PURPOSE_Repairs"]) + (np.minimum(((data["DAYS_LAST_DUE"])), ((np.maximum(((data["cnt_FLAG_DOCUMENT_10"])), ((data["DAYS_FIRST_DUE"]))))))))) > ((2.0)))*1.)) +
                    0.099570*np.tanh(np.where((((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((data["NAME_GOODS_CATEGORY_Insurance"])))) > (data["cnt_FLAG_DOCUMENT_10"]))*1.)>0, data["CODE_REJECT_REASON_XAP"], ((np.maximum(((((data["NAME_CONTRACT_STATUS_Refused"]) - ((3.88871765136718750))))), ((np.tanh((data["FLAG_DOCUMENT_10"])))))) * 2.0) )) +
                    0.099609*np.tanh(((data["NAME_CONTRACT_STATUS_Approved"]) * ((((((data["CODE_REJECT_REASON_SCO"]) + (1.0))) > (((1.0) - ((((-1.0*((3.0)))) * 2.0)))))*1.)))) +
                    0.099805*np.tanh(np.minimum(((((((((10.0)) - (data["CODE_REJECT_REASON_VERIF"]))) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))/2.0))), ((np.where(data["CODE_REJECT_REASON_VERIF"]>0, (-1.0*((data["CODE_REJECT_REASON_LIMIT"]))), data["NAME_GOODS_CATEGORY_Insurance"] ))))) +
                    0.098242*np.tanh(np.where(data["cc_bal_cc_bal_status__Refused"]>0, ((data["cc_bal_cc_bal_status__Refused"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"])), (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > ((0.0)))*1.) )) +
                    0.099375*np.tanh(np.minimum(((((np.where(data["cnt_FLAG_DOCUMENT_10"]>0, data["FLAG_DOCUMENT_10"], ((-3.0) - (2.0)) )) + (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * ((((-2.0) + (data["NAME_CONTRACT_STATUS_Refused"]))/2.0))))))), ((2.0)))) +
                    0.099922*np.tanh(np.minimum((((((data["AMT_CREDIT_x"]) + (1.0))/2.0))), ((((((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) - (np.tanh((data["NAME_GOODS_CATEGORY_Fitness"]))))) * (((data["AMT_CREDIT_x"]) * 2.0)))) * 2.0))))) +
                    0.096327*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, data["cnt_HOUR_APPR_PROCESS_START_x"], (-1.0*(((((((data["PRODUCT_COMBINATION_POS_mobile_without_interest"]) > (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))*1.)) / 2.0)))) )) +
                    0.092849*np.tanh(((((((data["AMT_CREDIT_MAX_OVERDUE"]) + (np.tanh((((data["NAME_GOODS_CATEGORY_Fitness"]) * (np.where(data["AMT_CREDIT_MAX_OVERDUE"]>0, (2.0), data["AMT_CREDIT_MAX_OVERDUE"] )))))))) * 2.0)) * 2.0)) +
                    0.099844*np.tanh(((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, (((-1.0*((data["cnt_FLAG_DOCUMENT_18"])))) * 2.0), ((data["cnt_FLAG_DOCUMENT_18"]) / 2.0) )) * (np.maximum(((-3.0)), ((((2.0) - (data["NAME_CLIENT_TYPE_Refreshed"])))))))) +
                    0.088042*np.tanh((((np.minimum(((((((data["NAME_GOODS_CATEGORY_Fitness"]) * (-1.0))) / 2.0))), ((data["cnt_FLAG_DOCUMENT_18"])))) + (((data["cnt_FLAG_DOCUMENT_18"]) * (data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]))))/2.0)) +
                    0.099218*np.tanh(np.tanh((((((((-1.0*((data["AMT_CREDIT_x"])))) + ((4.31017494201660156)))) < (((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) + (data["AMT_CREDIT_x"]))/2.0)))*1.)))) +
                    0.099805*np.tanh(((((-1.0*((((data["NAME_GOODS_CATEGORY_Education"]) * (((data["NAME_YIELD_GROUP_low_normal"]) * (((data["NAME_GOODS_CATEGORY_Insurance"]) + (3.0)))))))))) + (np.minimum(((data["cnt_FLAG_DOCUMENT_4"])), (((-1.0*((data["NAME_GOODS_CATEGORY_Weapon"]))))))))/2.0)) +
                    0.071786*np.tanh(((((np.where(data["inst_AMT_PAYMENT"]>0, data["NAME_GOODS_CATEGORY_Insurance"], (((((((3.0) * (-1.0))) - (data["te_NAME_FAMILY_STATUS"]))) > ((-1.0*((1.0)))))*1.) )) * 2.0)) * 2.0)) +
                    0.100000*np.tanh(((data["te_NAME_TYPE_SUITE"]) * (np.minimum((((((np.minimum(((data["FLAG_DOCUMENT_10"])), ((data["NAME_GOODS_CATEGORY_Education"])))) > (data["te_NAME_TYPE_SUITE"]))*1.))), ((((data["te_NAME_TYPE_SUITE"]) + ((1.57757556438446045))))))))) +
                    0.100000*np.tanh(np.minimum(((((np.tanh(((-1.0*((((((2.47150230407714844)) < (data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))*1.))))))) * 2.0))), (((((((data["te_NAME_FAMILY_STATUS"]) + (np.tanh((2.0))))/2.0)) / 2.0))))) +
                    0.098828*np.tanh((((((((8.32762813568115234)) * (((data["te_NAME_FAMILY_STATUS"]) * (data["cnt_FLAG_DOCUMENT_2"]))))) + ((((data["cnt_FLAG_DOCUMENT_10"]) > (data["cc_bal_SK_DPD_DEF"]))*1.)))) - (np.tanh(((4.71051406860351562)))))) +
                    0.098124*np.tanh(np.minimum(((((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * 2.0)) * ((-1.0*((data["cnt_FLAG_DOCUMENT_17"]))))))), ((np.minimum(((data["cnt_FLAG_DOCUMENT_10"])), (((-1.0*((((data["NAME_GOODS_CATEGORY_Education"]) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))))))))))) +
                    0.098750*np.tanh(np.maximum(((((((data["cc_bal_SK_DPD_DEF"]) * 2.0)) + (data["cnt_FLAG_DOCUMENT_10"])))), ((((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"]) * 2.0)) - ((((((((13.95753192901611328)) * 2.0)) + (-2.0))) * 2.0))))))) +
                    0.099805*np.tanh((((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"], (-1.0*(((((((data["cc_bal_SK_DPD_DEF"]) * (((1.0) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))) < (((data["AMT_REQ_CREDIT_BUREAU_DAY"]) * 2.0)))*1.)))) )) < (data["AMT_REQ_CREDIT_BUREAU_DAY"]))*1.)) +
                    0.094998*np.tanh(np.minimum((((((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) + (((data["cnt_FLAG_DOCUMENT_4"]) - (((data["YEARS_BEGINEXPLUATATION_MEDI"]) / 2.0)))))/2.0))), (((((-1.0*((data["YEARS_BEGINEXPLUATATION_MEDI"])))) + (np.tanh(((((data["NAME_GOODS_CATEGORY_Insurance"]) > (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))*1.))))))))) +
                    0.100000*np.tanh((-1.0*((np.where(data["ty__Credit_card"]>0, data["NONLIVINGAPARTMENTS_MODE"], (-1.0*((data["NONLIVINGAPARTMENTS_MODE"]))) ))))) +
                    0.099805*np.tanh((-1.0*((((((((np.where(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"], data["NONLIVINGAPARTMENTS_MODE"] )>0, data["NONLIVINGAPARTMENTS_MODE"], data["CHANNEL_TYPE_Car_dealer"] )) + (0.0))) - (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))) + (0.0)))))) +
                    0.076710*np.tanh(np.tanh(((((((data["AMT_CREDIT_SUM_DEBT"]) + (((((data["NONLIVINGAPARTMENTS_MEDI"]) + (data["NONLIVINGAREA_MEDI"]))) / 2.0)))/2.0)) + ((((data["AMT_CREDIT_SUM_DEBT"]) + ((((data["NONLIVINGAREA_MEDI"]) > (data["NONLIVINGAPARTMENTS_AVG"]))*1.)))/2.0)))))) +
                    0.085190*np.tanh(np.where(np.tanh((data["NONLIVINGAPARTMENTS_MODE"]))>0, data["DAYS_LAST_DUE"], np.where(data["ty__Mortgage"]>0, np.where(-2.0>0, 1.0, -1.0 ), ((((1.68957030773162842)) < (data["te_HOUR_APPR_PROCESS_START_x"]))*1.) ) )) +
                    0.099883*np.tanh((((((-1.0*((data["NONLIVINGAPARTMENTS_MEDI"])))) * (((data["NAME_SELLER_INDUSTRY_Connectivity"]) * (2.0))))) * ((((((data["NONLIVINGAPARTMENTS_MEDI"]) * 2.0)) < ((-1.0*((((((0.40902623534202576)) + (data["te_HOUR_APPR_PROCESS_START_x"]))/2.0))))))*1.)))) +
                    0.067370*np.tanh(np.maximum((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + ((((data["NONLIVINGAPARTMENTS_MODE"]) + (-2.0))/2.0)))/2.0))), (((((data["cnt_HOUR_APPR_PROCESS_START_x"]) + (np.tanh((np.tanh((data["HOUR_APPR_PROCESS_START_x"]))))))/2.0))))) +
                    0.099844*np.tanh(((data["HOUR_APPR_PROCESS_START_x"]) * (np.maximum((((-1.0*((0.0))))), (((((((np.maximum(((data["NAME_YIELD_GROUP_high"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) * (data["cnt_HOUR_APPR_PROCESS_START_x"]))) + (data["AMT_CREDIT_SUM"]))/2.0))))))) +
                    0.099687*np.tanh(np.where(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, np.minimum(((np.tanh((0.0)))), (((-1.0*((data["AMT_CREDIT_SUM"])))))), data["AMT_CREDIT_SUM"] )>0, data["REGION_RATING_CLIENT_W_CITY"], (((((data["AMT_CREDIT_SUM"]) / 2.0)) < (-3.0))*1.) )) +
                    0.100000*np.tanh(np.where(((data["NONLIVINGAPARTMENTS_MODE"]) * (np.maximum(((((((data["REGION_POPULATION_RELATIVE"]) - (-1.0))) / 2.0))), ((data["NONLIVINGAPARTMENTS_MODE"])))))>0, data["REGION_POPULATION_RELATIVE"], 0.0 )) +
                    0.088159*np.tanh((((((((5.0)) * (data["te_WALLSMATERIAL_MODE"]))) - ((-1.0*((data["te_HOUSETYPE_MODE"])))))) * (data["cc_bal_cc_bal_status__Sent_proposal"]))) +
                    0.098867*np.tanh((-1.0*((((np.where(((data["NONLIVINGAPARTMENTS_MODE"]) * (data["NONLIVINGAPARTMENTS_MODE"]))>0, data["PRODUCT_COMBINATION_Cash_Street__low"], ((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["NAME_SELLER_INDUSTRY_MLM_partners"])) )) / 2.0))))) +
                    0.099609*np.tanh(((((data["AMT_CREDIT_y"]) * (data["EMERGENCYSTATE_MODE"]))) * (np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])), ((((data["AMT_CREDIT_SUM"]) * (data["FONDKAPREMONT_MODE"])))))))) +
                    0.099922*np.tanh(((((((data["AMT_ANNUITY"]) + (data["CNT_INSTALMENT_FUTURE"]))/2.0)) > (3.0))*1.)) +
                    0.099726*np.tanh((((data["AMT_CREDIT_SUM"]) < (np.tanh(((((-1.0) + ((-1.0*((((data["AMT_CREDIT_SUM"]) / 2.0))))))/2.0)))))*1.)) +
                    0.099140*np.tanh(((((np.tanh((np.tanh((((data["AMT_CREDIT_x"]) * (((data["EMERGENCYSTATE_MODE"]) * 2.0)))))))) / 2.0)) / 2.0)) +
                    0.099805*np.tanh((((((2.0) < (((data["FONDKAPREMONT_MODE"]) / 2.0)))*1.)) * ((((((data["AMT_ANNUITY"]) + (data["NAME_YIELD_GROUP_low_normal"]))/2.0)) + (data["AMT_CREDIT_x"]))))) +
                    0.099687*np.tanh(np.minimum(((data["NAME_TYPE_SUITE_Group_of_people"])), ((((-3.0) * ((((3.0) < (((data["AMT_ANNUITY"]) * (data["SK_ID_PREV_x"]))))*1.))))))) +
                    0.099726*np.tanh(((data["SK_ID_PREV_x"]) * (((((((data["AMT_GOODS_PRICE_y"]) > (np.where(data["AMT_ANNUITY"]>0, 1.0, data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"] )))*1.)) + (((0.0) * 2.0)))/2.0)))) +
                    0.096522*np.tanh(((data["NAME_GOODS_CATEGORY_Audio_Video"]) * (((((-1.0*(((((data["NAME_GOODS_CATEGORY_Audio_Video"]) < (1.0))*1.))))) > (data["CODE_REJECT_REASON_XAP"]))*1.)))) +
                    0.100000*np.tanh(((((((((-1.0*((np.maximum(((data["AMT_ANNUITY"])), (((((data["AMT_ANNUITY"]) < (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))*1.)))))))) + ((6.0)))) - (data["CHANNEL_TYPE_AP___Cash_loan_"]))) < (1.0))*1.)) +
                    0.099726*np.tanh((((((-1.0*((((data["FONDKAPREMONT_MODE"]) * (data["NAME_GOODS_CATEGORY_Other"])))))) - (np.tanh(((((data["FONDKAPREMONT_MODE"]) > (1.0))*1.)))))) / 2.0)) +
                    0.099297*np.tanh(np.where(((np.where(data["SK_DPD"]>0, data["NAME_TYPE_SUITE_Group_of_people"], data["NAME_GOODS_CATEGORY_Direct_Sales"] )) + (data["NAME_TYPE_SUITE_Group_of_people"]))>0, data["te_FLAG_DOCUMENT_21"], (((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))) * 2.0) )) +
                    0.099961*np.tanh(((((data["NAME_TYPE_SUITE_Group_of_people"]) * ((-1.0*((np.tanh((data["AMT_ANNUITY"])))))))) - (np.where(data["NONLIVINGAPARTMENTS_MODE"]>0, data["AMT_CREDIT_y"], np.minimum((((((data["AMT_ANNUITY"]) < ((4.0)))*1.))), ((data["NAME_TYPE_SUITE_Group_of_people"]))) )))) +
                    0.100000*np.tanh(np.where(np.tanh((((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)))>0, ((-3.0) * (((data["NAME_GOODS_CATEGORY_Insurance"]) - ((-1.0*((data["te_REGION_RATING_CLIENT"]))))))), (((data["te_REGION_RATING_CLIENT"]) < (-2.0))*1.) )) +
                    0.074639*np.tanh((-1.0*((np.tanh(((((((((data["cnt_REGION_RATING_CLIENT_W_CITY"]) * (data["te_FLAG_DOCUMENT_4"]))) < ((((data["cnt_REGION_RATING_CLIENT_W_CITY"]) < (0.0))*1.)))*1.)) / 2.0))))))) +
                    0.099961*np.tanh(np.minimum(((0.0)), (((((((np.maximum(((((data["REGION_RATING_CLIENT"]) - (np.tanh((((((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)) * 2.0))))))), ((data["AMT_GOODS_PRICE_x"])))) > (2.0))*1.)) * (data["REGION_RATING_CLIENT"])))))) +
                    0.099961*np.tanh(((data["te_FLAG_DOCUMENT_20"]) * (np.maximum(((((np.where(data["te_REGION_RATING_CLIENT_W_CITY"]>0, data["te_FLAG_DOCUMENT_20"], (((-2.0) > (0.0))*1.) )) * 2.0))), ((((data["NAME_GOODS_CATEGORY_Insurance"]) - ((-1.0*((data["NAME_TYPE_SUITE_Group_of_people"]))))))))))) +
                    0.099844*np.tanh(((((((data["te_FLAG_DOCUMENT_5"]) + (((((data["te_FLAG_DOCUMENT_5"]) + (data["te_FLAG_DOCUMENT_5"]))) - (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))))) - (data["AMT_CREDIT_x"]))) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) +
                    0.099297*np.tanh((-1.0*(((((data["NAME_TYPE_SUITE_Group_of_people"]) > (((((data["NAME_SELLER_INDUSTRY_MLM_partners"]) - (np.minimum(((data["AMT_CREDIT_y"])), ((data["NONLIVINGAPARTMENTS_MODE"])))))) / 2.0)))*1.))))) +
                    0.099961*np.tanh(((((2.78377246856689453)) < ((((np.maximum(((data["NONLIVINGAPARTMENTS_MODE"])), ((data["AMT_ANNUITY"])))) + (np.where(((data["NAME_GOODS_CATEGORY_Fitness"]) * 2.0)>0, data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"], data["NONLIVINGAPARTMENTS_MODE"] )))/2.0)))*1.)) +
                    0.072802*np.tanh((((((data["NAME_YIELD_GROUP_middle"]) > (np.tanh((data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"]))))*1.)) * (np.minimum(((data["AMT_CREDIT_y"])), ((((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) * 2.0))))))) +
                    0.099922*np.tanh(((data["NAME_TYPE_SUITE_Group_of_people"]) * ((((-2.0) + (((data["SK_ID_PREV_y"]) * (np.where(data["AMT_GOODS_PRICE_x"]>0, data["NAME_TYPE_SUITE_Group_of_people"], (((3.0) + ((9.08674621582031250)))/2.0) )))))/2.0)))) +
                    0.099961*np.tanh(((data["NAME_CONTRACT_STATUS_Canceled"]) * ((((data["NAME_SELLER_INDUSTRY_MLM_partners"]) + ((((-1.0*((data["SK_DPD"])))) * (np.minimum(((((3.0) - (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) / 2.0))))), ((data["inst_AMT_PAYMENT"])))))))/2.0)))) +
                    0.097499*np.tanh(np.where(np.minimum(((data["cc_bal_cc_bal_status__Sent_proposal"])), (((((data["SK_DPD"]) + ((-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"])))))/2.0))))>0, data["NAME_FAMILY_STATUS"], (-1.0*((((data["NAME_YIELD_GROUP_high"]) * (data["SK_DPD"]))))) )) +
                    0.063501*np.tanh(((((data["SK_ID_PREV_x"]) / 2.0)) * (((data["NAME_YIELD_GROUP_high"]) * ((((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]) < (np.tanh((0.0))))*1.)))))) +
                    0.100000*np.tanh((((np.tanh((np.tanh(((((data["NAME_SELLER_INDUSTRY_Connectivity"]) < (1.0))*1.)))))) < (((data["NAME_SELLER_INDUSTRY_Connectivity"]) * (data["inst_AMT_PAYMENT"]))))*1.)) +
                    0.096170*np.tanh(((np.where(data["AMT_GOODS_PRICE_x"]>0, ((data["NAME_CLIENT_TYPE_Repeater"]) / 2.0), np.tanh((((data["NAME_CLIENT_TYPE_Repeater"]) * (data["inst_AMT_PAYMENT"])))) )) / 2.0)) +
                    0.099766*np.tanh(np.minimum(((np.maximum(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (-2.0)))), ((data["inst_AMT_PAYMENT"]))))), ((((data["NAME_GOODS_CATEGORY_Photo___Cinema_Equipment"]) * (np.maximum(((data["inst_AMT_PAYMENT"])), ((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))))))))) +
                    0.095037*np.tanh((((((-2.0) + ((-1.0*((data["SK_ID_PREV_x"])))))/2.0)) * (np.minimum(((((data["SK_DPD"]) * (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"])))), ((((data["SK_ID_PREV_x"]) * (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"])))))))) +
                    0.099570*np.tanh((((np.maximum(((np.where(data["Completed"]>0, data["NFLAG_LAST_APPL_IN_DAY"], data["cc_bal_cc_bal_status__Sent_proposal"] ))), ((-3.0)))) + (np.where(data["Completed"]>0, data["inst_AMT_INSTALMENT"], (((-1.0) > (0.0))*1.) )))/2.0)) +
                    0.099961*np.tanh((((((data["CNT_INSTALMENT"]) + (data["MONTHS_BALANCE"]))/2.0)) * (np.minimum((((((data["SK_DPD"]) > (((data["cc_bal_cc_bal_status__Sent_proposal"]) / 2.0)))*1.))), ((((data["NFLAG_INSURED_ON_APPROVAL"]) / 2.0))))))) +
                    0.079562*np.tanh(((data["DAYS_LAST_DUE"]) * (np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, data["NFLAG_INSURED_ON_APPROVAL"], np.tanh((np.maximum(((((data["NFLAG_INSURED_ON_APPROVAL"]) / 2.0))), ((((data["cc_bal_cc_bal_status__Sent_proposal"]) * 2.0)))))) )))) +
                    0.082298*np.tanh(np.tanh((np.minimum(((3.0)), ((((np.maximum(((data["inst_AMT_PAYMENT"])), ((((data["SK_DPD"]) * ((13.14832782745361328))))))) * (data["RATE_DOWN_PAYMENT"])))))))) +
                    0.079914*np.tanh(np.where(((np.minimum((((-1.0*((data["SK_DPD"]))))), ((-2.0)))) + (data["DAYS_TERMINATION"]))>0, data["NAME_CONTRACT_STATUS_Approved"], (-1.0*((((np.tanh((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))) / 2.0)))) )) +
                    0.099922*np.tanh(((np.minimum(((np.minimum(((((((data["cc_bal_cc_bal_status__Sent_proposal"]) * (data["DAYS_TERMINATION"]))) - (((data["cc_bal_cc_bal_status__Sent_proposal"]) / 2.0))))), ((np.maximum(((data["Completed"])), ((data["SK_DPD"])))))))), (((-1.0*((-1.0))))))) * 2.0)) +
                    0.099609*np.tanh(((((((data["PRODUCT_COMBINATION_Cash"]) / 2.0)) * ((((np.tanh((data["cc_bal_cc_bal_status__Sent_proposal"]))) + ((((data["cc_bal_cc_bal_status__Sent_proposal"]) > (0.0))*1.)))/2.0)))) * (data["CNT_INSTALMENT_FUTURE"]))) +
                    0.071473*np.tanh(np.maximum(((np.minimum(((data["SK_DPD"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))), ((((data["NAME_CONTRACT_TYPE_Consumer_loans"]) * (np.minimum(((((data["NAME_YIELD_GROUP_XNA"]) / 2.0))), ((np.maximum(((data["NAME_CONTRACT_TYPE_Consumer_loans"])), (((1.0))))))))))))) +
                    0.098828*np.tanh(((np.maximum(((((((((((((-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"])))) < (((data["NFLAG_INSURED_ON_APPROVAL"]) / 2.0)))*1.)) + (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))) * (data["NAME_YIELD_GROUP_XNA"]))) + (-1.0)))), ((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"])))) / 2.0)) +
                    0.099961*np.tanh(((((data["inst_AMT_PAYMENT"]) * (np.minimum(((data["SK_DPD"])), ((np.minimum(((data["SK_DPD"])), (((((7.0)) - (((3.0) * (data["SK_DPD"]))))))))))))) * 2.0)) +
                    0.099687*np.tanh((((((np.minimum(((-2.0)), ((((-1.0) - (data["inst_AMT_INSTALMENT"])))))) * 2.0)) > (((((1.0) + (((-1.0) - (data["AMT_CREDIT_y"]))))) - (data["CNT_INSTALMENT_FUTURE"]))))*1.)) +
                    0.099179*np.tanh(((data["PRODUCT_COMBINATION_Cash"]) * (((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * (np.minimum(((data["cnt_FLAG_DOCUMENT_21"])), ((((data["NFLAG_INSURED_ON_APPROVAL"]) * (((-3.0) * 2.0))))))))) - (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))))) +
                    0.099922*np.tanh((-1.0*((np.where(data["NAME_CONTRACT_STATUS_Canceled"]>0, (-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"]))), (((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) > ((0.0)))*1.)) * ((((data["DAYS_TERMINATION"]) + (((data["AMT_CREDIT_x"]) * 2.0)))/2.0))) ))))) +
                    0.099492*np.tanh((((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) * (data["cc_bal_cc_bal_status__Sent_proposal"]))) + ((((((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) * (((((data["cc_bal_cc_bal_status__Sent_proposal"]) * 2.0)) * 2.0)))) * 2.0)) + (((data["AMT_CREDIT_x"]) / 2.0)))/2.0)))/2.0)) +
                    0.099101*np.tanh((-1.0*((np.maximum(((data["cc_bal_cc_bal_status__Sent_proposal"])), ((((((((-1.0) + (data["AMT_GOODS_PRICE_x"]))/2.0)) + (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) - (data["cc_bal_cc_bal_status__Sent_proposal"]))))))/2.0)))))))) +
                    0.099414*np.tanh(((data["NAME_CONTRACT_STATUS_Canceled"]) * ((-1.0*((np.tanh((((data["cnt_FLAG_DOCUMENT_21"]) + ((((data["AMT_CREDIT_x"]) > (((3.0) - (np.tanh((data["cnt_FLAG_DOCUMENT_21"]))))))*1.))))))))))) +
                    0.091481*np.tanh(((((((((data["AMT_CREDIT_x"]) - (2.0))) < ((-1.0*((data["AMT_CREDIT_x"])))))*1.)) < (np.tanh((data["NFLAG_INSURED_ON_APPROVAL"]))))*1.)) +
                    0.095936*np.tanh(np.tanh((np.where(data["AMT_ANNUITY_x"]>0, np.where((((data["AMT_GOODS_PRICE_x"]) + (data["NFLAG_INSURED_ON_APPROVAL"]))/2.0)>0, data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"], (((-1.0*((data["inst_AMT_PAYMENT"])))) * 2.0) ), (((-2.0) + (2.0))/2.0) )))) +
                    0.099922*np.tanh(np.minimum(((((((((data["inst_AMT_INSTALMENT"]) * (np.maximum(((data["cnt_FLAG_DOCUMENT_21"])), ((data["AMT_GOODS_PRICE_x"])))))) * 2.0)) / 2.0))), ((np.minimum(((((1.0) * 2.0))), (((((3.0) < (data["inst_AMT_INSTALMENT"]))*1.)))))))) +
                    0.090504*np.tanh(((((((((((((-3.0) * 2.0)) * 2.0)) * (data["NFLAG_INSURED_ON_APPROVAL"]))) * (((data["inst_AMT_PAYMENT"]) * 2.0)))) * ((((data["inst_AMT_PAYMENT"]) > (2.0))*1.)))) * 2.0)) +
                    0.057718*np.tanh((((((np.minimum(((((3.0) * (((((data["inst_AMT_PAYMENT"]) / 2.0)) * (data["NFLAG_INSURED_ON_APPROVAL"])))))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))) + (((data["FLAG_DOCUMENT_21"]) * 2.0)))/2.0)) / 2.0)) +
                    0.099961*np.tanh(np.minimum(((((((data["AMT_CREDIT_x"]) * (data["AMT_ANNUITY_x"]))) * 2.0))), (((((-1.0*((data["NAME_GOODS_CATEGORY_Fitness"])))) * (data["AMT_ANNUITY_x"])))))) +
                    0.098867*np.tanh((((((data["NAME_SELLER_INDUSTRY_Auto_technology"]) + (((data["cnt_REGION_RATING_CLIENT_W_CITY"]) * (data["NAME_SELLER_INDUSTRY_Auto_technology"]))))/2.0)) * ((((data["NAME_SELLER_INDUSTRY_Auto_technology"]) > ((((data["cnt_REGION_RATING_CLIENT_W_CITY"]) > (data["cnt_REGION_RATING_CLIENT_W_CITY"]))*1.)))*1.)))) +
                    0.099961*np.tanh(((((((((((((((((((data["DAYS_TERMINATION"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) + (data["NFLAG_INSURED_ON_APPROVAL"]))/2.0)) + (data["cu__currency_3"]))/2.0)) > (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.)) + (data["cu__currency_3"]))/2.0)) * (data["te_WEEKDAY_APPR_PROCESS_START"]))) / 2.0)) +
                    0.088277*np.tanh((((((data["DAYS_TERMINATION"]) + ((-1.0*(((((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) / 2.0)) * 2.0)) > (((data["NAME_GOODS_CATEGORY_Fitness"]) * 2.0)))*1.)) / 2.0))))))/2.0)) * (((((data["cnt_HOUR_APPR_PROCESS_START_x"]) / 2.0)) / 2.0)))) +
                    0.099805*np.tanh((((-3.0) > (((data["te_WEEKDAY_APPR_PROCESS_START"]) + (np.minimum(((data["NFLAG_INSURED_ON_APPROVAL"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))))))*1.)) +
                    0.079445*np.tanh((((((((np.tanh((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((data["DAYS_TERMINATION"]) + (data["DAYS_TERMINATION"]))))))) * 2.0)) > (((((1.0)) + (np.minimum(((data["AMT_ANNUITY_x"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))/2.0)))*1.)) / 2.0)) +
                    0.099258*np.tanh(((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * ((-1.0*((((np.maximum(((data["AMT_CREDIT_x"])), ((data["cnt_FLAG_DOCUMENT_3"])))) + (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * (data["te_FLAG_DOCUMENT_3"])))))))))) +
                    0.099648*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) - (((((data["AMT_CREDIT_x"]) - ((-1.0*((data["ty__Microloan"])))))) * (data["ty__Microloan"]))))) +
                    0.099922*np.tanh(((np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((np.minimum(((data["ty__Microloan"])), (((((1.0) > (-1.0))*1.))))) * (data["AMT_GOODS_PRICE_x"])))))) * (data["AMT_ANNUITY_x"]))) +
                    0.072333*np.tanh(((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (np.minimum(((((data["AMT_CREDIT_x"]) * 2.0))), ((data["te_FLAG_DOCUMENT_19"])))))) +
                    0.074404*np.tanh((((((-3.0) > ((((-1.0*((data["NAME_GOODS_CATEGORY_Fitness"])))) / 2.0)))*1.)) - (((np.minimum(((data["NAME_GOODS_CATEGORY_Fitness"])), ((np.tanh((data["AMT_DOWN_PAYMENT"])))))) * (data["AMT_CREDIT_x"]))))) +
                    0.068581*np.tanh((-1.0*((((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, np.minimum(((((data["AMT_GOODS_PRICE_x"]) * 2.0))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0) )) * ((-1.0*((data["AMT_GOODS_PRICE_x"]))))))))) +
                    0.099726*np.tanh(((((data["NAME_GOODS_CATEGORY_Fitness"]) - (np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["cnt_FLAG_DOCUMENT_17"], 0.0 )))) * (((((data["nans"]) + (np.maximum(((((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0))), ((data["nans"])))))) * 2.0)))) +
                    0.098789*np.tanh(((((((data["FLAG_DOCUMENT_17"]) * (data["NAME_GOODS_CATEGORY_Fitness"]))) - ((-1.0*(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((1.0) / 2.0)))) > ((12.29629707336425781)))*1.))))))) * 2.0)) +
                    0.099922*np.tanh(np.tanh(((((((((-1.0) / 2.0)) + ((((((-2.0) * (data["AMT_GOODS_PRICE_x"]))) > (((np.tanh(((((-3.0) < (1.0))*1.)))) / 2.0)))*1.)))/2.0)) / 2.0)))) +
                    0.073466*np.tanh(np.tanh(((((((np.tanh((np.tanh((data["AMT_CREDIT_x"]))))) + (np.minimum((((-1.0*((data["AMT_DOWN_PAYMENT"]))))), ((((data["cnt_FLAG_DOCUMENT_17"]) + (data["cnt_FLAG_DOCUMENT_4"])))))))/2.0)) / 2.0)))) +
                    0.095975*np.tanh((((data["RATE_INTEREST_PRIMARY"]) + (((((((-1.0*((((data["NAME_GOODS_CATEGORY_Insurance"]) - (data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))) * (data["PRODUCT_COMBINATION_POS_industry_with_interest"]))) < (data["NAME_CASH_LOAN_PURPOSE_Hobby"]))*1.)))/2.0)) +
                    0.099218*np.tanh(((np.tanh((((((data["PRODUCT_COMBINATION_POS_industry_with_interest"]) * (((data["NAME_GOODS_CATEGORY_Furniture"]) * (np.minimum(((data["AMT_DOWN_PAYMENT"])), (((((data["NAME_GOODS_CATEGORY_Insurance"]) < (3.0))*1.))))))))) * (data["PRODUCT_COMBINATION_POS_industry_with_interest"]))))) * (data["PRODUCT_COMBINATION_POS_industry_with_interest"]))) +
                    0.099648*np.tanh(np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((data["PRODUCT_COMBINATION_POS_industry_with_interest"]) + ((((((((data["AMT_GOODS_PRICE_x"]) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))) < (((np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)))) / 2.0)))*1.)) * 2.0))))))) +
                    0.099922*np.tanh(((data["AMT_CREDIT_x"]) * ((((((((((((0.0) > (data["AMT_CREDIT_x"]))*1.)) < (data["NAME_SELLER_INDUSTRY_Clothing"]))*1.)) > (((data["AMT_CREDIT_x"]) / 2.0)))*1.)) * (data["NAME_SELLER_INDUSTRY_Clothing"]))))) +
                    0.099844*np.tanh(np.minimum(((np.minimum((((((((1.0) + (((data["AMT_ANNUITY_x"]) * 2.0)))/2.0)) * (data["AMT_CREDIT_x"])))), (((((1.0) < (data["AMT_DOWN_PAYMENT"]))*1.)))))), ((1.0)))) +
                    0.099492*np.tanh(np.tanh(((((1.0) < ((((((((((data["AMT_ANNUITY_x"]) > ((9.0)))*1.)) + (data["cnt_FLAG_DOCUMENT_3"]))/2.0)) + (data["AMT_ANNUITY_x"]))/2.0)))*1.)))) +
                    0.093982*np.tanh((-1.0*(((((1.0) < ((((data["MONTHS_BALANCE"]) + (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) + ((((((data["PRODUCT_COMBINATION_POS_industry_with_interest"]) * 2.0)) + (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) - (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))/2.0)))))/2.0)))*1.))))) +
                    0.100000*np.tanh(np.minimum((((((np.tanh(((((np.tanh(((1.0)))) < (data["NAME_SELLER_INDUSTRY_Furniture"]))*1.)))) > (data["AMT_DOWN_PAYMENT"]))*1.))), ((((((((-2.0) + (((data["NAME_SELLER_INDUSTRY_Furniture"]) * 2.0)))/2.0)) > (0.0))*1.))))) +
                    0.099492*np.tanh(((((np.tanh((-1.0))) * (data["AMT_DOWN_PAYMENT"]))) * ((((((data["NAME_SELLER_INDUSTRY_Furniture"]) > (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))*1.)) * (data["AMT_DOWN_PAYMENT"]))))) +
                    0.071786*np.tanh(((((data["te_FLAG_DOCUMENT_19"]) * (((data["AMT_CREDIT_x"]) * (data["AMT_DOWN_PAYMENT"]))))) * 2.0)) +
                    0.095506*np.tanh(((data["AMT_DOWN_PAYMENT"]) * ((((np.tanh((data["te_FLAG_DOCUMENT_19"]))) + (((-1.0) * ((((data["AMT_DOWN_PAYMENT"]) < (data["AMT_CREDIT_x"]))*1.)))))/2.0)))) +
                    0.099844*np.tanh(np.where((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) > (((2.0) - (data["NAME_GOODS_CATEGORY_Furniture"]))))*1.)>0, data["AMT_CREDIT_y"], np.tanh((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])) )) +
                    0.099414*np.tanh((((((((((data["NAME_CASH_LOAN_PURPOSE_Car_repairs"]) * ((((2.27680611610412598)) / 2.0)))) - (data["cnt_NAME_HOUSING_TYPE"]))) > ((9.0)))*1.)) - (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["AMT_DOWN_PAYMENT"]))))) +
                    0.053966*np.tanh(((np.maximum((((((data["cnt_FLAG_DOCUMENT_10"]) < (((data["te_FLAG_DOCUMENT_19"]) + ((-1.0*(((((((3.0) + ((1.60392558574676514)))/2.0)) - (data["AMT_CREDIT_x"])))))))))*1.))), ((data["FLAG_DOCUMENT_10"])))) * (data["AMT_CREDIT_x"]))) +
                    0.087339*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) - ((((3.0) < (((data["AMT_GOODS_PRICE_x"]) - ((-1.0*(((((data["AMT_DOWN_PAYMENT"]) < (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))*1.))))))))*1.)))) + (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["NAME_GOODS_CATEGORY_Insurance"]))))) +
                    0.099531*np.tanh((((((((data["AMT_DOWN_PAYMENT"]) > (np.maximum(((-3.0)), (((((((data["AMT_CREDIT_x"]) / 2.0)) < (((((1.59686958789825439)) < (3.0))*1.)))*1.))))))*1.)) * (data["ty__Consumer_credit"]))) * 2.0)) +
                    0.100000*np.tanh(((data["ty__Credit_card"]) * ((-1.0*((np.where(((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0)>0, data["te_FLAG_DOCUMENT_20"], np.where(-3.0>0, ((((-2.0) * 2.0)) / 2.0), data["NAME_CASH_LOAN_PURPOSE_Business_development"] ) ))))))) +
                    0.079445*np.tanh(((data["cnt_NAME_HOUSING_TYPE"]) * ((((data["ty__Credit_card"]) > ((((((((((0.0) < ((-1.0*((data["ty__Consumer_credit"])))))*1.)) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)) > (((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)))*1.)))*1.)))) +
                    0.077687*np.tanh((((data["te_NAME_HOUSING_TYPE"]) > (np.maximum((((((((((np.minimum(((((data["AMT_CREDIT_x"]) * 2.0))), ((data["cnt_FLAG_DOCUMENT_4"])))) < (-2.0))*1.)) * ((9.0)))) * 2.0))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))*1.)) +
                    0.099883*np.tanh((((-1.0*((np.maximum(((((((data["cnt_NAME_HOUSING_TYPE"]) * (data["te_NAME_HOUSING_TYPE"]))) * (2.0)))), (((((0.17687205970287323)) / 2.0)))))))) / 2.0)) +
                    0.099453*np.tanh((((2.0) < (((((data["AMT_CREDIT_x"]) - (np.where(data["cnt_REGION_RATING_CLIENT_W_CITY"]>0, (-1.0*((((data["cnt_REGION_RATING_CLIENT_W_CITY"]) * 2.0)))), (((((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) * 2.0)) < ((12.91717815399169922)))*1.) )))) / 2.0)))*1.)) +
                    0.084134*np.tanh(np.tanh(((-1.0*(((((((2.0) + (np.minimum(((data["FLAG_DOCUMENT_4"])), ((data["te_FLAG_DOCUMENT_19"])))))) < (((data["FLAG_DOCUMENT_4"]) + (data["te_WEEKDAY_APPR_PROCESS_START"]))))*1.))))))) +
                    0.098632*np.tanh((-1.0*(((((((2.0) / 2.0)) < ((((((data["AMT_GOODS_PRICE_x"]) + (data["te_NAME_HOUSING_TYPE"]))/2.0)) / 2.0)))*1.))))) +
                    0.070848*np.tanh((((-1.0*((((((((np.maximum(((((data["te_FLAG_DOCUMENT_19"]) / 2.0))), ((data["te_FLAG_CONT_MOBILE"])))) + (data["te_LIVE_REGION_NOT_WORK_REGION"]))/2.0)) > (3.0))*1.))))) * 2.0)) +
                    0.082493*np.tanh(((((((((data["te_REG_REGION_NOT_WORK_REGION"]) / 2.0)) + (((0.0) * ((-1.0*((data["REG_REGION_NOT_WORK_REGION"])))))))/2.0)) < (np.tanh((data["te_REG_REGION_NOT_WORK_REGION"]))))*1.)) +
                    0.082689*np.tanh(((np.where(data["inst_AMT_PAYMENT"]>0, data["te_FLAG_DOCUMENT_20"], np.minimum((((((((data["te_FLAG_DOCUMENT_20"]) < (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))*1.)) / 2.0))), ((2.0))) )) / 2.0)) +
                    0.099218*np.tanh((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"]))/2.0)) * (((((data["te_FLAG_CONT_MOBILE"]) + ((((data["te_FLAG_DOCUMENT_20"]) < (np.minimum((((((7.0)) / 2.0))), (((7.0))))))*1.)))) * 2.0)))) +
                    0.086518*np.tanh(((data["te_FLAG_DOCUMENT_19"]) * ((-1.0*((np.maximum(((np.maximum(((data["FLAG_DOCUMENT_17"])), ((data["te_REG_REGION_NOT_LIVE_REGION"]))))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))))))) +
                    0.098437*np.tanh((-1.0*((((((3.0) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) - (np.where((((data["AMT_CREDIT_MAX_OVERDUE"]) > ((1.90302777290344238)))*1.)>0, data["EMERGENCYSTATE_MODE"], ((data["NAME_GOODS_CATEGORY_Insurance"]) * (data["EMERGENCYSTATE_MODE"])) ))))))) +
                    0.083392*np.tanh(np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((((((data["te_FLAG_DOCUMENT_19"]) > (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))*1.)) + (data["NAME_INCOME_TYPE"]))/2.0))))) +
                    0.099961*np.tanh((((np.maximum(((data["te_ORGANIZATION_TYPE"])), ((((((((np.minimum((((1.0))), ((2.0)))) + ((-1.0*((data["te_ORGANIZATION_TYPE"])))))/2.0)) > (data["te_FLAG_DOCUMENT_19"]))*1.))))) + (-1.0))/2.0)) +
                    0.038023*np.tanh(((data["cnt_FLAG_WORK_PHONE"]) * (np.minimum((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))))), (((-1.0*(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)))))))))) +
                    0.099687*np.tanh((((np.where(((data["NAME_GOODS_CATEGORY_Insurance"]) * (((data["cnt_FLAG_DOCUMENT_11"]) / 2.0)))>0, ((data["cnt_FLAG_DOCUMENT_11"]) * 2.0), np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["FLAG_DOCUMENT_11"], ((data["cnt_FLAG_DOCUMENT_11"]) / 2.0) ) )) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))/2.0)) +
                    0.099609*np.tanh(((((data["te_FLAG_DOCUMENT_20"]) * (data["ca__Sold"]))) * ((((-2.0) > (((((((3.50255107879638672)) > (data["te_FLAG_DOCUMENT_19"]))*1.)) + (((-1.0) - (data["ca__Sold"]))))))*1.)))) +
                    0.099844*np.tanh((((((data["AMT_CREDIT_MAX_OVERDUE"]) > (np.where(data["AMT_CREDIT_MAX_OVERDUE"]>0, ((((6.0)) + (data["te_FLAG_CONT_MOBILE"]))/2.0), data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"] )))*1.)) * 2.0)) +
                    0.099961*np.tanh(((((data["te_FLAG_DOCUMENT_19"]) * (((data["avg_buro_buro_bal_status_2"]) * 2.0)))) + (((data["cnt_FLAG_DOCUMENT_11"]) / 2.0)))) +
                    0.093943*np.tanh(((data["EXT_SOURCE_3"]) * (np.tanh((np.maximum(((((data["avg_buro_buro_bal_status_1"]) + (np.maximum(((((data["avg_buro_buro_bal_status_1"]) + (data["avg_buro_buro_bal_status_4"])))), ((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"])), ((-2.0)))))))))), ((data["AMT_CREDIT_x"])))))))) +
                    0.080852*np.tanh(np.where((((data["EXT_SOURCE_3"]) + (data["DAYS_CREDIT"]))/2.0)>0, data["avg_buro_buro_bal_status_2"], np.where(data["EXT_SOURCE_3"]>0, 0.0, data["DAYS_CREDIT"] ) )) +
                    0.089019*np.tanh((((((data["DAYS_ENDDATE_FACT"]) * (data["te_FLAG_DOCUMENT_19"]))) > (np.where(np.maximum(((np.tanh((data["DAYS_ENDDATE_FACT"])))), ((data["DAYS_CREDIT"])))>0, data["DAYS_ENDDATE_FACT"], (-1.0*((data["avg_buro_MONTHS_BALANCE"]))) )))*1.)) +
                    0.100000*np.tanh(((data["avg_buro_buro_bal_status_2"]) * (((data["avg_buro_buro_bal_status_2"]) * (np.where(-1.0>0, 1.0, np.minimum(((((data["avg_buro_buro_bal_status_2"]) * (data["te_FLAG_DOCUMENT_17"])))), (((((2.0) < (data["avg_buro_buro_bal_status_2"]))*1.)))) )))))) +
                    0.098867*np.tanh(((data["te_FLAG_DOCUMENT_17"]) * ((((data["avg_buro_MONTHS_BALANCE"]) < ((((((data["te_FLAG_DOCUMENT_17"]) < (-1.0))*1.)) - (1.0))))*1.)))) +
                    0.095193*np.tanh(((np.where(data["te_FLAG_DOCUMENT_19"]>0, (((-2.0) > (data["DAYS_CREDIT"]))*1.), data["NAME_CASH_LOAN_PURPOSE_Business_development"] )) - (data["avg_buro_buro_bal_status_2"]))) +
                    0.099258*np.tanh(((data["avg_buro_MONTHS_BALANCE"]) * (np.where(data["AMT_CREDIT_x"]>0, data["te_FLAG_DOCUMENT_19"], np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((np.where(3.0>0, data["avg_buro_buro_bal_status_2"], data["te_FLAG_DOCUMENT_19"] )))) )))) +
                    0.085815*np.tanh(((data["te_FLAG_DOCUMENT_20"]) * ((((((((data["te_FLAG_DOCUMENT_20"]) < ((((data["te_FLAG_EMAIL"]) + (0.0))/2.0)))*1.)) / 2.0)) * (data["te_NAME_EDUCATION_TYPE"]))))) +
                    0.099140*np.tanh(np.tanh((np.tanh((np.where(data["avg_buro_buro_bal_status_5"]>0, data["te_FLAG_CONT_MOBILE"], np.tanh((((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["AMT_CREDIT_x"], data["NAME_CASH_LOAN_PURPOSE_Business_development"] )) * 2.0))) )))))) +
                    0.099883*np.tanh(((data["avg_buro_buro_bal_status_2"]) * (((data["te_FLAG_DOCUMENT_20"]) - ((((((3.0) < (data["avg_buro_buro_bal_status_2"]))*1.)) / 2.0)))))) +
                    0.099922*np.tanh((((((((np.where(data["avg_buro_buro_bal_status_3"]>0, data["te_FLAG_DOCUMENT_20"], data["te_FLAG_DOCUMENT_19"] )) > ((((((data["te_FLAG_DOCUMENT_19"]) * (data["avg_buro_buro_bal_status_5"]))) + (-1.0))/2.0)))*1.)) * (data["avg_buro_buro_bal_status_3"]))) / 2.0)) +
                    0.099883*np.tanh(((np.where(data["te_FLAG_DOCUMENT_11"]>0, (-1.0*((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]))), (((data["te_FLAG_DOCUMENT_11"]) > (-1.0))*1.) )) * (np.maximum(((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"])), (((-1.0*((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]))))))))) +
                    0.029386*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) - ((((data["PRODUCT_COMBINATION_POS_industry_without_interest"]) > (((((1.0) + (((data["cnt_FLAG_DOCUMENT_10"]) * (data["te_FLAG_DOCUMENT_19"]))))) + ((-1.0*((1.0)))))))*1.)))) +
                    0.098671*np.tanh(np.where(((((-2.0) + (data["avg_buro_buro_bal_status_2"]))) / 2.0)>0, data["avg_buro_buro_bal_status_2"], ((((-3.0) + (data["te_FLAG_DOCUMENT_20"]))) * (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((data["avg_buro_buro_bal_status_2"]))))) )) +
                    0.088941*np.tanh((((3.0) < (np.where(data["avg_buro_buro_bal_status_2"]>0, data["SK_ID_BUREAU"], ((((3.0) - (data["SK_ID_BUREAU"]))) - (1.0)) )))*1.)) +
                    0.057757*np.tanh(np.minimum(((np.minimum((((((data["avg_buro_buro_bal_status_2"]) + (data["SK_ID_BUREAU"]))/2.0))), ((0.0))))), (((((((-1.0*(((((-1.0) + (((data["avg_buro_buro_bal_status_2"]) - (3.0))))/2.0))))) * 2.0)) * 2.0))))) +
                    0.099844*np.tanh(np.where(np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["avg_buro_buro_bal_status_2"])))>0, data["avg_buro_buro_bal_status_2"], np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["te_FLAG_DOCUMENT_19"], ((data["avg_buro_buro_bal_status_2"]) * (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"])) ) )) +
                    0.094412*np.tanh(((((np.maximum(((data["EMERGENCYSTATE_MODE"])), ((((((data["EMERGENCYSTATE_MODE"]) + ((((((data["avg_buro_buro_bal_status_4"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) + (data["avg_buro_buro_bal_status_4"]))/2.0)))) * 2.0))))) * (data["avg_buro_buro_bal_status_4"]))) * (3.0))) +
                    0.072255*np.tanh(((data["ty__Consumer_credit"]) * ((((((np.maximum(((((((data["te_HOUSETYPE_MODE"]) - (data["cnt_WALLSMATERIAL_MODE"]))) * 2.0))), ((data["ty__Consumer_credit"])))) * (data["HOUSETYPE_MODE"]))) + (data["avg_buro_buro_bal_status_2"]))/2.0)))) +
                    0.099609*np.tanh((-1.0*((np.maximum(((((np.where(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((data["avg_buro_buro_bal_status_4"]) * 2.0)))) / 2.0)>0, (((0.0)) / 2.0), (6.77353763580322266) )) * (data["avg_buro_buro_bal_status_4"])))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))))) +
                    0.099453*np.tanh((((((data["avg_buro_buro_bal_status_2"]) + ((((((((data["avg_buro_buro_bal_status_2"]) + (data["avg_buro_buro_bal_status_2"]))/2.0)) * (data["AMT_GOODS_PRICE_x"]))) * (data["avg_buro_buro_bal_status_2"]))))/2.0)) * (np.tanh((data["avg_buro_buro_bal_status_2"]))))) +
                    0.099336*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) + (((((((data["te_FLAG_DOCUMENT_19"]) * 2.0)) * (np.maximum((((((((np.tanh((-1.0))) * (data["EMERGENCYSTATE_MODE"]))) < (-2.0))*1.))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))))) * 2.0)))) +
                    0.098867*np.tanh((-1.0*((((data["NAME_GOODS_CATEGORY_Insurance"]) + (np.where(-3.0>0, data["avg_buro_buro_bal_status_2"], ((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, ((data["AMT_GOODS_PRICE_x"]) * (data["EMERGENCYSTATE_MODE"])), data["cc_bal_SK_DPD_DEF"] )) * (data["AMT_GOODS_PRICE_x"])) ))))))) +
                    0.100000*np.tanh((-1.0*((np.where(data["avg_buro_buro_bal_status_4"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], np.where(data["avg_buro_buro_bal_status_2"]>0, data["NAME_GOODS_CATEGORY_Insurance"], (((((data["cc_bal_SK_DPD_DEF"]) > (0.0))*1.)) * (-2.0)) ) ))))) +
                    0.099922*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["avg_buro_buro_bal_status_2"], (((-1.0*((((((data["avg_buro_buro_bal_status_2"]) * (-3.0))) * (((data["avg_buro_buro_bal_status_2"]) * (data["cc_bal_SK_DPD_DEF"])))))))) / 2.0) )) +
                    0.098398*np.tanh((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["te_FLAG_DOCUMENT_19"]))) - (np.tanh((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))) + (((data["avg_buro_buro_bal_status_4"]) * (np.tanh((np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((data["te_FLAG_DOCUMENT_20"])))))))))/2.0)) +
                    0.077022*np.tanh(((((data["cnt_EMERGENCYSTATE_MODE"]) * (((np.maximum(((data["te_FLAG_DOCUMENT_11"])), ((((np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((np.tanh((data["te_FLAG_DOCUMENT_19"])))))) / 2.0))))) / 2.0)))) * 2.0)) +
                    0.099922*np.tanh((-1.0*((((np.where(np.minimum(((2.0)), ((((data["HOUSETYPE_MODE"]) / 2.0))))>0, (((data["te_EMERGENCYSTATE_MODE"]) > ((((2.77418088912963867)) / 2.0)))*1.), data["NAME_GOODS_CATEGORY_Insurance"] )) * (data["te_EMERGENCYSTATE_MODE"])))))) +
                    0.097421*np.tanh(((((data["cnt_EMERGENCYSTATE_MODE"]) * ((5.0)))) * (np.where(data["te_FLAG_DOCUMENT_19"]>0, ((((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)) * (((((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)) * 2.0))), data["NAME_CASH_LOAN_PURPOSE_Business_development"] )))) +
                    0.099101*np.tanh(np.minimum(((((data["avg_buro_buro_bal_status_2"]) * (data["avg_buro_buro_bal_status_4"])))), ((((data["avg_buro_buro_bal_status_4"]) * ((((-1.0*((((-3.0) + (data["avg_buro_buro_bal_status_2"])))))) * (data["avg_buro_buro_bal_status_2"])))))))) +
                    0.099531*np.tanh((((-3.0) > (np.minimum((((((((data["te_FLAG_DOCUMENT_11"]) / 2.0)) + (3.0))/2.0))), ((data["te_FLAG_DOCUMENT_20"])))))*1.)) +
                    0.099062*np.tanh(((np.where(np.maximum(((data["te_FLAG_DOCUMENT_19"])), ((((((-2.0) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) * 2.0))))>0, data["NAME_CASH_LOAN_PURPOSE_Business_development"], ((data["te_FLAG_DOCUMENT_19"]) - (data["cnt_FLAG_DOCUMENT_10"])) )) * (data["AMT_GOODS_PRICE_x"]))) +
                    0.099961*np.tanh(np.minimum(((((data["AMT_CREDIT_x"]) - (((np.where(data["AMT_CREDIT_x"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], 0.0 )) + (-1.0)))))), ((((data["NAME_GOODS_CATEGORY_Insurance"]) * (data["cu__currency_4"])))))) +
                    0.099218*np.tanh(((((data["FLAG_WORK_PHONE"]) * (np.tanh((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))) - ((((((data["EMERGENCYSTATE_MODE"]) * (((data["te_EMERGENCYSTATE_MODE"]) / 2.0)))) > (2.0))*1.)))) +
                    0.099414*np.tanh((((((data["cnt_EMERGENCYSTATE_MODE"]) < (np.minimum(((-2.0)), ((-2.0)))))*1.)) - (np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, ((data["cnt_FLAG_WORK_PHONE"]) * (data["cnt_EMERGENCYSTATE_MODE"])), data["NAME_CASH_LOAN_PURPOSE_Business_development"] )))) +
                    0.094334*np.tanh(np.tanh((((np.maximum(((3.0)), ((data["FLAG_DOCUMENT_21"])))) * (((((data["cnt_FLAG_WORK_PHONE"]) * (np.tanh((data["FLAG_DOCUMENT_21"]))))) - (((data["te_FLAG_WORK_PHONE"]) + (data["cnt_FLAG_WORK_PHONE"]))))))))) +
                    0.097577*np.tanh(((((((((0.0) + ((((0.0) < (-3.0))*1.)))/2.0)) * ((((5.0)) * 2.0)))) + (((((((data["te_FLAG_MOBIL"]) < (data["te_FLAG_DOCUMENT_12"]))*1.)) < (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)))/2.0)) +
                    0.098242*np.tanh((((((((data["te_FLAG_DOCUMENT_21"]) + (0.0))/2.0)) * 2.0)) * (((data["te_FLAG_DOCUMENT_2"]) - (np.minimum(((data["te_FLAG_DOCUMENT_7"])), ((data["te_FLAG_DOCUMENT_12"])))))))) +
                    0.079640*np.tanh(((-2.0) * (np.minimum((((5.0))), ((((data["te_FLAG_DOCUMENT_4"]) * (((np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((((3.0) > (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.))))) * 2.0))))))))) +
                    0.094568*np.tanh(((((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]))) * (np.minimum(((2.0)), ((np.tanh((data["te_FLAG_DOCUMENT_19"])))))))) / 2.0)) / 2.0)) +
                    0.083744*np.tanh(np.minimum((((-1.0*(((((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]) > ((3.0)))*1.)))))), ((((data["cnt_FLAG_DOCUMENT_10"]) - (np.maximum((((((0.0) > (data["cnt_FLAG_DOCUMENT_10"]))*1.))), (((((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]) + (-2.0))/2.0)))))))))) +
                    0.098671*np.tanh((((((2.0) < (np.maximum((((((data["inst_AMT_PAYMENT"]) + (0.0))/2.0))), ((((data["inst_AMT_PAYMENT"]) / 2.0))))))*1.)) + (((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["inst_AMT_PAYMENT"]))) * (data["inst_AMT_PAYMENT"]))))) +
                    0.098281*np.tanh((((((((data["inst_AMT_INSTALMENT"]) / 2.0)) > (2.0))*1.)) + (((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((-3.0)))) * (data["inst_AMT_INSTALMENT"]))))) +
                    0.098984*np.tanh(((np.where((((data["cc_bal_SK_DPD_DEF"]) + (data["Amortized_debt"]))/2.0)>0, ((data["inst_AMT_PAYMENT"]) - (data["NAME_CASH_LOAN_PURPOSE_Business_development"])), data["cc_bal_SK_DPD_DEF"] )) * 2.0)) +
                    0.099922*np.tanh(((np.minimum(((data["NAME_GOODS_CATEGORY_Fitness"])), (((-1.0*((data["EMERGENCYSTATE_MODE"]))))))) * (((data["NAME_CONTRACT_STATUS_Approved"]) * (np.where(data["NAME_GOODS_CATEGORY_Fitness"]>0, data["NAME_GOODS_CATEGORY_Fitness"], (((1.0) + ((-1.0*((data["EMERGENCYSTATE_MODE"])))))/2.0) )))))) +
                    0.099687*np.tanh((((data["Amortized_debt"]) + (np.where(-1.0>0, ((data["NAME_CONTRACT_STATUS_Canceled"]) / 2.0), (((((np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((data["inst_AMT_PAYMENT"])))) + (data["CODE_REJECT_REASON_LIMIT"]))/2.0)) * (data["inst_AMT_PAYMENT"])) )))/2.0)) +
                    0.072255*np.tanh((((((((((data["inst_AMT_INSTALMENT"]) * 2.0)) / 2.0)) / 2.0)) + ((((-1.0*((((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) * (np.tanh((((data["inst_AMT_INSTALMENT"]) - (data["cu__currency_3"])))))))))) / 2.0)))/2.0)) +
                    0.099453*np.tanh(((((((data["NAME_PORTFOLIO_Cash"]) < ((((data["NAME_PORTFOLIO_POS"]) + ((3.68294215202331543)))/2.0)))*1.)) < (((data["NAME_PORTFOLIO_POS"]) + (np.minimum(((data["NAME_SELLER_INDUSTRY_XNA"])), ((np.tanh((data["NFLAG_INSURED_ON_APPROVAL"])))))))))*1.)) +
                    0.100000*np.tanh(((((((data["NAME_CASH_LOAN_PURPOSE_XAP"]) < (((-3.0) / 2.0)))*1.)) > ((((data["NAME_SELLER_INDUSTRY_Connectivity"]) < (np.tanh((((np.tanh((((data["NAME_CASH_LOAN_PURPOSE_XAP"]) / 2.0)))) / 2.0)))))*1.)))*1.)) +
                    0.099453*np.tanh((-1.0*((np.tanh(((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["CNT_INSTALMENT"]))) > ((-1.0*((np.where(data["inst_AMT_PAYMENT"]>0, data["CNT_INSTALMENT"], np.tanh((data["inst_AMT_PAYMENT"])) ))))))*1.)) / 2.0))))))) +
                    0.099297*np.tanh(np.tanh(((((data["cc_bal_AMT_PAYMENT_CURRENT"]) < (np.where(((data["cc_bal_AMT_PAYMENT_CURRENT"]) + (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]))>0, ((((((data["NAME_CONTRACT_STATUS_Approved"]) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))/2.0)) + (data["EMERGENCYSTATE_MODE"]))/2.0), data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"] )))*1.)))) +
                    0.099922*np.tanh(((np.where(data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]>0, np.minimum(((np.tanh(((0.0))))), ((((data["DAYS_FIRST_DRAWING"]) / 2.0)))), np.minimum(((2.0)), ((data["DAYS_FIRST_DRAWING"]))) )) * (data["inst_NUM_INSTALMENT_NUMBER"]))) +
                    0.075459*np.tanh(((np.tanh((((((((data["cc_bal_AMT_RECIVABLE"]) < ((((data["cc_bal_AMT_PAYMENT_CURRENT"]) > (1.0))*1.)))*1.)) + (((((data["cc_bal_AMT_PAYMENT_CURRENT"]) * (data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]))) * (data["cc_bal_AMT_TOTAL_RECEIVABLE"]))))/2.0)))) / 2.0)) +
                    0.083353*np.tanh((-1.0*((((((data["cc_bal_SK_ID_PREV"]) * (np.minimum((((((data["cc_bal_SK_ID_PREV"]) + (data["cc_bal_AMT_TOTAL_RECEIVABLE"]))/2.0))), (((((np.maximum(((data["cc_bal_AMT_TOTAL_RECEIVABLE"])), ((-2.0)))) > (2.0))*1.))))))) * 2.0))))) +
                    0.087925*np.tanh((((data["DAYS_FIRST_DUE"]) > (np.where(((data["cc_bal_AMT_INST_MIN_REGULARITY"]) * 2.0)>0, 2.0, ((np.tanh(((-1.0*((-1.0)))))) * 2.0) )))*1.)) +
                    0.042009*np.tanh(((((np.tanh(((((data["te_EMERGENCYSTATE_MODE"]) + ((((((-2.0) > (data["NAME_CONTRACT_STATUS_Canceled"]))*1.)) - (data["NAME_GOODS_CATEGORY_House_Construction"]))))/2.0)))) / 2.0)) / 2.0)) +
                    0.099648*np.tanh((-1.0*((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) * ((((np.tanh((data["NFLAG_INSURED_ON_APPROVAL"]))) > (np.minimum((((((((data["NFLAG_INSURED_ON_APPROVAL"]) > (data["DAYS_DECISION"]))*1.)) / 2.0))), ((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])))))*1.))))))) +
                    0.099961*np.tanh((((((((1.0) / 2.0)) > (data["cc_bal_MONTHS_BALANCE"]))*1.)) * (np.where(((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) / 2.0)>0, data["NFLAG_INSURED_ON_APPROVAL"], data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"] )))) +
                    0.073740*np.tanh(((np.maximum((((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) < (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)))) > (((data["DAYS_FIRST_DRAWING"]) / 2.0)))*1.)) / 2.0))), ((((data["NAME_PORTFOLIO_Cards"]) - ((2.80302000045776367))))))) / 2.0)) +
                    0.099805*np.tanh(np.minimum(((np.maximum(((((data["DAYS_DECISION"]) * (np.minimum(((((data["NAME_PRODUCT_TYPE_walk_in"]) / 2.0))), ((1.0))))))), ((-3.0))))), (((((data["DAYS_DECISION"]) < (-3.0))*1.))))) +
                    0.038453*np.tanh(((data["NFLAG_INSURED_ON_APPROVAL"]) * ((-1.0*((np.maximum((((-1.0*((np.maximum(((data["NFLAG_INSURED_ON_APPROVAL"])), ((data["DAYS_LAST_DUE_1ST_VERSION"])))))))), ((data["cc_bal_AMT_PAYMENT_CURRENT"]))))))))) +
                    0.099766*np.tanh(((np.minimum(((((data["cu__currency_3"]) * (((((data["te_FLAG_DOCUMENT_19"]) + (-3.0))) * 2.0))))), ((data["cc_bal_AMT_PAYMENT_CURRENT"])))) * ((-1.0*((data["te_FLAG_DOCUMENT_19"])))))) +
                    0.089136*np.tanh(((((data["cc_bal_AMT_PAYMENT_CURRENT"]) * (np.tanh(((((data["CNT_INSTALMENT_FUTURE"]) + ((((-1.0) > (((np.tanh((data["cc_bal_AMT_PAYMENT_CURRENT"]))) * 2.0)))*1.)))/2.0)))))) * 2.0)) +
                    0.092849*np.tanh(((((((np.where(((data["cc_bal_cc_bal_status__Refused"]) - (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))>0, ((data["cc_bal_cc_bal_status__Refused"]) * (data["CNT_INSTALMENT_FUTURE"])), data["te_FLAG_EMAIL"] )) * (-3.0))) - (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) / 2.0)) +
                    0.099609*np.tanh(((np.minimum(((0.0)), ((((np.minimum(((0.0)), ((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) * (np.maximum(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0))), ((data["cc_bal_AMT_INST_MIN_REGULARITY"]))))))))) * 2.0))))) * 2.0)) +
                    0.095115*np.tanh(((((((data["te_FLAG_DOCUMENT_20"]) + (2.0))/2.0)) < ((((1.0) < (((data["CNT_INSTALMENT_FUTURE"]) - ((-1.0*((((data["cc_bal_cc_bal_status__Refused"]) * 2.0))))))))*1.)))*1.)))


In [17]:
def GPII(data):
    return Output(0.100000*np.tanh(((((((data["te_ORGANIZATION_TYPE"]) - (((data["EXT_SOURCE_3"]) * 2.0)))) - (((data["EXT_SOURCE_2"]) - ((-1.0*((data["EXT_SOURCE_2"])))))))) - (data["EXT_SOURCE_1"]))) +
                    0.100000*np.tanh(((((data["NAME_CONTRACT_STATUS_Refused"]) / 2.0)) + (((((((np.minimum(((data["te_OCCUPATION_TYPE"])), ((data["DAYS_BIRTH"])))) - (data["EXT_SOURCE_3"]))) * 2.0)) - (((((data["EXT_SOURCE_2"]) * 2.0)) + (data["EXT_SOURCE_3"]))))))) +
                    0.100000*np.tanh(((((((-3.0) * (data["EXT_SOURCE_2"]))) - (data["EXT_SOURCE_3"]))) + (((np.tanh((data["te_OCCUPATION_TYPE"]))) - (((data["EXT_SOURCE_1"]) + (((data["EXT_SOURCE_3"]) - (0.0))))))))) +
                    0.100000*np.tanh(((data["te_OCCUPATION_TYPE"]) + (((((((data["NAME_CONTRACT_STATUS_Refused"]) - ((((((data["EXT_SOURCE_1"]) + (np.minimum(((data["EXT_SOURCE_3"])), ((data["EXT_SOURCE_3"])))))/2.0)) * 2.0)))) - (data["EXT_SOURCE_2"]))) * 2.0)))) +
                    0.100000*np.tanh(((((((np.tanh((data["NAME_CONTRACT_STATUS_Refused"]))) - (data["EXT_SOURCE_3"]))) - (data["EXT_SOURCE_2"]))) + (((np.minimum(((((data["EXT_SOURCE_3"]) * (-3.0)))), ((data["ca__Active"])))) - (data["EXT_SOURCE_2"]))))) +
                    0.100000*np.tanh(((data["te_ORGANIZATION_TYPE"]) + (((((((data["NAME_CONTRACT_STATUS_Refused"]) - (data["EXT_SOURCE_2"]))) - (np.where(data["EXT_SOURCE_1"]>0, (9.09630203247070312), data["ca__Closed"] )))) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))))) +
                    0.100000*np.tanh(((((data["te_OCCUPATION_TYPE"]) + ((((((((-1.0*((data["EXT_SOURCE_2"])))) + (data["te_NAME_EDUCATION_TYPE"]))) - (((data["EXT_SOURCE_3"]) + (data["EXT_SOURCE_1"]))))) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))))) - (data["EXT_SOURCE_3"]))) +
                    0.100000*np.tanh(((((data["NAME_PRODUCT_TYPE_walk_in"]) + (((((((np.tanh(((-1.0*((((2.0) - (data["cc_bal_AMT_TOTAL_RECEIVABLE"])))))))) + (data["cc_bal_AMT_TOTAL_RECEIVABLE"]))) - (data["EXT_SOURCE_2"]))) - (data["EXT_SOURCE_3"]))))) * 2.0)) +
                    0.100000*np.tanh(((((((((((((data["te_NAME_EDUCATION_TYPE"]) - (data["CODE_GENDER"]))) - (data["DAYS_FIRST_DRAWING"]))) * 2.0)) * 2.0)) - (((data["EXT_SOURCE_3"]) * 2.0)))) * 2.0)) +
                    0.100000*np.tanh((((((((data["REGION_RATING_CLIENT"]) + (data["te_ORGANIZATION_TYPE"]))/2.0)) + (((np.maximum(((-3.0)), ((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])))) + (np.minimum(((data["DAYS_CREDIT"])), ((((data["cnt_FLAG_DOCUMENT_3"]) - (data["EXT_SOURCE_2"])))))))))) * 2.0)) +
                    0.100000*np.tanh((((((((7.0)) * (data["ty__Microloan"]))) - (data["EXT_SOURCE_3"]))) + ((((((data["CNT_INSTALMENT_FUTURE"]) * (data["CNT_INSTALMENT_FUTURE"]))) + (((data["EXT_SOURCE_1"]) * (((-1.0) * 2.0)))))/2.0)))) +
                    0.100000*np.tanh((((((-1.0*((((data["EXT_SOURCE_3"]) - (0.0)))))) + (data["NAME_YIELD_GROUP_high"]))) - ((-1.0*((np.where(((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) / 2.0)) / 2.0)>0, (8.0), data["te_FLAG_OWN_CAR"] ))))))) +
                    0.100000*np.tanh(((((data["CNT_PAYMENT"]) - (((((((data["EXT_SOURCE_1"]) + (data["SK_ID_PREV_y"]))) - ((((data["DEF_30_CNT_SOCIAL_CIRCLE"]) + (((data["CNT_PAYMENT"]) - (data["EXT_SOURCE_2"]))))/2.0)))) * 2.0)))) + (data["NAME_CLIENT_TYPE_New"]))) +
                    0.100000*np.tanh(((((((np.where(((((2.0) / 2.0)) - (data["EXT_SOURCE_2"]))>0, data["te_CODE_GENDER"], -2.0 )) + (data["te_NAME_CONTRACT_TYPE"]))) * 2.0)) + (data["cnt_CODE_GENDER"]))) +
                    0.100000*np.tanh(((((((((data["REG_CITY_NOT_LIVE_CITY"]) + (((((data["ty__Microloan"]) * 2.0)) * 2.0)))) + (np.maximum(((data["cc_bal_CNT_DRAWINGS_CURRENT"])), ((((data["NAME_CONTRACT_STATUS_Refused"]) - (data["cnt_REG_CITY_NOT_LIVE_CITY"])))))))) + (data["cc_bal_CNT_DRAWINGS_CURRENT"]))) * 2.0)) +
                    0.100000*np.tanh((((((((((data["te_NAME_EDUCATION_TYPE"]) - (((data["AMT_ANNUITY"]) - (((data["CNT_INSTALMENT_FUTURE"]) + (data["ty__Microloan"]))))))) + (data["DEF_30_CNT_SOCIAL_CIRCLE"]))/2.0)) * 2.0)) * 2.0)) +
                    0.100000*np.tanh(((data["te_FLAG_DOCUMENT_3"]) + (((((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) * 2.0)) + (((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) * 2.0)) + (((0.0) + (data["DAYS_ID_PUBLISH"]))))))) - (data["cnt_NAME_FAMILY_STATUS"]))))) +
                    0.100000*np.tanh(((((data["CNT_INSTALMENT_FUTURE"]) + (((data["NAME_CLIENT_TYPE_New"]) + (((data["avg_buro_buro_bal_status_1"]) + (((data["te_WALLSMATERIAL_MODE"]) - (data["PRODUCT_COMBINATION_POS_industry_with_interest"]))))))))) + (((data["NAME_CLIENT_TYPE_New"]) - (data["DAYS_FIRST_DRAWING"]))))) +
                    0.100000*np.tanh(((np.where(data["NAME_YIELD_GROUP_low_action"]>0, -3.0, ((data["REGION_RATING_CLIENT_W_CITY"]) + (((data["CNT_PAYMENT"]) + (np.maximum(((-3.0)), ((data["NAME_YIELD_GROUP_high"]))))))) )) + (data["ty__Microloan"]))) +
                    0.100000*np.tanh(((((data["AMT_ANNUITY_x"]) * 2.0)) + (((((((data["AMT_ANNUITY_x"]) * 2.0)) + (np.maximum(((data["NAME_YIELD_GROUP_XNA"])), ((((data["DAYS_LAST_PHONE_CHANGE"]) / 2.0))))))) + (((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) * (-3.0))))))) +
                    0.100000*np.tanh(((((data["cnt_FLAG_OWN_CAR"]) - (((data["inst_NUM_INSTALMENT_VERSION"]) - (np.where(data["EXT_SOURCE_1"]>0, -3.0, data["CODE_REJECT_REASON_SCOFR"] )))))) - (data["RATE_DOWN_PAYMENT"]))) +
                    0.100000*np.tanh(((((((data["OWN_CAR_AGE"]) - (((((data["CODE_GENDER"]) - (((data["OWN_CAR_AGE"]) + (data["OWN_CAR_AGE"]))))) / 2.0)))) * 2.0)) + (np.maximum(((data["CNT_PAYMENT"])), ((data["ty__Microloan"])))))) +
                    0.100000*np.tanh(((((((data["DEF_60_CNT_SOCIAL_CIRCLE"]) - (((data["NAME_YIELD_GROUP_low_action"]) + ((((data["avg_buro_buro_bal_status_1"]) < (data["PRODUCT_COMBINATION_Cash_Street__low"]))*1.)))))) + (data["avg_buro_buro_bal_status_1"]))) - (data["AMT_ANNUITY"]))) +
                    0.100000*np.tanh(np.where(((data["ty__Mortgage"]) * 2.0)>0, -3.0, (((data["cnt_DAYS_BIRTH"]) + ((((((data["NAME_PORTFOLIO_XNA"]) + (np.maximum(((data["CODE_REJECT_REASON_HC"])), ((0.0)))))/2.0)) - (data["CHANNEL_TYPE_Channel_of_corporate_sales"]))))/2.0) )) +
                    0.100000*np.tanh(((((((data["DAYS_LAST_PHONE_CHANGE"]) + (data["nans"]))) + (((((((data["SK_DPD"]) + (data["CNT_PAYMENT"]))) / 2.0)) * 2.0)))) + (((((-1.0*((data["PRODUCT_COMBINATION_Cash_Street__low"])))) + (data["cnt_ORGANIZATION_TYPE"]))/2.0)))) +
                    0.100000*np.tanh(np.where(data["inst_AMT_PAYMENT"]>0, (-1.0*((data["Active"]))), (((-1.0) > (((np.minimum((((1.0))), ((data["inst_AMT_PAYMENT"])))) * 2.0)))*1.) )) +
                    0.100000*np.tanh(((np.where(np.tanh((np.where((10.0)>0, data["cc_bal_AMT_RECIVABLE"], data["cc_bal_AMT_BALANCE"] )))>0, data["cc_bal_CNT_DRAWINGS_CURRENT"], ((data["SK_DPD"]) * ((6.0))) )) + ((-1.0*((data["FLAG_DOCUMENT_18"])))))) +
                    0.100000*np.tanh((((((((((((data["cnt_FLAG_DOCUMENT_16"]) + (data["AMT_ANNUITY_x"]))/2.0)) - (data["ty__Mortgage"]))) + (data["ty__Microloan"]))) * 2.0)) + (np.tanh(((((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) > (data["ty__Microloan"]))*1.)))))) +
                    0.099961*np.tanh(np.where(data["ca__Sold"]>0, 2.0, (((-1.0*((((data["RATE_DOWN_PAYMENT"]) + (((data["AMT_GOODS_PRICE_x"]) * (((((data["AMT_GOODS_PRICE_x"]) * 2.0)) - (-1.0)))))))))) + (data["cnt_NAME_INCOME_TYPE"])) )) +
                    0.099961*np.tanh(((((((3.0) * ((((data["CHANNEL_TYPE_AP___Cash_loan_"]) > (((data["inst_AMT_PAYMENT"]) / 2.0)))*1.)))) * 2.0)) - ((-1.0*((np.minimum(((data["cnt_WEEKDAY_APPR_PROCESS_START"])), (((((data["CHANNEL_TYPE_AP___Cash_loan_"]) > (data["MONTHS_BALANCE"]))*1.)))))))))) +
                    0.099961*np.tanh(((((((np.maximum(((data["CNT_INSTALMENT_FUTURE"])), (((((((-1.0*(((((((data["te_FLAG_DOCUMENT_6"]) * 2.0)) + (data["CNT_INSTALMENT_FUTURE"]))/2.0))))) * 2.0)) * 2.0))))) - (((data["te_FLAG_DOCUMENT_6"]) * 2.0)))) / 2.0)) * 2.0)) +
                    0.100000*np.tanh((((((data["te_NAME_FAMILY_STATUS"]) - (data["PRODUCT_COMBINATION_Cash_Street__low"]))) + (np.minimum((((((-1.0*((data["CHANNEL_TYPE_Channel_of_corporate_sales"])))) - (((((data["AMT_CREDIT_SUM_LIMIT"]) * 2.0)) * 2.0))))), ((((data["DAYS_REGISTRATION"]) / 2.0))))))/2.0)) +
                    0.099805*np.tanh(((((-1.0*((data["Active"])))) + ((((((data["HOUSETYPE_MODE"]) - (data["FLAG_DOCUMENT_18"]))) + ((((((data["CODE_REJECT_REASON_SCO"]) + (data["CHANNEL_TYPE_Channel_of_corporate_sales"]))/2.0)) * (-2.0))))/2.0)))/2.0)) +
                    0.100000*np.tanh((((((((-1.0*((((data["inst_NUM_INSTALMENT_VERSION"]) * (data["inst_NUM_INSTALMENT_VERSION"])))))) + (data["NFLAG_INSURED_ON_APPROVAL"]))/2.0)) + ((((((((data["cnt_FLAG_PHONE"]) + (data["NFLAG_INSURED_ON_APPROVAL"]))/2.0)) - (data["inst_AMT_PAYMENT"]))) / 2.0)))/2.0)) +
                    0.091051*np.tanh((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) + (np.maximum(((((-1.0) - ((-1.0*((data["FLAG_DOCUMENT_2"]))))))), ((np.where(data["cc_bal_AMT_TOTAL_RECEIVABLE"]>0, data["FLAG_WORK_PHONE"], data["cc_bal_AMT_TOTAL_RECEIVABLE"] ))))))/2.0)) +
                    0.099961*np.tanh(np.minimum((((((((((data["NAME_PAYMENT_TYPE_XNA"]) / 2.0)) + (data["cnt_FLAG_DOCUMENT_13"]))/2.0)) - (np.tanh(((((((data["cnt_FLAG_DOCUMENT_2"]) < (data["NAME_PAYMENT_TYPE_XNA"]))*1.)) * (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"])))))))), (((5.0))))) +
                    0.099179*np.tanh(((((-1.0*((((data["DAYS_ENDDATE_FACT"]) + (np.minimum(((data["YEARS_BUILD_MODE"])), (((11.45870018005371094)))))))))) + (((((-1.0*((((data["avg_buro_buro_bal_status_0"]) + (data["FLAG_PHONE"])))))) + ((-1.0*(((0.0))))))/2.0)))/2.0)) +
                    0.100000*np.tanh(((np.minimum((((((-2.0) > (data["EXT_SOURCE_3"]))*1.))), ((((data["avg_buro_buro_bal_status_1"]) - (np.minimum(((data["AMT_REQ_CREDIT_BUREAU_QRT"])), ((((data["EXT_SOURCE_3"]) - (np.tanh((-3.0))))))))))))) * (3.0))) +
                    0.094959*np.tanh((((data["SK_DPD"]) > ((((data["ty__Microloan"]) + (np.where((-1.0*((data["NAME_GOODS_CATEGORY_Furniture"])))>0, (((data["te_REG_REGION_NOT_LIVE_REGION"]) + (((0.0) / 2.0)))/2.0), (((-1.0*((data["ty__Microloan"])))) / 2.0) )))/2.0)))*1.)) +
                    0.099883*np.tanh(((np.where((((((data["NAME_GOODS_CATEGORY_Insurance"]) - (data["cnt_FLAG_DOCUMENT_16"]))) > (data["ty__Microloan"]))*1.)>0, -2.0, data["ty__Microloan"] )) - (np.tanh((((data["cnt_FLAG_DOCUMENT_2"]) * 2.0)))))) +
                    0.099922*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NAME_FAMILY_STATUS"], (((((((2.0)) + (np.where(np.minimum(((-3.0)), ((data["FLAG_DOCUMENT_2"])))>0, data["cnt_FLAG_DOCUMENT_2"], data["NAME_YIELD_GROUP_middle"] )))/2.0)) < (data["NAME_YIELD_GROUP_middle"]))*1.) )) +
                    0.099570*np.tanh(np.where((((data["DAYS_LAST_DUE"]) < (data["RATE_DOWN_PAYMENT"]))*1.)>0, np.minimum(((0.0)), (((-1.0*((data["RATE_DOWN_PAYMENT"])))))), (((((data["DAYS_LAST_DUE"]) + (data["te_CNT_CHILDREN"]))/2.0)) / 2.0) )) +
                    0.100000*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], ((data["AMT_ANNUITY_x"]) - ((((((data["AMT_ANNUITY_x"]) + (data["cnt_FLAG_DOCUMENT_2"]))/2.0)) * (((data["inst_DAYS_INSTALMENT"]) + (data["AMT_ANNUITY_x"])))))) )) +
                    0.089644*np.tanh(((((((-1.0*((data["AMT_GOODS_PRICE_x"])))) < ((((data["CHANNEL_TYPE_Channel_of_corporate_sales"]) > (((data["AMT_GOODS_PRICE_x"]) / 2.0)))*1.)))*1.)) * (np.where(data["AMT_GOODS_PRICE_x"]>0, data["FLAG_DOCUMENT_2"], 3.0 )))) +
                    0.100000*np.tanh((-1.0*(((((((((((((-1.0*((((np.tanh((((-1.0) * 2.0)))) / 2.0))))) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)) + (data["DAYS_EMPLOYED"]))) < (data["CODE_REJECT_REASON_SCO"]))*1.)) / 2.0))))) +
                    0.099883*np.tanh((((((data["inst_DAYS_INSTALMENT"]) < (data["inst_DAYS_ENTRY_PAYMENT"]))*1.)) + (np.minimum(((((data["inst_DAYS_INSTALMENT"]) * (-1.0)))), ((data["AMT_CREDIT_x"])))))) +
                    0.099961*np.tanh(np.where(data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]>0, (-1.0*((np.maximum((((2.0))), ((((0.0) * 2.0))))))), (((((data["DAYS_ID_PUBLISH"]) - ((-1.0*((data["DAYS_DECISION"])))))) > (2.0))*1.) )) +
                    0.099883*np.tanh(np.where((((np.maximum(((data["NAME_FAMILY_STATUS"])), ((((data["NAME_TYPE_SUITE_Unaccompanied"]) * 2.0))))) < (-1.0))*1.)>0, 2.0, (((np.where(data["NAME_FAMILY_STATUS"]>0, -1.0, data["NAME_GOODS_CATEGORY_Direct_Sales"] )) + (data["NAME_FAMILY_STATUS"]))/2.0) )) +
                    0.099805*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, 1.0, np.minimum(((data["cc_bal_AMT_BALANCE"])), ((np.where(data["cc_bal_SK_DPD_DEF"]>0, np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((3.0))), (-1.0*((data["CHANNEL_TYPE_Channel_of_corporate_sales"]))) )))) )) +
                    0.092927*np.tanh(((((np.where(((np.minimum(((data["cnt_FLAG_DOCUMENT_13"])), ((data["FLAG_DOCUMENT_2"])))) - (data["cc_bal_SK_DPD_DEF"]))>0, ((data["CNT_INSTALMENT_FUTURE"]) * 2.0), data["cnt_FLAG_DOCUMENT_13"] )) - (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) + (data["NAME_GOODS_CATEGORY_Insurance"]))) +
                    0.086753*np.tanh(np.minimum((((((-1.0*((data["NAME_GOODS_CATEGORY_Furniture"])))) - (data["inst_AMT_PAYMENT"])))), (((((((data["inst_AMT_PAYMENT"]) / 2.0)) < (data["NAME_GOODS_CATEGORY_Furniture"]))*1.))))) +
                    0.091794*np.tanh((((((np.maximum(((data["cc_bal_SK_DPD_DEF"])), ((data["AMT_GOODS_PRICE_y"])))) / 2.0)) > (((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]) * (data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]))))*1.)) +
                    0.099922*np.tanh((-1.0*((np.where(data["CNT_INSTALMENT"]>0, data["Active"], np.minimum(((data["SK_ID_PREV_y"])), ((2.0))) ))))) +
                    0.100000*np.tanh(np.where(data["CNT_INSTALMENT_FUTURE"]>0, ((data["SK_DPD"]) + (np.minimum(((data["CNT_INSTALMENT_FUTURE"])), ((data["SK_ID_PREV_x"]))))), np.tanh(((-1.0*(((((data["SK_ID_PREV_x"]) > (data["NAME_PRODUCT_TYPE_x_sell"]))*1.)))))) )) +
                    0.099961*np.tanh(np.tanh(((((((((data["SK_DPD"]) - (data["FLAG_DOCUMENT_18"]))) * 2.0)) > ((((data["NAME_GOODS_CATEGORY_Insurance"]) + ((-1.0*((data["SK_DPD"])))))/2.0)))*1.)))) +
                    0.100000*np.tanh(np.where((-1.0*((data["SK_DPD"])))>0, ((np.minimum((((-1.0*(((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) + (data["FLAG_DOCUMENT_16"]))/2.0)))))), (((-1.0*((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))))))) / 2.0), -2.0 )) +
                    0.100000*np.tanh((((((data["cnt_FLAG_DOCUMENT_2"]) + ((((data["SK_DPD"]) + (((data["cnt_FLAG_DOCUMENT_2"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))/2.0)))) > (((data["FLAG_DOCUMENT_2"]) * (2.0))))*1.)) +
                    0.100000*np.tanh((((np.where(np.where(data["CNT_INSTALMENT_FUTURE"]>0, -2.0, data["SK_DPD"] )>0, -2.0, data["SK_DPD"] )) + ((((data["SK_DPD"]) + ((-1.0*((data["te_REG_REGION_NOT_LIVE_REGION"])))))/2.0)))/2.0)) +
                    0.099961*np.tanh((-1.0*((np.maximum(((((((((((((data["WEEKDAY_APPR_PROCESS_START_SATURDAY"]) * 2.0)) + (data["ty__Car_loan"]))/2.0)) + (data["inst_NUM_INSTALMENT_VERSION"]))/2.0)) / 2.0))), ((data["ty__Mortgage"]))))))) +
                    0.100000*np.tanh(((((((((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (((data["DAYS_REGISTRATION"]) / 2.0)))/2.0)) / 2.0)) + ((((data["DAYS_REGISTRATION"]) + (data["CNT_INSTALMENT_FUTURE"]))/2.0)))) - (2.0))) > ((-1.0*((data["DAYS_REGISTRATION"])))))*1.)) +
                    0.095858*np.tanh(np.where(data["cc_bal_SK_DPD_DEF"]>0, 1.0, ((((data["FLAG_DOCUMENT_2"]) + ((((((data["DAYS_FIRST_DUE"]) + ((-1.0*((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"])))))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)))) / 2.0) )) +
                    0.099726*np.tanh((((13.15089130401611328)) * (np.where(((data["ca__Sold"]) - (data["cc_bal_SK_DPD_DEF"]))>0, np.minimum(((data["FLAG_DOCUMENT_2"])), ((((np.minimum(((((data["cc_bal_cc_bal_status__Sent_proposal"]) * 2.0))), ((data["ca__Sold"])))) * 2.0)))), data["cc_bal_SK_DPD_DEF"] )))) +
                    0.100000*np.tanh(np.maximum(((np.maximum((((((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)) + (data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"])))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))), (((((((((data["cc_bal_CNT_DRAWINGS_OTHER_CURRENT"]) + (data["ca__Sold"]))) > ((5.0)))*1.)) * 2.0))))) +
                    0.099922*np.tanh((-1.0*(((((data["CHANNEL_TYPE_Channel_of_corporate_sales"]) + (np.where((-1.0*((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])))>0, data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"], data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"] )))/2.0))))) +
                    0.093787*np.tanh((((((((6.87359857559204102)) * ((((6.87359476089477539)) * (((((data["cc_bal_AMT_BALANCE"]) - (data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]))) * (np.where(data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]>0, 1.0, 3.0 )))))))) * 2.0)) / 2.0)) +
                    0.099922*np.tanh(((np.minimum(((((1.0) - (data["NAME_YIELD_GROUP_high"])))), (((((((data["NAME_YIELD_GROUP_high"]) > ((((data["NAME_YIELD_GROUP_high"]) > (((data["NAME_YIELD_GROUP_high"]) + (data["cnt_HOUR_APPR_PROCESS_START_x"]))))*1.)))*1.)) * 2.0))))) / 2.0)) +
                    0.099805*np.tanh(((data["NONLIVINGAPARTMENTS_MODE"]) - ((((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) + ((((data["FLAG_DOCUMENT_14"]) > ((-1.0*((((data["cc_bal_cc_bal_status__Sent_proposal"]) * ((-1.0*(((-1.0*((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])))))))))))))*1.)))) + (data["FLAG_DOCUMENT_14"]))/2.0)))) +
                    0.099883*np.tanh((((((data["cnt_FLAG_DOCUMENT_13"]) + (np.minimum(((data["cc_bal_SK_ID_PREV"])), ((((((((data["cnt_FLAG_DOCUMENT_18"]) + (data["cc_bal_SK_ID_PREV"]))/2.0)) + (((data["cnt_FLAG_DOCUMENT_18"]) * 2.0)))/2.0))))))) + ((-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"])))))/2.0)) +
                    0.094959*np.tanh((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) + ((-1.0*((np.where(((((data["AMT_REQ_CREDIT_BUREAU_MON"]) / 2.0)) + (data["AMT_CREDIT_SUM_LIMIT"]))>0, np.maximum(((data["AMT_CREDIT_SUM_LIMIT"])), ((((data["AMT_REQ_CREDIT_BUREAU_MON"]) / 2.0)))), data["FLOORSMAX_MODE"] ))))))/2.0)) +
                    0.099805*np.tanh((((data["cnt_FLAG_DOCUMENT_2"]) < ((((data["NAME_GOODS_CATEGORY_Insurance"]) + ((((data["DAYS_FIRST_DUE"]) > (((np.tanh(((((1.0) < (data["DAYS_FIRST_DUE"]))*1.)))) * 2.0)))*1.)))/2.0)))*1.)) +
                    0.100000*np.tanh(((((((data["NAME_GOODS_CATEGORY_Gardening"]) < (data["SK_DPD"]))*1.)) + (np.minimum(((np.where(data["SK_DPD"]>0, data["cnt_ORGANIZATION_TYPE"], data["REGION_POPULATION_RELATIVE"] ))), (((((1.0) > (data["REGION_POPULATION_RELATIVE"]))*1.))))))/2.0)) +
                    0.098710*np.tanh(((((data["FLAG_DOCUMENT_16"]) * (((data["CODE_REJECT_REASON_SCO"]) + ((-1.0*((data["REGION_RATING_CLIENT_W_CITY"])))))))) * 2.0)) +
                    0.099766*np.tanh(((np.maximum(((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["te_ORGANIZATION_TYPE"]))/2.0)) + (((data["FLAG_DOCUMENT_2"]) * 2.0)))/2.0))), ((data["cu__currency_3"])))) + ((-1.0*((((((data["NAME_GOODS_CATEGORY_Gardening"]) + (data["cnt_FLAG_DOCUMENT_2"]))) / 2.0))))))) +
                    0.099883*np.tanh(np.where(np.where(data["cnt_NAME_HOUSING_TYPE"]>0, data["cnt_FLAG_DOCUMENT_16"], data["cnt_NAME_HOUSING_TYPE"] )>0, data["FLAG_DOCUMENT_16"], np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["FLAG_DOCUMENT_16"], data["cnt_FLAG_DOCUMENT_16"] ) )) +
                    0.087261*np.tanh(np.minimum(((((data["cnt_FLAG_DOCUMENT_16"]) - ((((np.tanh((data["NAME_GOODS_CATEGORY_Clothing_and_Accessories"]))) > (np.tanh((np.minimum((((((data["cnt_FLAG_DOCUMENT_16"]) + (data["cnt_FLAG_DOCUMENT_16"]))/2.0))), ((data["FLAG_DOCUMENT_16"])))))))*1.))))), ((data["te_FLAG_DOCUMENT_16"])))) +
                    0.100000*np.tanh((((((((-1.0*((np.minimum(((data["AMT_CREDIT_SUM_DEBT"])), (((0.37224778532981873)))))))) * 2.0)) * (data["AMT_CREDIT_SUM_DEBT"]))) - (np.where(data["AMT_CREDIT_SUM_DEBT"]>0, -3.0, (((-1.0*((data["AMT_CREDIT_SUM_DEBT"])))) * 2.0) )))) +
                    0.099922*np.tanh((((((data["DAYS_EMPLOYED"]) > (np.minimum(((np.tanh((np.minimum(((data["DAYS_EMPLOYED"])), ((((0.0) + ((((-1.0*((1.0)))) / 2.0)))))))))), ((data["avg_buro_buro_bal_status_0"])))))*1.)) / 2.0)) +
                    0.100000*np.tanh(np.where(data["EXT_SOURCE_3"]>0, data["EXT_SOURCE_3"], ((data["EXT_SOURCE_3"]) * (((data["EXT_SOURCE_3"]) + (2.0)))) )) +
                    0.100000*np.tanh(((((np.maximum(((np.maximum(((-3.0)), ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0)))))), (((((-1.0*((((data["EXT_SOURCE_3"]) * 2.0))))) + (-3.0)))))) * 2.0)) * 2.0)) +
                    0.098085*np.tanh(np.minimum(((((((-1.0*((data["ty__Car_loan"])))) + (np.maximum((((((-1.0*((data["inst_AMT_PAYMENT"])))) / 2.0))), ((data["AMT_REQ_CREDIT_BUREAU_DAY"])))))/2.0))), ((((((2.0) * 2.0)) - (data["te_LIVE_REGION_NOT_WORK_REGION"])))))) +
                    0.099922*np.tanh(((np.minimum(((np.where((((data["te_OCCUPATION_TYPE"]) < (1.0))*1.)>0, data["FLAG_DOCUMENT_2"], np.minimum(((data["AMT_CREDIT_y"])), ((data["AMT_CREDIT_y"]))) ))), ((np.where(data["cc_bal_cc_bal_status__Sent_proposal"]>0, data["FLAG_DOCUMENT_2"], data["AMT_CREDIT_y"] ))))) / 2.0)) +
                    0.099844*np.tanh((((-1.0*(((((-1.0) > (((data["PRODUCT_COMBINATION_Cash_Street__low"]) * (data["inst_AMT_PAYMENT"]))))*1.))))) - ((((np.maximum(((-2.0)), ((data["PRODUCT_COMBINATION_Cash_Street__low"])))) > (((2.0) * 2.0)))*1.)))) +
                    0.099648*np.tanh(((np.maximum(((((data["Returned_to_the_store"]) + (np.tanh((((data["cc_bal_cc_bal_status__Sent_proposal"]) * ((((data["Returned_to_the_store"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0))))))))), ((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["te_CNT_CHILDREN"]))))))) * 2.0)) +
                    0.099648*np.tanh((((((data["cnt_FLAG_DOCUMENT_15"]) + (data["FLAG_DOCUMENT_2"]))/2.0)) + (np.where((((1.49800336360931396)) - (((-3.0) + (data["ca__Sold"]))))>0, data["NAME_GOODS_CATEGORY_Insurance"], data["ca__Sold"] )))) +
                    0.099922*np.tanh((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) > ((((((3.23952984809875488)) * (((((((data["cnt_FLAG_DOCUMENT_5"]) / 2.0)) - (np.where(data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]>0, data["CODE_REJECT_REASON_SCOFR"], -1.0 )))) * 2.0)))) * 2.0)))*1.)) +
                    0.080813*np.tanh((((np.tanh((np.tanh(((4.0)))))) + (np.minimum(((data["AMT_CREDIT_x"])), ((((((data["NAME_GOODS_CATEGORY_Insurance"]) - (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) * (2.0)))))))/2.0)) +
                    0.099961*np.tanh((-1.0*((((((((6.0)) * (data["cc_bal_SK_DPD_DEF"]))) > ((((1.0) + (data["AMT_GOODS_PRICE_x"]))/2.0)))*1.))))) +
                    0.083783*np.tanh((-1.0*(((((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]) > ((((((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]) < (data["inst_AMT_PAYMENT"]))*1.)) * (data["cnt_HOUR_APPR_PROCESS_START_x"]))))*1.))))) +
                    0.099922*np.tanh(np.tanh((((np.where(((np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["AMT_GOODS_PRICE_x"])))) * (data["AMT_CREDIT_y"]))>0, data["cc_bal_CNT_DRAWINGS_CURRENT"], (-1.0*((data["AMT_GOODS_PRICE_x"]))) )) / 2.0)))) +
                    0.099766*np.tanh(((((((((((2.0) > (data["PRODUCT_COMBINATION_Cash_X_Sell__low"]))*1.)) - (1.0))) + ((((((data["NAME_GOODS_CATEGORY_Insurance"]) > ((((data["SK_DPD"]) > (data["cc_bal_cc_bal_status__Sent_proposal"]))*1.)))*1.)) * 2.0)))/2.0)) * 2.0)) +
                    0.076983*np.tanh(((((((((((data["SK_DPD"]) * 2.0)) > (data["PRODUCT_COMBINATION_POS_industry_without_interest"]))*1.)) + (np.tanh((np.tanh((np.tanh(((((data["FLAG_DOCUMENT_2"]) + (((((data["HOUSETYPE_MODE"]) / 2.0)) / 2.0)))/2.0)))))))))/2.0)) * 2.0)) +
                    0.099922*np.tanh(((np.where((((np.maximum(((((2.0) * 2.0))), (((((data["te_FLAG_OWN_REALTY"]) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.))))) < (data["NAME_TYPE_SUITE_Children"]))*1.)>0, data["NAME_TYPE_SUITE_Children"], ((data["CODE_REJECT_REASON_VERIF"]) / 2.0) )) * (data["te_FLAG_OWN_REALTY"]))) +
                    0.099961*np.tanh(np.minimum((((((((((2.68808555603027344)) < (data["AMT_CREDIT_y"]))*1.)) + (0.0))/2.0))), (((((((2.0) > (data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))*1.)) - (data["te_NAME_EDUCATION_TYPE"])))))) +
                    0.099766*np.tanh(((((((data["cu__currency_3"]) * 2.0)) - (data["YEARS_BEGINEXPLUATATION_MODE"]))) * (np.maximum(((data["ORGANIZATION_TYPE"])), (((((-1.0*((((data["YEARS_BEGINEXPLUATATION_MODE"]) + (((data["cc_bal_cc_bal_status__Sent_proposal"]) * 2.0))))))) + (data["ORGANIZATION_TYPE"])))))))) +
                    0.099922*np.tanh((-1.0*(((((np.maximum((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (np.maximum((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (1.0))/2.0))), ((data["WEEKDAY_APPR_PROCESS_START_TUESDAY"])))))/2.0))), ((data["cc_bal_cc_bal_status__Sent_proposal"])))) + (np.minimum(((data["SK_ID_PREV_y"])), ((0.0)))))/2.0))))) +
                    0.099961*np.tanh((((((((((np.maximum(((0.0)), ((data["NAME_CLIENT_TYPE_Repeater"])))) < (data["AMT_CREDIT_y"]))*1.)) * (((data["NAME_CLIENT_TYPE_Repeater"]) / 2.0)))) * 2.0)) * (data["AMT_CREDIT_y"]))) +
                    0.094216*np.tanh(((((-1.0*((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"])))) + ((-1.0*(((((data["NAME_CONTRACT_STATUS_Refused"]) > (np.where(data["cc_bal_SK_DPD_DEF"]>0, data["CODE_REJECT_REASON_XAP"], 1.0 )))*1.))))))/2.0)) +
                    0.098515*np.tanh(np.maximum((((((((data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]) * 2.0)) + (np.minimum(((data["SK_ID_PREV_x"])), ((1.0)))))/2.0))), (((((((1.0) - (data["NAME_GOODS_CATEGORY_Insurance"]))) < (((data["cc_bal_AMT_RECIVABLE"]) / 2.0)))*1.))))) +
                    0.100000*np.tanh(np.minimum(((np.minimum(((data["cc_bal_SK_DPD_DEF"])), ((data["cc_bal_SK_DPD_DEF"]))))), (((-1.0*((np.minimum(((((data["inst_SK_ID_PREV"]) * 2.0))), (((-1.0*((np.tanh(((-1.0*(((((data["AMT_REQ_CREDIT_BUREAU_YEAR"]) + (data["DAYS_LAST_DUE_1ST_VERSION"]))/2.0))))))))))))))))))) +
                    0.099922*np.tanh((-1.0*(((((((data["cc_bal_SK_DPD_DEF"]) < (data["cu__currency_3"]))*1.)) - ((((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) > (np.tanh(((((((data["cc_bal_SK_DPD_DEF"]) > (-1.0))*1.)) + (1.0))))))*1.))))))) +
                    0.099883*np.tanh(((np.minimum(((((data["NONLIVINGAPARTMENTS_MODE"]) * (np.maximum(((data["NAME_TYPE_SUITE_Unaccompanied"])), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))))), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"])))) * (((data["cc_bal_SK_DPD_DEF"]) + (((data["NAME_TYPE_SUITE_Unaccompanied"]) * 2.0)))))) +
                    0.099687*np.tanh(((data["cu__currency_3"]) + (((((data["inst_AMT_PAYMENT"]) * (((data["FLAG_DOCUMENT_14"]) - (np.maximum(((((data["cu__currency_3"]) - (data["inst_AMT_PAYMENT"])))), ((data["cc_bal_SK_DPD_DEF"])))))))) - (data["cnt_FLAG_DOCUMENT_2"]))))) +
                    0.100000*np.tanh(np.tanh((((((data["EXT_SOURCE_2"]) * (data["EXT_SOURCE_2"]))) + (((data["EXT_SOURCE_2"]) * 2.0)))))) +
                    0.100000*np.tanh((-1.0*((((np.minimum((((0.67251223325729370))), ((((((((data["EXT_SOURCE_2"]) + ((-1.0*((-2.0)))))) * 2.0)) * 2.0))))) / 2.0))))) +
                    0.093083*np.tanh(((np.where(data["NAME_GOODS_CATEGORY_Photo___Cinema_Equipment"]>0, data["AMT_CREDIT_y"], (((((data["HOUR_APPR_PROCESS_START_y"]) < (np.tanh((-2.0))))*1.)) / 2.0) )) - ((((1.0) < (data["NAME_GOODS_CATEGORY_Photo___Cinema_Equipment"]))*1.)))) +
                    0.099297*np.tanh((((data["cc_bal_SK_DPD_DEF"]) + (((data["cnt_FLAG_DOCUMENT_14"]) + (np.maximum(((((((np.tanh((data["ENTRANCES_AVG"]))) / 2.0)) - (1.0)))), ((((data["cu__currency_3"]) - (data["ENTRANCES_MEDI"])))))))))/2.0)) +
                    0.099570*np.tanh((((np.where(data["PRODUCT_COMBINATION_POS_industry_without_interest"]>0, np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), (((-1.0*((data["LIVINGAPARTMENTS_AVG"])))))), data["LIVINGAPARTMENTS_AVG"] )) + ((((data["CNT_INSTALMENT_FUTURE"]) > (3.0))*1.)))/2.0)) +
                    0.099687*np.tanh((((((data["FLAG_CONT_MOBILE"]) < (data["cc_bal_SK_DPD_DEF"]))*1.)) + (np.where(np.maximum(((data["cc_bal_SK_DPD_DEF"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))>0, np.tanh((data["FLAG_WORK_PHONE"])), data["NAME_GOODS_CATEGORY_Direct_Sales"] )))) +
                    0.100000*np.tanh(((((((data["FLAG_DOCUMENT_11"]) - (((np.maximum(((data["FLAG_DOCUMENT_11"])), ((data["SK_DPD"])))) / 2.0)))) * (((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (3.0))))) * 2.0)) +
                    0.099570*np.tanh(((((((data["NAME_TYPE_SUITE_Children"]) * (((((data["NAME_TYPE_SUITE_Children"]) * (np.tanh((data["ty__Microloan"]))))) - (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Car_repairs"])), ((np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Car_repairs"])), ((-2.0))))))))))) * 2.0)) * 2.0)) +
                    0.100000*np.tanh(np.minimum(((data["cnt_FLAG_DOCUMENT_13"])), ((((((((3.0) - (data["ty__Mortgage"]))) * 2.0)) + (((data["cnt_FLAG_DOCUMENT_15"]) + (data["cnt_FLAG_DOCUMENT_15"])))))))) +
                    0.099609*np.tanh((((data["NAME_GOODS_CATEGORY_Insurance"]) + ((((((((data["PRODUCT_COMBINATION_POS_industry_without_interest"]) * 2.0)) + (((((10.99971294403076172)) < ((14.19601345062255859)))*1.)))/2.0)) * (np.minimum(((2.0)), (((((data["PRODUCT_COMBINATION_Card_X_Sell"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0))))))))/2.0)) +
                    0.090895*np.tanh((((((((2.0) * (((((((-1.0*((data["avg_buro_MONTHS_BALANCE"])))) + (data["cu__currency_3"]))/2.0)) / 2.0)))) + ((((data["cnt_FLAG_DOCUMENT_11"]) + (((data["cc_bal_SK_DPD_DEF"]) / 2.0)))/2.0)))/2.0)) + (data["cu__currency_3"]))) +
                    0.077999*np.tanh(((((((((0.0) < (data["DAYS_LAST_DUE_1ST_VERSION"]))*1.)) * 2.0)) < (data["DAYS_CREDIT_ENDDATE"]))*1.)) +
                    0.100000*np.tanh(((((((-1.0*(((((np.tanh((data["NAME_CONTRACT_STATUS_Approved"]))) < (np.tanh((data["te_OCCUPATION_TYPE"]))))*1.))))) / 2.0)) + (data["cnt_FLAG_DOCUMENT_17"]))/2.0)) +
                    0.074951*np.tanh(np.where(data["NAME_CLIENT_TYPE_Repeater"]>0, ((data["CODE_GENDER"]) * ((-1.0*(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((data["CODE_GENDER"]) * 2.0)))) + (data["te_ORGANIZATION_TYPE"]))/2.0)))))), data["NAME_GOODS_CATEGORY_Direct_Sales"] )) +
                    0.099375*np.tanh(np.minimum(((((((-1.0*((data["DAYS_BIRTH"])))) + (2.0))/2.0))), ((((data["DAYS_BIRTH"]) * (((data["cnt_DAYS_BIRTH"]) + (np.tanh((2.0)))))))))) +
                    0.099062*np.tanh((-1.0*(((((((-1.0) - (data["DAYS_ID_PUBLISH"]))) > ((((((-1.0) / 2.0)) + (np.tanh((data["cnt_FLAG_EMAIL"]))))/2.0)))*1.))))) +
                    0.083314*np.tanh(np.where((-1.0*(((((data["FLAG_EMP_PHONE"]) + (data["DAYS_EMPLOYED"]))/2.0))))>0, -1.0, 1.0 )) +
                    0.084252*np.tanh(((data["DAYS_BIRTH"]) * (np.maximum(((((-3.0) * (((((6.0)) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0))))), ((((1.0) - (np.maximum(((data["DAYS_BIRTH"])), ((data["AMT_CREDIT_y"]))))))))))) +
                    0.099766*np.tanh(((data["CNT_FAM_MEMBERS"]) * (((((-1.0) + (data["DAYS_BIRTH"]))) / 2.0)))) +
                    0.089293*np.tanh((((((0.0) + (np.where(data["NAME_EDUCATION_TYPE"]>0, data["AMT_CREDIT_x"], ((data["FLAG_WORK_PHONE"]) / 2.0) )))) + (((data["AMT_CREDIT_x"]) * ((-1.0*((data["FLAG_WORK_PHONE"])))))))/2.0)) +
                    0.081907*np.tanh(np.where(np.maximum(((data["te_FLAG_OWN_REALTY"])), ((data["NAME_GOODS_CATEGORY_Insurance"])))>0, ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * 2.0), (((0.0) > (data["cnt_NAME_CONTRACT_TYPE"]))*1.) )) +
                    0.099922*np.tanh(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + ((((data["MONTHS_BALANCE"]) > (np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((np.maximum(((0.0)), ((((2.0) + (data["WALLSMATERIAL_MODE"]))))))))))*1.)))) * 2.0)) * 2.0)) +
                    0.100000*np.tanh(((((data["inst_AMT_PAYMENT"]) - (-1.0))) * (((data["MONTHS_BALANCE"]) * ((-1.0*(((((data["inst_DAYS_ENTRY_PAYMENT"]) < (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NAME_CASH_LOAN_PURPOSE_Furniture"], data["inst_DAYS_INSTALMENT"] )))*1.))))))))) +
                    0.098632*np.tanh(np.where(((data["CNT_INSTALMENT_FUTURE"]) / 2.0)>0, (((data["NAME_GOODS_CATEGORY_Construction_Materials"]) < (data["NAME_EDUCATION_TYPE"]))*1.), ((data["NAME_EDUCATION_TYPE"]) * (((data["NAME_EDUCATION_TYPE"]) * (data["NAME_GOODS_CATEGORY_Construction_Materials"])))) )) +
                    0.100000*np.tanh((-1.0*((((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["AMT_GOODS_PRICE_x"], ((((((np.tanh((-1.0))) + (data["PRODUCT_COMBINATION_Cash_Street__low"]))/2.0)) > (((((9.59368038177490234)) > (-1.0))*1.)))*1.) )) * 2.0))))) +
                    0.099375*np.tanh(((((np.tanh((-3.0))) - (data["NAME_SELLER_INDUSTRY_Connectivity"]))) * (np.tanh((((data["NONLIVINGAREA_MODE"]) / 2.0)))))) +
                    0.071317*np.tanh(((data["CODE_GENDER"]) * (np.minimum(((np.minimum(((data["NONLIVINGAPARTMENTS_MODE"])), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))), ((np.tanh((np.minimum(((data["NONLIVINGAPARTMENTS_MODE"])), ((((((data["NONLIVINGAPARTMENTS_MODE"]) * (data["WALLSMATERIAL_MODE"]))) * 2.0)))))))))))) +
                    0.071317*np.tanh(((np.where(data["NAME_CASH_LOAN_PURPOSE_Furniture"]>0, ((((1.0) + (data["cnt_HOUR_APPR_PROCESS_START_x"]))) + (data["NAME_GOODS_CATEGORY_Insurance"])), np.minimum(((0.0)), ((((((data["WALLSMATERIAL_MODE"]) * (-1.0))) / 2.0)))) )) / 2.0)) +
                    0.091364*np.tanh(np.minimum(((((data["cnt_FLAG_DOCUMENT_16"]) - ((((((((0.0) * 2.0)) / 2.0)) > ((-1.0*((data["NAME_GOODS_CATEGORY_House_Construction"])))))*1.))))), (((-1.0*(((((data["FLAG_DOCUMENT_16"]) + (data["ty__Another_type_of_loan"]))/2.0)))))))) +
                    0.099805*np.tanh(np.where(((1.0) * ((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) > (data["NONLIVINGAPARTMENTS_MODE"]))*1.)))>0, ((data["cc_bal_SK_ID_PREV"]) / 2.0), np.maximum(((data["FLAG_DOCUMENT_21"])), ((data["NAME_SELLER_INDUSTRY_Auto_technology"]))) )) +
                    0.087339*np.tanh(((data["AMT_CREDIT_x"]) * (np.maximum(((-3.0)), ((((((-1.0*((data["DAYS_ID_PUBLISH"])))) + (((((((-2.0) > (data["DAYS_ID_PUBLISH"]))*1.)) < (data["NONLIVINGAPARTMENTS_MODE"]))*1.)))/2.0))))))) +
                    0.099023*np.tanh(((((np.minimum(((((2.0) - (data["NONLIVINGAPARTMENTS_MODE"])))), ((data["NONLIVINGAPARTMENTS_MODE"])))) * (((data["te_LIVE_CITY_NOT_WORK_CITY"]) - (data["NONLIVINGAPARTMENTS_MODE"]))))) * 2.0)) +
                    0.099844*np.tanh((((((-1.0*(((((data["cc_bal_cc_bal_status__Sent_proposal"]) + (((((2.0) + ((5.66302442550659180)))) * (data["AMT_CREDIT_x"]))))/2.0))))) * (((data["cc_bal_cc_bal_status__Sent_proposal"]) / 2.0)))) - (data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]))) +
                    0.099844*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Medicine"]) - (np.where(data["NAME_CASH_LOAN_PURPOSE_Medicine"]>0, data["Active"], ((((((np.tanh(((5.0)))) > (data["Active"]))*1.)) < (np.tanh((data["inst_AMT_PAYMENT"]))))*1.) )))) +
                    0.099961*np.tanh(((2.0) * (((((np.maximum(((((data["AMT_CREDIT_MAX_OVERDUE"]) + (data["cnt_FLAG_DOCUMENT_2"])))), ((((data["AMT_CREDIT_MAX_OVERDUE"]) * ((9.91402626037597656))))))) * ((9.91402626037597656)))) * 2.0)))) +
                    0.093318*np.tanh(np.where(data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]>0, data["SK_ID_BUREAU"], np.where(((((((1.0) + (2.0))/2.0)) < (data["SK_ID_BUREAU"]))*1.)>0, data["SK_ID_PREV_x"], ((data["SK_ID_BUREAU"]) * (data["SK_ID_PREV_x"])) ) )) +
                    0.076045*np.tanh(np.where(np.minimum(((1.0)), ((((((-1.0*((-3.0)))) > (data["avg_buro_buro_count"]))*1.))))>0, np.minimum(((((data["NAME_YIELD_GROUP_high"]) * (data["avg_buro_buro_count"])))), ((((data["PRODUCT_COMBINATION_Cash"]) / 2.0)))), 1.0 )) +
                    0.099961*np.tanh((((((((data["AMT_CREDIT_SUM"]) * 2.0)) < ((((1.0) + (np.where((-1.0*(((6.0))))>0, data["avg_buro_buro_count"], ((((-1.0) / 2.0)) - (2.0)) )))/2.0)))*1.)) * 2.0)) +
                    0.099961*np.tanh(((((((-1.0*((-2.0)))) < (((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["SK_ID_BUREAU"]))))*1.)) - (np.where(((data["avg_buro_buro_count"]) * (-1.0))>0, data["PRODUCT_COMBINATION_Cash_X_Sell__low"], (0.0) )))) +
                    0.099805*np.tanh(np.minimum((((((((2.0) < (data["CNT_INSTALMENT_FUTURE"]))*1.)) / 2.0))), (((((data["cnt_FLAG_EMAIL"]) + (np.maximum(((3.0)), ((data["CNT_INSTALMENT_FUTURE"])))))/2.0))))) +
                    0.099922*np.tanh(((((((((((np.maximum(((1.0)), (((((-3.0) > ((12.31715393066406250)))*1.))))) < (data["CNT_INSTALMENT"]))*1.)) * 2.0)) < (np.where(data["DAYS_BIRTH"]>0, data["EXT_SOURCE_3"], data["AMT_CREDIT_x"] )))*1.)) / 2.0)) +
                    0.078077*np.tanh(np.minimum(((((data["NAME_GOODS_CATEGORY_Insurance"]) - ((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["te_WALLSMATERIAL_MODE"]))/2.0))))), ((((((-1.0) * ((((data["NAME_CLIENT_TYPE_Repeater"]) < (data["DAYS_BIRTH"]))*1.)))) / 2.0))))) +
                    0.095506*np.tanh((((((((data["SK_ID_PREV_y"]) * (data["inst_AMT_PAYMENT"]))) * 2.0)) + (((data["SK_ID_PREV_y"]) * (np.minimum(((data["EXT_SOURCE_2"])), (((((data["inst_AMT_PAYMENT"]) > (0.0))*1.))))))))/2.0)) +
                    0.100000*np.tanh(((((data["AMT_GOODS_PRICE_x"]) * (np.tanh((np.minimum((((-1.0*((data["CNT_INSTALMENT_FUTURE"]))))), (((((((data["NAME_CLIENT_TYPE_Repeater"]) + ((-1.0*((data["AMT_GOODS_PRICE_x"])))))/2.0)) / 2.0))))))))) / 2.0)) +
                    0.099961*np.tanh(np.tanh((((((((((data["NAME_CLIENT_TYPE_Repeater"]) + (data["PRODUCT_COMBINATION_Cash_X_Sell__high"]))/2.0)) + (np.where((((((data["AMT_CREDIT_x"]) / 2.0)) > (data["AMT_CREDIT_y"]))*1.)>0, data["AMT_CREDIT_x"], data["NAME_GOODS_CATEGORY_Insurance"] )))/2.0)) / 2.0)))) +
                    0.099922*np.tanh(((((-1.0*(((((((data["DAYS_LAST_DUE_1ST_VERSION"]) > (((((2.0)) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)))*1.)) * (data["inst_NUM_INSTALMENT_NUMBER"])))))) + ((((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) < (data["inst_NUM_INSTALMENT_NUMBER"]))*1.)) * (data["NAME_CONTRACT_STATUS_Approved"]))))/2.0)) +
                    0.069949*np.tanh(np.where(((data["AMT_GOODS_PRICE_x"]) * 2.0)>0, data["Completed"], (((((np.tanh((data["SK_ID_PREV_x"]))) + (np.tanh((1.0))))/2.0)) / 2.0) )) +
                    0.099883*np.tanh(((3.0) * (np.minimum(((np.minimum(((((data["AMT_CREDIT_x"]) + (1.0)))), ((((data["NAME_GOODS_CATEGORY_Insurance"]) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))))), (((((data["AMT_CREDIT_x"]) + (1.0))/2.0))))))) +
                    0.099687*np.tanh(((data["CNT_INSTALMENT_FUTURE"]) * ((((((((data["AMT_GOODS_PRICE_x"]) * ((-1.0*((data["SK_DPD"])))))) < (data["SK_DPD"]))*1.)) * 2.0)))) +
                    0.099375*np.tanh(((((((-1.0*((np.tanh((np.minimum(((data["AMT_GOODS_PRICE_x"])), ((((data["FLAG_DOCUMENT_2"]) * (data["SK_DPD"]))))))))))) + (((((data["SK_DPD"]) - (((data["AMT_GOODS_PRICE_x"]) / 2.0)))) / 2.0)))/2.0)) / 2.0)) +
                    0.099922*np.tanh(np.where(data["SK_DPD"]>0, (-1.0*((data["te_OCCUPATION_TYPE"]))), ((((np.where(data["AMT_CREDIT_x"]>0, (-1.0*((data["te_OCCUPATION_TYPE"]))), ((((-2.0) / 2.0)) / 2.0) )) / 2.0)) / 2.0) )) +
                    0.099414*np.tanh((((((data["AMT_CREDIT_x"]) > (np.maximum((((-1.0*((data["cnt_NAME_EDUCATION_TYPE"]))))), ((((((((-1.0*((np.minimum(((data["AMT_CREDIT_x"])), ((np.tanh((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))))))) / 2.0)) > (data["SK_DPD"]))*1.))))))*1.)) / 2.0)) +
                    0.087612*np.tanh(np.minimum((((((data["cnt_FLAG_DOCUMENT_17"]) + (data["cnt_FLAG_DOCUMENT_15"]))/2.0))), ((np.tanh((((np.maximum(((data["cnt_FLAG_DOCUMENT_17"])), (((-1.0*((data["AMT_GOODS_PRICE_x"]))))))) - (np.where(-3.0>0, data["cnt_FLAG_DOCUMENT_17"], data["cnt_NAME_HOUSING_TYPE"] ))))))))) +
                    0.099609*np.tanh(((data["NAME_SELLER_INDUSTRY_MLM_partners"]) * (((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], (-1.0*((data["NAME_GOODS_CATEGORY_Direct_Sales"]))) )) - (np.maximum(((np.maximum(((3.0)), ((data["SK_DPD"]))))), ((0.0)))))))) +
                    0.099805*np.tanh(np.minimum(((((np.minimum(((data["FLAG_DOCUMENT_21"])), ((((data["AMT_CREDIT_x"]) / 2.0))))) * (data["AMT_REQ_CREDIT_BUREAU_MON"])))), ((np.tanh((((np.minimum(((((data["AMT_CREDIT_x"]) / 2.0))), ((0.0)))) / 2.0))))))) +
                    0.099101*np.tanh(((((((((((data["AMT_GOODS_PRICE_x"]) < (((np.tanh(((10.0)))) / 2.0)))*1.)) > (((((((data["inst_AMT_PAYMENT"]) + (0.0))/2.0)) > ((-1.0*((0.0)))))*1.)))*1.)) / 2.0)) / 2.0)) +
                    0.099570*np.tanh((((((((3.0) < ((((((data["NAME_YIELD_GROUP_high"]) + ((((-2.0) > (np.tanh((data["FLAG_DOCUMENT_21"]))))*1.)))/2.0)) + (data["AMT_CREDIT_y"]))))*1.)) - (data["cnt_FLAG_DOCUMENT_21"]))) * 2.0)) +
                    0.099766*np.tanh(((((((data["FLAG_DOCUMENT_2"]) + ((-1.0*(((((data["AMT_GOODS_PRICE_x"]) > (np.tanh((np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((((data["AMT_GOODS_PRICE_x"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"])))))))))*1.))))))) / 2.0)) / 2.0)) +
                    0.088081*np.tanh(((((((data["AMT_CREDIT_x"]) + (data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]))/2.0)) + ((((-3.0) > (((data["AMT_CREDIT_x"]) + (((data["AMT_CREDIT_x"]) * 2.0)))))*1.)))/2.0)) +
                    0.098515*np.tanh(((data["AMT_GOODS_PRICE_x"]) * (np.where(((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))>0, -1.0, ((data["AMT_GOODS_PRICE_x"]) * (np.tanh((data["CODE_REJECT_REASON_VERIF"])))) )))) +
                    0.098163*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Furniture"]>0, ((data["OCCUPATION_TYPE"]) / 2.0), ((((((data["te_WEEKDAY_APPR_PROCESS_START"]) / 2.0)) / 2.0)) / 2.0) )) +
                    0.099961*np.tanh((-1.0*((((3.0) * ((((data["CODE_REJECT_REASON_VERIF"]) > ((((9.0)) + ((-1.0*((np.maximum(((np.minimum(((data["te_REG_REGION_NOT_LIVE_REGION"])), ((data["te_REG_REGION_NOT_LIVE_REGION"]))))), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))))))))*1.))))))) +
                    0.099961*np.tanh((((data["cnt_FLAG_DOCUMENT_17"]) > (((data["CODE_REJECT_REASON_VERIF"]) - (np.where(data["SK_DPD"]>0, -3.0, ((data["SK_DPD"]) * 2.0) )))))*1.)) +
                    0.099375*np.tanh(((((data["NAME_EDUCATION_TYPE"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) + (((((np.minimum(((data["cnt_FLAG_DOCUMENT_17"])), (((((((10.18762969970703125)) - (data["CODE_REJECT_REASON_VERIF"]))) + ((3.18121862411499023))))))) / 2.0)) / 2.0)))) +
                    0.078351*np.tanh((((((-1.0*((data["NAME_YIELD_GROUP_high"])))) * (((((-1.0*((-1.0)))) < (((data["cnt_HOUR_APPR_PROCESS_START_x"]) * (2.0))))*1.)))) / 2.0)) +
                    0.099570*np.tanh((((-1.0*((((np.minimum(((((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (data["PRODUCT_COMBINATION_Cash_Street__high"]))) * 2.0))), ((2.0)))) + ((((3.0) < (data["PRODUCT_COMBINATION_Cash_X_Sell__low"]))*1.))))))) * 2.0)) +
                    0.098437*np.tanh(((data["SK_DPD"]) * (((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]) + (((((data["PRODUCT_COMBINATION_Card_Street"]) + (((data["PRODUCT_COMBINATION_Card_Street"]) - (((data["NAME_SELLER_INDUSTRY_Connectivity"]) * 2.0)))))) - (data["NAME_GOODS_CATEGORY_Insurance"]))))))) +
                    0.099726*np.tanh(((data["AMT_CREDIT_x"]) * (((((((data["ty__Mortgage"]) > ((((data["NAME_GOODS_CATEGORY_Insurance"]) + (np.where(data["ty__Mortgage"]>0, 3.0, 1.0 )))/2.0)))*1.)) < (((data["ty__Mortgage"]) / 2.0)))*1.)))) +
                    0.099961*np.tanh((-1.0*((np.where(data["ty__Mortgage"]>0, data["cc_bal_CNT_DRAWINGS_CURRENT"], (((((3.0) < ((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) + (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))/2.0)))*1.)) * ((-1.0*((data["cc_bal_CNT_DRAWINGS_CURRENT"]))))) ))))) +
                    0.099766*np.tanh(np.where(0.0>0, -3.0, np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, (((data["DEF_30_CNT_SOCIAL_CIRCLE"]) > (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.), np.where(data["cu__currency_3"]>0, 2.0, (((data["DEF_30_CNT_SOCIAL_CIRCLE"]) > (2.0))*1.) ) ) )) +
                    0.099609*np.tanh(np.minimum((((-1.0*((np.where(((data["NAME_YIELD_GROUP_high"]) / 2.0)>0, data["ty__Mortgage"], data["NAME_GOODS_CATEGORY_Weapon"] )))))), ((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["cnt_HOUR_APPR_PROCESS_START_x"], data["NAME_GOODS_CATEGORY_Direct_Sales"] ))))) +
                    0.093591*np.tanh((-1.0*((((((data["CHANNEL_TYPE_Car_dealer"]) + (((np.maximum(((data["CHANNEL_TYPE_Car_dealer"])), ((np.tanh((-2.0)))))) + (data["NAME_GOODS_CATEGORY_Fitness"]))))) + (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NONLIVINGAPARTMENTS_MODE"], data["CHANNEL_TYPE_Car_dealer"] ))))))) +
                    0.099297*np.tanh(np.where(np.where(((-2.0) + (data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]))>0, data["WEEKDAY_APPR_PROCESS_START_MONDAY"], data["NONLIVINGAPARTMENTS_MODE"] )>0, data["NAME_GOODS_CATEGORY_Computers"], np.minimum(((((data["NONLIVINGAPARTMENTS_MODE"]) / 2.0))), ((((data["NAME_GOODS_CATEGORY_Computers"]) * (data["NAME_GOODS_CATEGORY_Gardening"]))))) )) +
                    0.066862*np.tanh((((-1.0*((np.where(data["NAME_YIELD_GROUP_high"]>0, data["NONLIVINGAPARTMENTS_MODE"], (-1.0*((np.where((12.60592746734619141)>0, data["NONLIVINGAPARTMENTS_MODE"], ((data["NONLIVINGAPARTMENTS_MODE"]) * 2.0) )))) ))))) * 2.0)) +
                    0.099687*np.tanh((-1.0*((((np.tanh(((((((data["FLAG_DOCUMENT_17"]) < (data["WEEKDAY_APPR_PROCESS_START_MONDAY"]))*1.)) - (data["NAME_GOODS_CATEGORY_Weapon"]))))) * (np.where(data["WEEKDAY_APPR_PROCESS_START_MONDAY"]>0, ((data["AMT_REQ_CREDIT_BUREAU_WEEK"]) + (data["AMT_REQ_CREDIT_BUREAU_WEEK"])), data["FLAG_DOCUMENT_17"] ))))))) +
                    0.099531*np.tanh(np.where(((((10.0)) < (data["NONLIVINGAPARTMENTS_MODE"]))*1.)>0, 3.0, np.where(data["NONLIVINGAPARTMENTS_MODE"]>0, ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * 2.0), data["AMT_REQ_CREDIT_BUREAU_DAY"] ) )) +
                    0.078077*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["avg_buro_buro_count"], np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((((data["NAME_YIELD_GROUP_high"]) * (np.tanh((((data["avg_buro_buro_count"]) * 2.0)))))))) )) +
                    0.082884*np.tanh(((((-1.0*((((-1.0) - (data["avg_buro_buro_count"])))))) < (np.minimum(((data["te_HOUR_APPR_PROCESS_START_x"])), ((((data["DAYS_ENDDATE_FACT"]) * 2.0))))))*1.)) +
                    0.099766*np.tanh(np.minimum((((((3.0) < (data["AMT_CREDIT_x"]))*1.))), ((((((((3.0) < (data["AMT_CREDIT_x"]))*1.)) + (np.maximum(((data["AMT_CREDIT_x"])), ((data["cnt_HOUR_APPR_PROCESS_START_x"])))))/2.0))))) +
                    0.081125*np.tanh((-1.0*(((((((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["NAME_GOODS_CATEGORY_Insurance"], data["NAME_GOODS_CATEGORY_Direct_Sales"] )) > (((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (data["AMT_GOODS_PRICE_x"]))) * 2.0)))*1.)) / 2.0))))) +
                    0.099492*np.tanh((((((data["FLAG_CONT_MOBILE"]) + (np.tanh((np.tanh((np.tanh((np.tanh((data["CODE_GENDER"]))))))))))/2.0)) * (data["AMT_CREDIT_x"]))) +
                    0.094373*np.tanh(((((((((((((np.minimum(((data["cc_bal_SK_DPD_DEF"])), ((0.0)))) * 2.0)) < (0.0))*1.)) < ((-1.0*((data["AMT_GOODS_PRICE_x"])))))*1.)) / 2.0)) / 2.0)) +
                    0.099883*np.tanh(((data["NAME_YIELD_GROUP_high"]) * ((((data["cc_bal_SK_DPD_DEF"]) < (((((-1.0) + (data["inst_NUM_INSTALMENT_VERSION"]))) + (((((0.0)) < (((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)))*1.)))))*1.)))) +
                    0.099609*np.tanh((((np.tanh(((((((data["cc_bal_SK_DPD_DEF"]) - (data["AMT_DOWN_PAYMENT"]))) < (np.where((((5.78417444229125977)) + ((-1.0*((data["cc_bal_SK_DPD_DEF"])))))>0, data["NAME_GOODS_CATEGORY_Insurance"], data["NAME_GOODS_CATEGORY_Insurance"] )))*1.)))) < (data["AMT_DOWN_PAYMENT"]))*1.)) +
                    0.098476*np.tanh((((np.tanh((((data["AMT_CREDIT_x"]) / 2.0)))) + (np.tanh((((data["RATE_DOWN_PAYMENT"]) * ((((-1.0*(((((data["CNT_INSTALMENT_FUTURE"]) < (((data["AMT_CREDIT_x"]) * 2.0)))*1.))))) / 2.0)))))))/2.0)) +
                    0.099961*np.tanh(np.minimum(((((((((((data["AMT_ANNUITY_x"]) > (data["AMT_GOODS_PRICE_x"]))*1.)) / 2.0)) + (np.minimum(((data["NAME_PORTFOLIO_Cards"])), ((data["NAME_GOODS_CATEGORY_Insurance"])))))/2.0))), (((6.0))))) +
                    0.099922*np.tanh((((((((np.minimum(((data["AMT_GOODS_PRICE_x"])), (((((8.0)) + (data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"])))))) + ((-1.0*((3.0)))))/2.0)) * (((((-1.0*((data["AMT_GOODS_PRICE_x"])))) < ((0.21274571120738983)))*1.)))) / 2.0)) +
                    0.100000*np.tanh(((((((np.maximum((((((((((((data["SK_DPD"]) + (data["SK_DPD"]))/2.0)) < (data["cc_bal_SK_ID_PREV"]))*1.)) + ((((data["CNT_INSTALMENT_FUTURE"]) + (data["SK_DPD"]))/2.0)))/2.0))), ((data["NAME_GOODS_CATEGORY_Insurance"])))) / 2.0)) * 2.0)) / 2.0)) +
                    0.072216*np.tanh(((((-1.0) / 2.0)) * (np.where(data["CODE_REJECT_REASON_HC"]>0, data["NFLAG_INSURED_ON_APPROVAL"], np.minimum(((np.minimum(((data["MONTHS_BALANCE"])), ((2.0))))), ((0.0))) )))) +
                    0.099531*np.tanh(((((((2.39028859138488770)) > (np.maximum(((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])), (((-1.0*((((data["AMT_CREDIT_x"]) * 2.0)))))))))*1.)) + (-1.0))) +
                    0.097890*np.tanh(np.where(np.tanh((data["NAME_YIELD_GROUP_high"]))>0, np.maximum((((((np.tanh(((((data["CHANNEL_TYPE_Car_dealer"]) + (data["DAYS_DECISION"]))/2.0)))) + (data["CHANNEL_TYPE_Car_dealer"]))/2.0))), ((data["SK_ID_PREV_x"]))), (-1.0*((data["CHANNEL_TYPE_Car_dealer"]))) )) +
                    0.099726*np.tanh(np.where(data["SK_ID_PREV_y"]>0, ((np.minimum(((np.minimum(((data["NAME_YIELD_GROUP_high"])), ((((data["NAME_YIELD_GROUP_high"]) + (3.0))))))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) * (data["NAME_CONTRACT_STATUS_Refused"])), data["NAME_GOODS_CATEGORY_Direct_Sales"] )) +
                    0.079562*np.tanh(np.minimum(((1.0)), (((-1.0*((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((data["DAYS_BIRTH"]) * (np.tanh((data["AMT_REQ_CREDIT_BUREAU_YEAR"])))))))))))))) +
                    0.099726*np.tanh((((((np.minimum(((data["CNT_INSTALMENT"])), ((((((-1.0) * 2.0)) * (((data["CNT_INSTALMENT"]) / 2.0))))))) > (data["SK_ID_PREV_y"]))*1.)) / 2.0)) +
                    0.100000*np.tanh((-1.0*(((((data["DAYS_BIRTH"]) > (((3.0) - ((((((data["DAYS_BIRTH"]) + (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (-3.0))))/2.0)) * 2.0)))))*1.))))) +
                    0.099805*np.tanh(((((((((((data["cnt_DAYS_BIRTH"]) / 2.0)) + ((3.49354934692382812)))/2.0)) < (data["DAYS_ID_PUBLISH"]))*1.)) * (data["DAYS_ID_PUBLISH"]))) +
                    0.099531*np.tanh((((np.maximum(((((data["NAME_INCOME_TYPE"]) + (-2.0)))), (((((data["cnt_DAYS_BIRTH"]) > ((((data["NAME_INCOME_TYPE"]) < (0.0))*1.)))*1.))))) < (data["NAME_GOODS_CATEGORY_House_Construction"]))*1.)) +
                    0.099961*np.tanh(((np.minimum((((7.0))), ((((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((np.tanh((((np.tanh((((data["cnt_FLAG_EMP_PHONE"]) + (data["DAYS_EMPLOYED"]))))) * ((9.89645957946777344))))))))) * 2.0))))) * 2.0)) +
                    0.090543*np.tanh(((np.tanh(((((data["SK_ID_BUREAU"]) < (-1.0))*1.)))) * ((((data["NAME_YIELD_GROUP_high"]) > (((data["SK_ID_BUREAU"]) - (np.maximum(((data["te_FLAG_DOCUMENT_20"])), ((((-3.0) * 2.0))))))))*1.)))) +
                    0.100000*np.tanh(((np.minimum(((((data["NAME_CLIENT_TYPE_New"]) * (((data["DAYS_LAST_PHONE_CHANGE"]) * (data["SK_ID_BUREAU"])))))), ((((((((3.0) < (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.)) > (data["DAYS_LAST_PHONE_CHANGE"]))*1.))))) / 2.0)) +
                    0.099140*np.tanh((-1.0*((((np.maximum((((((-2.0) > (data["DAYS_BIRTH"]))*1.))), ((((np.where(((3.0) - (data["te_LIVE_REGION_NOT_WORK_REGION"]))>0, -3.0, data["DAYS_BIRTH"] )) * 2.0))))) * 2.0))))) +
                    0.099297*np.tanh((-1.0*((np.where(np.minimum(((1.0)), ((((data["SK_ID_BUREAU"]) + (-2.0)))))>0, -1.0, ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"]) * 2.0) ))))) +
                    0.098320*np.tanh(np.minimum((((-1.0*((np.where(data["te_FLAG_DOCUMENT_20"]>0, data["NAME_GOODS_CATEGORY_Weapon"], data["NAME_GOODS_CATEGORY_Direct_Sales"] )))))), ((((((((data["NAME_GOODS_CATEGORY_Weapon"]) + (((data["NAME_CONTRACT_STATUS_Approved"]) * (data["NFLAG_LAST_APPL_IN_DAY"]))))) * 2.0)) / 2.0))))) +
                    0.068894*np.tanh(((((data["te_FLAG_DOCUMENT_20"]) * (np.where(data["te_FLAG_DOCUMENT_20"]>0, data["te_FLAG_DOCUMENT_20"], (6.34485292434692383) )))) * (np.where(np.tanh(((4.0)))>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], data["NAME_GOODS_CATEGORY_Insurance"] )))) +
                    0.099922*np.tanh(((data["SK_ID_PREV_x"]) * (((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) - ((-1.0*(((((((data["SK_ID_PREV_x"]) * 2.0)) > ((14.96901893615722656)))*1.))))))) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))) +
                    0.089293*np.tanh(((data["cnt_FLAG_DOCUMENT_10"]) + (((((np.where(data["cnt_FLAG_DOCUMENT_17"]>0, data["NAME_YIELD_GROUP_high"], data["NAME_GOODS_CATEGORY_Insurance"] )) * (data["FLAG_DOCUMENT_17"]))) * ((8.24418640136718750)))))) +
                    0.099609*np.tanh(np.where(((-1.0) + (data["CNT_INSTALMENT"]))>0, data["REG_REGION_NOT_WORK_REGION"], (-1.0*((data["NAME_GOODS_CATEGORY_Medical_Supplies"]))) )) +
                    0.072255*np.tanh(np.where(((data["AMT_ANNUITY_x"]) * (data["FLAG_DOCUMENT_15"]))>0, np.minimum((((0.83881276845932007))), ((((-1.0) - (data["AMT_ANNUITY_x"]))))), ((data["FLAG_DOCUMENT_15"]) * (data["AMT_ANNUITY_x"])) )) +
                    0.095819*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["te_FLAG_DOCUMENT_3"], (((-1.0*((((0.0) - ((((data["AMT_GOODS_PRICE_x"]) < (((data["AMT_ANNUITY_x"]) - (((((9.0)) < (3.0))*1.)))))*1.))))))) / 2.0) )) +
                    0.100000*np.tanh(((((-2.0) / 2.0)) + ((((data["AMT_CREDIT_x"]) > (((np.where(((3.0) * (data["AMT_ANNUITY_x"]))>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], data["AMT_ANNUITY_x"] )) * 2.0)))*1.)))) +
                    0.100000*np.tanh((-1.0*((((((((((2.0) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)) - (np.minimum(((-2.0)), (((12.27902317047119141))))))) < (((data["AMT_GOODS_PRICE_x"]) + (data["AMT_ANNUITY_x"]))))*1.))))) +
                    0.099922*np.tanh((((((-3.0) > (((np.where((((((((data["AMT_CREDIT_x"]) / 2.0)) + (data["CNT_PAYMENT"]))/2.0)) / 2.0)>0, data["DAYS_BIRTH"], ((data["CNT_PAYMENT"]) / 2.0) )) * 2.0)))*1.)) * 2.0)) +
                    0.097265*np.tanh(np.minimum(((((((data["AMT_CREDIT_x"]) / 2.0)) / 2.0))), (((((((-1.0) * 2.0)) > (((data["AMT_CREDIT_x"]) - ((3.90542602539062500)))))*1.))))) +
                    0.084017*np.tanh(((((np.tanh(((-1.0*((np.minimum(((data["AMT_GOODS_PRICE_x"])), ((((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0)))))))))) / 2.0)) + ((((data["CHANNEL_TYPE_AP___Cash_loan_"]) > ((((4.0)) + (np.tanh((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))))*1.)))) +
                    0.076671*np.tanh((((-1.0*(((((((np.minimum(((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * 2.0))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) * ((6.0)))) > (data["HOUR_APPR_PROCESS_START_x"]))*1.))))) / 2.0)) +
                    0.092497*np.tanh((((np.where(data["FLAG_DOCUMENT_2"]>0, (((-1.0) < (-2.0))*1.), ((((-1.0*((data["cnt_FLAG_DOCUMENT_2"])))) + (3.0))/2.0) )) < ((((data["REGION_RATING_CLIENT_W_CITY"]) + (data["te_HOUR_APPR_PROCESS_START_x"]))/2.0)))*1.)) +
                    0.099453*np.tanh(((3.0) * (np.tanh((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((np.minimum(((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > ((((data["cnt_REGION_RATING_CLIENT"]) < (data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]))*1.)))*1.)) > (data["cnt_FLAG_DOCUMENT_2"]))*1.))), ((data["cnt_HOUR_APPR_PROCESS_START_x"]))))))))))) +
                    0.099883*np.tanh(((data["te_FLAG_DOCUMENT_18"]) * (((data["cnt_REGION_RATING_CLIENT_W_CITY"]) + (data["AMT_CREDIT_x"]))))) +
                    0.099922*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, (-1.0*((((((((-1.0*((-1.0)))) * (((((((3.0)) + ((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"])))))/2.0)) / 2.0)))) + (data["cnt_FLAG_DOCUMENT_18"]))/2.0)))), data["cnt_FLAG_DOCUMENT_18"] )) +
                    0.099883*np.tanh(((data["cnt_FLAG_DOCUMENT_18"]) - (np.tanh(((((0.0) > (np.maximum(((((((((data["REGION_POPULATION_RELATIVE"]) + (data["te_FLAG_DOCUMENT_18"]))) + (data["te_FLAG_DOCUMENT_18"]))) - (data["cc_bal_SK_DPD_DEF"])))), ((data["cc_bal_SK_DPD_DEF"])))))*1.)))))) +
                    0.087182*np.tanh(((data["inst_NUM_INSTALMENT_VERSION"]) * (((data["SK_ID_PREV_x"]) * ((-1.0*(((((((((((((data["NAME_CLIENT_TYPE_Repeater"]) < (((-1.0) / 2.0)))*1.)) * 2.0)) > (data["AMT_GOODS_PRICE_x"]))*1.)) > (data["NAME_PORTFOLIO_Cards"]))*1.))))))))) +
                    0.099922*np.tanh((((-1.0) + ((((((data["AMT_GOODS_PRICE_x"]) * (data["NAME_CLIENT_TYPE_New"]))) < ((((2.0) + (((data["SK_ID_PREV_x"]) / 2.0)))/2.0)))*1.)))/2.0)) +
                    0.099101*np.tanh(np.where(data["HOUR_APPR_PROCESS_START_y"]>0, (-1.0*((np.minimum(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])), ((((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * 2.0))))))), data["NAME_GOODS_CATEGORY_Insurance"] )) +
                    0.099609*np.tanh(((((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]) * (data["cc_bal_CNT_DRAWINGS_CURRENT"]))) * ((-1.0*((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, ((data["cc_bal_CNT_DRAWINGS_CURRENT"]) * (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, -2.0, data["inst_NUM_INSTALMENT_VERSION"] ))), data["inst_NUM_INSTALMENT_VERSION"] ))))))) +
                    0.099766*np.tanh(((data["AMT_ANNUITY_x"]) * ((((np.minimum((((((((1.0) / 2.0)) < (((data["CNT_INSTALMENT_FUTURE"]) / 2.0)))*1.))), ((data["AMT_ANNUITY_x"])))) > (data["avg_buro_buro_bal_status_C"]))*1.)))) +
                    0.099961*np.tanh((-1.0*(((((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) + ((((data["AMT_GOODS_PRICE_x"]) > (((((((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["NAME_GOODS_CATEGORY_Insurance"])))) > ((2.34558510780334473)))*1.)) + (data["AMT_ANNUITY_x"]))/2.0)))*1.)))/2.0))))) +
                    0.085619*np.tanh(np.tanh((((((np.tanh(((((((data["avg_buro_buro_bal_status_C"]) + ((((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) + (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]))/2.0)))) > (np.tanh((3.0))))*1.)))) - (data["AMT_ANNUITY_x"]))) + (data["AMT_CREDIT_x"]))))) +
                    0.084447*np.tanh(np.minimum(((((((data["NAME_GOODS_CATEGORY_Insurance"]) - ((((data["AMT_GOODS_PRICE_x"]) > (1.0))*1.)))) / 2.0))), (((-1.0*((((((-1.0*((-2.0)))) < (data["AMT_GOODS_PRICE_x"]))*1.)))))))) +
                    0.099609*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, np.where(data["AMT_CREDIT_x"]>0, (2.79278469085693359), data["te_CODE_GENDER"] ), (((-2.0) > (((data["te_FLAG_DOCUMENT_8"]) / 2.0)))*1.) )) +
                    0.099883*np.tanh((((((((((((3.0) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)) > (data["AMT_CREDIT_x"]))*1.)) < (((-3.0) * (data["cc_bal_SK_DPD_DEF"]))))*1.)) - (data["CODE_REJECT_REASON_SYSTEM"]))) +
                    0.099687*np.tanh(np.where(data["cc_bal_SK_DPD_DEF"]>0, data["HOUR_APPR_PROCESS_START_y"], (((-1.0*(((((((np.maximum(((data["HOUR_APPR_PROCESS_START_y"])), ((data["CNT_INSTALMENT"])))) + (data["NAME_GOODS_CATEGORY_Weapon"]))/2.0)) / 2.0))))) / 2.0) )) +
                    0.098750*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"]) * (data["FLAG_DOCUMENT_17"]))) + (np.maximum(((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"])), (((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"])))))))), (((((((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) / 2.0)) > ((6.61112833023071289)))*1.))))))) +
                    0.099687*np.tanh(np.minimum((((((np.maximum(((data["CNT_INSTALMENT_FUTURE"])), ((data["cnt_FLAG_DOCUMENT_13"])))) > (3.0))*1.))), ((((data["AMT_CREDIT_x"]) - ((((((((data["cnt_FLAG_DOCUMENT_13"]) * 2.0)) > (data["CNT_INSTALMENT_FUTURE"]))*1.)) + (-2.0)))))))) +
                    0.099844*np.tanh((-1.0*((np.where(data["NAME_GOODS_CATEGORY_Fitness"]>0, data["EXT_SOURCE_3"], (((np.minimum(((data["NAME_GOODS_CATEGORY_Fitness"])), ((np.tanh(((-1.0*((data["EXT_SOURCE_3"]))))))))) + (((data["cnt_FLAG_DOCUMENT_10"]) / 2.0)))/2.0) ))))) +
                    0.078429*np.tanh(np.minimum(((((2.0) - (data["CNT_INSTALMENT"])))), ((np.tanh((np.where(data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]>0, data["NAME_CASH_LOAN_PURPOSE_Furniture"], np.where(data["cc_bal_SK_DPD_DEF"]>0, -3.0, ((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) * ((7.0))) ) ))))))) +
                    0.098554*np.tanh(np.where(data["avg_buro_buro_count"]>0, np.where(data["cc_bal_SK_DPD_DEF"]>0, data["avg_buro_buro_count"], data["cnt_FLAG_DOCUMENT_13"] ), np.minimum(((data["cc_bal_SK_DPD_DEF"])), ((data["cnt_FLAG_DOCUMENT_13"]))) )) +
                    0.093669*np.tanh(np.where(data["NAME_GOODS_CATEGORY_House_Construction"]>0, data["avg_buro_buro_count"], (((((-2.0) / 2.0)) + ((((data["NAME_GOODS_CATEGORY_House_Construction"]) < (((((data["avg_buro_buro_count"]) - (data["NAME_GOODS_CATEGORY_Fitness"]))) * (data["NAME_GOODS_CATEGORY_Fitness"]))))*1.)))/2.0) )) +
                    0.098554*np.tanh(((((((((((data["cnt_FLAG_DOCUMENT_14"]) / 2.0)) + ((-1.0*((data["nans"])))))/2.0)) / 2.0)) + (((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) * (data["nans"]))))/2.0)) +
                    0.079601*np.tanh(np.tanh(((((((((((data["CNT_INSTALMENT_FUTURE"]) * (data["CNT_INSTALMENT_FUTURE"]))) / 2.0)) > ((((data["REG_REGION_NOT_WORK_REGION"]) < (data["cc_bal_SK_DPD_DEF"]))*1.)))*1.)) / 2.0)))) +
                    0.100000*np.tanh(((data["AMT_CREDIT_MAX_OVERDUE"]) * ((((12.82818222045898438)) + ((((((12.73882579803466797)) + ((((12.73882579803466797)) * ((13.13270282745361328)))))) * (((data["cc_bal_SK_DPD_DEF"]) * 2.0)))))))) +
                    0.099179*np.tanh(((data["FLAG_LAST_APPL_PER_CONTRACT_N"]) * ((((((data["te_REG_REGION_NOT_LIVE_REGION"]) + (data["FONDKAPREMONT_MODE"]))/2.0)) + ((((data["FONDKAPREMONT_MODE"]) > ((((-1.0*((-3.0)))) / 2.0)))*1.)))))) +
                    0.073896*np.tanh((-1.0*((np.tanh((((data["inst_AMT_PAYMENT"]) * (((data["inst_AMT_PAYMENT"]) * (np.maximum(((-1.0)), ((data["CNT_INSTALMENT"]))))))))))))) +
                    0.096444*np.tanh(((np.where(((np.tanh((data["AMT_CREDIT_y"]))) + (((((1.0)) + (((data["cnt_FLAG_DOCUMENT_10"]) * 2.0)))/2.0)))>0, (((data["CNT_INSTALMENT_FUTURE"]) + (-1.0))/2.0), data["NAME_GOODS_CATEGORY_Insurance"] )) * (data["AMT_CREDIT_y"]))) +
                    0.099531*np.tanh((((data["cnt_FLAG_DOCUMENT_10"]) + (np.minimum(((np.maximum(((data["AMT_CREDIT_y"])), ((data["CNT_PAYMENT"]))))), (((((data["cnt_FLAG_DOCUMENT_10"]) > (((((2.70514798164367676)) > (data["AMT_CREDIT_y"]))*1.)))*1.))))))/2.0)) +
                    0.099805*np.tanh((((((((np.tanh((data["cnt_FLAG_DOCUMENT_3"]))) + (np.minimum((((((data["FLAG_DOCUMENT_3"]) < ((((data["cc_bal_SK_DPD_DEF"]) + (data["cnt_FLAG_DOCUMENT_3"]))/2.0)))*1.))), (((-1.0*((data["CNT_INSTALMENT"]))))))))/2.0)) / 2.0)) / 2.0)) +
                    0.072450*np.tanh(np.maximum(((data["cnt_FLAG_DOCUMENT_5"])), ((((data["cc_bal_SK_DPD_DEF"]) + ((((((((data["cnt_FLAG_DOCUMENT_5"]) / 2.0)) / 2.0)) < (data["cc_bal_SK_DPD_DEF"]))*1.))))))) +
                    0.098984*np.tanh((((((data["cc_bal_cc_bal_status__Refused"]) * 2.0)) + (((((data["NAME_SELLER_INDUSTRY_Construction"]) * ((4.0)))) * ((-1.0*(((((((data["NAME_SELLER_INDUSTRY_Construction"]) / 2.0)) > ((((7.86149215698242188)) / 2.0)))*1.))))))))/2.0)) +
                    0.099453*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]) * (((((data["cc_bal_SK_DPD_DEF"]) * (data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]))) - ((-1.0*((data["NAME_GOODS_CATEGORY_Fitness"])))))))) +
                    0.099336*np.tanh(((((((data["FLAG_DOCUMENT_17"]) * 2.0)) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))) - (((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) - (data["NAME_GOODS_CATEGORY_Weapon"]))))))) +
                    0.100000*np.tanh(np.tanh((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["avg_buro_buro_bal_status_2"], (-1.0*(((((((data["avg_buro_buro_bal_status_2"]) + (data["cc_bal_SK_DPD_DEF"]))) > (data["cnt_FLAG_DOCUMENT_10"]))*1.)))) )))) +
                    0.099258*np.tanh(((((((10.22499179840087891)) < (data["NONLIVINGAPARTMENTS_MODE"]))*1.)) - ((((data["CHANNEL_TYPE_Car_dealer"]) + (((data["YEARS_BEGINEXPLUATATION_MODE"]) * 2.0)))/2.0)))) +
                    0.100000*np.tanh(np.where(data["te_WALLSMATERIAL_MODE"]>0, data["YEARS_BUILD_AVG"], np.where(data["cnt_EMERGENCYSTATE_MODE"]>0, ((0.0) - (data["NAME_GOODS_CATEGORY_Direct_Sales"])), data["YEARS_BUILD_MEDI"] ) )) +
                    0.098046*np.tanh((-1.0*((np.where(data["cc_bal_SK_DPD_DEF"]>0, data["NAME_GOODS_CATEGORY_Computers"], (((np.maximum(((-2.0)), (((1.0))))) < ((-1.0*(((((-1.0*((data["NAME_GOODS_CATEGORY_Computers"])))) + (2.0)))))))*1.) ))))) +
                    0.099961*np.tanh(((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["avg_buro_buro_bal_status_5"], ((((data["DAYS_LAST_DUE_1ST_VERSION"]) * (data["cc_bal_SK_DPD_DEF"]))) * (data["DAYS_LAST_DUE_1ST_VERSION"])) )) * 2.0)) +
                    0.062407*np.tanh((((np.minimum(((((np.tanh((np.minimum(((data["NFLAG_INSURED_ON_APPROVAL"])), (((12.13233852386474609))))))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"])))), ((data["avg_buro_buro_bal_status_5"])))) > (np.where(data["avg_buro_buro_bal_status_5"]>0, data["cnt_FLAG_DOCUMENT_10"], data["cc_bal_SK_DPD_DEF"] )))*1.)) +
                    0.099805*np.tanh(((data["NFLAG_INSURED_ON_APPROVAL"]) * ((((((((-1.0*((data["inst_SK_ID_PREV"])))) * (data["SK_ID_PREV_y"]))) / 2.0)) / 2.0)))) +
                    0.099336*np.tanh(np.minimum((((((((((data["cc_bal_SK_DPD_DEF"]) + (data["SK_ID_PREV_x"]))) - (((((np.tanh((data["te_FLAG_EMAIL"]))) * (2.0))) / 2.0)))) > (2.0))*1.))), (((0.31483063101768494))))) +
                    0.099297*np.tanh((-1.0*((np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((((6.30967998504638672)) + ((((7.0)) - ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + ((((data["FLAG_DOCUMENT_10"]) > ((-1.0*((0.0)))))*1.)))/2.0)))))/2.0)))))))) +
                    0.099805*np.tanh(np.minimum(((((((((-1.0*((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) / 2.0)) < (0.0))*1.))), (((((((((((0.0) > (data["cc_bal_SK_DPD_DEF"]))*1.)) * (data["cnt_EMERGENCYSTATE_MODE"]))) / 2.0)) * 2.0))))) +
                    0.091403*np.tanh(((((((((data["NAME_GOODS_CATEGORY_Insurance"]) < (data["NAME_PORTFOLIO_Cards"]))*1.)) / 2.0)) + ((((((8.0)) / 2.0)) * ((((3.0) < (data["NAME_PORTFOLIO_Cards"]))*1.)))))/2.0)) +
                    0.099922*np.tanh(np.where(data["FLAG_LAST_APPL_PER_CONTRACT_N"]>0, np.where((((data["DAYS_LAST_DUE_1ST_VERSION"]) < (((np.tanh((np.tanh((data["FLAG_LAST_APPL_PER_CONTRACT_N"]))))) / 2.0)))*1.)>0, data["DAYS_LAST_DUE_1ST_VERSION"], -3.0 ), (-1.0*((data["NAME_GOODS_CATEGORY_Education"]))) )) +
                    0.097069*np.tanh(np.minimum(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((((((data["ty__Another_type_of_loan"]) + (data["DAYS_LAST_DUE_1ST_VERSION"]))) - (data["cc_bal_cc_bal_status__Refused"]))) + (data["DAYS_LAST_DUE_1ST_VERSION"]))))) * 2.0))), (((((10.71418285369873047)) - (data["ty__Another_type_of_loan"])))))) +
                    0.077882*np.tanh((((((((-1.0) * 2.0)) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) > (np.maximum(((data["DAYS_FIRST_DRAWING"])), ((((data["CNT_INSTALMENT_FUTURE"]) * (np.minimum(((3.0)), ((data["inst_AMT_PAYMENT"]))))))))))*1.)) +
                    0.099961*np.tanh(np.minimum((((((-2.0) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.))), (((((((((data["NAME_INCOME_TYPE"]) + (((-1.0) + (data["DAYS_BIRTH"]))))) + (2.0))) + (data["AMT_CREDIT_x"]))/2.0))))) +
                    0.092106*np.tanh(((data["DAYS_BIRTH"]) * (np.minimum(((np.minimum(((data["FLAG_DOCUMENT_10"])), (((-1.0*((np.maximum(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])), ((np.minimum(((data["AMT_GOODS_PRICE_x"])), ((data["DAYS_BIRTH"])))))))))))))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))))) +
                    0.099687*np.tanh((((((data["AMT_CREDIT_x"]) - ((((-3.0) + (((data["cnt_REG_CITY_NOT_LIVE_CITY"]) * 2.0)))/2.0)))) > ((4.65822219848632812)))*1.)) +
                    0.064361*np.tanh(((np.where(data["te_REG_CITY_NOT_WORK_CITY"]>0, data["FLAG_DOCUMENT_10"], (((data["te_FLAG_WORK_PHONE"]) > (((-3.0) * ((((data["cnt_FLAG_DOCUMENT_10"]) + (data["OCCUPATION_TYPE"]))/2.0)))))*1.) )) / 2.0)) +
                    0.099531*np.tanh(np.tanh(((-1.0*((np.tanh(((((((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["AMT_REQ_CREDIT_BUREAU_MON"], 2.0 )) < (data["cnt_DAYS_BIRTH"]))*1.)) * (((((data["cnt_DAYS_BIRTH"]) * 2.0)) / 2.0))))))))))) +
                    0.094177*np.tanh(np.where(((((1.0) - (data["AMT_CREDIT_SUM_DEBT"]))) + (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]))>0, ((((((data["AMT_CREDIT_SUM_DEBT"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * 2.0), ((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0) )) +
                    0.099961*np.tanh((((np.minimum(((((((data["ca__Closed"]) + (data["ca__Active"]))) / 2.0))), ((((((data["ca__Active"]) / 2.0)) + (data["AMT_CREDIT_SUM"])))))) < (-1.0))*1.)) +
                    0.099844*np.tanh((((data["AMT_CREDIT_SUM"]) < ((((((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)) + (np.tanh((((np.where(np.tanh((2.0))>0, data["NAME_GOODS_CATEGORY_Insurance"], data["cnt_FLAG_DOCUMENT_10"] )) - (np.tanh((3.0))))))))/2.0)))*1.)) +
                    0.099805*np.tanh(np.where(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["SK_ID_BUREAU"], data["AMT_CREDIT_SUM"] )>0, ((data["SK_ID_BUREAU"]) + ((((data["AMT_CREDIT_SUM"]) < (data["SK_ID_BUREAU"]))*1.))), data["cnt_FLAG_DOCUMENT_10"] )) +
                    0.099375*np.tanh(np.minimum(((np.minimum(((data["CODE_REJECT_REASON_VERIF"])), (((((11.77319049835205078)) - (data["CODE_REJECT_REASON_VERIF"]))))))), ((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * (((data["inst_NUM_INSTALMENT_NUMBER"]) * 2.0))))))) +
                    0.099766*np.tanh(((data["OWN_CAR_AGE"]) * (np.maximum(((data["inst_AMT_PAYMENT"])), ((((np.where((-1.0*((data["inst_AMT_PAYMENT"])))>0, -3.0, (((5.90897226333618164)) * 2.0) )) * (np.tanh((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) / 2.0))))))))))) +
                    0.099766*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["AMT_REQ_CREDIT_BUREAU_MON"], ((data["inst_AMT_PAYMENT"]) * ((((np.tanh((data["AMT_REQ_CREDIT_BUREAU_MON"]))) + (((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["inst_AMT_PAYMENT"]))) / 2.0)))/2.0))) )) +
                    0.099961*np.tanh(((2.0) * (((np.where((-1.0*((data["cc_bal_SK_DPD_DEF"])))>0, data["FLAG_DOCUMENT_10"], data["inst_NUM_INSTALMENT_NUMBER"] )) * (np.maximum(((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])), ((np.where(3.0>0, data["inst_NUM_INSTALMENT_NUMBER"], (1.0) ))))))))) +
                    0.099922*np.tanh(((((((3.0)) < (data["inst_SK_ID_PREV"]))*1.)) - (((np.maximum(((np.maximum(((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])), ((data["FLAG_DOCUMENT_10"]))))), ((np.tanh((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])))))) / 2.0)))) +
                    0.076710*np.tanh(((np.minimum(((data["cnt_FLAG_DOCUMENT_10"])), ((((1.0) - ((((((((0.0) - (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))) + (data["cc_bal_cc_bal_status__Sent_proposal"]))/2.0)) * ((8.0))))))))) - (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))) +
                    0.097460*np.tanh(np.maximum(((np.where((((3.0) > (((data["NAME_YIELD_GROUP_XNA"]) * 2.0)))*1.)>0, (((((data["NAME_YIELD_GROUP_XNA"]) - ((-1.0*((0.0)))))) > ((5.0)))*1.), data["NAME_PORTFOLIO_Cards"] ))), ((-1.0)))) +
                    0.100000*np.tanh(((np.where((((data["NAME_CONTRACT_TYPE_Cash_loans"]) < (data["CODE_REJECT_REASON_SCOFR"]))*1.)>0, (-1.0*((((data["avg_buro_buro_bal_status_5"]) / 2.0)))), ((data["avg_buro_buro_bal_status_5"]) * 2.0) )) * (-2.0))) +
                    0.081712*np.tanh(((data["cc_bal_cc_bal_status__Refused"]) * ((((((((((1.0) < (data["PRODUCT_COMBINATION_Card_Street"]))*1.)) * ((6.0)))) * ((6.0)))) - (np.tanh((((data["cc_bal_cc_bal_status__Refused"]) - (data["FLAG_DOCUMENT_17"]))))))))) +
                    0.093357*np.tanh(((np.minimum(((data["DAYS_FIRST_DRAWING"])), ((((((data["AMT_REQ_CREDIT_BUREAU_MON"]) * (np.where(data["NAME_CONTRACT_STATUS_Approved"]>0, data["DAYS_FIRST_DRAWING"], ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["AMT_REQ_CREDIT_BUREAU_MON"]))) / 2.0) )))) * 2.0))))) * (data["AMT_REQ_CREDIT_BUREAU_MON"]))) +
                    0.098046*np.tanh(((((((data["cnt_FLAG_DOCUMENT_10"]) - ((-1.0*((np.where(data["cc_bal_SK_DPD_DEF"]>0, -1.0, data["NAME_GOODS_CATEGORY_Direct_Sales"] ))))))) + (np.where(data["cc_bal_SK_DPD_DEF"]>0, -1.0, data["cnt_FLAG_DOCUMENT_10"] )))) - (data["FLAG_DOCUMENT_17"]))) +
                    0.100000*np.tanh(((((((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"])))) < (((data["cc_bal_SK_DPD_DEF"]) / 2.0)))*1.)) * ((((((((data["cc_bal_SK_DPD_DEF"]) + (((1.0) / 2.0)))/2.0)) - (data["cc_bal_SK_ID_PREV"]))) - (data["cc_bal_SK_ID_PREV"]))))) +
                    0.099570*np.tanh((((11.08083438873291016)) * (((data["cc_bal_SK_DPD_DEF"]) * (np.minimum(((np.where(data["cc_bal_MONTHS_BALANCE"]>0, (11.08083438873291016), np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"])), ((-3.0))) ))), ((np.tanh((data["cc_bal_MONTHS_BALANCE"])))))))))) +
                    0.098476*np.tanh(((((data["cc_bal_SK_DPD"]) * (((((((data["AMT_GOODS_PRICE_x"]) * ((3.0)))) * 2.0)) * (((((((((data["cc_bal_AMT_PAYMENT_CURRENT"]) < (0.0))*1.)) > (data["cc_bal_AMT_PAYMENT_CURRENT"]))*1.)) * 2.0)))))) / 2.0)) +
                    0.099140*np.tanh((((((((np.minimum(((data["AMT_CREDIT_x"])), ((data["cc_bal_SK_DPD_DEF"])))) + (((((((((((data["cc_bal_SK_DPD_DEF"]) / 2.0)) + (data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]))/2.0)) + (data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]))/2.0)) - (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]))))/2.0)) / 2.0)) / 2.0)) +
                    0.096639*np.tanh((((((data["cnt_FLAG_DOCUMENT_10"]) + (((((((8.0)) < ((((((data["cc_bal_SK_DPD_DEF"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) / 2.0)))*1.)) * ((((9.73687171936035156)) * 2.0)))))/2.0)) + (data["cc_bal_cc_bal_status__Refused"]))) +
                    0.099062*np.tanh((((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) > (np.where(data["PRODUCT_COMBINATION_POS_industry_without_interest"]>0, data["cc_bal_AMT_PAYMENT_CURRENT"], (((((3.0) - (data["cc_bal_AMT_PAYMENT_CURRENT"]))) + ((3.74398207664489746)))/2.0) )))*1.)) +
                    0.099883*np.tanh((((np.minimum((((((((10.36883640289306641)) - (((data["PRODUCT_COMBINATION_POS_industry_without_interest"]) * 2.0)))) * 2.0))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) + (((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["avg_buro_buro_bal_status_5"], (-1.0*((data["avg_buro_buro_bal_status_5"]))) )) / 2.0)))/2.0)) +
                    0.099062*np.tanh(((np.where(((data["avg_buro_buro_bal_status_3"]) * (((data["cnt_FLAG_DOCUMENT_15"]) - (data["NAME_GOODS_CATEGORY_Other"]))))>0, data["avg_buro_buro_bal_status_5"], np.minimum(((((data["cnt_FLAG_DOCUMENT_15"]) / 2.0))), ((((data["cnt_FLAG_DOCUMENT_15"]) - (data["NAME_GOODS_CATEGORY_Other"]))))) )) * 2.0)) +
                    0.099726*np.tanh(((data["NAME_GOODS_CATEGORY_Other"]) * (((np.tanh((np.where(-3.0>0, (((14.68852424621582031)) / 2.0), ((data["NAME_GOODS_CATEGORY_Other"]) - ((((14.23720550537109375)) / 2.0))) )))) / 2.0)))) +
                    0.098984*np.tanh(np.where((((6.69561433792114258)) - (data["NAME_GOODS_CATEGORY_Other"]))>0, ((((((7.0)) < (data["NAME_GOODS_CATEGORY_Other"]))*1.)) - ((-1.0*(((-1.0*((np.maximum(((data["NAME_SELLER_INDUSTRY_MLM_partners"])), ((data["NAME_GOODS_CATEGORY_Weapon"]))))))))))), data["NAME_GOODS_CATEGORY_Other"] )) +
                    0.099492*np.tanh(np.minimum(((((((np.where(data["NAME_GOODS_CATEGORY_Auto_Accessories"]>0, (-1.0*((3.0))), np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Furniture"])), ((data["NAME_GOODS_CATEGORY_Auto_Accessories"]))) )) / 2.0)) * (data["DAYS_LAST_DUE_1ST_VERSION"])))), ((data["cnt_FLAG_DOCUMENT_4"])))) +
                    0.099062*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["cc_bal_CNT_DRAWINGS_POS_CURRENT"], (((((np.where(np.where((7.0)>0, data["DAYS_FIRST_DUE"], data["DAYS_FIRST_DUE"] )>0, data["NAME_GOODS_CATEGORY_Insurance"], -1.0 )) > (((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) * 2.0)))*1.)) * 2.0) )) +
                    0.099883*np.tanh((((7.0)) * (np.tanh((((np.where(data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]>0, ((data["AMT_CREDIT_MAX_OVERDUE"]) / 2.0), (-1.0*((((np.maximum(((data["AMT_CREDIT_MAX_OVERDUE"])), ((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"])))) * 2.0)))) )) * 2.0)))))) +
                    0.099609*np.tanh(((data["AMT_GOODS_PRICE_x"]) * (np.minimum(((data["NAME_GOODS_CATEGORY_Fitness"])), ((np.minimum(((data["FLAG_DOCUMENT_10"])), (((((((5.34786462783813477)) - (data["NAME_GOODS_CATEGORY_Fitness"]))) / 2.0)))))))))) +
                    0.098828*np.tanh(np.where(np.maximum(((((data["DAYS_ENDDATE_FACT"]) * (((((data["DAYS_ENDDATE_FACT"]) * ((4.0)))) / 2.0))))), ((((data["AMT_CREDIT_x"]) + (-3.0)))))>0, data["FLAG_DOCUMENT_10"], ((data["cc_bal_CNT_DRAWINGS_CURRENT"]) * 2.0) )) +
                    0.088238*np.tanh((((data["AMT_CREDIT_x"]) > (((3.0) - (((data["DAYS_FIRST_DUE"]) - (np.maximum((((-1.0*((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))))), ((np.minimum(((-3.0)), (((-1.0*((data["cnt_FLAG_DOCUMENT_10"])))))))))))))))*1.)) +
                    0.099844*np.tanh(((((((3.0) + ((((1.0) < (((data["NAME_EDUCATION_TYPE"]) / 2.0)))*1.)))/2.0)) < ((-1.0*((np.minimum(((data["cc_bal_AMT_DRAWINGS_CURRENT"])), ((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) * 2.0)))))))))*1.)) +
                    0.099922*np.tanh(np.minimum(((((((np.tanh((np.minimum(((data["NAME_PAYMENT_TYPE_Cash_through_the_bank"])), ((0.0)))))) / 2.0)) / 2.0))), ((np.where(data["NAME_CONTRACT_TYPE_Revolving_loans"]>0, data["NAME_PAYMENT_TYPE_XNA"], 0.0 ))))) +
                    0.081985*np.tanh(np.tanh((np.minimum(((((data["PRODUCT_COMBINATION_Cash"]) * (((data["SK_ID_PREV_x"]) - (data["AMT_GOODS_PRICE_x"])))))), ((((((((1.0) > ((-1.0*((data["PRODUCT_COMBINATION_Cash"])))))*1.)) + (data["SK_ID_PREV_x"]))/2.0))))))) +
                    0.099492*np.tanh((-1.0*(((((((((3.0) - (data["AMT_GOODS_PRICE_x"]))) / 2.0)) > (((3.0) - (np.maximum(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])), ((data["AMT_GOODS_PRICE_x"])))))))*1.))))) +
                    0.099687*np.tanh(np.where(data["cc_bal_AMT_PAYMENT_CURRENT"]>0, data["DAYS_ENDDATE_FACT"], ((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) * ((((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) < (data["DAYS_ENDDATE_FACT"]))*1.)) / 2.0)))) * ((-1.0*((data["cc_bal_AMT_PAYMENT_CURRENT"]))))) )) +
                    0.068152*np.tanh(((((2.0) * 2.0)) * (np.where(((data["NAME_TYPE_SUITE_Unaccompanied"]) / 2.0)>0, data["NAME_CASH_LOAN_PURPOSE_Furniture"], ((data["cnt_FLAG_DOCUMENT_10"]) * 2.0) )))) +
                    0.067565*np.tanh(((np.tanh((((((data["NAME_CONTRACT_STATUS_Approved"]) * ((((data["DAYS_LAST_DUE_1ST_VERSION"]) > ((((-1.0*((1.0)))) / 2.0)))*1.)))) / 2.0)))) - (((data["NAME_CONTRACT_STATUS_Approved"]) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))) +
                    0.095741*np.tanh(np.where(data["SK_ID_PREV_x"]>0, ((((((data["cc_bal_AMT_RECIVABLE"]) < (data["NAME_GOODS_CATEGORY_Fitness"]))*1.)) + (data["NAME_GOODS_CATEGORY_Fitness"]))/2.0), data["cc_bal_AMT_RECIVABLE"] )) +
                    0.067018*np.tanh(((((8.43800640106201172)) < (np.where((5.17690801620483398)>0, (((((11.66382503509521484)) / 2.0)) - ((-1.0*(((-1.0*((((data["DAYS_LAST_DUE_1ST_VERSION"]) - (data["CODE_REJECT_REASON_LIMIT"])))))))))), data["cc_bal_SK_DPD"] )))*1.)) +
                    0.099805*np.tanh(((data["cc_bal_SK_DPD"]) * ((-1.0*((np.minimum(((data["DAYS_LAST_DUE_1ST_VERSION"])), ((np.tanh((((np.minimum(((2.0)), ((data["cc_bal_SK_DPD"])))) - (data["DAYS_LAST_DUE_1ST_VERSION"]))))))))))))) +
                    0.097421*np.tanh(((((((np.tanh((1.0))) + ((((data["FLAG_DOCUMENT_16"]) + (2.0))/2.0)))) * ((5.0)))) * (((data["NAME_GOODS_CATEGORY_Insurance"]) * (((data["FLAG_DOCUMENT_16"]) - (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))))))) +
                    0.099531*np.tanh(((((data["NAME_SELLER_INDUSTRY_Tourism"]) * (np.where((((data["NAME_SELLER_INDUSTRY_Tourism"]) < (data["NAME_TYPE_SUITE_Group_of_people"]))*1.)>0, ((data["NAME_TYPE_SUITE_Group_of_people"]) * 2.0), data["PRODUCT_COMBINATION_POS_mobile_without_interest"] )))) * 2.0)) +
                    0.099531*np.tanh((((((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, np.minimum(((data["CNT_INSTALMENT"])), ((((data["NAME_GOODS_CATEGORY_Insurance"]) * (data["FLAG_DOCUMENT_4"]))))), data["NAME_GOODS_CATEGORY_Education"] )) < (np.minimum(((-1.0)), ((data["NAME_GOODS_CATEGORY_Education"])))))*1.)) * 2.0)) +
                    0.099766*np.tanh(((((((np.where(((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) / 2.0)>0, data["NAME_CASH_LOAN_PURPOSE_Furniture"], data["CNT_INSTALMENT"] )) * (data["NAME_GOODS_CATEGORY_Education"]))) - ((((data["FLAG_DOCUMENT_4"]) + (data["FLAG_DOCUMENT_10"]))/2.0)))) * 2.0)) +
                    0.099766*np.tanh(np.where(((data["PRODUCT_COMBINATION_Cash"]) - (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))>0, np.minimum(((np.tanh((data["cnt_FLAG_DOCUMENT_10"])))), ((((data["CNT_INSTALMENT_FUTURE"]) / 2.0)))), (((-1.0*(((((data["NAME_GOODS_CATEGORY_Fitness"]) > (1.0))*1.))))) / 2.0) )) +
                    0.088277*np.tanh(((((np.tanh((np.where(data["NAME_YIELD_GROUP_XNA"]>0, data["PRODUCT_COMBINATION_Cash"], data["NAME_CONTRACT_TYPE_Cash_loans"] )))) / 2.0)) / 2.0)) +
                    0.098867*np.tanh(np.minimum((((-1.0*(((((np.where(-1.0>0, 2.0, ((data["NAME_SELLER_INDUSTRY_XNA"]) + (data["NAME_PORTFOLIO_Cash"])) )) > ((3.0)))*1.)))))), ((data["NAME_GOODS_CATEGORY_Insurance"])))) +
                    0.099687*np.tanh(np.where(data["CHANNEL_TYPE_AP___Cash_loan_"]>0, (((((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0)) > (((np.tanh((np.minimum(((0.0)), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"])))))) + (2.0))))*1.), (-1.0*((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * 2.0)))) )) +
                    0.070066*np.tanh(np.maximum((((((np.where(np.where(data["DAYS_LAST_DUE_1ST_VERSION"]>0, data["CHANNEL_TYPE_Stone"], data["cnt_FLAG_DOCUMENT_10"] )>0, 3.0, data["CNT_INSTALMENT"] )) < (((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)))*1.))), (((-1.0*((0.0))))))) +
                    0.099531*np.tanh((-1.0*(((((np.where(((data["NAME_PORTFOLIO_Cards"]) + (data["DAYS_FIRST_DRAWING"]))>0, data["NAME_YIELD_GROUP_XNA"], (-1.0*((data["NAME_GOODS_CATEGORY_Insurance"]))) )) > ((2.0)))*1.))))) +
                    0.074717*np.tanh(np.tanh((np.where(data["DAYS_LAST_DUE_1ST_VERSION"]>0, ((((data["EXT_SOURCE_3"]) * 2.0)) - (1.0)), data["NAME_GOODS_CATEGORY_Insurance"] )))) +
                    0.099531*np.tanh(((((np.tanh((((((((data["ca__Active"]) * 2.0)) * ((4.0)))) * 2.0)))) / 2.0)) / 2.0)) +
                    0.085619*np.tanh((((data["DAYS_CREDIT_ENDDATE"]) < ((-1.0*(((((((data["NAME_GOODS_CATEGORY_Fitness"]) > (data["DAYS_CREDIT_ENDDATE"]))*1.)) / 2.0))))))*1.)) +
                    0.099961*np.tanh((((((((np.where((((1.0) + (data["CNT_INSTALMENT_FUTURE"]))/2.0)>0, data["DAYS_CREDIT"], -2.0 )) + (data["DAYS_CREDIT"]))) * (data["SK_ID_BUREAU"]))) + (data["DAYS_CREDIT"]))/2.0)) +
                    0.079132*np.tanh(((data["SK_ID_BUREAU"]) * ((((((((data["CNT_INSTALMENT_FUTURE"]) * 2.0)) < (data["SK_ID_BUREAU"]))*1.)) * (data["AMT_CREDIT_x"]))))) +
                    0.098828*np.tanh((((((np.where(data["SK_ID_BUREAU"]>0, data["CNT_INSTALMENT_FUTURE"], data["FLAG_DOCUMENT_2"] )) + (np.where(data["SK_ID_BUREAU"]>0, data["cnt_DAYS_ID_PUBLISH"], np.tanh(((((1.0) > ((11.27696132659912109)))*1.))) )))/2.0)) / 2.0)) +
                    0.099492*np.tanh(np.minimum(((0.0)), ((((data["DAYS_CREDIT"]) * (np.maximum(((data["CNT_INSTALMENT"])), ((np.maximum(((data["inst_NUM_INSTALMENT_VERSION"])), ((np.tanh(((((-2.0) > (0.0))*1.))))))))))))))) +
                    0.099922*np.tanh(np.tanh(((((np.tanh((np.tanh(((-1.0*((np.maximum(((-3.0)), ((data["ca__Closed"]))))))))))) + (((data["ca__Closed"]) * ((-1.0*((data["SK_ID_BUREAU"])))))))/2.0)))) +
                    0.099883*np.tanh(np.maximum(((((((((np.where(data["DAYS_ENDDATE_FACT"]>0, -1.0, data["SK_ID_BUREAU"] )) + (1.0))) * (data["AMT_CREDIT_y"]))) * (data["CNT_INSTALMENT_FUTURE"])))), ((0.0)))) +
                    0.089840*np.tanh(((((((data["inst_AMT_PAYMENT"]) * (((((data["SK_ID_BUREAU"]) / 2.0)) * (data["inst_AMT_PAYMENT"]))))) + ((((0.0) < (data["PRODUCT_COMBINATION_POS_household_without_interest"]))*1.)))) / 2.0)) +
                    0.100000*np.tanh(np.minimum((((((data["AMT_CREDIT_SUM_DEBT"]) + (np.maximum(((data["EXT_SOURCE_3"])), ((data["AMT_CREDIT_SUM_DEBT"])))))/2.0))), (((((0.0)) - (np.where(data["AMT_CREDIT_SUM_DEBT"]>0, data["EXT_SOURCE_3"], data["NAME_GOODS_CATEGORY_Insurance"] ))))))) +
                    0.099922*np.tanh((-1.0*((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) + ((((np.tanh((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((data["EXT_SOURCE_3"]) / 2.0)))))) < (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)))) * (data["EXT_SOURCE_3"])))))) +
                    0.100000*np.tanh(((((np.minimum(((((data["DAYS_LAST_DUE"]) * (data["CNT_INSTALMENT"])))), (((((10.0)) * (1.0)))))) / 2.0)) * (data["inst_AMT_PAYMENT"]))) +
                    0.100000*np.tanh(((data["EXT_SOURCE_3"]) * ((((((data["AMT_GOODS_PRICE_x"]) < (((np.tanh((((data["AMT_GOODS_PRICE_x"]) * 2.0)))) * 2.0)))*1.)) / 2.0)))) +
                    0.100000*np.tanh(((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((((data["DAYS_CREDIT_UPDATE"]) < (-3.0))*1.))))) * (data["DAYS_CREDIT"]))) +
                    0.084447*np.tanh(np.minimum(((((np.minimum((((-1.0*((data["CHANNEL_TYPE_Stone"]))))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))) * (((data["inst_AMT_PAYMENT"]) / 2.0))))), ((((((-1.0*((data["NAME_YIELD_GROUP_low_action"])))) + (2.0))/2.0))))) +
                    0.099922*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Repairs"]) * (((((data["NAME_CASH_LOAN_PURPOSE_Repairs"]) * ((((((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((((data["EXT_SOURCE_3"]) + (1.0))/2.0))))) + (data["inst_DAYS_INSTALMENT"]))/2.0)) / 2.0)))) / 2.0)))) +
                    0.099726*np.tanh(np.where(np.maximum(((data["te_DAYS_ID_PUBLISH"])), ((np.tanh((data["te_DAYS_ID_PUBLISH"])))))>0, (((((data["te_DAYS_ID_PUBLISH"]) > (3.0))*1.)) * (data["te_DAYS_ID_PUBLISH"])), data["AMT_CREDIT_x"] )) +
                    0.100000*np.tanh(((data["NAME_GOODS_CATEGORY_Medicine"]) + ((((data["NAME_GOODS_CATEGORY_Medicine"]) > ((((((data["EXT_SOURCE_3"]) + (2.0))/2.0)) + ((-1.0*((np.tanh((data["NAME_GOODS_CATEGORY_Medicine"])))))))))*1.)))) +
                    0.100000*np.tanh(np.where((-1.0*((((((((((0.41248092055320740)) / 2.0)) + (data["AMT_GOODS_PRICE_x"]))/2.0)) * (data["AMT_GOODS_PRICE_x"])))))>0, -2.0, ((((data["cnt_FLAG_DOCUMENT_10"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * 2.0) )) +
                    0.099883*np.tanh(((((data["inst_AMT_PAYMENT"]) * (((np.where(0.0>0, (-1.0*((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))), np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((-2.0) + (((data["inst_AMT_PAYMENT"]) / 2.0)))))) )) * 2.0)))) * 2.0)) +
                    0.096717*np.tanh(((np.where(data["CNT_INSTALMENT_FUTURE"]>0, (((-1.0*((2.0)))) + (((data["CODE_REJECT_REASON_VERIF"]) - (((data["CODE_REJECT_REASON_VERIF"]) * 2.0))))), np.tanh((data["inst_AMT_PAYMENT"])) )) * (data["CODE_REJECT_REASON_VERIF"]))) +
                    0.098828*np.tanh(np.where((((data["NAME_CASH_LOAN_PURPOSE_Repairs"]) < (((data["AMT_DOWN_PAYMENT"]) * 2.0)))*1.)>0, data["NAME_CASH_LOAN_PURPOSE_Repairs"], np.tanh(((-1.0*((((((((data["cc_bal_AMT_PAYMENT_CURRENT"]) + ((5.25137329101562500)))/2.0)) < (data["NAME_CASH_LOAN_PURPOSE_Repairs"]))*1.)))))) )) +
                    0.099570*np.tanh(((((((data["cc_bal_AMT_PAYMENT_CURRENT"]) + (-1.0))/2.0)) > ((((np.maximum(((-1.0)), ((((data["inst_AMT_PAYMENT"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"])))))) + (data["AMT_DOWN_PAYMENT"]))/2.0)))*1.)) +
                    0.099609*np.tanh(((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((((np.tanh((np.where((((data["AMT_GOODS_PRICE_x"]) > (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))*1.)>0, data["inst_AMT_PAYMENT"], -2.0 )))) * 2.0)) * 2.0)))) +
                    0.098476*np.tanh(((np.maximum(((((data["cc_bal_AMT_DRAWINGS_CURRENT"]) - (-1.0)))), ((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])))) * (np.tanh(((-1.0*((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) - (data["cc_bal_AMT_DRAWINGS_CURRENT"]))))))))))))) +
                    0.067917*np.tanh(((((data["inst_AMT_INSTALMENT"]) * 2.0)) * ((((((0.0) / 2.0)) > ((((((data["AMT_CREDIT_x"]) + (1.0))) + (((data["inst_AMT_INSTALMENT"]) / 2.0)))/2.0)))*1.)))) +
                    0.064478*np.tanh(((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, (((data["inst_AMT_PAYMENT"]) < (data["AMT_GOODS_PRICE_x"]))*1.), data["NAME_GOODS_CATEGORY_Direct_Sales"] )) * (((data["inst_AMT_PAYMENT"]) + (((-2.0) * 2.0)))))) +
                    0.100000*np.tanh(((data["AMT_GOODS_PRICE_x"]) * (np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((data["NAME_GOODS_CATEGORY_Insurance"]) - ((((data["cu__currency_3"]) > ((-1.0*((np.tanh((data["NAME_GOODS_CATEGORY_Direct_Sales"])))))))*1.))))))))) +
                    0.099570*np.tanh(np.maximum(((((((data["AMT_DOWN_PAYMENT"]) * (data["cu__currency_3"]))) * (-3.0)))), (((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > (np.maximum((((14.77923965454101562))), ((data["AMT_DOWN_PAYMENT"])))))*1.))))) +
                    0.099453*np.tanh((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) * 2.0)) < (((((((-1.0*((2.0)))) + (((data["AMT_DOWN_PAYMENT"]) / 2.0)))/2.0)) / 2.0)))*1.)) +
                    0.062173*np.tanh((((((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) - (((data["CHANNEL_TYPE_Car_dealer"]) / 2.0)))) + ((((((data["CHANNEL_TYPE_Car_dealer"]) - (data["NAME_GOODS_CATEGORY_Insurance"]))) > (data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]))*1.)))/2.0)) +
                    0.098867*np.tanh((-1.0*((np.maximum(((((np.where(data["NAME_GOODS_CATEGORY_Fitness"]>0, data["NAME_GOODS_CATEGORY_Fitness"], data["NAME_GOODS_CATEGORY_Weapon"] )) + (np.tanh((data["FLAG_DOCUMENT_10"])))))), (((((((data["NAME_GOODS_CATEGORY_Fitness"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) < (-1.0))*1.)))))))) +
                    0.098671*np.tanh(((data["cnt_FLAG_DOCUMENT_9"]) * ((((data["cnt_FLAG_DOCUMENT_4"]) + (np.where(np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((data["cnt_FLAG_DOCUMENT_9"])))>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], -3.0 )))/2.0)))) +
                    0.086088*np.tanh(np.minimum(((data["FLAG_DOCUMENT_17"])), ((((np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((((data["YEARS_BUILD_AVG"]) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"]))/2.0))))) + ((-1.0*((np.where((-1.0*((data["te_FLAG_DOCUMENT_8"])))>0, data["YEARS_BUILD_AVG"], data["FLAG_DOCUMENT_17"] )))))))))) +
                    0.098867*np.tanh(np.where((10.90770244598388672)>0, ((((((2.0)) < (data["HOUSETYPE_MODE"]))*1.)) * (((data["nans"]) * (((-3.0) * 2.0))))), (-1.0*((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) )) +
                    0.099531*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((((data["cnt_WALLSMATERIAL_MODE"]) - (np.minimum(((3.0)), (((-1.0*((((data["te_FONDKAPREMONT_MODE"]) * 2.0)))))))))) - ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) < (1.0))*1.)))))) +
                    0.098867*np.tanh(np.tanh(((((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["te_FLAG_DOCUMENT_8"])))) < (((data["inst_AMT_PAYMENT"]) - ((((((((3.0) + (data["inst_AMT_PAYMENT"]))) + (data["cnt_HOUSETYPE_MODE"]))/2.0)) * 2.0)))))*1.)))) +
                    0.098710*np.tanh(np.minimum(((((((data["cnt_FLAG_DOCUMENT_8"]) * (((1.0) - (np.maximum(((data["AMT_GOODS_PRICE_x"])), ((np.maximum(((data["FLAG_DOCUMENT_8"])), ((0.0))))))))))) / 2.0))), ((((data["cnt_FLAG_DOCUMENT_8"]) * (data["cnt_HOUSETYPE_MODE"])))))) +
                    0.069637*np.tanh(((((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["te_REG_REGION_NOT_LIVE_REGION"], np.maximum(((data["CODE_REJECT_REASON_SCOFR"])), ((data["NAME_GOODS_CATEGORY_Insurance"]))) )) * (data["te_REG_REGION_NOT_LIVE_REGION"]))) * 2.0)) +
                    0.099766*np.tanh(np.where(data["AMT_CREDIT_MAX_OVERDUE"]>0, data["AMT_CREDIT_MAX_OVERDUE"], (((((data["AMT_CREDIT_MAX_OVERDUE"]) * 2.0)) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.) )) +
                    0.086635*np.tanh(((((-1.0*((((data["te_REG_REGION_NOT_LIVE_REGION"]) * (((((((3.0) * (data["te_FLAG_EMAIL"]))) / 2.0)) / 2.0))))))) + (np.where(data["te_FLAG_EMAIL"]>0, 0.0, data["AMT_CREDIT_x"] )))/2.0)) +
                    0.099492*np.tanh((((((((np.minimum(((-2.0)), (((((data["te_FLAG_EMAIL"]) < (np.tanh(((7.92169857025146484)))))*1.))))) - (3.0))) > (data["te_FLAG_EMAIL"]))*1.)) - (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0)))) +
                    0.096678*np.tanh(np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((data["CNT_INSTALMENT"]) * ((((((data["AMT_GOODS_PRICE_x"]) * (data["cnt_DAYS_BIRTH"]))) + (data["cnt_DAYS_BIRTH"]))/2.0))))))) +
                    0.099961*np.tanh(((np.where(data["NAME_GOODS_CATEGORY_Education"]>0, ((data["AMT_CREDIT_x"]) - (((((data["NAME_GOODS_CATEGORY_Education"]) + (data["cnt_FLAG_DOCUMENT_18"]))) * 2.0))), ((data["cnt_FLAG_DOCUMENT_18"]) - (((data["NAME_GOODS_CATEGORY_Education"]) * 2.0))) )) / 2.0)) +
                    0.099922*np.tanh(((data["cnt_FLAG_DOCUMENT_14"]) * (np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["FLAG_DOCUMENT_18"], ((data["FLAG_DOCUMENT_18"]) + (((data["FLAG_DOCUMENT_14"]) - (((1.0) - (((data["cnt_FLAG_DOCUMENT_18"]) * ((7.0))))))))) )))) +
                    0.060297*np.tanh(np.where((((data["CNT_INSTALMENT_FUTURE"]) > (np.minimum(((1.0)), ((1.0)))))*1.)>0, data["nans"], (((data["cnt_FLAG_DOCUMENT_14"]) + (np.tanh(((-1.0*((data["nans"])))))))/2.0) )) +
                    0.099922*np.tanh(((data["DAYS_CREDIT"]) * (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((((((data["CNT_INSTALMENT_FUTURE"]) + ((((-1.0*((((data["te_FLAG_DOCUMENT_20"]) * (data["WALLSMATERIAL_MODE"])))))) * (((data["CNT_INSTALMENT_FUTURE"]) * 2.0)))))/2.0)) / 2.0))))))) +
                    0.099883*np.tanh((((((3.0) / 2.0)) < (((data["avg_buro_buro_bal_status_C"]) - ((((np.where(data["AMT_CREDIT_y"]>0, data["te_FLAG_DOCUMENT_20"], data["avg_buro_MONTHS_BALANCE"] )) + (data["WALLSMATERIAL_MODE"]))/2.0)))))*1.)) +
                    0.096600*np.tanh(((((data["WALLSMATERIAL_MODE"]) * (((np.where(2.0>0, ((data["AMT_CREDIT_y"]) * 2.0), -3.0 )) * (data["NAME_GOODS_CATEGORY_Auto_Accessories"]))))) - ((((data["te_FLAG_DOCUMENT_20"]) > (np.tanh((2.0))))*1.)))) +
                    0.099648*np.tanh((-1.0*((((((2.71130990982055664)) < ((((((data["NAME_GOODS_CATEGORY_Auto_Accessories"]) - ((((-1.0*((1.0)))) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))) + (((1.0) + (data["AMT_CREDIT_y"]))))/2.0)))*1.))))) +
                    0.099726*np.tanh(((((((-1.0) + (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"]) * (data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"]))))/2.0)) < (((((data["AMT_CREDIT_y"]) * 2.0)) - ((((3.0)) * 2.0)))))*1.)) +
                    0.053576*np.tanh(np.minimum(((((data["AMT_GOODS_PRICE_x"]) * (((data["inst_AMT_PAYMENT"]) - (data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (np.maximum(((data["inst_AMT_PAYMENT"])), ((((np.tanh((data["CNT_INSTALMENT_FUTURE"]))) / 2.0)))))))))) +
                    0.099258*np.tanh(((((((data["CNT_INSTALMENT_FUTURE"]) * ((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"])))))) * 2.0)) + (((-2.0) * (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * ((((2.0) > (-3.0))*1.)))))))) +
                    0.099258*np.tanh(((((np.maximum((((-1.0*((data["inst_AMT_PAYMENT"]))))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"])))) * (data["inst_AMT_PAYMENT"]))) * (data["CNT_INSTALMENT"]))) +
                    0.099961*np.tanh(((data["inst_AMT_INSTALMENT"]) * ((((((np.where(-2.0>0, ((-1.0) * (data["inst_AMT_INSTALMENT"])), data["CNT_INSTALMENT_FUTURE"] )) < ((((data["NAME_YIELD_GROUP_high"]) + (data["inst_AMT_INSTALMENT"]))/2.0)))*1.)) / 2.0)))) +
                    0.099961*np.tanh(((((((data["inst_AMT_INSTALMENT"]) - (data["inst_AMT_PAYMENT"]))) * 2.0)) * ((4.81916522979736328)))) +
                    0.099922*np.tanh((-1.0*(((((((((((((np.where(data["AMT_CREDIT_y"]>0, (0.0), data["inst_AMT_PAYMENT"] )) > (data["inst_AMT_INSTALMENT"]))*1.)) / 2.0)) > ((((0.0) > ((11.85620403289794922)))*1.)))*1.)) + (data["inst_AMT_PAYMENT"]))/2.0))))) +
                    0.099883*np.tanh((-1.0*((((((np.where(((data["NAME_PORTFOLIO_Cash"]) + ((-1.0*((np.minimum(((np.tanh((data["AMT_ANNUITY"])))), ((data["inst_AMT_PAYMENT"]))))))))>0, data["AMT_ANNUITY"], 0.0 )) / 2.0)) / 2.0))))) +
                    0.077061*np.tanh(np.where(data["cc_bal_SK_ID_PREV"]>0, np.tanh((1.0)), ((data["inst_NUM_INSTALMENT_NUMBER"]) * ((((((1.0) < (data["inst_NUM_INSTALMENT_NUMBER"]))*1.)) + (data["inst_NUM_INSTALMENT_NUMBER"])))) )) +
                    0.099922*np.tanh(((np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((((data["avg_buro_buro_bal_status_1"]) * (((data["inst_NUM_INSTALMENT_VERSION"]) - (data["PRODUCT_COMBINATION_Cash_X_Sell__high"])))))))) + ((((((data["inst_NUM_INSTALMENT_VERSION"]) * 2.0)) > (1.0))*1.)))) +
                    0.100000*np.tanh(np.minimum(((((data["Completed"]) * (((data["PRODUCT_COMBINATION_Cash_Street__high"]) * (1.0)))))), ((np.tanh((((((((((data["NFLAG_INSURED_ON_APPROVAL"]) * (data["CNT_INSTALMENT_FUTURE"]))) * 2.0)) / 2.0)) / 2.0))))))) +
                    0.087026*np.tanh(np.maximum(((np.where(data["PRODUCT_COMBINATION_Cash_X_Sell__high"]>0, data["NAME_YIELD_GROUP_low_action"], (((np.maximum(((data["NAME_YIELD_GROUP_low_action"])), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))) > (1.0))*1.) ))), (((((data["SK_ID_PREV_y"]) < (((2.0) - ((3.0)))))*1.))))) +
                    0.056389*np.tanh(((np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["MONTHS_BALANCE"])))) * (((data["CNT_INSTALMENT_FUTURE"]) - (np.minimum(((data["inst_AMT_PAYMENT"])), (((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)) + ((14.19008731842041016))))))))))) +
                    0.075889*np.tanh(((data["MONTHS_BALANCE"]) * (np.tanh((np.minimum((((-1.0*((data["NAME_YIELD_GROUP_low_action"]))))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))))))) +
                    0.080227*np.tanh((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + ((-1.0*((((((np.tanh((data["SK_ID_PREV_y"]))) / 2.0)) * 2.0))))))/2.0)) / 2.0)) +
                    0.091755*np.tanh(np.minimum(((np.tanh((((data["NAME_SELLER_INDUSTRY_Connectivity"]) * (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (data["NAME_SELLER_INDUSTRY_Connectivity"])))))))), ((((data["NAME_SELLER_INDUSTRY_Connectivity"]) * (((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * 2.0))))))) +
                    0.099570*np.tanh(((data["NAME_SELLER_INDUSTRY_Connectivity"]) * ((-1.0*((((((((data["NAME_SELLER_INDUSTRY_Connectivity"]) > (data["DAYS_FIRST_DRAWING"]))*1.)) < (np.maximum(((0.0)), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) - (((data["cc_bal_AMT_PAYMENT_CURRENT"]) * 2.0))))))))*1.))))))) +
                    0.090621*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, data["DAYS_FIRST_DRAWING"], ((((data["cc_bal_AMT_PAYMENT_CURRENT"]) * (data["CNT_INSTALMENT"]))) / 2.0) )) +
                    0.099766*np.tanh(np.where(((data["DAYS_FIRST_DRAWING"]) * ((((1.90255212783813477)) * (-2.0))))>0, (((((data["cc_bal_AMT_PAYMENT_CURRENT"]) / 2.0)) > (((3.0) + (data["DAYS_FIRST_DRAWING"]))))*1.), np.tanh((data["cc_bal_AMT_PAYMENT_CURRENT"])) )) +
                    0.099844*np.tanh(((((data["SK_ID_BUREAU"]) * 2.0)) * ((((np.tanh((3.0))) < (((data["SK_ID_BUREAU"]) + (((((((data["CNT_INSTALMENT"]) + (-2.0))) * 2.0)) * 2.0)))))*1.)))) +
                    0.079797*np.tanh((((data["NAME_SELLER_INDUSTRY_Connectivity"]) > (((((2.0)) + (((((((3.0) / 2.0)) + (data["MONTHS_BALANCE"]))) + (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["MONTHS_BALANCE"], np.tanh((2.0)) )))))/2.0)))*1.)) +
                    0.096561*np.tanh(np.minimum((((((((((((((data["SK_DPD"]) + (data["WALLSMATERIAL_MODE"]))/2.0)) * (data["SK_ID_PREV_x"]))) * (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))) * 2.0)) * 2.0))), ((np.where(data["WALLSMATERIAL_MODE"]>0, data["NAME_CLIENT_TYPE_Repeater"], (2.05001640319824219) ))))) +
                    0.081594*np.tanh((-1.0*((((((((data["CNT_INSTALMENT"]) < (2.0))*1.)) < (data["te_WALLSMATERIAL_MODE"]))*1.))))) +
                    0.099492*np.tanh(((((((((data["CNT_INSTALMENT_FUTURE"]) / 2.0)) + (((np.tanh((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, 3.0, data["cnt_WALLSMATERIAL_MODE"] )))) + (data["AMT_CREDIT_x"]))))/2.0)) > (2.0))*1.)) +
                    0.076163*np.tanh((((((((2.0) - ((((data["WALLSMATERIAL_MODE"]) > ((11.67886257171630859)))*1.)))) - ((((data["cnt_WALLSMATERIAL_MODE"]) > ((-1.0*((data["DAYS_LAST_DUE_1ST_VERSION"])))))*1.)))) < (data["WALLSMATERIAL_MODE"]))*1.)) +
                    0.091755*np.tanh(((np.where((((data["NAME_CONTRACT_TYPE_Revolving_loans"]) > (data["DAYS_LAST_DUE_1ST_VERSION"]))*1.)>0, np.where(-1.0>0, np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])), ((0.0))), data["NAME_PORTFOLIO_Cards"] ), data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"] )) * (data["AMT_CREDIT_x"]))) +
                    0.093513*np.tanh(((((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) * (((((((data["DAYS_LAST_DUE_1ST_VERSION"]) > ((((data["DAYS_LAST_DUE_1ST_VERSION"]) > (((((data["inst_NUM_INSTALMENT_VERSION"]) * 2.0)) / 2.0)))*1.)))*1.)) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))/2.0)))) * 2.0)) +
                    0.098202*np.tanh(((-2.0) * (((np.tanh((data["avg_buro_buro_count"]))) * (np.where(np.where((5.0)>0, np.maximum(((data["CNT_INSTALMENT_FUTURE"])), ((data["avg_buro_buro_count"]))), data["CNT_INSTALMENT_FUTURE"] )>0, data["NAME_CASH_LOAN_PURPOSE_Business_development"], data["AMT_GOODS_PRICE_x"] )))))) +
                    0.091872*np.tanh(np.tanh((((data["FLAG_OWN_CAR"]) * (np.minimum(((data["inst_AMT_PAYMENT"])), ((((((-1.0*((data["avg_buro_buro_count"])))) > (((data["AMT_GOODS_PRICE_x"]) * 2.0)))*1.))))))))) +
                    0.098789*np.tanh(((3.0) * ((((-1.0*((((data["FLAG_OWN_CAR"]) + (data["te_FLAG_OWN_CAR"])))))) + ((((data["avg_buro_buro_count"]) < (((((3.0) / 2.0)) + ((-1.0*((3.0)))))))*1.)))))) +
                    0.099570*np.tanh(((((np.tanh((((np.minimum(((((data["te_FLAG_DOCUMENT_2"]) / 2.0))), ((np.minimum(((data["te_FLAG_DOCUMENT_12"])), ((data["te_DAYS_BIRTH"]))))))) - (((np.tanh((((1.0) / 2.0)))) / 2.0)))))) / 2.0)) / 2.0)) +
                    0.098476*np.tanh(((((((1.0) * (((np.where(data["AMT_GOODS_PRICE_x"]>0, -1.0, data["te_CODE_GENDER"] )) / 2.0)))) / 2.0)) / 2.0)) +
                    0.099531*np.tanh((((np.where((((data["AMT_CREDIT_y"]) + (((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) / 2.0)))/2.0)>0, ((data["AMT_CREDIT_y"]) + (1.0)), np.minimum(((((2.0) / 2.0))), ((data["avg_buro_buro_count"]))) )) < (-1.0))*1.)) +
                    0.093630*np.tanh(np.minimum(((((np.maximum(((data["CNT_INSTALMENT_FUTURE"])), (((-1.0*((data["avg_buro_MONTHS_BALANCE"]))))))) / 2.0))), (((((data["FLAG_OWN_CAR"]) > (np.minimum(((data["CNT_INSTALMENT_FUTURE"])), ((np.maximum(((data["te_FLAG_OWN_CAR"])), ((data["avg_buro_MONTHS_BALANCE"]))))))))*1.))))) +
                    0.096991*np.tanh((-1.0*(((((((((data["CNT_INSTALMENT"]) - (np.where((13.12239170074462891)>0, np.where(data["NAME_YIELD_GROUP_low_normal"]>0, -1.0, data["AMT_GOODS_PRICE_x"] ), data["CNT_INSTALMENT"] )))) > (2.0))*1.)) / 2.0))))) +
                    0.099844*np.tanh((((2.0) < (((data["CNT_INSTALMENT_FUTURE"]) - ((((data["AMT_GOODS_PRICE_x"]) < (2.0))*1.)))))*1.)) +
                    0.082689*np.tanh((((3.0) < (((data["inst_AMT_PAYMENT"]) * (data["inst_AMT_PAYMENT"]))))*1.)) +
                    0.099961*np.tanh(((np.tanh(((((2.0) < (np.minimum(((((((((data["CHANNEL_TYPE_Car_dealer"]) * 2.0)) * 2.0)) + ((((0.91963672637939453)) - (data["AMT_ANNUITY_x"])))))), (((6.00309991836547852))))))*1.)))) / 2.0)) +
                    0.099570*np.tanh(np.tanh((np.where(np.tanh(((((data["inst_AMT_PAYMENT"]) < (data["inst_AMT_INSTALMENT"]))*1.)))>0, ((data["CHANNEL_TYPE_Car_dealer"]) - (data["inst_AMT_PAYMENT"])), ((data["NAME_GOODS_CATEGORY_Insurance"]) - (0.0)) )))) +
                    0.069676*np.tanh((((((((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) / 2.0)) > (data["AMT_APPLICATION"]))*1.)) * ((((-1.0*((((data["AMT_APPLICATION"]) + ((-1.0*((data["AMT_CREDIT_y"]))))))))) * 2.0)))) * 2.0)) +
                    0.098593*np.tanh(np.tanh((np.tanh((np.minimum((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) < (data["NAME_YIELD_GROUP_low_normal"]))*1.))), (((((data["AMT_CREDIT_y"]) > (data["AMT_APPLICATION"]))*1.))))))))) +
                    0.100000*np.tanh((((-1.0*((((data["NAME_PRODUCT_TYPE_x_sell"]) * (data["inst_NUM_INSTALMENT_NUMBER"])))))) * (((((((np.maximum(((data["DAYS_FIRST_DRAWING"])), ((data["NAME_PRODUCT_TYPE_x_sell"])))) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) / 2.0)) * (1.0))))) +
                    0.099961*np.tanh((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) > (np.maximum((((((((-2.0) < (data["DAYS_LAST_DUE_1ST_VERSION"]))*1.)) + (data["DAYS_LAST_DUE_1ST_VERSION"])))), ((((((data["DAYS_LAST_DUE_1ST_VERSION"]) + (data["DAYS_LAST_DUE_1ST_VERSION"]))) + (data["DAYS_LAST_DUE_1ST_VERSION"])))))))*1.)) +
                    0.076827*np.tanh(((data["NAME_PRODUCT_TYPE_x_sell"]) * (np.where(((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) / 2.0)>0, data["NAME_CLIENT_TYPE_New"], ((np.minimum(((((data["NAME_CONTRACT_STATUS_Approved"]) + (2.0)))), ((((((8.13994503021240234)) < (3.0))*1.))))) * 2.0) )))) +
                    0.099179*np.tanh(np.minimum(((((data["AMT_CREDIT_x"]) * ((((((((-1.0*((data["CHANNEL_TYPE_Car_dealer"])))) * 2.0)) * 2.0)) - (0.0)))))), (((-1.0*((data["CHANNEL_TYPE_Car_dealer"]))))))) +
                    0.099726*np.tanh(((((((data["NAME_CONTRACT_STATUS_Approved"]) / 2.0)) * (data["PRODUCT_COMBINATION_Card_X_Sell"]))) * (np.minimum(((((data["DAYS_LAST_DUE_1ST_VERSION"]) * (data["PRODUCT_COMBINATION_Card_X_Sell"])))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))))) +
                    0.099883*np.tanh(np.maximum(((((data["inst_DAYS_ENTRY_PAYMENT"]) + (((-1.0) - ((((data["CODE_REJECT_REASON_HC"]) > (0.0))*1.))))))), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["CODE_REJECT_REASON_HC"])))))) +
                    0.099453*np.tanh(((((data["DAYS_FIRST_DUE"]) * (((((((data["DAYS_FIRST_DUE"]) * (data["cc_bal_AMT_PAYMENT_CURRENT"]))) / 2.0)) + ((-1.0*((np.tanh((np.tanh((data["PRODUCT_COMBINATION_Card_X_Sell"])))))))))))) / 2.0)) +
                    0.099844*np.tanh((((np.minimum(((((((((1.0) * 2.0)) + (data["DAYS_FIRST_DUE"]))) * (data["DAYS_LAST_DUE"])))), ((np.tanh((0.0)))))) > (data["DAYS_TERMINATION"]))*1.)) +
                    0.099375*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((data["DAYS_FIRST_DUE"]) + (((data["DAYS_FIRST_DUE"]) + (data["DAYS_TERMINATION"]))))))) * 2.0)) +
                    0.099922*np.tanh((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) > ((-1.0*((np.minimum(((data["NAME_CONTRACT_STATUS_Approved"])), (((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["SK_DPD_DEF"]))/2.0)))))))))*1.)) * 2.0)) * 2.0)) +
                    0.088707*np.tanh(((np.where(data["PRODUCT_COMBINATION_Card_X_Sell"]>0, np.minimum(((np.tanh((data["AMT_CREDIT_y"])))), ((np.minimum((((-1.0*((data["avg_buro_buro_bal_status_1"]))))), ((data["SK_DPD_DEF"])))))), data["SK_DPD_DEF"] )) * 2.0)) +
                    0.075811*np.tanh((-1.0*((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, np.maximum(((data["AMT_GOODS_PRICE_x"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))), data["cc_bal_cc_bal_status__Sent_proposal"] ))))) +
                    0.099570*np.tanh((((-1.0*((np.where((((13.84500789642333984)) * (data["CHANNEL_TYPE_Car_dealer"]))>0, data["OBS_60_CNT_SOCIAL_CIRCLE"], (-1.0*((np.maximum(((data["SK_DPD_DEF"])), ((((((3.0)) < (data["OBS_60_CNT_SOCIAL_CIRCLE"]))*1.))))))) ))))) * 2.0)) +
                    0.099805*np.tanh((-1.0*(((((data["NAME_GOODS_CATEGORY_Insurance"]) + (((((np.minimum(((data["OBS_30_CNT_SOCIAL_CIRCLE"])), ((np.maximum(((((-1.0) + (data["NAME_YIELD_GROUP_high"])))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))))) / 2.0)) + (data["SK_DPD_DEF"]))))/2.0))))) +
                    0.099648*np.tanh(((((data["EXT_SOURCE_3"]) * (np.tanh(((((((((11.57633495330810547)) > (((data["SK_DPD_DEF"]) * ((5.0)))))*1.)) < (((np.minimum(((data["AMT_DOWN_PAYMENT"])), (((11.51627063751220703))))) * 2.0)))*1.)))))) * 2.0)) +
                    0.100000*np.tanh(np.where(data["CNT_INSTALMENT"]>0, ((((data["SK_DPD_DEF"]) - ((((3.0) < (data["CHANNEL_TYPE_Car_dealer"]))*1.)))) * ((8.51310443878173828))), data["NAME_GOODS_CATEGORY_Insurance"] )) +
                    0.099922*np.tanh(np.where(data["SK_DPD_DEF"]>0, ((np.minimum((((2.84313511848449707))), ((((((data["inst_DAYS_INSTALMENT"]) - (0.0))) * 2.0))))) * 2.0), (-1.0*((0.0))) )) +
                    0.099844*np.tanh(((((np.minimum(((data["AMT_CREDIT_x"])), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))) + (data["SK_DPD_DEF"]))) * (np.where(data["SK_DPD_DEF"]>0, ((data["CNT_INSTALMENT_FUTURE"]) * 2.0), data["NAME_GOODS_CATEGORY_Insurance"] )))) +
                    0.068660*np.tanh(((data["CNT_INSTALMENT_FUTURE"]) * (np.maximum(((np.minimum(((((1.0) - (data["SK_DPD_DEF"])))), ((data["SK_DPD_DEF"]))))), ((((data["SK_ID_PREV_x"]) / 2.0))))))) +
                    0.099179*np.tanh(np.where(data["CHANNEL_TYPE_AP___Cash_loan_"]>0, (-1.0*((((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((data["CNT_INSTALMENT"]) / 2.0))))) / 2.0)))), np.where(data["AMT_DOWN_PAYMENT"]>0, (((data["AMT_CREDIT_x"]) < (-1.0))*1.), data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"] ) )) +
                    0.099844*np.tanh(np.minimum((((((((data["SK_DPD_DEF"]) * 2.0)) + (((np.tanh((data["REGION_POPULATION_RELATIVE"]))) / 2.0)))/2.0))), ((np.tanh((((data["SK_DPD_DEF"]) * (data["CNT_INSTALMENT"])))))))) +
                    0.099726*np.tanh(((data["cnt_FLAG_DOCUMENT_10"]) + ((-1.0*((np.maximum(((np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, data["te_REGION_RATING_CLIENT_W_CITY"], data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"] ))), (((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) + (((data["SK_DPD_DEF"]) / 2.0)))/2.0)))))))))) +
                    0.099648*np.tanh((((((-1.0*((np.where(np.where(((-1.0) / 2.0)>0, data["NAME_GOODS_CATEGORY_Insurance"], data["NAME_CASH_LOAN_PURPOSE_Business_development"] )>0, ((data["AMT_CREDIT_x"]) * 2.0), data["NAME_CASH_LOAN_PURPOSE_Business_development"] ))))) * 2.0)) * (data["REGION_RATING_CLIENT_W_CITY"]))) +
                    0.099687*np.tanh(np.minimum((((-1.0*(((((data["AMT_CREDIT_x"]) > (((2.0) * 2.0)))*1.)))))), (((((((((-1.0*((data["cnt_FLAG_DOCUMENT_13"])))) < (np.tanh((data["cnt_REGION_RATING_CLIENT_W_CITY"]))))*1.)) + (data["AMT_CREDIT_x"]))/2.0))))) +
                    0.075225*np.tanh(((((data["inst_AMT_PAYMENT"]) * 2.0)) * (np.where(((np.where(-3.0>0, data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"], data["cnt_NAME_HOUSING_TYPE"] )) - (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))>0, data["CHANNEL_TYPE_Car_dealer"], data["inst_AMT_PAYMENT"] )))) +
                    0.099844*np.tanh(((-3.0) * ((((((((data["REGION_RATING_CLIENT_W_CITY"]) + (data["NAME_HOUSING_TYPE"]))) + (data["CHANNEL_TYPE_Car_dealer"]))) > (((((9.0)) + ((((data["CHANNEL_TYPE_Car_dealer"]) + (-1.0))/2.0)))/2.0)))*1.)))) +
                    0.094881*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * 2.0)) * ((((((-1.0*((data["te_REGION_RATING_CLIENT_W_CITY"])))) - ((((2.0) + (0.0))/2.0)))) - (data["REGION_RATING_CLIENT_W_CITY"]))))) +
                    0.084525*np.tanh(((np.tanh((data["HOUR_APPR_PROCESS_START_x"]))) * ((((data["AMT_DOWN_PAYMENT"]) + (((((((data["AMT_CREDIT_x"]) < (2.0))*1.)) + (data["AMT_CREDIT_x"]))/2.0)))/2.0)))) +
                    0.085698*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, (((data["cnt_FLAG_DOCUMENT_4"]) < ((-1.0*((data["te_HOUR_APPR_PROCESS_START_x"])))))*1.), (((-3.0) > (data["HOUR_APPR_PROCESS_START_y"]))*1.) )) +
                    0.084369*np.tanh(((np.minimum(((np.minimum(((((data["HOUR_APPR_PROCESS_START_x"]) * (data["nans"])))), (((0.75266021490097046)))))), ((((((data["HOUR_APPR_PROCESS_START_x"]) * (np.tanh((data["NAME_GOODS_CATEGORY_Insurance"]))))) * 2.0))))) / 2.0)) +
                    0.098828*np.tanh(((data["AMT_GOODS_PRICE_x"]) * (((data["FLAG_DOCUMENT_14"]) - ((((0.0) < (data["NAME_GOODS_CATEGORY_Fitness"]))*1.)))))) +
                    0.094998*np.tanh(np.where(data["AMT_CREDIT_y"]>0, data["te_FLAG_DOCUMENT_14"], np.tanh(((-1.0*((np.where(3.0>0, ((((-1.0*((((data["AMT_CREDIT_x"]) / 2.0))))) + (0.0))/2.0), data["NAME_CASH_LOAN_PURPOSE_Business_development"] )))))) )) +
                    0.099492*np.tanh((-1.0*(((-1.0*((((((data["te_FLAG_DOCUMENT_5"]) + (data["AMT_GOODS_PRICE_x"]))) * (data["FLAG_DOCUMENT_14"]))))))))) +
                    0.099961*np.tanh(((np.minimum(((np.maximum(((data["FLAG_DOCUMENT_14"])), (((((data["AMT_GOODS_PRICE_x"]) < (data["te_FLAG_DOCUMENT_14"]))*1.)))))), ((data["cnt_FLAG_DOCUMENT_14"])))) * (((2.0) - (data["AMT_GOODS_PRICE_x"]))))) +
                    0.094216*np.tanh(np.minimum((((((np.tanh(((-1.0*((((data["nans"]) / 2.0))))))) + (data["te_FLAG_DOCUMENT_14"]))/2.0))), ((0.0)))) +
                    0.099570*np.tanh(np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])), (((((((((-2.0) - ((((data["FLAG_DOCUMENT_4"]) > (1.0))*1.)))) < ((((-1.0*((data["AMT_DOWN_PAYMENT"])))) * (2.0))))*1.)) + (data["AMT_CREDIT_y"])))))) +
                    0.080070*np.tanh((((((data["CNT_INSTALMENT"]) + (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) - (np.maximum(((data["AMT_CREDIT_x"])), ((((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))) + (1.0))/2.0))))))))) < (((((-2.0) / 2.0)) * 2.0)))*1.)) +
                    0.080070*np.tanh((((data["AMT_DOWN_PAYMENT"]) > (((((2.39543843269348145)) + (((np.tanh((data["AMT_GOODS_PRICE_x"]))) * ((-1.0*(((((((data["AMT_GOODS_PRICE_x"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) < (data["AMT_DOWN_PAYMENT"]))*1.))))))))/2.0)))*1.)) +
                    0.099883*np.tanh(((data["cnt_DAYS_BIRTH"]) * (((((data["CHANNEL_TYPE_Car_dealer"]) + (np.where(data["NAME_GOODS_CATEGORY_Fitness"]>0, data["AMT_GOODS_PRICE_x"], (((data["te_FLAG_DOCUMENT_20"]) + (data["NAME_GOODS_CATEGORY_Fitness"]))/2.0) )))) / 2.0)))) +
                    0.084134*np.tanh(np.minimum(((data["SK_DPD_DEF"])), ((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["te_FLAG_DOCUMENT_20"], ((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["SK_DPD_DEF"]))) - (data["CHANNEL_TYPE_Car_dealer"]))) - (data["CHANNEL_TYPE_Car_dealer"])) ))))) +
                    0.097499*np.tanh(np.minimum(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["AMT_GOODS_PRICE_x"])))), ((((0.0) - (((-2.0) + (np.maximum(((((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * 2.0)) * 2.0))), ((data["AMT_GOODS_PRICE_x"]))))))))))) +
                    0.079015*np.tanh(((data["te_FLAG_CONT_MOBILE"]) * (np.where(2.0>0, np.where(((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["AMT_CREDIT_y"]))>0, ((data["AMT_CREDIT_x"]) / 2.0), data["NAME_GOODS_CATEGORY_Fitness"] ), ((data["NAME_GOODS_CATEGORY_Fitness"]) - (data["te_FLAG_CONT_MOBILE"])) )))) +
                    0.099101*np.tanh((((((data["te_FLAG_DOCUMENT_19"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) < (np.minimum(((((((data["te_FLAG_DOCUMENT_20"]) - (data["te_FLAG_EMAIL"]))) * 2.0))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))))*1.)) +
                    0.099766*np.tanh(((data["FLAG_DOCUMENT_15"]) * (np.minimum(((3.0)), ((np.minimum(((data["FLAG_DOCUMENT_15"])), ((((data["te_FLAG_DOCUMENT_20"]) - (((np.minimum((((5.04014062881469727))), (((((0.19776110351085663)) / 2.0))))) * 2.0)))))))))))) +
                    0.099492*np.tanh(((np.minimum(((data["cnt_FLAG_DOCUMENT_4"])), ((((((1.0) - (data["te_FLAG_DOCUMENT_20"]))) - (np.tanh((np.tanh((data["NAME_GOODS_CATEGORY_Fitness"])))))))))) + (((data["FLAG_DOCUMENT_15"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))) +
                    0.098085*np.tanh(np.where(data["FLAG_DOCUMENT_15"]>0, data["te_FLAG_DOCUMENT_20"], (((((-1.0*((data["te_FLAG_DOCUMENT_20"])))) * ((((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) < (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)))) / 2.0) )) +
                    0.099297*np.tanh(((data["FLAG_DOCUMENT_15"]) * (np.where(np.tanh((((data["FLAG_DOCUMENT_15"]) / 2.0)))>0, (((np.where(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) / 2.0)>0, data["FLAG_DOCUMENT_15"], -2.0 )) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0), data["NAME_GOODS_CATEGORY_Fitness"] )))) +
                    0.099531*np.tanh(((((((np.where(((((12.33196353912353516)) > (data["NAME_GOODS_CATEGORY_Fitness"]))*1.)>0, data["cnt_FLAG_DOCUMENT_10"], (((8.0)) * 2.0) )) * 2.0)) + (((np.tanh((data["cnt_FLAG_DOCUMENT_15"]))) - (data["NAME_GOODS_CATEGORY_Fitness"]))))) / 2.0)) +
                    0.099883*np.tanh(((((np.minimum(((data["ty__Real_estate_loan"])), ((data["ty__Loan_for_working_capital_replenishment"])))) + (((data["AMT_CREDIT_y"]) * (data["ty__Loan_for_working_capital_replenishment"]))))) - (np.tanh((((((0.58618438243865967)) < (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))*1.)))))) +
                    0.099726*np.tanh((((np.minimum(((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"])), ((np.where(np.where(-3.0>0, data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"], ((data["cc_bal_AMT_RECIVABLE"]) - (1.0)) )>0, data["NAME_GOODS_CATEGORY_Insurance"], ((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) * 2.0) ))))) > (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))*1.)) +
                    0.072646*np.tanh((-1.0*((((np.where(data["AMT_CREDIT_y"]>0, data["AMT_GOODS_PRICE_x"], data["CNT_INSTALMENT_FUTURE"] )) * (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])))))) +
                    0.099453*np.tanh(((np.maximum(((np.where(data["ty__Loan_for_working_capital_replenishment"]>0, data["AMT_CREDIT_y"], data["ty__Loan_for_working_capital_replenishment"] ))), ((data["ty__Loan_for_working_capital_replenishment"])))) * (((np.tanh((data["AMT_CREDIT_y"]))) * 2.0)))) +
                    0.098828*np.tanh(np.minimum(((np.tanh((((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) * ((10.98657798767089844))))))), (((((-1.0*((((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))) < (np.minimum(((data["AMT_GOODS_PRICE_x"])), ((data["ty__Real_estate_loan"])))))*1.))))) * ((10.98657798767089844))))))) +
                    0.098281*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"]) - (np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal"])), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) - ((((((((data["ty__Interbank_credit"]) * (data["AMT_GOODS_PRICE_x"]))) * 2.0)) < (((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * 2.0)))*1.))))))))) +
                    0.087691*np.tanh(((((data["AMT_GOODS_PRICE_x"]) * (np.minimum(((((0.0) * 2.0))), (((((((data["NAME_GOODS_CATEGORY_House_Construction"]) > (0.0))*1.)) - (np.minimum(((data["nans"])), ((data["AMT_GOODS_PRICE_x"]))))))))))) * 2.0)) +
                    0.094138*np.tanh(np.where(data["te_HOUR_APPR_PROCESS_START_x"]>0, ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * 2.0), ((np.tanh(((((((data["AMT_CREDIT_x"]) > ((((((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * 2.0)) * 2.0)) < (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.)))*1.)) * 2.0)))) / 2.0) )) +
                    0.081321*np.tanh(np.minimum(((((data["HOUR_APPR_PROCESS_START_x"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"])))), ((np.maximum((((-1.0*((np.where(1.0>0, data["HOUR_APPR_PROCESS_START_y"], data["HOUR_APPR_PROCESS_START_x"] )))))), ((np.tanh((data["HOUR_APPR_PROCESS_START_x"]))))))))) +
                    0.088589*np.tanh((-1.0*((np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, data["EMERGENCYSTATE_MODE"], ((((((data["WALLSMATERIAL_MODE"]) * (data["HOUR_APPR_PROCESS_START_x"]))) * (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))) * ((-1.0*((((data["HOUR_APPR_PROCESS_START_x"]) * 2.0)))))) ))))) +
                    0.067097*np.tanh(np.minimum(((np.where(np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["NAME_GOODS_CATEGORY_Insurance"])))>0, data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"], data["cnt_FLAG_DOCUMENT_18"] ))), (((-1.0*(((((((data["te_HOUR_APPR_PROCESS_START_x"]) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))) < (-1.0))*1.)))))))) +
                    0.099961*np.tanh(((data["EXT_SOURCE_1"]) * (((((((1.0) * 2.0)) + (data["EXT_SOURCE_1"]))) - ((-1.0*((np.minimum(((((data["EXT_SOURCE_1"]) / 2.0))), ((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))))))))))) +
                    0.099062*np.tanh((((10.0)) * (((np.minimum(((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]) * (data["EXT_SOURCE_1"])))), (((-1.0*((((((data["EXT_SOURCE_1"]) + ((((0.0) > (data["EXT_SOURCE_1"]))*1.)))) / 2.0)))))))) / 2.0)))) +
                    0.099414*np.tanh(((np.maximum(((((data["te_DAYS_EMPLOYED"]) * ((0.0))))), ((np.tanh((np.where((8.0)>0, (((data["cnt_FLAG_DOCUMENT_3"]) > (data["te_ORGANIZATION_TYPE"]))*1.), 0.0 ))))))) / 2.0)) +
                    0.077765*np.tanh(np.tanh((np.where(np.tanh((data["te_NAME_CONTRACT_TYPE"]))>0, np.where((((np.minimum(((data["te_NAME_CONTRACT_TYPE"])), ((-3.0)))) < (data["te_FLAG_DOCUMENT_8"]))*1.)>0, data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"], data["te_DAYS_EMPLOYED"] ), data["cnt_NAME_INCOME_TYPE"] )))) +
                    0.074795*np.tanh(((((((data["te_ORGANIZATION_TYPE"]) * (np.tanh((data["AMT_CREDIT_x"]))))) / 2.0)) * (data["AMT_CREDIT_x"]))) +
                    0.079328*np.tanh(((data["te_OCCUPATION_TYPE"]) * (np.minimum(((((data["cnt_FLAG_DOCUMENT_6"]) / 2.0))), (((-1.0*((np.minimum(((3.0)), ((np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, data["te_OCCUPATION_TYPE"], ((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0) ))))))))))))) +
                    0.099922*np.tanh(((np.where(data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]>0, (-1.0*((((3.0) - (data["cnt_FLAG_DOCUMENT_3"]))))), data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"] )) * ((((np.where(data["cnt_FLAG_DOCUMENT_3"]>0, 3.0, data["AMT_CREDIT_x"] )) < (data["cnt_FLAG_DOCUMENT_6"]))*1.)))) +
                    0.099101*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["FLAG_DOCUMENT_6"], ((data["te_REGION_RATING_CLIENT"]) * ((-1.0*(((((np.minimum(((data["te_REGION_RATING_CLIENT"])), ((((data["te_FLAG_DOCUMENT_6"]) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"])))))) < (-2.0))*1.)))))) )) +
                    0.099140*np.tanh(((np.where(data["cnt_FLAG_DOCUMENT_6"]>0, data["te_FLAG_DOCUMENT_6"], (-1.0*(((-1.0*((3.0)))))) )) * (((((((data["NAME_GOODS_CATEGORY_Insurance"]) < (data["te_REGION_RATING_CLIENT_W_CITY"]))*1.)) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)))) +
                    0.099609*np.tanh(np.where(data["FLAG_DOCUMENT_6"]>0, data["CNT_FAM_MEMBERS"], ((((data["te_REGION_RATING_CLIENT_W_CITY"]) * (data["te_FLAG_DOCUMENT_6"]))) * (data["te_DAYS_EMPLOYED"])) )) +
                    0.099883*np.tanh(np.where(((data["te_NAME_HOUSING_TYPE"]) - (1.0))>0, data["SK_ID_BUREAU"], np.maximum(((np.where(data["te_NAME_HOUSING_TYPE"]>0, 1.0, 0.0 ))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) )) +
                    0.097304*np.tanh(np.minimum(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["te_NAME_CONTRACT_TYPE"]))) * 2.0))), ((((((((data["cnt_NAME_HOUSING_TYPE"]) > ((((data["NAME_CONTRACT_TYPE"]) + (((data["te_NAME_CONTRACT_TYPE"]) * (data["NAME_GOODS_CATEGORY_Insurance"]))))/2.0)))*1.)) + (data["NAME_CONTRACT_TYPE"]))/2.0))))) +
                    0.058734*np.tanh(((np.where(((data["ty__Interbank_credit"]) * (data["NAME_GOODS_CATEGORY_Insurance"]))>0, data["NAME_GOODS_CATEGORY_Insurance"], np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((-1.0*((data["cnt_REGION_RATING_CLIENT_W_CITY"])))))) )) * ((((((data["NAME_GOODS_CATEGORY_Insurance"]) * 2.0)) > (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.)))) +
                    0.098828*np.tanh(np.where(((data["cnt_REGION_RATING_CLIENT"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))>0, (-1.0*((((data["cnt_REGION_RATING_CLIENT"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))), (-1.0*((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"]) * 2.0)))) )) +
                    0.098945*np.tanh(np.where(data["ty__Interbank_credit"]>0, ((data["NAME_GOODS_CATEGORY_Insurance"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((data["ty__Loan_for_working_capital_replenishment"]) * 2.0)) - (data["NAME_GOODS_CATEGORY_Insurance"])) )) +
                    0.063892*np.tanh(((np.minimum(((((np.maximum(((((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]) - (1.0)))), ((np.maximum(((data["ty__Loan_for_working_capital_replenishment"])), (((-1.0*((-3.0)))))))))) * (data["ty__Loan_for_working_capital_replenishment"])))), ((data["ty__Loan_for_working_capital_replenishment"])))) - (data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"]))) +
                    0.099570*np.tanh((-1.0*(((((np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((np.where(data["ty__Loan_for_working_capital_replenishment"]>0, data["REGION_RATING_CLIENT_W_CITY"], 0.0 ))))) + ((((((data["DAYS_CREDIT"]) < (-3.0))*1.)) * 2.0)))/2.0))))) +
                    0.099179*np.tanh(np.minimum(((np.where(data["cc_bal_CNT_DRAWINGS_CURRENT"]>0, data["REGION_RATING_CLIENT_W_CITY"], ((data["REGION_RATING_CLIENT_W_CITY"]) * (np.tanh((data["cc_bal_CNT_DRAWINGS_CURRENT"])))) ))), ((np.maximum(((-3.0)), ((0.0))))))) +
                    0.071043*np.tanh(((np.maximum(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"])))), ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (np.minimum((((((-3.0) + (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))/2.0))), ((((-3.0) * 2.0)))))))))) * (data["te_REGION_RATING_CLIENT_W_CITY"]))) +
                    0.099726*np.tanh((((np.where(np.tanh((data["NAME_GOODS_CATEGORY_Direct_Sales"]))>0, data["cc_bal_CNT_DRAWINGS_CURRENT"], ((((-2.0) + (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))) / 2.0) )) > (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))*1.)) +
                    0.100000*np.tanh((((np.minimum(((data["ty__Loan_for_working_capital_replenishment"])), ((data["cc_bal_CNT_DRAWINGS_CURRENT"])))) > ((((np.tanh((np.tanh((data["cc_bal_CNT_DRAWINGS_CURRENT"]))))) < (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))*1.)))*1.)) +
                    0.099961*np.tanh((((((np.where(np.where((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)>0, data["AMT_CREDIT_SUM_DEBT"], -3.0 )>0, data["NAME_GOODS_CATEGORY_Insurance"], -3.0 )) > (data["ty__Loan_for_working_capital_replenishment"]))*1.)) * 2.0)) +
                    0.099961*np.tanh(np.tanh((np.where((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) + (np.tanh((-2.0))))/2.0)>0, data["cnt_REGION_RATING_CLIENT_W_CITY"], (((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) > (3.0))*1.) )))) +
                    0.099570*np.tanh(np.where(data["DAYS_LAST_DUE_1ST_VERSION"]>0, ((data["inst_NUM_INSTALMENT_NUMBER"]) * (np.minimum(((np.where(data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]>0, -1.0, data["NAME_CASH_LOAN_PURPOSE_Business_development"] ))), ((data["NAME_GOODS_CATEGORY_Insurance"]))))), data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"] )) +
                    0.099609*np.tanh((((((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) > (2.0))*1.)) + (data["cc_bal_AMT_DRAWINGS_POS_CURRENT"]))) * ((((0.0) < ((((-1.0*((((2.0) - (data["cc_bal_AMT_TOTAL_RECEIVABLE"])))))) * 2.0)))*1.)))) +
                    0.099805*np.tanh(((data["cc_bal_AMT_DRAWINGS_CURRENT"]) * (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) - ((-1.0*(((-1.0*(((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) > (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + ((((2.0)) * 2.0)))))*1.)))))))))))) +
                    0.099687*np.tanh(((((data["DAYS_LAST_DUE_1ST_VERSION"]) * (data["cc_bal_AMT_DRAWINGS_POS_CURRENT"]))) * ((((((8.0)) * (data["ty__Loan_for_working_capital_replenishment"]))) - (data["ty__Loan_for_working_capital_replenishment"]))))) +
                    0.099648*np.tanh((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) > (np.maximum(((np.where(3.0>0, ((data["cc_bal_AMT_PAYMENT_CURRENT"]) / 2.0), data["NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home___land"] ))), ((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) * 2.0))))))*1.)) +
                    0.099648*np.tanh((((((((data["cc_bal_AMT_PAYMENT_CURRENT"]) * (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))) < ((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) + (np.minimum(((3.0)), ((-3.0)))))/2.0)))*1.)) * 2.0)) +
                    0.094099*np.tanh(((((data["DAYS_LAST_DUE_1ST_VERSION"]) * ((2.0)))) * (np.where(data["ty__Loan_for_working_capital_replenishment"]>0, data["cc_bal_AMT_PAYMENT_CURRENT"], ((np.tanh((data["DAYS_LAST_DUE_1ST_VERSION"]))) * (data["NAME_GOODS_CATEGORY_Direct_Sales"])) )))) +
                    0.099609*np.tanh(np.where(((((((6.0)) * 2.0)) > (((data["ty__Loan_for_working_capital_replenishment"]) - ((-1.0*((data["FLAG_DOCUMENT_4"])))))))*1.)>0, np.maximum(((data["ty__Interbank_credit"])), ((data["ty__Loan_for_working_capital_replenishment"]))), -2.0 )) +
                    0.100000*np.tanh(np.where(data["ty__Loan_for_working_capital_replenishment"]>0, (((2.0) < (data["AMT_CREDIT_x"]))*1.), np.tanh((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, (((2.0) < (data["AMT_CREDIT_x"]))*1.), 0.0 ))) )) +
                    0.099844*np.tanh((-1.0*((np.where(((((((data["nans"]) + (1.0))/2.0)) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))/2.0)>0, data["NAME_GOODS_CATEGORY_Insurance"], data["AMT_GOODS_PRICE_x"] ))))) +
                    0.099766*np.tanh(((data["ELEVATORS_MODE"]) * (((np.where(((np.where((6.0)>0, data["cc_bal_CNT_INSTALMENT_MATURE_CUM"], np.maximum(((0.0)), (((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"])))))) )) / 2.0)>0, data["cc_bal_CNT_INSTALMENT_MATURE_CUM"], data["ty__Loan_for_working_capital_replenishment"] )) * 2.0)))) +
                    0.099766*np.tanh(((((2.09928321838378906)) < (np.maximum(((((data["cc_bal_SK_ID_PREV"]) * (-2.0)))), ((np.where(data["AMT_CREDIT_x"]>0, data["cc_bal_CNT_INSTALMENT_MATURE_CUM"], np.tanh((1.0)) ))))))*1.)) +
                    0.094685*np.tanh(((((data["AMT_CREDIT_x"]) * (((data["cc_bal_SK_ID_PREV"]) * (data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))))) / 2.0)) +
                    0.055021*np.tanh((-1.0*((np.maximum((((((np.tanh((((((((data["ty__Loan_for_working_capital_replenishment"]) * (data["nans"]))) / 2.0)) + (data["ty__Interbank_credit"]))))) < (data["ty__Loan_for_working_capital_replenishment"]))*1.))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))))) +
                    0.099414*np.tanh(np.where(((np.tanh((np.maximum(((((((data["ty__Interbank_credit"]) * (data["AMT_GOODS_PRICE_x"]))) - (data["te_HOUSETYPE_MODE"])))), ((data["cnt_WALLSMATERIAL_MODE"])))))) / 2.0)>0, ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["AMT_GOODS_PRICE_x"])), data["AMT_GOODS_PRICE_x"] )) +
                    0.081360*np.tanh(np.minimum(((0.0)), (((((((data["te_HOUSETYPE_MODE"]) < ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) < (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)))*1.)) - ((((data["cnt_WALLSMATERIAL_MODE"]) + (data["AMT_GOODS_PRICE_x"]))/2.0))))))) +
                    0.099726*np.tanh(((np.maximum(((((data["te_FLAG_DOCUMENT_7"]) + (-1.0)))), (((((-1.0*((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["te_FLAG_DOCUMENT_10"])))))) * (-1.0)))))) * 2.0)))

In [18]:
def GPIII(data):
    return Output( 0.100000*np.tanh((((6.72599077224731445)) - (np.where((((((-1.0*(((((((((-1.0*((data["EXT_SOURCE_3"])))) - (data["EXT_SOURCE_1"]))) / 2.0)) * 2.0))))) * 2.0)) * 2.0)>0, (11.37177181243896484), data["EXT_SOURCE_3"] )))) +
                    0.100000*np.tanh(((data["DAYS_BIRTH"]) + (((((((((data["NAME_CONTRACT_STATUS_Refused"]) + (((data["DAYS_CREDIT"]) + (((-3.0) * (data["EXT_SOURCE_1"]))))))) * 2.0)) + (((data["te_OCCUPATION_TYPE"]) * 2.0)))) * 2.0)))) +
                    0.100000*np.tanh(((data["te_OCCUPATION_TYPE"]) - (((((data["EXT_SOURCE_3"]) + (np.maximum(((np.maximum(((data["EXT_SOURCE_2"])), ((((((data["EXT_SOURCE_3"]) + (((data["EXT_SOURCE_2"]) * 2.0)))) * 2.0)))))), ((-1.0)))))) * 2.0)))) +
                    0.100000*np.tanh(((data["DAYS_CREDIT"]) - ((((5.0)) * (np.tanh((((((data["EXT_SOURCE_2"]) * 2.0)) - (((data["te_ORGANIZATION_TYPE"]) - (((data["EXT_SOURCE_3"]) * 2.0)))))))))))) +
                    0.100000*np.tanh(((((np.tanh((data["te_OCCUPATION_TYPE"]))) + ((((((data["REGION_RATING_CLIENT_W_CITY"]) + (((data["DAYS_BIRTH"]) + (data["REGION_RATING_CLIENT_W_CITY"]))))/2.0)) - (((data["EXT_SOURCE_3"]) * 2.0)))))) - (data["EXT_SOURCE_1"]))) +
                    0.100000*np.tanh(((((data["NAME_CONTRACT_STATUS_Refused"]) - (data["EXT_SOURCE_1"]))) - (np.where(((((data["te_NAME_EDUCATION_TYPE"]) - (data["EXT_SOURCE_1"]))) - (data["EXT_SOURCE_3"]))>0, ((data["EXT_SOURCE_3"]) * 2.0), 3.0 )))) +
                    0.100000*np.tanh(((((data["cc_bal_AMT_BALANCE"]) + (((data["cc_bal_AMT_RECIVABLE"]) + (((data["NAME_PRODUCT_TYPE_walk_in"]) - (((data["EXT_SOURCE_3"]) + (((data["EXT_SOURCE_2"]) * 2.0)))))))))) * 2.0)) +
                    0.100000*np.tanh(((-2.0) + (((((np.where(data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]>0, 3.0, (-1.0*((((data["EXT_SOURCE_2"]) + (data["EXT_SOURCE_1"]))))) )) - (data["EXT_SOURCE_3"]))) * 2.0)))) +
                    0.100000*np.tanh(((((((((data["te_NAME_EDUCATION_TYPE"]) - (data["EXT_SOURCE_2"]))) - (((data["EXT_SOURCE_3"]) / 2.0)))) + (np.tanh((np.where(data["cnt_CODE_GENDER"]>0, -2.0, (3.21376276016235352) )))))) * 2.0)) +
                    0.100000*np.tanh(np.where(data["EXT_SOURCE_1"]>0, -2.0, ((data["te_ORGANIZATION_TYPE"]) - (((((np.minimum(((data["FLAG_DOCUMENT_3"])), ((np.minimum(((data["te_ORGANIZATION_TYPE"])), ((2.0))))))) - (data["EXT_SOURCE_2"]))) * (-3.0)))) )) +
                    0.100000*np.tanh(((((data["NAME_PRODUCT_TYPE_walk_in"]) - (((data["DAYS_FIRST_DRAWING"]) - (((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) * 2.0)))))) + (((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) + ((-1.0*((data["EXT_SOURCE_3"])))))) - (data["CODE_REJECT_REASON_XAP"]))))) +
                    0.100000*np.tanh(((((((data["te_REGION_RATING_CLIENT_W_CITY"]) + (data["cnt_FLAG_OWN_CAR"]))) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))) + (np.maximum(((((((((data["ty__Microloan"]) * 2.0)) + (data["REGION_RATING_CLIENT_W_CITY"]))) * 2.0))), ((data["CNT_INSTALMENT_FUTURE"])))))) +
                    0.100000*np.tanh(((((data["DAYS_LAST_PHONE_CHANGE"]) + ((((-1.0*(((0.0))))) + (((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) - (((data["cnt_CODE_GENDER"]) + (data["EXT_SOURCE_3"]))))))))) + (((data["NAME_YIELD_GROUP_high"]) - (data["EXT_SOURCE_2"]))))) +
                    0.100000*np.tanh(((((((data["DEF_30_CNT_SOCIAL_CIRCLE"]) - ((((-1.0*(((((14.40037250518798828)) * (data["ty__Microloan"])))))) + (data["NAME_CONTRACT_TYPE"]))))) - (data["EXT_SOURCE_2"]))) - (data["SK_ID_PREV_y"]))) +
                    0.100000*np.tanh(((((data["CNT_INSTALMENT_FUTURE"]) - (((((data["cnt_REG_CITY_NOT_LIVE_CITY"]) - (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))) - (((data["CNT_PAYMENT"]) - (data["EXT_SOURCE_2"]))))))) - (data["ca__Closed"]))) +
                    0.100000*np.tanh(((((((((data["te_NAME_CONTRACT_TYPE"]) - (data["AMT_ANNUITY"]))) - (1.0))) + ((-1.0*(((-1.0*((((0.0) - (((data["SK_ID_PREV_y"]) * 2.0)))))))))))) + (data["te_ORGANIZATION_TYPE"]))) +
                    0.100000*np.tanh(((data["NAME_CONTRACT_STATUS_Refused"]) + (((((np.minimum(((data["te_NAME_EDUCATION_TYPE"])), ((((np.maximum(((data["NAME_YIELD_GROUP_high"])), ((data["ty__Microloan"])))) + (data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])))))) * 2.0)) - (((data["PRODUCT_COMBINATION_POS_industry_with_interest"]) * 2.0)))))) +
                    0.100000*np.tanh(((((data["CNT_PAYMENT"]) + (((data["cc_bal_CNT_DRAWINGS_CURRENT"]) + (((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) - (data["RATE_DOWN_PAYMENT"]))) - ((-1.0*((data["ty__Microloan"])))))))))) + (data["avg_buro_buro_bal_status_1"]))) +
                    0.100000*np.tanh(((data["DAYS_ID_PUBLISH"]) - (((((data["cnt_NAME_FAMILY_STATUS"]) + (((data["DAYS_FIRST_DRAWING"]) + (np.tanh((data["inst_NUM_INSTALMENT_VERSION"]))))))) + (data["AMT_ANNUITY"]))))) +
                    0.100000*np.tanh(((np.minimum(((np.where(data["NAME_YIELD_GROUP_low_action"]>0, -3.0, np.maximum(((data["NAME_CLIENT_TYPE_New"])), ((data["DEF_60_CNT_SOCIAL_CIRCLE"]))) ))), (((7.0))))) + (data["te_CODE_GENDER"]))) +
                    0.100000*np.tanh(((data["AMT_ANNUITY_x"]) + (((((np.maximum(((data["AMT_ANNUITY_x"])), ((((data["CNT_INSTALMENT_FUTURE"]) + (data["AMT_ANNUITY_x"])))))) * 2.0)) + ((((data["te_FLAG_DOCUMENT_3"]) + (((data["cnt_FLAG_OWN_CAR"]) * 2.0)))/2.0)))))) +
                    0.100000*np.tanh(((((((((((data["NAME_YIELD_GROUP_high"]) + ((((8.0)) * (((data["ty__Microloan"]) * 2.0)))))/2.0)) - (((data["AMT_ANNUITY"]) - (data["SK_DPD"]))))) + (data["avg_buro_buro_bal_status_1"]))) + (data["te_WALLSMATERIAL_MODE"]))/2.0)) +
                    0.100000*np.tanh((((((((data["CNT_PAYMENT"]) + ((-1.0*((data["ty__Mortgage"])))))) + (data["cnt_DAYS_BIRTH"]))/2.0)) + ((((((((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) + (data["NAME_PAYMENT_TYPE_XNA"]))) + (data["CNT_PAYMENT"]))/2.0)) + (data["CODE_REJECT_REASON_SCOFR"]))))) +
                    0.100000*np.tanh((-1.0*((np.where((-1.0*(((-1.0*((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]))))))>0, data["PRODUCT_COMBINATION_Cash_X_Sell__low"], (-1.0*((((((data["inst_AMT_PAYMENT"]) * (((data["SK_ID_PREV_y"]) * 2.0)))) * 2.0)))) ))))) +
                    0.100000*np.tanh(((data["CODE_REJECT_REASON_HC"]) + (((((np.where(data["OWN_CAR_AGE"]>0, data["NAME_YIELD_GROUP_low_action"], data["cnt_FLAG_OWN_CAR"] )) - (data["PRODUCT_COMBINATION_Cash_Street__low"]))) + (((data["OWN_CAR_AGE"]) - (data["NAME_YIELD_GROUP_low_action"]))))))) +
                    0.099961*np.tanh(np.where(data["ty__Mortgage"]>0, -2.0, ((((data["SK_DPD"]) - (((data["CODE_GENDER"]) - (data["CNT_PAYMENT"]))))) + ((-1.0*((data["inst_NUM_INSTALMENT_VERSION"]))))) )) +
                    0.100000*np.tanh(((data["DEF_30_CNT_SOCIAL_CIRCLE"]) - (((data["CHANNEL_TYPE_Channel_of_corporate_sales"]) - (((((((((data["FLAG_DOCUMENT_18"]) * (data["cnt_FLAG_DOCUMENT_18"]))) + (data["DEF_60_CNT_SOCIAL_CIRCLE"]))/2.0)) < (((data["SK_DPD"]) * 2.0)))*1.)))))) +
                    0.100000*np.tanh((((np.minimum(((((((data["cnt_FLAG_DOCUMENT_18"]) - ((((((-1.0) / 2.0)) < (data["inst_SK_ID_PREV"]))*1.)))) / 2.0))), ((((data["cnt_FLAG_DOCUMENT_16"]) / 2.0))))) + ((-1.0*((data["inst_SK_ID_PREV"])))))/2.0)) +
                    0.099961*np.tanh(((((data["ty__Microloan"]) + (np.minimum((((((((((data["cnt_FLAG_DOCUMENT_18"]) - (data["RATE_DOWN_PAYMENT"]))) - (data["CHANNEL_TYPE_Channel_of_corporate_sales"]))) + (data["cnt_WEEKDAY_APPR_PROCESS_START"]))/2.0))), ((data["cnt_FLAG_DOCUMENT_18"])))))) * 2.0)) +
                    0.100000*np.tanh((((data["SK_DPD"]) > (((np.minimum(((((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0))), (((-1.0*((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))))))) / 2.0)))*1.)) +
                    0.100000*np.tanh(np.minimum(((((data["AMT_ANNUITY_x"]) - (np.where((-1.0*((data["CHANNEL_TYPE_Channel_of_corporate_sales"])))>0, data["FLAG_DOCUMENT_16"], 1.0 ))))), ((((((2.0) - (data["cnt_FLAG_DOCUMENT_16"]))) - (data["AMT_ANNUITY_x"])))))) +
                    0.098789*np.tanh((((((data["te_NAME_FAMILY_STATUS"]) + (((np.maximum(((data["CNT_INSTALMENT_FUTURE"])), ((((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0))))) + (((data["NAME_SELLER_INDUSTRY_Connectivity"]) - (((data["cnt_NAME_FAMILY_STATUS"]) - (data["CNT_INSTALMENT_FUTURE"]))))))))) + (data["cnt_ORGANIZATION_TYPE"]))/2.0)) +
                    0.100000*np.tanh((-1.0*((((((-1.0*((((data["SK_DPD"]) + ((-1.0*((((data["Active"]) + (data["AMT_CREDIT_SUM_LIMIT"]))))))))))) + ((((data["MONTHS_BALANCE"]) + (np.maximum(((data["CHANNEL_TYPE_Channel_of_corporate_sales"])), ((data["PRODUCT_COMBINATION_Cash_X_Sell__low"])))))/2.0)))/2.0))))) +
                    0.099961*np.tanh((((((np.where(-1.0>0, 3.0, data["PRODUCT_COMBINATION_Cash_Street__low"] )) < (data["ty__Car_loan"]))*1.)) + ((((-1.0*((np.maximum(((data["CODE_REJECT_REASON_SCO"])), ((((data["ty__Car_loan"]) - (data["cnt_FLAG_DOCUMENT_13"]))))))))) / 2.0)))) +
                    0.099883*np.tanh((((((data["DAYS_REGISTRATION"]) + (data["ca__Sold"]))) + (np.minimum((((4.0))), (((-1.0*((((data["FLOORSMAX_MEDI"]) + ((((data["DAYS_REGISTRATION"]) > (np.maximum(((data["FLOORSMAX_MEDI"])), ((-2.0)))))*1.)))))))))))/2.0)) +
                    0.100000*np.tanh(((np.minimum(((data["avg_buro_buro_bal_status_1"])), ((data["cnt_FLAG_DOCUMENT_18"])))) - ((-1.0*((np.maximum((((((-1.0*(((((-1.0) > (((data["inst_AMT_PAYMENT"]) * 2.0)))*1.))))) * (-3.0)))), ((data["cc_bal_CNT_DRAWINGS_CURRENT"]))))))))) +
                    0.100000*np.tanh((((np.where(data["NFLAG_INSURED_ON_APPROVAL"]>0, data["CNT_PAYMENT"], (((-1.0) + (np.where((9.0)>0, data["cnt_FLAG_DOCUMENT_2"], data["CNT_PAYMENT"] )))/2.0) )) + ((-1.0*((np.maximum(((data["ty__Mortgage"])), ((data["PRODUCT_COMBINATION_Cash_Street__low"]))))))))/2.0)) +
                    0.100000*np.tanh(((((((((data["cnt_FLAG_PHONE"]) + (data["cnt_FLAG_DOCUMENT_13"]))/2.0)) + (((data["cnt_CNT_CHILDREN"]) * ((((-1.0) + ((((-3.0) > (((-3.0) / 2.0)))*1.)))/2.0)))))/2.0)) - (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) +
                    0.099922*np.tanh(((((np.where((((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]) > (np.maximum(((data["cnt_FLAG_DOCUMENT_2"])), ((data["cc_bal_AMT_TOTAL_RECEIVABLE"])))))*1.)>0, 0.0, data["cc_bal_AMT_RECIVABLE"] )) / 2.0)) + (((np.minimum(((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])), ((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"])))) / 2.0)))) +
                    0.093748*np.tanh(((np.tanh((np.maximum((((((np.minimum(((data["CHANNEL_TYPE_AP___Cash_loan_"])), ((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])))) + (data["FLAG_WORK_PHONE"]))/2.0))), ((np.tanh((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["CHANNEL_TYPE_AP___Cash_loan_"])))))))))) - (((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) / 2.0)))) +
                    0.100000*np.tanh(np.tanh(((-1.0*((np.where((((np.maximum(((data["FLAG_DOCUMENT_2"])), ((data["cnt_FLAG_EMP_PHONE"])))) < ((-1.0*((data["DAYS_EMPLOYED"])))))*1.)>0, (6.0), -1.0 ))))))) +
                    0.099922*np.tanh(np.tanh((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) - (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))) - ((((((1.0) + (data["te_LIVE_REGION_NOT_WORK_REGION"]))/2.0)) + (((data["DAYS_ENDDATE_FACT"]) / 2.0)))))))) +
                    0.099844*np.tanh(np.where(((data["cc_bal_AMT_BALANCE"]) / 2.0)>0, (((data["FLAG_DOCUMENT_2"]) < (((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) - ((((data["CNT_INSTALMENT_FUTURE"]) > (-1.0))*1.)))))*1.), data["CNT_INSTALMENT_FUTURE"] )) +
                    0.100000*np.tanh(((np.minimum(((np.minimum((((((data["AMT_ANNUITY_x"]) > (data["AMT_REQ_CREDIT_BUREAU_QRT"]))*1.))), (((-1.0*((data["AMT_REQ_CREDIT_BUREAU_QRT"])))))))), ((data["AMT_ANNUITY_x"])))) - (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, ((data["AMT_ANNUITY_x"]) * 2.0), data["AMT_REQ_CREDIT_BUREAU_QRT"] )))) +
                    0.099961*np.tanh(np.where(((((-1.0*((data["AMT_GOODS_PRICE_x"])))) < (((data["AMT_GOODS_PRICE_x"]) * (data["AMT_GOODS_PRICE_x"]))))*1.)>0, -1.0, (((data["AMT_GOODS_PRICE_x"]) + ((1.39743840694427490)))/2.0) )) +
                    0.088081*np.tanh((-1.0*((((data["te_REG_REGION_NOT_LIVE_REGION"]) + ((((data["AMT_CREDIT_SUM_LIMIT"]) + (np.minimum(((data["te_FLAG_OWN_REALTY"])), (((((((((data["NAME_GOODS_CATEGORY_Insurance"]) - (data["RATE_DOWN_PAYMENT"]))) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)) + (data["RATE_DOWN_PAYMENT"])))))))/2.0))))))) +
                    0.100000*np.tanh((((((((((data["inst_DAYS_ENTRY_PAYMENT"]) + (data["cnt_FLAG_DOCUMENT_2"]))/2.0)) - (data["AMT_CREDIT_x"]))) * (data["MONTHS_BALANCE"]))) + (np.minimum(((0.0)), ((((data["AMT_CREDIT_x"]) * 2.0))))))) +
                    0.100000*np.tanh(((((((data["avg_buro_buro_bal_status_1"]) + (data["CNT_INSTALMENT_FUTURE"]))/2.0)) + ((-1.0*(((-1.0*((((data["CNT_INSTALMENT_FUTURE"]) * (((data["SK_ID_PREV_x"]) * 2.0)))))))))))/2.0)) +
                    0.100000*np.tanh(((((np.where(data["SK_DPD"]>0, data["SK_DPD"], (((((data["SK_DPD"]) - (data["NAME_GOODS_CATEGORY_Insurance"]))) > (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0)))*1.) )) * 2.0)) * 2.0)) +
                    0.099922*np.tanh(((((((data["cc_bal_SK_DPD_DEF"]) * ((14.15333747863769531)))) + ((((((data["cnt_NAME_INCOME_TYPE"]) + (np.minimum(((data["cc_bal_SK_DPD_DEF"])), (((-1.0*((data["inst_NUM_INSTALMENT_VERSION"]))))))))/2.0)) / 2.0)))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) +
                    0.099883*np.tanh((((((data["SK_ID_PREV_x"]) > (np.tanh((np.where(data["SK_ID_PREV_x"]>0, data["SK_ID_PREV_x"], data["NAME_SELLER_INDUSTRY_XNA"] )))))*1.)) * (((data["NAME_SELLER_INDUSTRY_XNA"]) / 2.0)))) +
                    0.099883*np.tanh(((((((((data["NAME_YIELD_GROUP_middle"]) > (2.0))*1.)) * 2.0)) + ((-1.0*((((((((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) + (data["avg_buro_buro_bal_status_0"]))/2.0)) + (data["ty__Mortgage"]))/2.0))))))/2.0)) +
                    0.099922*np.tanh((((data["DAYS_EMPLOYED"]) + (np.tanh(((((((((np.where(data["DAYS_EMPLOYED"]>0, data["FLAG_DOCUMENT_2"], data["cnt_DAYS_EMPLOYED"] )) < (data["DAYS_EMPLOYED"]))*1.)) * 2.0)) * (data["FLAG_EMP_PHONE"]))))))/2.0)) +
                    0.099883*np.tanh((((data["cnt_FLAG_DOCUMENT_2"]) + (np.minimum((((-1.0*((np.where((((data["cnt_DAYS_BIRTH"]) < (-1.0))*1.)>0, (3.64491915702819824), data["FLAG_DOCUMENT_16"] )))))), (((((-1.0*((data["CHANNEL_TYPE_Channel_of_corporate_sales"])))) * (data["CHANNEL_TYPE_Channel_of_corporate_sales"])))))))/2.0)) +
                    0.099922*np.tanh(np.where(data["ca__Active"]>0, np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), (((-1.0*((data["te_OCCUPATION_TYPE"])))))), ((np.minimum(((data["cnt_FLAG_DOCUMENT_13"])), ((data["cnt_HOUR_APPR_PROCESS_START_x"])))) * 2.0) )) +
                    0.096483*np.tanh((-1.0*((np.maximum((((((data["DAYS_ID_PUBLISH"]) < (-1.0))*1.))), ((data["YEARS_BUILD_MODE"]))))))) +
                    0.100000*np.tanh((-1.0*(((((((data["EXT_SOURCE_1"]) / 2.0)) > (np.tanh((np.tanh((data["EXT_SOURCE_1"]))))))*1.))))) +
                    0.099805*np.tanh((-1.0*(((((data["cnt_NAME_HOUSING_TYPE"]) + (np.minimum(((data["WEEKDAY_APPR_PROCESS_START_SATURDAY"])), ((np.where(data["cnt_NAME_HOUSING_TYPE"]>0, data["EXT_SOURCE_1"], ((2.0) * 2.0) ))))))/2.0))))) +
                    0.076749*np.tanh(((((((np.tanh((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]))/2.0)) < (data["SK_ID_PREV_x"]))*1.)))) - (((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]) / 2.0)))) + (data["ty__Microloan"]))) / 2.0)) +
                    0.100000*np.tanh((((np.tanh((data["inst_AMT_PAYMENT"]))) < (((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["cc_bal_SK_DPD_DEF"]))) + ((((data["FLAG_DOCUMENT_2"]) + (-1.0))/2.0)))))*1.)) +
                    0.100000*np.tanh((((np.tanh((((1.0) - (np.maximum(((data["REGION_POPULATION_RELATIVE"])), (((((8.08363628387451172)) * ((-1.0*((data["REGION_POPULATION_RELATIVE"]))))))))))))) + ((-1.0*((data["FLAG_DOCUMENT_14"])))))/2.0)) +
                    0.099883*np.tanh((-1.0*(((((data["avg_buro_buro_bal_status_0"]) + ((((((data["cnt_FLAG_DOCUMENT_13"]) < ((-1.0*((data["avg_buro_buro_bal_status_0"])))))*1.)) + (data["ty__Car_loan"]))))/2.0))))) +
                    0.099805*np.tanh((-1.0*(((((data["cc_bal_cc_bal_status__Sent_proposal"]) + ((((-1.0) + ((((np.minimum((((-1.0*((data["NAME_TYPE_SUITE_Unaccompanied"]))))), ((data["cc_bal_SK_DPD_DEF"])))) < (data["FLAG_DOCUMENT_2"]))*1.)))/2.0)))/2.0))))) +
                    0.099687*np.tanh(np.maximum(((data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"])), (((((((((data["DAYS_FIRST_DUE"]) / 2.0)) - (np.minimum(((data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"])), ((data["te_NAME_EDUCATION_TYPE"])))))) + (((data["HOUSETYPE_MODE"]) - ((-1.0*((-1.0)))))))/2.0))))) +
                    0.099961*np.tanh(np.minimum(((((data["cc_bal_CNT_DRAWINGS_CURRENT"]) - (data["FLAG_DOCUMENT_11"])))), (((((-1.0*((((((((np.maximum(((data["cnt_FLAG_DOCUMENT_2"])), ((data["cnt_FLAG_DOCUMENT_2"])))) < (data["cc_bal_SK_DPD_DEF"]))*1.)) < (data["cc_bal_CNT_DRAWINGS_CURRENT"]))*1.))))) + (data["cc_bal_CNT_DRAWINGS_CURRENT"])))))) +
                    0.099922*np.tanh((-1.0*((np.maximum((((((np.maximum(((np.tanh(((((-3.0) + ((-1.0*((0.0)))))/2.0))))), ((data["inst_NUM_INSTALMENT_VERSION"])))) + (((-3.0) / 2.0)))/2.0))), ((data["cc_bal_AMT_DRAWINGS_POS_CURRENT"]))))))) +
                    0.100000*np.tanh(np.minimum(((np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((data["NAME_GOODS_CATEGORY_Furniture"]))))), ((((((((data["SK_ID_PREV_x"]) / 2.0)) * 2.0)) - (((-3.0) + (data["NAME_GOODS_CATEGORY_Furniture"])))))))) +
                    0.086010*np.tanh(((np.minimum((((-1.0*(((((((data["AMT_REQ_CREDIT_BUREAU_MON"]) * ((((data["NAME_GOODS_CATEGORY_Gardening"]) + (data["cnt_FLAG_DOCUMENT_15"]))/2.0)))) < (data["NAME_GOODS_CATEGORY_Gardening"]))*1.)))))), (((((2.42437767982482910)) - (data["AMT_REQ_CREDIT_BUREAU_MON"])))))) * 2.0)) +
                    0.099726*np.tanh((((((((((7.96912765502929688)) < (data["ca__Sold"]))*1.)) + (np.tanh((((((((((data["cnt_FLAG_DOCUMENT_15"]) + (np.tanh((data["REG_CITY_NOT_LIVE_CITY"]))))/2.0)) + (np.tanh((data["REG_CITY_NOT_LIVE_CITY"]))))/2.0)) / 2.0)))))/2.0)) * 2.0)) +
                    0.097929*np.tanh(np.where((((np.maximum(((data["CNT_INSTALMENT_FUTURE"])), ((np.minimum(((2.0)), ((0.0))))))) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)>0, 1.0, (((np.maximum(((2.0)), ((data["NAME_GOODS_CATEGORY_Insurance"])))) < (data["CNT_INSTALMENT_FUTURE"]))*1.) )) +
                    0.099883*np.tanh(np.minimum(((((((data["PRODUCT_COMBINATION_Cash_Street__low"]) * (data["inst_AMT_PAYMENT"]))) / 2.0))), ((np.where(((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)>0, (-1.0*((data["cnt_FLAG_DOCUMENT_2"]))), data["cnt_FLAG_DOCUMENT_18"] ))))) +
                    0.068308*np.tanh(((((((np.maximum(((((((((-2.0) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)) + (data["SK_ID_PREV_x"]))/2.0))), ((data["HOUR_APPR_PROCESS_START_y"])))) * (data["SK_ID_PREV_x"]))) / 2.0)) + (data["NONLIVINGAPARTMENTS_MODE"]))) +
                    0.090113*np.tanh(np.where(((((((((data["EXT_SOURCE_2"]) - (-2.0))) < (((((data["FLAG_DOCUMENT_2"]) / 2.0)) * 2.0)))*1.)) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)>0, 3.0, ((data["EXT_SOURCE_2"]) * 2.0) )) +
                    0.099961*np.tanh((((((((data["NAME_GOODS_CATEGORY_House_Construction"]) + (data["NAME_CONTRACT_STATUS_Approved"]))/2.0)) / 2.0)) - ((((-2.0) > (((-1.0) - (np.maximum(((data["EXT_SOURCE_2"])), ((data["AMT_REQ_CREDIT_BUREAU_YEAR"])))))))*1.)))) +
                    0.099844*np.tanh(((((((((data["EXT_SOURCE_2"]) < (((-1.0) * 2.0)))*1.)) * 2.0)) + ((((-1.0*(((((data["EXT_SOURCE_2"]) > (((((12.74159717559814453)) > (data["EXT_SOURCE_2"]))*1.)))*1.))))) * 2.0)))/2.0)) +
                    0.086127*np.tanh((((((data["SK_ID_PREV_x"]) / 2.0)) + ((((np.minimum(((np.minimum(((((data["FLAG_DOCUMENT_2"]) - (data["nans"])))), ((data["cnt_FLAG_DOCUMENT_13"]))))), ((data["NAME_GOODS_CATEGORY_House_Construction"])))) + (np.tanh((data["PRODUCT_COMBINATION_Cash"]))))/2.0)))/2.0)) +
                    0.098320*np.tanh((((((((np.minimum(((3.0)), ((data["cc_bal_SK_DPD_DEF"])))) + ((((data["cnt_FLAG_WORK_PHONE"]) < (np.maximum(((data["PRODUCT_COMBINATION_Cash_X_Sell__low"])), ((((data["cnt_FLAG_DOCUMENT_2"]) / 2.0))))))*1.)))/2.0)) - (((data["PRODUCT_COMBINATION_Cash_X_Sell__low"]) / 2.0)))) / 2.0)) +
                    0.099922*np.tanh(((((((((np.tanh((-1.0))) + (data["AMT_CREDIT_y"]))/2.0)) > (((((((data["AMT_CREDIT_y"]) + (((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * 2.0)))/2.0)) > (data["FLAG_DOCUMENT_2"]))*1.)))*1.)) - (((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) / 2.0)))) +
                    0.094685*np.tanh(((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) * (data["cc_bal_CNT_DRAWINGS_CURRENT"]))) - ((((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) < (np.maximum(((((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]) - (((data["cc_bal_AMT_TOTAL_RECEIVABLE"]) - (data["cc_bal_AMT_TOTAL_RECEIVABLE"])))))), ((data["PRODUCT_COMBINATION_Cash_Street__low"])))))*1.)))) +
                    0.079015*np.tanh(((data["cnt_FLAG_DOCUMENT_14"]) + (((np.where(((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * (data["DAYS_FIRST_DRAWING"]))>0, (8.0), ((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * (data["DAYS_FIRST_DRAWING"])) )) / 2.0)))) +
                    0.071004*np.tanh(((((-1.0*((np.minimum(((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"])), ((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))))))) + ((-1.0*((np.where(data["NAME_PORTFOLIO_Cards"]>0, data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"], np.tanh(((((2.0) > (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))*1.))) ))))))/2.0)) +
                    0.099961*np.tanh(np.tanh((((((data["cnt_FLAG_DOCUMENT_15"]) - (np.minimum(((data["EXT_SOURCE_1"])), ((((data["ENTRANCES_MEDI"]) / 2.0))))))) - ((((0.0) > (data["EXT_SOURCE_1"]))*1.)))))) +
                    0.097343*np.tanh(np.maximum(((((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0))), (((((data["NAME_GOODS_CATEGORY_Insurance"]) < (((data["NAME_GOODS_CATEGORY_Construction_Materials"]) - ((-1.0*(((((((((data["AMT_CREDIT_y"]) * 2.0)) * 2.0)) + (-3.0))/2.0))))))))*1.))))) +
                    0.100000*np.tanh((-1.0*((((((data["inst_AMT_PAYMENT"]) + (((((data["CNT_INSTALMENT"]) * 2.0)) * (data["Active"]))))) * (np.maximum(((data["Active"])), ((((data["cnt_FLAG_DOCUMENT_2"]) + (data["FLAG_DOCUMENT_2"]))))))))))) +
                    0.083353*np.tanh(np.minimum((((((1.0) + (data["CODE_GENDER"]))/2.0))), (((((np.where(data["cnt_FLAG_DOCUMENT_2"]>0, (((data["cnt_FLAG_DOCUMENT_2"]) < (data["RATE_DOWN_PAYMENT"]))*1.), data["CNT_PAYMENT"] )) < (data["CNT_INSTALMENT_FUTURE"]))*1.))))) +
                    0.100000*np.tanh(((np.minimum(((1.0)), ((np.where(data["inst_AMT_PAYMENT"]>0, ((data["Completed"]) * 2.0), ((data["PRODUCT_COMBINATION_POS_household_without_interest"]) - (data["FLAG_DOCUMENT_2"])) ))))) / 2.0)) +
                    0.099844*np.tanh((((-1.0*((np.minimum(((((data["inst_AMT_PAYMENT"]) * (np.maximum((((((data["inst_AMT_PAYMENT"]) > (2.0))*1.))), ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) / 2.0)))))))), (((((data["inst_AMT_PAYMENT"]) + (data["avg_buro_MONTHS_BALANCE"]))/2.0)))))))) / 2.0)) +
                    0.099844*np.tanh(((((((data["cu__currency_3"]) + (np.maximum(((np.tanh((data["FLAG_DOCUMENT_2"])))), (((((3.0) < (data["CNT_INSTALMENT_FUTURE"]))*1.))))))) * 2.0)) + ((-1.0*(((((data["cc_bal_SK_DPD_DEF"]) < (data["cu__currency_3"]))*1.))))))) +
                    0.084603*np.tanh((((-1.0*(((((data["CHANNEL_TYPE_Channel_of_corporate_sales"]) + (((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) + ((((data["Active"]) + (((data["WEEKDAY_APPR_PROCESS_START_MONDAY"]) * (-2.0))))/2.0)))))/2.0))))) / 2.0)) +
                    0.099961*np.tanh((((((data["NAME_CASH_LOAN_PURPOSE_Car_repairs"]) + ((((data["NAME_CASH_LOAN_PURPOSE_Car_repairs"]) + ((((data["DAYS_LAST_DUE"]) + (data["NAME_GOODS_CATEGORY_Insurance"]))/2.0)))/2.0)))/2.0)) + (np.where(data["cc_bal_SK_DPD_DEF"]>0, data["DAYS_LAST_DUE"], data["NAME_CASH_LOAN_PURPOSE_Car_repairs"] )))) +
                    0.099766*np.tanh(np.minimum(((data["cnt_FLAG_DOCUMENT_13"])), ((np.where((-1.0*((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) - (data["cc_bal_cc_bal_status__Sent_proposal"])))))>0, ((np.maximum(((2.0)), (((11.47983264923095703))))) - (data["cc_bal_cc_bal_status__Sent_proposal"])), ((data["FLAG_DOCUMENT_13"]) * 2.0) ))))) +
                    0.099922*np.tanh((((-1.0*(((((((1.0) + (data["inst_AMT_PAYMENT"]))/2.0)) / 2.0))))) * (np.where(data["cc_bal_SK_DPD_DEF"]>0, ((data["NAME_CLIENT_TYPE_Refreshed"]) - (1.0)), data["MONTHS_BALANCE"] )))) +
                    0.085737*np.tanh(np.where(0.0>0, 1.0, (((((data["cc_bal_SK_DPD_DEF"]) + (((data["NAME_GOODS_CATEGORY_Insurance"]) + ((-1.0*((np.tanh((data["NAME_CLIENT_TYPE_Refreshed"])))))))))) + (((((data["AMT_CREDIT_y"]) / 2.0)) / 2.0)))/2.0) )) +
                    0.099805*np.tanh(np.minimum(((((((data["cnt_FLAG_EMAIL"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) + (3.0)))), ((np.maximum(((data["NAME_SELLER_INDUSTRY_Auto_technology"])), (((((((data["DAYS_FIRST_DUE"]) - (2.0))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)))))))) +
                    0.100000*np.tanh(((((((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["FLAG_DOCUMENT_18"], data["cnt_FLAG_DOCUMENT_18"] )) * 2.0)) / 2.0)) + (np.tanh(((((((10.0)) * (data["cc_bal_SK_DPD_DEF"]))) * 2.0)))))) +
                    0.099922*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, (((data["FLAG_DOCUMENT_18"]) > ((1.66978633403778076)))*1.), ((((data["FLAG_DOCUMENT_2"]) - (np.maximum(((((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * 2.0)) - (2.0)))), ((data["FLAG_DOCUMENT_18"])))))) / 2.0) )) +
                    0.099687*np.tanh(((((((data["NAME_PORTFOLIO_Cards"]) > (((data["cc_bal_SK_ID_PREV"]) * (((data["cc_bal_SK_ID_PREV"]) * 2.0)))))*1.)) + (data["cc_bal_SK_ID_PREV"]))/2.0)) +
                    0.099648*np.tanh(((((np.minimum(((data["cnt_FLAG_DOCUMENT_18"])), ((((((data["cnt_FLAG_DOCUMENT_18"]) * (data["FLAG_DOCUMENT_18"]))) - ((((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * 2.0)) + (data["NAME_TYPE_SUITE_Unaccompanied"]))/2.0))))))) / 2.0)) / 2.0)) +
                    0.099805*np.tanh((-1.0*((np.maximum(((((((data["ty__Mortgage"]) * (((data["cnt_FLAG_DOCUMENT_15"]) * 2.0)))) * 2.0))), (((((data["NAME_GOODS_CATEGORY_Photo___Cinema_Equipment"]) > ((((2.0) + (data["ty__Mortgage"]))/2.0)))*1.)))))))) +
                    0.100000*np.tanh(((((((((data["AMT_CREDIT_SUM_DEBT"]) - (((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * 2.0)))) * 2.0)) + ((((((data["AMT_CREDIT_SUM_DEBT"]) > (np.tanh((0.0))))*1.)) / 2.0)))) * 2.0)) +
                    0.099961*np.tanh((((((0.0)) / 2.0)) + ((((((-1.0) > (((((np.tanh(((((((data["cnt_FLAG_DOCUMENT_2"]) * 2.0)) < ((2.0)))*1.)))) - (data["ca__Closed"]))) * 2.0)))*1.)) * 2.0)))) +
                    0.099922*np.tanh((((((np.tanh((np.minimum(((-1.0)), ((((((data["AMT_CREDIT_SUM"]) * 2.0)) * (((data["inst_AMT_PAYMENT"]) * 2.0))))))))) / 2.0)) > (data["AMT_CREDIT_SUM"]))*1.)) +
                    0.099922*np.tanh(np.maximum(((((-1.0) + (((-1.0) + (data["SK_ID_BUREAU"])))))), ((((data["inst_AMT_PAYMENT"]) * (((data["SK_ID_BUREAU"]) + (data["ty__Mortgage"])))))))) +
                    0.099922*np.tanh(np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((np.minimum(((((data["NAME_GOODS_CATEGORY_Insurance"]) + (0.0)))), ((((((data["NAME_EDUCATION_TYPE"]) * ((-1.0*((((np.tanh((data["te_NAME_FAMILY_STATUS"]))) * (data["NAME_EDUCATION_TYPE"])))))))) / 2.0)))))))) +
                    0.095467*np.tanh(((((((((1.0) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) + ((-1.0*((((((((((((12.27627658843994141)) < ((-1.0*((data["FLAG_DOCUMENT_2"])))))*1.)) + (data["AMT_GOODS_PRICE_x"]))/2.0)) > (data["ca__Sold"]))*1.))))))/2.0)) * 2.0)) +
                    0.100000*np.tanh((((((10.33813381195068359)) * (((data["AMT_CREDIT_x"]) - (data["AMT_GOODS_PRICE_x"]))))) + (data["AMT_CREDIT_x"]))) +
                    0.100000*np.tanh(((data["te_NAME_CONTRACT_TYPE"]) * (np.minimum(((np.where(data["cnt_FLAG_WORK_PHONE"]>0, ((data["te_NAME_CONTRACT_TYPE"]) - (data["cnt_NAME_CONTRACT_TYPE"])), data["te_NAME_CONTRACT_TYPE"] ))), ((data["NAME_GOODS_CATEGORY_Insurance"])))))) +
                    0.099961*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["FLAG_DOCUMENT_16"], (((((((np.maximum(((((-1.0) + (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"])))), ((0.0)))) - (data["ty__Another_type_of_loan"]))) + (data["cnt_FLAG_DOCUMENT_16"]))/2.0)) / 2.0) )) +
                    0.098320*np.tanh((((((((data["NONLIVINGAPARTMENTS_MODE"]) / 2.0)) + ((((((data["cnt_FLAG_WORK_PHONE"]) < (data["SK_ID_BUREAU"]))*1.)) - ((((np.where(2.0>0, data["cnt_FLAG_WORK_PHONE"], data["NAME_GOODS_CATEGORY_Insurance"] )) > (data["SK_ID_BUREAU"]))*1.)))))/2.0)) / 2.0)) +
                    0.075498*np.tanh(np.where(((((2.0)) < ((((((np.maximum(((data["CODE_REJECT_REASON_SCO"])), ((data["CODE_REJECT_REASON_XAP"])))) + (-1.0))) + (data["NAME_GOODS_CATEGORY_Gardening"]))/2.0)))*1.)>0, -3.0, np.maximum(((data["FLAG_DOCUMENT_2"])), ((data["CODE_REJECT_REASON_SCO"]))) )) +
                    0.100000*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["FLAG_DOCUMENT_6"], np.where(data["cnt_FLAG_DOCUMENT_6"]>0, data["NAME_GOODS_CATEGORY_Insurance"], (((((-3.0) + (((((((data["CODE_REJECT_REASON_XAP"]) - (data["cnt_FLAG_DOCUMENT_6"]))) / 2.0)) * 2.0)))/2.0)) * 2.0) ) )) +
                    0.075147*np.tanh((((-1.0*((((data["cnt_FLAG_DOCUMENT_2"]) - (((((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"])))) * 2.0)) + (((data["REGION_POPULATION_RELATIVE"]) - (((data["CODE_REJECT_REASON_SCO"]) / 2.0)))))/2.0))))))) / 2.0)) +
                    0.099844*np.tanh(((((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]) * (((np.minimum(((((-1.0) * (data["NAME_GOODS_CATEGORY_Sport_and_Leisure"])))), ((np.maximum(((data["inst_AMT_PAYMENT"])), ((data["inst_AMT_PAYMENT"]))))))) - (data["NAME_YIELD_GROUP_middle"]))))) * 2.0)) +
                    0.099492*np.tanh((((((np.tanh((data["PRODUCT_COMBINATION_Cash"]))) + (((((data["FLAG_DOCUMENT_2"]) - (((((-3.0) * (((data["PRODUCT_COMBINATION_Cash"]) * (data["cc_bal_cc_bal_status__Sent_proposal"]))))) / 2.0)))) / 2.0)))/2.0)) / 2.0)) +
                    0.100000*np.tanh((((data["AMT_REQ_CREDIT_BUREAU_DAY"]) + ((((data["cc_bal_SK_DPD_DEF"]) + ((((data["NAME_CONTRACT_STATUS_Approved"]) + (np.maximum((((((data["AMT_REQ_CREDIT_BUREAU_DAY"]) + (data["SK_DPD"]))/2.0))), (((((np.tanh((data["NAME_CONTRACT_STATUS_Approved"]))) + (data["FLAG_DOCUMENT_2"]))/2.0))))))/2.0)))/2.0)))/2.0)) +
                    0.099961*np.tanh(((data["Returned_to_the_store"]) + (np.maximum(((((data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]) + ((((np.tanh((data["Returned_to_the_store"]))) + ((-1.0*((data["inst_AMT_PAYMENT"])))))/2.0))))), ((np.maximum(((data["FLAG_DOCUMENT_2"])), ((np.tanh((data["Returned_to_the_store"]))))))))))) +
                    0.097812*np.tanh(np.maximum(((np.minimum(((((((((((data["NAME_CONTRACT_TYPE_Consumer_loans"]) + (data["CNT_INSTALMENT_FUTURE"]))/2.0)) - (-1.0))) > (data["NAME_GOODS_CATEGORY_XNA"]))*1.))), ((data["NAME_PRODUCT_TYPE_x_sell"]))))), ((((-3.0) - (-3.0)))))) +
                    0.099726*np.tanh(np.minimum(((((((data["NAME_PORTFOLIO_XNA"]) / 2.0)) / 2.0))), ((np.minimum(((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * (np.minimum(((data["PRODUCT_COMBINATION_Cash"])), ((((data["cnt_FLAG_DOCUMENT_2"]) * 2.0)))))))), ((data["NAME_GOODS_CATEGORY_Insurance"]))))))) +
                    0.099922*np.tanh(np.maximum(((data["FLAG_DOCUMENT_2"])), ((np.minimum((((6.0))), (((((np.minimum(((data["CODE_REJECT_REASON_LIMIT"])), ((data["inst_AMT_PAYMENT"])))) > (((data["Active"]) - (((data["CODE_REJECT_REASON_LIMIT"]) - ((3.66125917434692383)))))))*1.)))))))) +
                    0.100000*np.tanh(((np.where(((1.0) - (data["RATE_DOWN_PAYMENT"]))>0, np.minimum(((2.0)), ((data["NAME_GOODS_CATEGORY_Insurance"]))), (-1.0*(((((data["inst_AMT_PAYMENT"]) < (data["FLAG_CONT_MOBILE"]))*1.)))) )) / 2.0)) +
                    0.066510*np.tanh(np.where(data["FLAG_CONT_MOBILE"]>0, (((np.minimum(((data["cnt_FLAG_DOCUMENT_17"])), ((np.minimum(((((data["AMT_REQ_CREDIT_BUREAU_MON"]) * (data["inst_AMT_PAYMENT"])))), (((12.75508689880371094)))))))) + ((((data["FLAG_CONT_MOBILE"]) < (data["OCCUPATION_TYPE"]))*1.)))/2.0), (10.0) )) +
                    0.092106*np.tanh(np.where(data["te_NAME_CONTRACT_TYPE"]>0, (((data["cnt_FLAG_DOCUMENT_11"]) + (((((0.0) + (0.0))) - (np.tanh((data["te_NAME_EDUCATION_TYPE"]))))))/2.0), data["te_NAME_EDUCATION_TYPE"] )) +
                    0.099961*np.tanh(((((np.where(data["NAME_CASH_LOAN_PURPOSE_Medicine"]>0, data["NAME_CASH_LOAN_PURPOSE_Medicine"], np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Medicine"])), (((((-1.0*((data["inst_AMT_PAYMENT"])))) / 2.0)))) )) / 2.0)) + ((-1.0*((data["YEARS_BEGINEXPLUATATION_MODE"])))))) +
                    0.099531*np.tanh((((np.where((((data["NAME_GOODS_CATEGORY_Insurance"]) > (data["cnt_NAME_EDUCATION_TYPE"]))*1.)>0, data["LIVINGAPARTMENTS_MEDI"], data["cu__currency_3"] )) + (np.where(data["LIVINGAPARTMENTS_MEDI"]>0, data["cnt_NAME_EDUCATION_TYPE"], data["CODE_REJECT_REASON_VERIF"] )))/2.0)) +
                    0.099844*np.tanh(np.minimum(((((((data["FLAG_DOCUMENT_15"]) * 2.0)) * (((data["cnt_FLAG_DOCUMENT_2"]) - ((((-1.0*((data["CODE_REJECT_REASON_VERIF"])))) + (data["te_WEEKDAY_APPR_PROCESS_START"])))))))), ((((3.0) - (-1.0)))))) +
                    0.099961*np.tanh(((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["FLAG_DOCUMENT_16"], np.minimum(((data["cc_bal_cc_bal_status__Sent_proposal"])), ((np.minimum(((data["cnt_FLAG_DOCUMENT_16"])), ((data["cnt_FLAG_DOCUMENT_14"])))))) )) / 2.0)) +
                    0.099726*np.tanh(np.minimum(((data["cc_bal_cc_bal_status__Sent_proposal"])), ((np.minimum((((-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"]))))), ((((np.minimum(((np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["CODE_REJECT_REASON_VERIF"], data["cnt_FLAG_DOCUMENT_16"] ))), ((((data["CODE_REJECT_REASON_VERIF"]) * (data["FLAG_DOCUMENT_16"])))))) / 2.0)))))))) +
                    0.099844*np.tanh((((data["AMT_DOWN_PAYMENT"]) > (np.tanh(((((np.minimum(((data["cnt_FLAG_DOCUMENT_15"])), ((((-3.0) * (data["AMT_DOWN_PAYMENT"])))))) < (np.minimum(((1.0)), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))))*1.)))))*1.)) +
                    0.090660*np.tanh(((data["NAME_GOODS_CATEGORY_Insurance"]) + (np.minimum(((data["cnt_FLAG_DOCUMENT_15"])), ((np.where(((data["cc_bal_SK_DPD_DEF"]) * (data["cc_bal_SK_DPD_DEF"]))>0, ((data["NFLAG_INSURED_ON_APPROVAL"]) + (data["FLAG_DOCUMENT_15"])), data["cc_bal_SK_DPD_DEF"] ))))))) +
                    0.086245*np.tanh(((((((np.minimum((((((np.minimum(((0.0)), ((data["NAME_CONTRACT_TYPE_Revolving_loans"])))) > (data["inst_AMT_PAYMENT"]))*1.))), ((data["NAME_CONTRACT_TYPE_Revolving_loans"])))) > (data["inst_AMT_PAYMENT"]))*1.)) > ((((data["NAME_PORTFOLIO_Cards"]) > (1.0))*1.)))*1.)) +
                    0.072763*np.tanh(np.minimum((((-1.0*((data["RATE_DOWN_PAYMENT"]))))), (((((data["Completed"]) + (((((((data["cc_bal_SK_DPD_DEF"]) * 2.0)) / 2.0)) + ((((data["DAYS_FIRST_DRAWING"]) < (((data["DAYS_FIRST_DRAWING"]) / 2.0)))*1.)))))/2.0))))) +
                    0.099766*np.tanh(((np.where(data["DAYS_LAST_DUE"]>0, (((((((((((1.0) < (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]))*1.)) > (data["DAYS_TERMINATION"]))*1.)) + (data["cnt_FLAG_DOCUMENT_2"]))/2.0)) * ((13.01487827301025391))), np.tanh((data["NAME_GOODS_CATEGORY_Direct_Sales"])) )) * 2.0)) +
                    0.095115*np.tanh(np.where((((((-3.0) > ((((4.0)) * (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))))*1.)) / 2.0)>0, (((6.0)) * 2.0), data["cc_bal_SK_DPD"] )) +
                    0.086792*np.tanh(np.minimum(((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"])), ((np.tanh((((data["SK_ID_PREV_y"]) * (np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((data["FLAG_DOCUMENT_17"]) * (((((data["SK_ID_PREV_y"]) * 2.0)) * 2.0)))))))))))))) +
                    0.099766*np.tanh((((((np.tanh(((((data["AMT_DOWN_PAYMENT"]) > ((-1.0*((data["FLAG_DOCUMENT_2"])))))*1.)))) / 2.0)) < (((data["AMT_DOWN_PAYMENT"]) - (np.tanh((data["AMT_DOWN_PAYMENT"]))))))*1.)) +
                    0.099531*np.tanh(((data["FLAG_DOCUMENT_17"]) * ((((((7.0)) * 2.0)) * ((((data["NAME_GOODS_CATEGORY_Furniture"]) + (np.minimum(((((data["NAME_GOODS_CATEGORY_Furniture"]) + (data["NAME_GOODS_CATEGORY_Furniture"])))), ((data["cnt_FLAG_DOCUMENT_2"])))))/2.0)))))) +
                    0.099570*np.tanh(np.where(((data["CODE_REJECT_REASON_VERIF"]) * (data["NAME_CONTRACT_STATUS_Approved"]))>0, (((((data["NAME_SELLER_INDUSTRY_MLM_partners"]) + (((((((data["NAME_CONTRACT_STATUS_Approved"]) / 2.0)) + (data["CODE_REJECT_REASON_VERIF"]))) / 2.0)))/2.0)) / 2.0), (-1.0*((data["NAME_SELLER_INDUSTRY_MLM_partners"]))) )) +
                    0.099766*np.tanh(((((((((data["NAME_GOODS_CATEGORY_Insurance"]) * (data["cnt_FLAG_DOCUMENT_2"]))) + ((((data["cc_bal_SK_DPD_DEF"]) < (data["NAME_CASH_LOAN_PURPOSE_Car_repairs"]))*1.)))) * 2.0)) + (np.where(data["FLAG_DOCUMENT_2"]>0, (12.16825199127197266), ((data["cc_bal_SK_DPD_DEF"]) * 2.0) )))) +
                    0.099648*np.tanh(np.minimum(((data["cnt_FLAG_DOCUMENT_13"])), ((np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"])), ((((((((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"])))) < (data["NAME_TYPE_SUITE_Children"]))*1.)) - ((((data["NAME_TYPE_SUITE_Children"]) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"]))/2.0)))) / 2.0)))))))) +
                    0.098476*np.tanh((-1.0*((((np.where(((data["NAME_GOODS_CATEGORY_Direct_Sales"]) / 2.0)>0, np.where((((3.0)) / 2.0)>0, data["NONLIVINGAREA_MODE"], (((((data["cc_bal_AMT_BALANCE"]) > (3.0))*1.)) / 2.0) ), data["NAME_GOODS_CATEGORY_Fitness"] )) * 2.0))))) +
                    0.099805*np.tanh(((np.tanh((((np.minimum(((data["cnt_FLAG_DOCUMENT_17"])), ((((((data["te_REG_CITY_NOT_WORK_CITY"]) * (((data["te_REG_CITY_NOT_WORK_CITY"]) * 2.0)))) - (data["FLAG_DOCUMENT_14"])))))) + (((data["NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer"]) * (data["te_REG_CITY_NOT_WORK_CITY"]))))))) * 2.0)) +
                    0.075733*np.tanh(((data["OWN_CAR_AGE"]) * (((((-1.0*((data["cu__currency_3"])))) > (((np.tanh((data["OWN_CAR_AGE"]))) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))*1.)))) +
                    0.100000*np.tanh(((((data["AMT_ANNUITY_x"]) * (((-2.0) * (((data["avg_buro_buro_bal_status_5"]) * 2.0)))))) + (((np.where(2.0>0, data["CODE_REJECT_REASON_VERIF"], (-1.0*((data["NAME_HOUSING_TYPE"]))) )) * (data["NAME_HOUSING_TYPE"]))))) +
                    0.099531*np.tanh((-1.0*((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["AMT_GOODS_PRICE_x"], ((0.0) + ((((1.0) < (((data["NAME_SELLER_INDUSTRY_MLM_partners"]) - (data["AMT_GOODS_PRICE_x"]))))*1.))) ))))) +
                    0.099961*np.tanh(np.tanh((((data["cc_bal_AMT_BALANCE"]) * (((data["CHANNEL_TYPE_AP___Cash_loan_"]) + ((((data["cc_bal_AMT_BALANCE"]) > (((2.0) - (((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0)))))*1.)))))))) +
                    0.099961*np.tanh(np.where(((data["EXT_SOURCE_1"]) - (-2.0))>0, (((-1.0*(((((np.tanh((np.tanh((-2.0))))) > (data["EXT_SOURCE_1"]))*1.))))) / 2.0), (13.55623054504394531) )) +
                    0.099844*np.tanh((((((((((-1.0*((np.maximum(((3.0)), ((data["NAME_GOODS_CATEGORY_Insurance"]))))))) + (np.where(data["DAYS_CREDIT_ENDDATE"]>0, 0.0, np.minimum((((13.47132968902587891))), ((data["CODE_REJECT_REASON_SCOFR"]))) )))/2.0)) > (data["EXT_SOURCE_1"]))*1.)) * 2.0)) +
                    0.074717*np.tanh((((((data["FONDKAPREMONT_MODE"]) * (np.minimum(((((1.0) + (data["cu__currency_3"])))), ((data["cnt_FLAG_DOCUMENT_5"])))))) < (data["SK_DPD"]))*1.)) +
                    0.100000*np.tanh((-1.0*(((-1.0*((((data["EXT_SOURCE_3"]) * (((((data["EXT_SOURCE_3"]) / 2.0)) + ((-1.0*(((((-1.0*((data["EXT_SOURCE_3"])))) - (2.0)))))))))))))))) +
                    0.100000*np.tanh((-1.0*(((((((((((((2.0) + (((data["DAYS_CREDIT_ENDDATE"]) * 2.0)))/2.0)) < (data["EXT_SOURCE_2"]))*1.)) > (np.where(data["DAYS_CREDIT_ENDDATE"]>0, 2.0, ((data["EXT_SOURCE_2"]) / 2.0) )))*1.)) / 2.0))))) +
                    0.093005*np.tanh((-1.0*(((((data["AMT_GOODS_PRICE_x"]) > (((data["ty__Mortgage"]) + ((((((data["AMT_CREDIT_SUM"]) * (data["AMT_CREDIT_SUM"]))) < (np.where(2.0>0, data["AMT_GOODS_PRICE_x"], np.tanh(((9.0))) )))*1.)))))*1.))))) +
                    0.093669*np.tanh(np.where((((((data["CNT_INSTALMENT_FUTURE"]) > (((((((8.70177841186523438)) * 2.0)) > (-3.0))*1.)))*1.)) / 2.0)>0, ((((data["AMT_CREDIT_y"]) * (data["CNT_INSTALMENT_FUTURE"]))) / 2.0), (-1.0*((data["ty__Mortgage"]))) )) +
                    0.086440*np.tanh(np.where(data["inst_AMT_PAYMENT"]>0, np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((data["AMT_CREDIT_x"]) / 2.0)))), data["NAME_CASH_LOAN_PURPOSE_Furniture"] )) +
                    0.098984*np.tanh(np.where(np.tanh((data["ca__Sold"]))>0, (((-1.0*(((((((-1.0*((np.tanh((np.tanh((data["inst_AMT_PAYMENT"])))))))) * (data["ca__Sold"]))) * 2.0))))) - (data["inst_AMT_PAYMENT"])), data["ca__Sold"] )) +
                    0.099961*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["avg_buro_buro_bal_status_5"], ((np.tanh(((((-1.0*((np.where(data["ca__Sold"]>0, data["CNT_INSTALMENT"], ((data["cu__currency_3"]) * 2.0) ))))) / 2.0)))) * (((data["ca__Sold"]) / 2.0))) )) +
                    0.091989*np.tanh(((data["cu__currency_3"]) + ((((data["avg_buro_buro_count"]) < (np.where((((data["PRODUCT_COMBINATION_Cash"]) > (np.tanh((1.0))))*1.)>0, ((data["NAME_GOODS_CATEGORY_Insurance"]) * ((0.83815473318099976))), ((-2.0) / 2.0) )))*1.)))) +
                    0.100000*np.tanh(((((data["CNT_INSTALMENT"]) * (((((data["ca__Sold"]) - (data["DAYS_CREDIT"]))) / 2.0)))) * (((np.where(data["DAYS_CREDIT"]>0, data["avg_buro_buro_bal_status_0"], -2.0 )) / 2.0)))) +
                    0.099766*np.tanh(np.minimum(((((data["AMT_REQ_CREDIT_BUREAU_WEEK"]) * ((((-1.0) + (data["WEEKDAY_APPR_PROCESS_START_WEDNESDAY"]))/2.0))))), (((((((data["AMT_REQ_CREDIT_BUREAU_WEEK"]) / 2.0)) + (data["AMT_REQ_CREDIT_BUREAU_WEEK"]))/2.0))))) +
                    0.084564*np.tanh(((np.tanh(((((data["CNT_INSTALMENT_FUTURE"]) > (np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, (13.89491844177246094), (((((data["HOUR_APPR_PROCESS_START_y"]) > (data["NAME_GOODS_CATEGORY_Gardening"]))*1.)) * 2.0) )))*1.)))) / 2.0)) +
                    0.100000*np.tanh(((((((data["inst_AMT_PAYMENT"]) * ((((data["te_WALLSMATERIAL_MODE"]) < (data["avg_buro_MONTHS_BALANCE"]))*1.)))) * 2.0)) * ((((data["avg_buro_MONTHS_BALANCE"]) + (-1.0))/2.0)))) +
                    0.099414*np.tanh(np.where(data["OWN_CAR_AGE"]>0, ((data["AMT_CREDIT_y"]) - (data["avg_buro_MONTHS_BALANCE"])), (((((data["AMT_CREDIT_x"]) - (((3.0) / 2.0)))) > (data["cc_bal_SK_DPD_DEF"]))*1.) )) +
                    0.100000*np.tanh(np.where(data["AMT_GOODS_PRICE_x"]>0, data["FLAG_DOCUMENT_5"], (-1.0*((((((((6.79844999313354492)) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))) < (np.minimum(((data["AMT_GOODS_PRICE_x"])), ((3.0)))))*1.)))) )) +
                    0.069637*np.tanh((-1.0*((np.maximum(((data["FLAG_DOCUMENT_17"])), ((((np.where(data["NAME_CASH_LOAN_PURPOSE_Furniture"]>0, data["WALLSMATERIAL_MODE"], ((((data["WALLSMATERIAL_MODE"]) * 2.0)) - ((4.0))) )) * 2.0)))))))) +
                    0.100000*np.tanh((((((-2.0) > (data["EXT_SOURCE_1"]))*1.)) - ((((data["EXT_SOURCE_1"]) > ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0)))*1.)))*1.)))) +
                    0.085307*np.tanh((((data["te_FLAG_DOCUMENT_20"]) > (((((((((np.maximum(((data["cc_bal_SK_DPD_DEF"])), ((data["ENTRANCES_MEDI"])))) < (((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)))*1.)) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)) + (data["ENTRANCES_MODE"]))))*1.)) +
                    0.098554*np.tanh(((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((data["te_FLAG_DOCUMENT_5"]) * ((3.66830086708068848)))))) +
                    0.099961*np.tanh((((data["cnt_FLAG_DOCUMENT_11"]) + (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["FONDKAPREMONT_MODE"], (-1.0*((((((((2.0) > (-1.0))*1.)) < (data["FONDKAPREMONT_MODE"]))*1.)))) )))/2.0)) +
                    0.099453*np.tanh(((data["cc_bal_SK_DPD_DEF"]) + (np.where(data["cc_bal_SK_DPD_DEF"]>0, data["CODE_REJECT_REASON_XAP"], ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (((((data["CODE_REJECT_REASON_XAP"]) + ((((data["NAME_GOODS_CATEGORY_Gardening"]) > (data["NAME_GOODS_CATEGORY_Gardening"]))*1.)))) * (-2.0)))) )))) +
                    0.099648*np.tanh(np.tanh((np.minimum((((((data["WEEKDAY_APPR_PROCESS_START_WEDNESDAY"]) > (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) / 2.0)))*1.))), ((((((data["NONLIVINGAPARTMENTS_MODE"]) * (data["WEEKDAY_APPR_PROCESS_START_WEDNESDAY"]))) - (data["NAME_GOODS_CATEGORY_Gardening"])))))))) +
                    0.099687*np.tanh(((((((np.where(data["cc_bal_cc_bal_status__Sent_proposal"]>0, data["NAME_GOODS_CATEGORY_Fitness"], data["PRODUCT_COMBINATION_POS_industry_without_interest"] )) * (np.maximum(((data["NAME_YIELD_GROUP_high"])), ((0.0)))))) * 2.0)) * 2.0)) +
                    0.098281*np.tanh(((((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) * ((-1.0*((data["NONLIVINGAPARTMENTS_MODE"])))))) + ((-1.0*((np.tanh((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) - ((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) < (data["PRODUCT_COMBINATION_Cash_X_Sell__high"]))*1.)) / 2.0))))))))))) +
                    0.099883*np.tanh(((data["CNT_INSTALMENT"]) * (((np.maximum(((data["NAME_GOODS_CATEGORY_Other"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) * (np.where((7.0)>0, data["NAME_TYPE_SUITE_Unaccompanied"], (((data["NAME_TYPE_SUITE_Unaccompanied"]) + ((((3.0) + (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))/2.0)))/2.0) )))))) +
                    0.099140*np.tanh(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0)) * (np.where(((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["CNT_INSTALMENT"]))>0, data["NAME_TYPE_SUITE_Unaccompanied"], data["NAME_GOODS_CATEGORY_Education"] )))) +
                    0.099883*np.tanh(np.where((-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"])))>0, np.tanh((data["PRODUCT_COMBINATION_POS_industry_without_interest"])), (((((data["PRODUCT_COMBINATION_POS_industry_without_interest"]) * (data["inst_AMT_PAYMENT"]))) + (((((0.0) * 2.0)) * 2.0)))/2.0) )) +
                    0.099687*np.tanh(((((((2.0) < (((data["CHANNEL_TYPE_AP___Cash_loan_"]) / 2.0)))*1.)) + ((((np.maximum(((2.0)), ((data["cc_bal_cc_bal_status__Sent_proposal"])))) < (data["PRODUCT_COMBINATION_Card_X_Sell"]))*1.)))/2.0)) +
                    0.099844*np.tanh((-1.0*((np.where(data["NAME_GOODS_CATEGORY_Fitness"]>0, data["ORGANIZATION_TYPE"], (-1.0*(((((((2.0) < ((((data["ORGANIZATION_TYPE"]) + (np.tanh((data["NAME_GOODS_CATEGORY_Insurance"]))))/2.0)))*1.)) * 2.0)))) ))))) +
                    0.099844*np.tanh((((((0.0)) - ((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) < (np.where(((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) + (-1.0))>0, ((3.0) - (data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"])), np.tanh((-2.0)) )))*1.)))) * 2.0)) +
                    0.099687*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, np.maximum(((data["cnt_FLAG_DOCUMENT_3"])), ((np.where(np.tanh((data["SK_ID_BUREAU"]))>0, data["SK_ID_BUREAU"], -3.0 )))), data["NAME_GOODS_CATEGORY_Insurance"] )) +
                    0.075967*np.tanh((-1.0*((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + ((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) > ((((((((data["NAME_GOODS_CATEGORY_Insurance"]) > (2.0))*1.)) - (data["FLAG_DOCUMENT_16"]))) * (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (-3.0))))))*1.))))))) +
                    0.099414*np.tanh((((((np.maximum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["cc_bal_SK_DPD_DEF"])))) > ((((-2.0) < (np.minimum(((data["REG_CITY_NOT_LIVE_CITY"])), ((data["cnt_REG_CITY_NOT_LIVE_CITY"])))))*1.)))*1.)) * 2.0)) +
                    0.099766*np.tanh(((np.minimum(((data["cnt_FLAG_DOCUMENT_10"])), ((((np.minimum((((-1.0*((data["cc_bal_cc_bal_status__Sent_proposal"]))))), ((((data["NAME_GOODS_CATEGORY_Fitness"]) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"])))))) - (data["NAME_GOODS_CATEGORY_Fitness"])))))) * 2.0)) +
                    0.099726*np.tanh(np.minimum(((data["avg_buro_buro_bal_status_5"])), ((((3.0) - (np.where((((((data["NAME_GOODS_CATEGORY_Insurance"]) > (((0.0) - (data["NAME_GOODS_CATEGORY_Education"]))))*1.)) / 2.0)>0, data["NAME_GOODS_CATEGORY_Education"], data["avg_buro_buro_bal_status_5"] ))))))) +
                    0.099492*np.tanh(np.minimum(((data["cnt_FLAG_DOCUMENT_17"])), (((-1.0*((((np.where(np.tanh(((((((((2.0) < (data["PRODUCT_COMBINATION_POS_industry_without_interest"]))*1.)) * 2.0)) * 2.0)))>0, data["avg_buro_buro_bal_status_5"], data["PRODUCT_COMBINATION_POS_industry_without_interest"] )) * 2.0)))))))) +
                    0.099766*np.tanh((((((data["ca__Sold"]) / 2.0)) > (((3.0) + (((data["AMT_REQ_CREDIT_BUREAU_MON"]) - (((data["NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans"]) / 2.0)))))))*1.)) +
                    0.098281*np.tanh((((data["AMT_INCOME_TOTAL"]) > ((((data["ty__Another_type_of_loan"]) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)))*1.)) +
                    0.093982*np.tanh((((-1.0*(((((data["AMT_GOODS_PRICE_x"]) > (((np.where(((data["FLAG_EMP_PHONE"]) / 2.0)>0, np.tanh((data["cnt_FLAG_EMP_PHONE"])), 2.0 )) * (data["cnt_DAYS_EMPLOYED"]))))*1.))))) / 2.0)) +
                    0.100000*np.tanh(((np.where(np.minimum(((data["AMT_CREDIT_x"])), (((((data["AMT_CREDIT_x"]) < (data["AMT_ANNUITY_x"]))*1.))))>0, ((data["AMT_ANNUITY_x"]) * ((13.44352436065673828))), 0.0 )) / 2.0)) +
                    0.089683*np.tanh(((np.where(data["cnt_NAME_INCOME_TYPE"]>0, data["NAME_GOODS_CATEGORY_Insurance"], np.minimum(((np.maximum(((((((np.maximum(((data["avg_buro_buro_bal_status_5"])), ((data["cnt_NAME_INCOME_TYPE"])))) * 2.0)) * 2.0))), ((data["FLAG_DOCUMENT_8"]))))), ((np.tanh((data["avg_buro_buro_bal_status_5"]))))) )) * 2.0)) +
                    0.099844*np.tanh(((np.where(((data["ty__Another_type_of_loan"]) + ((((-1.0*((data["FLAG_DOCUMENT_8"])))) - (-3.0))))>0, ((((data["FLAG_DOCUMENT_8"]) + (data["MONTHS_BALANCE"]))) * (data["ty__Another_type_of_loan"])), data["NAME_FAMILY_STATUS"] )) * 2.0)) +
                    0.098202*np.tanh(np.where((((data["EXT_SOURCE_3"]) < (-1.0))*1.)>0, np.maximum(((-2.0)), (((((((data["EXT_SOURCE_3"]) * (((data["EXT_SOURCE_3"]) / 2.0)))) + (data["ty__Another_type_of_loan"]))/2.0)))), data["EXT_SOURCE_3"] )) +
                    0.099883*np.tanh(np.where(data["SK_ID_BUREAU"]>0, ((data["ca__Active"]) * (2.0)), np.where(data["ca__Active"]>0, data["SK_ID_BUREAU"], data["NAME_GOODS_CATEGORY_Insurance"] ) )) +
                    0.099570*np.tanh((((((np.where(((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * ((5.70599699020385742)))) + (data["DAYS_CREDIT_UPDATE"]))>0, (((data["ca__Active"]) + (-1.0))/2.0), data["DAYS_CREDIT"] )) > (data["ca__Closed"]))*1.)) / 2.0)) +
                    0.077100*np.tanh(((((np.tanh((np.tanh(((-1.0*((((np.maximum(((data["NAME_CONTRACT_STATUS_Refused"])), ((((((5.17349958419799805)) < (data["CODE_REJECT_REASON_HC"]))*1.))))) * 2.0))))))))) / 2.0)) + (((data["CODE_REJECT_REASON_HC"]) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))) +
                    0.099844*np.tanh(((np.tanh((data["SK_ID_PREV_x"]))) * (np.where(((((((data["NAME_YIELD_GROUP_XNA"]) / 2.0)) * (data["SK_ID_PREV_x"]))) + (((data["PRODUCT_COMBINATION_Cash"]) * 2.0)))>0, (-1.0*((data["ty__Another_type_of_loan"]))), data["SK_ID_BUREAU"] )))) +
                    0.099766*np.tanh(((((np.where(((3.0) / 2.0)>0, ((data["SK_ID_PREV_x"]) * ((((((((((0.0)) / 2.0)) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)) < (data["NAME_PORTFOLIO_POS"]))*1.))), data["SK_ID_PREV_x"] )) / 2.0)) / 2.0)) +
                    0.100000*np.tanh((((((data["EXT_SOURCE_3"]) < (-2.0))*1.)) + (((np.tanh(((((data["EXT_SOURCE_3"]) < (-2.0))*1.)))) * 2.0)))) +
                    0.075576*np.tanh(((np.where(np.maximum(((data["SK_ID_BUREAU"])), ((np.where(data["ty__Another_type_of_loan"]>0, -2.0, data["ty__Another_type_of_loan"] ))))>0, data["PRODUCT_COMBINATION_Cash"], data["avg_buro_buro_bal_status_C"] )) / 2.0)) +
                    0.099766*np.tanh(np.where(((-1.0) - ((((data["ty__Another_type_of_loan"]) + (-2.0))/2.0)))>0, np.tanh(((((data["SK_ID_BUREAU"]) > (2.0))*1.))), data["CODE_REJECT_REASON_LIMIT"] )) +
                    0.099883*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Furniture"]>0, data["avg_buro_buro_count"], (((-1.0*(((((1.0) < (data["avg_buro_buro_count"]))*1.))))) * 2.0) )) +
                    0.099883*np.tanh((((((data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]) > (3.0))*1.)) + ((((np.where(((data["CHANNEL_TYPE_Credit_and_cash_offices"]) + (-2.0))>0, np.tanh((0.0)), 2.0 )) + (-2.0))/2.0)))) +
                    0.099453*np.tanh((((2.0) < (((((((data["HOUR_APPR_PROCESS_START_y"]) * (data["HOUR_APPR_PROCESS_START_y"]))) * ((((data["PRODUCT_COMBINATION_Cash_Street__high"]) > (data["ty__Another_type_of_loan"]))*1.)))) * 2.0)))*1.)) +
                    0.099844*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Furniture"]) * (((np.where(-1.0>0, -3.0, data["NAME_CONTRACT_STATUS_Canceled"] )) * (np.maximum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((3.0)))))))) - (data["CHANNEL_TYPE_Car_dealer"]))) +
                    0.099140*np.tanh(((((1.0) - (data["CNT_INSTALMENT_FUTURE"]))) * ((((0.0) + ((((-1.0*(((((data["CNT_INSTALMENT_FUTURE"]) > (2.0))*1.))))) * (data["DAYS_LAST_DUE"]))))/2.0)))) +
                    0.099375*np.tanh((((np.maximum(((np.maximum(((data["CNT_INSTALMENT"])), ((data["NAME_CASH_LOAN_PURPOSE_Furniture"]))))), ((data["cnt_FLAG_OWN_REALTY"])))) < (np.tanh((data["inst_AMT_INSTALMENT"]))))*1.)) +
                    0.099844*np.tanh(((np.where((((data["ty__Another_type_of_loan"]) + (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))))/2.0)>0, data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"], data["NAME_GOODS_CATEGORY_Insurance"] )) + (((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * (np.tanh((data["DAYS_CREDIT"]))))))) +
                    0.099922*np.tanh(np.where(((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]) * 2.0)>0, ((data["NAME_CASH_LOAN_PURPOSE_Everyday_expenses"]) * (data["PRODUCT_COMBINATION_Cash_X_Sell__high"])), np.tanh(((((((-1.0*((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])))) * (data["PRODUCT_COMBINATION_Cash_X_Sell__high"]))) * (data["PRODUCT_COMBINATION_Cash_X_Sell__high"])))) )) +
                    0.099961*np.tanh(((np.minimum(((((data["NAME_YIELD_GROUP_high"]) * (((((data["NAME_YIELD_GROUP_high"]) * (data["ty__Another_type_of_loan"]))) * 2.0))))), ((data["NAME_GOODS_CATEGORY_Insurance"])))) * 2.0)) +
                    0.082493*np.tanh(np.tanh((np.tanh(((-1.0*((np.where(data["NAME_CONTRACT_STATUS_Approved"]>0, data["inst_AMT_PAYMENT"], 0.0 ))))))))) +
                    0.099883*np.tanh(((data["te_FLAG_PHONE"]) * (np.minimum(((((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) * ((((data["te_FLAG_PHONE"]) > (((data["te_FLAG_PHONE"]) + (data["ty__Another_type_of_loan"]))))*1.))))), ((((((1.0) * 2.0)) / 2.0))))))) +
                    0.099609*np.tanh(((((data["te_FLAG_PHONE"]) * ((((2.0) < (data["avg_buro_buro_bal_status_5"]))*1.)))) - (((data["ty__Another_type_of_loan"]) * (((data["cnt_FLAG_PHONE"]) + (data["cnt_FLAG_PHONE"]))))))) +
                    0.098632*np.tanh((((data["NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex"]) + (((((((0.0) > (data["NONLIVINGAPARTMENTS_MODE"]))*1.)) > ((((2.0)) - (data["WEEKDAY_APPR_PROCESS_START_WEDNESDAY"]))))*1.)))/2.0)) +
                    0.099922*np.tanh(np.minimum(((((((((1.0) > (data["NONLIVINGAPARTMENTS_MODE"]))*1.)) < (((data["NAME_GOODS_CATEGORY_Fitness"]) * (data["ty__Another_type_of_loan"]))))*1.))), (((((-1.0*((((data["PRODUCT_COMBINATION_Cash_Street__high"]) * (data["NONLIVINGAPARTMENTS_MODE"])))))) * 2.0))))) +
                    0.099922*np.tanh(((((data["NAME_GOODS_CATEGORY_Fitness"]) * (((((np.tanh(((-1.0*((data["PRODUCT_COMBINATION_Cash_Street__high"])))))) - (data["HOUSETYPE_MODE"]))) * 2.0)))) * 2.0)) +
                    0.095115*np.tanh(np.where(data["HOUR_APPR_PROCESS_START_y"]>0, ((data["ty__Another_type_of_loan"]) * 2.0), ((data["cnt_REG_CITY_NOT_LIVE_CITY"]) * (((((data["REG_CITY_NOT_LIVE_CITY"]) * (data["ty__Another_type_of_loan"]))) * 2.0))) )) +
                    0.100000*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["te_REG_REGION_NOT_LIVE_REGION"], ((np.minimum(((data["ty__Another_type_of_loan"])), (((((13.32924747467041016)) - (np.maximum(((((data["te_REG_REGION_NOT_LIVE_REGION"]) * 2.0))), ((data["ty__Another_type_of_loan"]))))))))) * ((0.66566365957260132))) )) +
                    0.098906*np.tanh(np.where(data["AMT_REQ_CREDIT_BUREAU_DAY"]>0, data["AMT_REQ_CREDIT_BUREAU_DAY"], ((((data["NAME_GOODS_CATEGORY_Fitness"]) + (((data["PRODUCT_COMBINATION_POS_industry_without_interest"]) / 2.0)))) * (data["NAME_CASH_LOAN_PURPOSE_Furniture"])) )) +
                    0.099844*np.tanh(np.minimum((((((-1.0*(((((np.maximum((((14.60459995269775391))), ((data["FLAG_DOCUMENT_4"])))) < (((data["NAME_GOODS_CATEGORY_Clothing_and_Accessories"]) * 2.0)))*1.))))) - (data["ty__Another_type_of_loan"])))), ((data["cnt_FLAG_DOCUMENT_4"])))) +
                    0.085932*np.tanh(np.tanh(((-1.0*((((((((-1.0*((np.maximum(((np.minimum(((3.0)), ((((data["PRODUCT_COMBINATION_POS_mobile_without_interest"]) + (data["NONLIVINGAREA_MODE"]))))))), ((data["PRODUCT_COMBINATION_POS_mobile_without_interest"]))))))) < (data["NAME_GOODS_CATEGORY_Weapon"]))*1.)) / 2.0))))))) +
                    0.073701*np.tanh((((((((data["Active"]) * ((((np.tanh((data["NAME_SELLER_INDUSTRY_Connectivity"]))) + (data["Active"]))/2.0)))) < ((((data["te_ORGANIZATION_TYPE"]) > (((((7.06093883514404297)) + (((-1.0) * 2.0)))/2.0)))*1.)))*1.)) / 2.0)) +
                    0.070379*np.tanh(((np.minimum(((data["NAME_SELLER_INDUSTRY_Connectivity"])), ((data["cnt_FLAG_DOCUMENT_10"])))) * ((((((data["Active"]) * ((7.0)))) > ((2.79707622528076172)))*1.)))) +
                    0.099336*np.tanh(np.where(data["DAYS_LAST_PHONE_CHANGE"]>0, ((np.tanh(((((((data["DAYS_LAST_PHONE_CHANGE"]) > (1.0))*1.)) * 2.0)))) / 2.0), (((((0.0) * (1.0))) < (((data["TOTALAREA_MODE"]) * 2.0)))*1.) )) +
                    0.100000*np.tanh(np.tanh(((((((data["inst_DAYS_ENTRY_PAYMENT"]) < (np.maximum(((data["inst_DAYS_INSTALMENT"])), ((-2.0)))))*1.)) * (((np.tanh((-3.0))) / 2.0)))))) +
                    0.100000*np.tanh((((data["inst_DAYS_ENTRY_PAYMENT"]) > (np.maximum((((((data["DAYS_TERMINATION"]) > ((((3.0) < (data["CNT_INSTALMENT_FUTURE"]))*1.)))*1.))), ((data["inst_DAYS_INSTALMENT"])))))*1.)) +
                    0.076788*np.tanh(np.tanh(((((((0.0) + ((((((6.0)) * 2.0)) * ((((data["EXT_SOURCE_2"]) + (data["EXT_SOURCE_3"]))/2.0)))))) + ((((data["DAYS_LAST_DUE_1ST_VERSION"]) < (0.0))*1.)))/2.0)))) +
                    0.084408*np.tanh((-1.0*(((((data["EXT_SOURCE_3"]) + (((((-1.0*((data["te_REGION_RATING_CLIENT_W_CITY"])))) > (((((data["EXT_SOURCE_3"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) * 2.0)))*1.)))/2.0))))) +
                    0.093904*np.tanh(np.where(-2.0>0, np.maximum(((3.0)), (((((1.0) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)))), np.maximum((((0.0))), ((((data["NAME_SELLER_INDUSTRY_Connectivity"]) * (data["inst_AMT_PAYMENT"]))))) )) +
                    0.100000*np.tanh(((np.maximum(((-3.0)), ((3.0)))) * (((((2.0)) < (((np.tanh((np.tanh((data["NAME_SELLER_INDUSTRY_Connectivity"]))))) - (data["EXT_SOURCE_2"]))))*1.)))) +
                    0.093357*np.tanh(np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((np.where(data["MONTHS_BALANCE"]>0, np.tanh((data["MONTHS_BALANCE"])), data["NAME_PRODUCT_TYPE_walk_in"] )) * (data["CNT_INSTALMENT_FUTURE"])))))) +
                    0.099922*np.tanh(np.where(((((((((((-1.0) + ((((2.0) + (-3.0))/2.0)))/2.0)) - (-2.0))) < (data["EXT_SOURCE_2"]))*1.)) / 2.0)>0, -1.0, ((data["cnt_FLAG_DOCUMENT_10"]) * 2.0) )) +
                    0.076319*np.tanh(np.minimum(((((data["WALLSMATERIAL_MODE"]) * (((data["EXT_SOURCE_2"]) / 2.0))))), ((((data["FLAG_DOCUMENT_16"]) * (((data["EXT_SOURCE_2"]) + (data["WALLSMATERIAL_MODE"])))))))) +
                    0.085190*np.tanh(((np.maximum(((data["FLAG_DOCUMENT_21"])), (((((((((0.0) * 2.0)) + (((((((data["FLAG_DOCUMENT_21"]) > (data["PRODUCT_COMBINATION_Cash_X_Sell__high"]))*1.)) > (data["DAYS_LAST_DUE_1ST_VERSION"]))*1.)))/2.0)) + (data["FLAG_DOCUMENT_21"])))))) / 2.0)) +
                    0.099883*np.tanh(((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) * (np.maximum(((((data["NAME_GOODS_CATEGORY_Computers"]) * (data["NAME_GOODS_CATEGORY_Computers"])))), (((-1.0*((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))))))) +
                    0.085932*np.tanh(((np.maximum((((-1.0*((((data["cc_bal_SK_DPD"]) * (((data["DAYS_LAST_DUE"]) + (data["DAYS_LAST_DUE"]))))))))), ((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])))) * ((-1.0*((((((data["te_HOUSETYPE_MODE"]) * 2.0)) / 2.0))))))) +
                    0.081243*np.tanh(((data["DAYS_FIRST_DUE"]) * (((np.maximum(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])), ((((((data["NAME_GOODS_CATEGORY_Computers"]) - (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))) * (((data["NAME_GOODS_CATEGORY_Computers"]) / 2.0))))))) / 2.0)))) +
                    0.099922*np.tanh(((((((((data["inst_AMT_PAYMENT"]) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) / 2.0)) - (np.where(data["PRODUCT_COMBINATION_Cash_X_Sell__high"]>0, data["inst_AMT_PAYMENT"], data["NAME_GOODS_CATEGORY_Direct_Sales"] )))) / 2.0)) +
                    0.099922*np.tanh((-1.0*(((((((np.maximum(((-2.0)), (((((data["CNT_INSTALMENT_FUTURE"]) < ((((2.0) < (data["cc_bal_SK_DPD"]))*1.)))*1.))))) + (-2.0))) > (data["AMT_CREDIT_x"]))*1.))))) +
                    0.088824*np.tanh(np.minimum(((2.0)), ((((np.tanh((((((np.tanh(((-1.0*((np.tanh((data["avg_buro_buro_bal_status_2"])))))))) * 2.0)) - (((((data["avg_buro_buro_bal_status_2"]) * (data["EXT_SOURCE_2"]))) * 2.0)))))) * 2.0))))) +
                    0.100000*np.tanh((((((data["AMT_ANNUITY_x"]) + (data["EXT_SOURCE_2"]))/2.0)) * (np.minimum((((((data["te_NAME_CONTRACT_TYPE"]) > (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["AMT_ANNUITY_x"], data["te_NAME_CONTRACT_TYPE"] )))*1.))), ((data["te_NAME_CONTRACT_TYPE"])))))) +
                    0.099101*np.tanh(np.where(data["cnt_NAME_CONTRACT_TYPE"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], np.tanh((((((((data["Active"]) * (data["cnt_NAME_CONTRACT_TYPE"]))) - ((((((-2.0) < (data["Active"]))*1.)) * 2.0)))) * 2.0))) )) +
                    0.098242*np.tanh(((np.minimum(((((np.where(data["avg_buro_buro_bal_status_5"]>0, ((data["avg_buro_buro_bal_status_5"]) * 2.0), ((data["EXT_SOURCE_2"]) * (data["avg_buro_buro_bal_status_5"])) )) * (((data["EXT_SOURCE_2"]) * 2.0))))), ((data["cnt_FLAG_DOCUMENT_10"])))) * 2.0)) +
                    0.098984*np.tanh((((((((np.tanh((data["EXT_SOURCE_2"]))) / 2.0)) + (((((np.minimum((((6.63713836669921875))), ((-2.0)))) * (data["NAME_SELLER_INDUSTRY_MLM_partners"]))) - (data["EXT_SOURCE_2"]))))/2.0)) / 2.0)) +
                    0.099570*np.tanh(((((((((((data["NAME_GOODS_CATEGORY_Education"]) + (((data["HOUR_APPR_PROCESS_START_x"]) * (((data["HOUR_APPR_PROCESS_START_x"]) + (((data["cnt_REG_REGION_NOT_WORK_REGION"]) - (np.tanh((3.0))))))))))/2.0)) < (data["cc_bal_SK_DPD"]))*1.)) / 2.0)) / 2.0)) +
                    0.100000*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["DEF_30_CNT_SOCIAL_CIRCLE"], (-1.0*(((((np.maximum((((-1.0*((data["cnt_FLAG_DOCUMENT_10"]))))), (((((data["NAME_GOODS_CATEGORY_Insurance"]) > ((-1.0*((data["DEF_30_CNT_SOCIAL_CIRCLE"])))))*1.))))) + (data["NAME_CASH_LOAN_PURPOSE_Buying_a_home"]))/2.0)))) )) +
                    0.099883*np.tanh(((((((4.0)) < (((data["CNT_INSTALMENT_FUTURE"]) * (((((data["SK_DPD"]) + (data["AMT_CREDIT_y"]))) + (data["NAME_GOODS_CATEGORY_Insurance"]))))))*1.)) * (np.maximum((((5.46688318252563477))), ((3.0)))))) +
                    0.074248*np.tanh((-1.0*(((((-1.0) < ((((-3.0) + ((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["NAME_PRODUCT_TYPE_walk_in"]))/2.0)))/2.0)))*1.))))) +
                    0.091989*np.tanh(((data["CNT_INSTALMENT"]) * (((np.where((((-1.0*((data["SK_DPD"])))) / 2.0)>0, data["SK_DPD"], (((6.79659414291381836)) * 2.0) )) * 2.0)))) +
                    0.099961*np.tanh((-1.0*((((((data["PRODUCT_COMBINATION_Cash_Street__high"]) * ((((data["SK_DPD"]) + (data["DAYS_TERMINATION"]))/2.0)))) / 2.0))))) +
                    0.100000*np.tanh((((-1.0) > ((((((-2.0) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))/2.0)) - ((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (((data["SK_DPD"]) - (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))/2.0)) / 2.0)))))*1.)) +
                    0.099961*np.tanh(np.where(data["DEF_60_CNT_SOCIAL_CIRCLE"]>0, data["nans"], ((((((np.tanh((((data["CNT_INSTALMENT_FUTURE"]) + (data["DEF_60_CNT_SOCIAL_CIRCLE"]))))) / 2.0)) / 2.0)) + (((((14.63262367248535156)) > ((((10.0)) * 2.0)))*1.))) )) +
                    0.099922*np.tanh((((((((data["SK_ID_PREV_y"]) < (((np.minimum(((1.0)), ((data["avg_buro_buro_bal_status_5"])))) - (((((0.71843880414962769)) > ((-1.0*(((2.00759816169738770))))))*1.)))))*1.)) * 2.0)) * 2.0)) +
                    0.081633*np.tanh(np.maximum(((np.minimum(((((data["inst_AMT_PAYMENT"]) * (data["DEF_60_CNT_SOCIAL_CIRCLE"])))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))), ((((((-3.0) * (1.0))) + (data["DEF_60_CNT_SOCIAL_CIRCLE"])))))) +
                    0.061391*np.tanh(np.where(np.where(data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"]>0, ((data["cnt_WALLSMATERIAL_MODE"]) * 2.0), data["cnt_FLAG_DOCUMENT_10"] )>0, data["cnt_FLAG_DOCUMENT_10"], (((np.where(data["SK_ID_PREV_y"]>0, data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"], -2.0 )) > (data["cnt_WALLSMATERIAL_MODE"]))*1.) )) +
                    0.097460*np.tanh((((((-1.0*((np.tanh((((((data["SK_ID_PREV_y"]) * ((((0.0) > ((-1.0*((data["YEARS_BEGINEXPLUATATION_MODE"])))))*1.)))) / 2.0))))))) - (data["YEARS_BEGINEXPLUATATION_MODE"]))) / 2.0)) +
                    0.067956*np.tanh(np.where(data["AMT_ANNUITY"]>0, np.where(data["AMT_CREDIT_x"]>0, data["NAME_GOODS_CATEGORY_Direct_Sales"], data["PRODUCT_COMBINATION_Cash"] ), np.maximum(((np.minimum(((data["AMT_CREDIT_y"])), ((data["NAME_GOODS_CATEGORY_Direct_Sales"]))))), ((((data["AMT_CREDIT_y"]) / 2.0)))) )) +
                    0.066393*np.tanh((((((data["DAYS_REGISTRATION"]) < (np.minimum(((data["CNT_INSTALMENT"])), (((-1.0*((2.0))))))))*1.)) - ((((-1.0) > (data["DAYS_REGISTRATION"]))*1.)))) +
                    0.099297*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_XNA"]) * ((-1.0*((np.maximum(((data["NAME_GOODS_CATEGORY_Other"])), ((np.where(data["SK_DPD"]>0, data["NAME_CASH_LOAN_PURPOSE_XNA"], (-1.0*((data["FLAG_DOCUMENT_15"]))) )))))))))) - (np.tanh((data["SK_DPD"]))))) +
                    0.098984*np.tanh((((data["cnt_FLAG_DOCUMENT_15"]) + (np.where((((3.0) < (data["SK_DPD"]))*1.)>0, data["SK_DPD"], ((((data["SK_DPD"]) * ((-1.0*((data["SK_DPD"])))))) / 2.0) )))/2.0)) +
                    0.099961*np.tanh(np.where(data["MONTHS_BALANCE"]>0, (((((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]) < (data["SK_DPD"]))*1.)) * 2.0), ((((((((data["SK_DPD"]) > (data["AMT_CREDIT_x"]))*1.)) * (3.0))) < (data["NAME_GOODS_CATEGORY_Education"]))*1.) )) +
                    0.097577*np.tanh(((((((data["NAME_GOODS_CATEGORY_Insurance"]) - (data["NAME_GOODS_CATEGORY_Medical_Supplies"]))) * 2.0)) * ((((np.maximum(((((data["MONTHS_BALANCE"]) * (data["NAME_GOODS_CATEGORY_Insurance"])))), ((data["NAME_GOODS_CATEGORY_Direct_Sales"])))) > (((3.0) * (0.0))))*1.)))) +
                    0.098202*np.tanh(((np.minimum(((data["cnt_FLAG_DOCUMENT_10"])), ((np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((((-1.0) * ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["NAME_GOODS_CATEGORY_Education"]))/2.0)))))))))) - (data["FLAG_CONT_MOBILE"]))) +
                    0.099766*np.tanh(((data["FLAG_DOCUMENT_18"]) * (np.maximum(((np.where(data["NAME_YIELD_GROUP_low_action"]>0, (1.0), data["NAME_YIELD_GROUP_low_action"] ))), ((data["NAME_YIELD_GROUP_low_action"])))))) +
                    0.099883*np.tanh(np.minimum(((data["NAME_GOODS_CATEGORY_Direct_Sales"])), ((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, (((np.tanh((data["NAME_GOODS_CATEGORY_Direct_Sales"]))) < (((((data["cnt_FLAG_DOCUMENT_18"]) / 2.0)) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))*1.), data["cnt_FLAG_DOCUMENT_18"] ))))) +
                    0.099140*np.tanh(np.minimum(((((np.minimum(((np.where(data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]>0, 0.0, data["cnt_FLAG_DOCUMENT_18"] ))), (((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"]))))))) * 2.0))), ((((data["NAME_GOODS_CATEGORY_Weapon"]) * (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"])))))) +
                    0.094724*np.tanh(((((((data["NAME_GOODS_CATEGORY_Fitness"]) * (data["NAME_GOODS_CATEGORY_Education"]))) + (((data["FLAG_DOCUMENT_4"]) * (np.minimum(((data["NAME_GOODS_CATEGORY_Fitness"])), ((data["FLAG_DOCUMENT_10"])))))))) + ((-1.0*((data["NAME_GOODS_CATEGORY_Direct_Sales"])))))) +
                    0.099101*np.tanh(np.minimum(((((data["cnt_FLAG_DOCUMENT_17"]) - (data["FLAG_DOCUMENT_10"])))), ((((data["cnt_FLAG_DOCUMENT_17"]) - (((np.maximum(((data["NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer"])), ((((((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) - (((3.0) * 2.0)))) / 2.0))))) * 2.0))))))) +
                    0.099609*np.tanh(((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"])), ((((((((data["WEEKDAY_APPR_PROCESS_START_FRIDAY"]) + ((((data["NAME_CASH_LOAN_PURPOSE_Gasification___water_supply"]) > (((data["cu__currency_3"]) * 2.0)))*1.)))/2.0)) > (2.0))*1.))))) + (((data["cu__currency_3"]) * 2.0)))) +
                    0.076514*np.tanh((-1.0*(((((((-1.0) - (-2.0))) < ((-1.0*((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["SK_ID_BUREAU"], (((data["WEEKDAY_APPR_PROCESS_START_SUNDAY"]) + (((2.0) * (data["AMT_CREDIT_y"]))))/2.0) ))))))*1.))))) +
                    0.099336*np.tanh((-1.0*(((((data["CNT_PAYMENT"]) > (((2.0) - ((((((data["NAME_GOODS_CATEGORY_Insurance"]) * (np.where(data["inst_AMT_PAYMENT"]>0, (2.21301603317260742), data["NAME_PORTFOLIO_Cash"] )))) > (0.0))*1.)))))*1.))))) +
                    0.089097*np.tanh(np.tanh((np.tanh(((((data["WEEKDAY_APPR_PROCESS_START_THURSDAY"]) > ((((1.0) > (((data["CNT_PAYMENT"]) * (data["CNT_PAYMENT"]))))*1.)))*1.)))))) +
                    0.099883*np.tanh(np.where(data["SK_DPD"]>0, data["DAYS_EMPLOYED"], (((((np.tanh((((data["DAYS_EMPLOYED"]) / 2.0)))) < (((data["AMT_CREDIT_y"]) - (3.0))))*1.)) * 2.0) )) +
                    0.099922*np.tanh(((np.tanh((np.where(data["WEEKDAY_APPR_PROCESS_START_THURSDAY"]>0, ((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]) * 2.0), (-1.0*((np.where(data["NAME_PRODUCT_TYPE_x_sell"]>0, data["inst_AMT_PAYMENT"], data["PRODUCT_COMBINATION_Cash_X_Sell__high"] )))) )))) / 2.0)) +
                    0.099883*np.tanh(((((data["SK_DPD"]) * ((-1.0*((((data["Demand"]) + (data["SK_ID_BUREAU"])))))))) + (data["Demand"]))) +
                    0.096913*np.tanh(((((data["CNT_INSTALMENT_FUTURE"]) * (((((((((data["PRODUCT_COMBINATION_Cash"]) > (1.0))*1.)) + (np.tanh((data["PRODUCT_COMBINATION_Cash_X_Sell__high"]))))/2.0)) / 2.0)))) * 2.0)) +
                    0.099336*np.tanh((-1.0*((np.where(data["DAYS_LAST_DUE"]>0, data["SK_DPD"], (((((data["cc_bal_cc_bal_status__Refused"]) * 2.0)) + (((np.where(data["SK_DPD"]>0, data["te_FLAG_DOCUMENT_11"], data["PRODUCT_COMBINATION_Cash_X_Sell__high"] )) / 2.0)))/2.0) ))))) +
                    0.099766*np.tanh(((0.0) + (np.where(data["cc_bal_cc_bal_status__Refused"]>0, np.where(data["SK_DPD"]>0, data["CNT_INSTALMENT_FUTURE"], np.minimum((((5.0))), ((data["Demand"]))) ), np.minimum(((data["SK_DPD"])), ((3.0))) )))) +
                    0.099844*np.tanh((-1.0*((((-3.0) * ((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) > ((((np.where(1.0>0, (((data["cc_bal_cc_bal_status__Refused"]) + (data["SK_DPD"]))/2.0), (-1.0*((data["avg_buro_buro_bal_status_5"]))) )) > (data["avg_buro_buro_bal_status_5"]))*1.)))*1.))))))) +
                    0.099805*np.tanh(np.minimum(((np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["AMT_REQ_CREDIT_BUREAU_MON"], data["cc_bal_cc_bal_status__Refused"] ))), ((((np.maximum(((((data["cnt_FLAG_DOCUMENT_10"]) + (data["cc_bal_cc_bal_status__Refused"])))), (((((7.0)) - (data["FLAG_DOCUMENT_10"])))))) - (data["AMT_REQ_CREDIT_BUREAU_MON"])))))) +
                    0.099336*np.tanh((((((data["te_ORGANIZATION_TYPE"]) > ((((3.34467959403991699)) - (np.maximum((((-1.0*((((data["SK_DPD"]) * (data["te_ORGANIZATION_TYPE"]))))))), ((data["te_ORGANIZATION_TYPE"])))))))*1.)) + (((data["cc_bal_cc_bal_status__Refused"]) / 2.0)))) +
                    0.099609*np.tanh(np.minimum(((np.tanh((np.maximum((((-1.0*((data["ORGANIZATION_TYPE"]))))), ((data["CNT_INSTALMENT_FUTURE"]))))))), ((data["cnt_FLAG_DOCUMENT_4"])))) +
                    0.099570*np.tanh(np.maximum(((((data["SK_DPD"]) * (((0.0) - (data["cc_bal_AMT_DRAWINGS_OTHER_CURRENT"])))))), (((((data["SK_DPD"]) > ((((13.89245414733886719)) + (np.tanh((0.0))))))*1.))))) +
                    0.099766*np.tanh(((data["HOUR_APPR_PROCESS_START_x"]) * (np.minimum(((2.0)), (((((data["NAME_GOODS_CATEGORY_Insurance"]) + ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (((data["cnt_FLAG_OWN_REALTY"]) * (np.minimum(((0.0)), ((data["cnt_ORGANIZATION_TYPE"])))))))/2.0)))/2.0))))))) +
                    0.083548*np.tanh(((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) / 2.0)) + (data["cc_bal_CNT_DRAWINGS_POS_CURRENT"]))/2.0)) > ((((((data["CNT_INSTALMENT"]) < (data["HOUR_APPR_PROCESS_START_x"]))*1.)) * 2.0)))*1.)) +
                    0.099883*np.tanh((((-1.0*((((np.where(3.0>0, data["te_REGION_RATING_CLIENT"], -3.0 )) * ((((((-2.0) > (np.minimum(((data["te_REGION_RATING_CLIENT"])), ((data["te_REGION_RATING_CLIENT"])))))*1.)) - (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])))))))) / 2.0)) +
                    0.099844*np.tanh(np.where(((data["NAME_GOODS_CATEGORY_Insurance"]) * (np.maximum(((0.0)), ((data["cnt_ORGANIZATION_TYPE"])))))>0, data["cnt_REGION_RATING_CLIENT"], data["NAME_GOODS_CATEGORY_Direct_Sales"] )) +
                    0.056467*np.tanh(((data["HOUR_APPR_PROCESS_START_x"]) * (np.maximum(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (((data["cnt_REGION_RATING_CLIENT"]) / 2.0))))), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))) +
                    0.099844*np.tanh(((((((data["CNT_INSTALMENT"]) * (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["HOUR_APPR_PROCESS_START_y"]))))) * (data["HOUR_APPR_PROCESS_START_y"]))) * ((((3.0) > (-3.0))*1.)))) +
                    0.095662*np.tanh(np.minimum((((((data["NAME_GOODS_CATEGORY_Insurance"]) > (data["te_HOUR_APPR_PROCESS_START_x"]))*1.))), (((((((data["te_HOUR_APPR_PROCESS_START_x"]) + (np.tanh((data["cnt_REGION_RATING_CLIENT"]))))/2.0)) / 2.0))))) +
                    0.099179*np.tanh(np.where(np.tanh((data["te_FLAG_DOCUMENT_17"]))>0, (0.04655243083834648), (-1.0*(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) < ((-1.0*((((((-1.0*((-3.0)))) + (((data["te_FLAG_DOCUMENT_17"]) - (data["cnt_ORGANIZATION_TYPE"]))))/2.0))))))*1.)))) )) +
                    0.099609*np.tanh((-1.0*(((((3.0) < (((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) + ((-1.0*((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["te_FLAG_CONT_MOBILE"]))) * (data["te_FLAG_CONT_MOBILE"])))))))))*1.))))) +
                    0.099922*np.tanh(np.minimum(((0.0)), ((((data["FLAG_DOCUMENT_3"]) + (np.minimum(((((((1.0) - (data["te_LIVE_REGION_NOT_WORK_REGION"]))) + (3.0)))), ((((data["te_LIVE_REGION_NOT_WORK_REGION"]) - (((-2.0) * 2.0)))))))))))) +
                    0.099648*np.tanh((((np.maximum(((data["te_REG_REGION_NOT_WORK_REGION"])), (((((np.tanh((2.0))) + ((((3.0) + (data["te_FLAG_DOCUMENT_20"]))/2.0)))/2.0))))) < (data["REG_REGION_NOT_WORK_REGION"]))*1.)) +
                    0.099687*np.tanh(((((14.74264430999755859)) < (((data["te_FLAG_EMAIL"]) * (np.minimum((((14.74264430999755859))), ((np.minimum((((5.0))), ((((data["te_FLAG_EMAIL"]) * (np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, (14.74264430999755859), 0.0 )))))))))))))*1.)) +
                    0.098398*np.tanh(np.where(data["te_FLAG_DOCUMENT_20"]>0, ((((4.54344129562377930)) < ((-1.0*((data["te_FLAG_EMAIL"])))))*1.), (-1.0*((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))) )) +
                    0.098750*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["FLAG_EMAIL"], ((((((data["FLAG_EMAIL"]) * 2.0)) * 2.0)) * (data["NAME_GOODS_CATEGORY_Direct_Sales"])) )) +
                    0.099648*np.tanh((((((data["te_FLAG_CONT_MOBILE"]) + (((data["te_FLAG_EMAIL"]) * (np.where(data["te_FLAG_EMAIL"]>0, -1.0, ((2.0) * 2.0) )))))/2.0)) * (data["NAME_GOODS_CATEGORY_Direct_Sales"]))) +
                    0.098710*np.tanh((-1.0*((((((data["NAME_SELLER_INDUSTRY_Tourism"]) - (((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (((np.tanh((((((((14.94352722167968750)) < (data["NAME_GOODS_CATEGORY_Direct_Sales"]))*1.)) / 2.0)))) - (data["cnt_FLAG_DOCUMENT_10"]))))) / 2.0)))) / 2.0))))) +
                    0.078429*np.tanh(((np.maximum((((((data["PRODUCT_COMBINATION_POS_household_without_interest"]) > ((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"])))))*1.))), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) / 2.0))))) * (((data["te_FLAG_DOCUMENT_20"]) * (data["te_FLAG_DOCUMENT_20"]))))) +
                    0.099726*np.tanh((((((((((((data["te_REG_REGION_NOT_LIVE_REGION"]) * (data["NAME_GOODS_CATEGORY_Insurance"]))) * (data["te_REG_REGION_NOT_LIVE_REGION"]))) / 2.0)) < (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)) * ((((data["CNT_INSTALMENT"]) < (-1.0))*1.)))) +
                    0.099258*np.tanh(((((data["te_FLAG_DOCUMENT_12"]) * (((0.0) - ((((((data["te_FLAG_DOCUMENT_7"]) - (data["te_FLAG_DOCUMENT_17"]))) < (((((data["NAME_GOODS_CATEGORY_Insurance"]) / 2.0)) / 2.0)))*1.)))))) * (data["te_REG_REGION_NOT_LIVE_REGION"]))) +
                    0.097538*np.tanh(np.where(data["NAME_TYPE_SUITE_Other_A"]>0, data["CNT_INSTALMENT"], np.tanh((np.where(data["CNT_INSTALMENT"]>0, np.maximum(((-3.0)), ((data["NAME_TYPE_SUITE_Other_A"]))), data["NAME_GOODS_CATEGORY_Fitness"] ))) )) +
                    0.099062*np.tanh(np.tanh((((np.where(data["CNT_INSTALMENT_FUTURE"]>0, data["NAME_GOODS_CATEGORY_Insurance"], ((data["inst_AMT_INSTALMENT"]) * (np.maximum(((data["FLAG_DOCUMENT_2"])), ((np.maximum(((data["inst_AMT_INSTALMENT"])), ((data["NAME_GOODS_CATEGORY_Fitness"])))))))) )) / 2.0)))) +
                    0.099297*np.tanh(np.where(np.where(data["NAME_TYPE_SUITE_Other_A"]>0, ((data["NAME_TYPE_SUITE_Other_A"]) - ((5.69756412506103516))), data["FLAG_DOCUMENT_2"] )>0, (10.03394317626953125), ((((((((data["inst_AMT_PAYMENT"]) * 2.0)) + ((11.95959758758544922)))/2.0)) > ((10.03394317626953125)))*1.) )) +
                    0.099414*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Direct_Sales"]>0, data["inst_AMT_PAYMENT"], ((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * ((3.0)))) * 2.0)) * (data["inst_AMT_PAYMENT"]))) + (0.0)) )) +
                    0.099922*np.tanh(np.minimum(((((data["CNT_INSTALMENT_FUTURE"]) * (np.tanh((np.maximum(((data["AMT_CREDIT_y"])), ((((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]) + (data["AMT_CREDIT_y"]))))))))))), (((((((data["CNT_INSTALMENT_FUTURE"]) * (data["AMT_CREDIT_y"]))) > ((4.0)))*1.))))) +
                    0.098945*np.tanh(((((-1.0*((data["NAME_PORTFOLIO_XNA"])))) > (((3.0) + (((data["NAME_PORTFOLIO_XNA"]) - (data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]))))))*1.)) +
                    0.046698*np.tanh(((np.tanh((np.where(((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["PRODUCT_COMBINATION_Cash"]))>0, np.tanh(((((((-1.0*((data["NAME_YIELD_GROUP_XNA"])))) * 2.0)) * (data["inst_AMT_PAYMENT"])))), data["NAME_GOODS_CATEGORY_Insurance"] )))) * 2.0)) +
                    0.091403*np.tanh(((np.minimum((((((data["inst_AMT_INSTALMENT"]) > (data["NAME_GOODS_CATEGORY_Insurance"]))*1.))), ((data["inst_AMT_INSTALMENT"])))) * ((((0.0) + ((((data["NAME_YIELD_GROUP_middle"]) < (1.0))*1.)))/2.0)))) +
                    0.099726*np.tanh(((((np.minimum(((0.0)), (((((((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((data["AMT_ANNUITY"])))) < ((3.0)))*1.)) - (1.0)))))) * (((data["AMT_GOODS_PRICE_y"]) / 2.0)))) * 2.0)) +
                    0.099844*np.tanh(((((((((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]) > (-1.0))*1.)) < (((data["NAME_SELLER_INDUSTRY_Connectivity"]) * (((((((data["CNT_INSTALMENT"]) < (data["NAME_YIELD_GROUP_middle"]))*1.)) + ((((data["inst_AMT_PAYMENT"]) > (0.0))*1.)))/2.0)))))*1.)) / 2.0)) +
                    0.099766*np.tanh(((data["CNT_INSTALMENT"]) * (((np.minimum(((((data["NAME_PORTFOLIO_Cash"]) * (data["inst_AMT_PAYMENT"])))), ((((((((-2.0) * 2.0)) * (data["NAME_GOODS_CATEGORY_Insurance"]))) / 2.0))))) + (data["NAME_GOODS_CATEGORY_Direct_Sales"]))))) +
                    0.098945*np.tanh((-1.0*((((((((0.0) > (((data["NAME_YIELD_GROUP_XNA"]) * 2.0)))*1.)) < (((-2.0) + ((-1.0*((((data["Completed"]) * ((-1.0*((data["inst_AMT_PAYMENT"]))))))))))))*1.))))) +
                    0.098202*np.tanh((((((np.maximum(((data["CNT_INSTALMENT_FUTURE"])), ((data["CNT_INSTALMENT_FUTURE"])))) > (((((((1.0) / 2.0)) + (2.0))) + (data["NAME_YIELD_GROUP_middle"]))))*1.)) / 2.0)) +
                    0.099297*np.tanh((((((data["NAME_YIELD_GROUP_middle"]) + ((2.0)))/2.0)) * (np.where(data["NAME_YIELD_GROUP_middle"]>0, np.tanh((data["SK_DPD"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (((np.tanh(((-1.0*((data["inst_AMT_PAYMENT"])))))) / 2.0))) )))) +
                    0.100000*np.tanh(((data["NAME_GOODS_CATEGORY_Insurance"]) - (np.where(-3.0>0, data["NAME_GOODS_CATEGORY_Fitness"], ((((2.52537202835083008)) < (np.where(data["DAYS_FIRST_DUE"]>0, data["CNT_INSTALMENT_FUTURE"], data["FLAG_DOCUMENT_4"] )))*1.) )))) +
                    0.099687*np.tanh((((np.where(data["inst_AMT_INSTALMENT"]>0, data["inst_AMT_INSTALMENT"], data["NAME_GOODS_CATEGORY_Medicine"] )) > ((((np.tanh((data["NAME_GOODS_CATEGORY_Medicine"]))) + ((((4.78901720046997070)) - (1.0))))/2.0)))*1.)) +
                    0.098828*np.tanh(((((((((((((data["DAYS_FIRST_DUE"]) > ((((data["NAME_GOODS_CATEGORY_Education"]) < (((data["PRODUCT_COMBINATION_Cash_Street__low"]) * (3.0))))*1.)))*1.)) * (data["te_WEEKDAY_APPR_PROCESS_START"]))) / 2.0)) > (0.0))*1.)) * (data["te_WEEKDAY_APPR_PROCESS_START"]))) +
                    0.091989*np.tanh(((((data["PRODUCT_COMBINATION_POS_household_with_interest"]) * ((-1.0*((((data["PRODUCT_COMBINATION_Cash_Street__low"]) * ((((data["te_FLAG_DOCUMENT_20"]) + ((((((data["PRODUCT_COMBINATION_Cash_Street__low"]) < ((-1.0*((data["cnt_FLAG_DOCUMENT_10"])))))*1.)) / 2.0)))/2.0))))))))) * 2.0)) +
                    0.081516*np.tanh(np.minimum(((0.0)), ((((np.where(data["NAME_PORTFOLIO_XNA"]>0, 3.0, (-1.0*((data["NAME_GOODS_CATEGORY_Fitness"]))) )) - (((1.0) * (data["inst_AMT_PAYMENT"])))))))) +
                    0.097773*np.tanh((((((np.where(data["inst_AMT_INSTALMENT"]>0, data["te_FLAG_DOCUMENT_20"], (-1.0*(((((data["AMT_ANNUITY"]) > (np.tanh((data["CNT_INSTALMENT_FUTURE"]))))*1.)))) )) + (data["inst_AMT_INSTALMENT"]))/2.0)) / 2.0)) +
                    0.099883*np.tanh((((np.minimum((((((((-1.0*((np.minimum(((0.0)), ((((data["AMT_GOODS_PRICE_y"]) - (-1.0))))))))) + (data["inst_AMT_INSTALMENT"]))) / 2.0))), ((data["inst_AMT_INSTALMENT"])))) > (data["inst_AMT_PAYMENT"]))*1.)) +
                    0.099961*np.tanh(np.minimum(((data["PRODUCT_COMBINATION_Cash_Street__low"])), (((((-1.0*(((((2.0) < (np.where(data["PRODUCT_COMBINATION_Cash_Street__low"]>0, ((data["PRODUCT_COMBINATION_Cash_Street__low"]) / 2.0), np.tanh((-2.0)) )))*1.))))) * 2.0))))) +
                    0.098906*np.tanh((((((data["te_FLAG_DOCUMENT_20"]) - (data["inst_AMT_PAYMENT"]))) > (((((np.minimum(((np.tanh((((data["te_REG_REGION_NOT_LIVE_REGION"]) / 2.0))))), ((data["te_FLAG_DOCUMENT_20"])))) + ((2.18707370758056641)))) / 2.0)))*1.)) +
                    0.097890*np.tanh((((((((0.0) + ((-1.0*((data["te_FLAG_CONT_MOBILE"])))))/2.0)) * ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) > ((-1.0*(((((5.0)) * (data["te_SK_ID_CURR"])))))))*1.)))) / 2.0)) +
                    0.075615*np.tanh((((((data["SK_ID_PREV_y"]) < (np.tanh(((((-1.0*(((((3.0) < (-2.0))*1.))))) - (np.tanh((((3.0) / 2.0)))))))))*1.)) / 2.0)) +
                    0.099883*np.tanh(((((((data["NAME_CLIENT_TYPE_Repeater"]) * (np.maximum(((data["AMT_CREDIT_x"])), (((((data["NAME_YIELD_GROUP_middle"]) > (2.0))*1.))))))) / 2.0)) / 2.0)) +
                    0.099726*np.tanh(((data["te_FLAG_CONT_MOBILE"]) * ((((((data["avg_buro_buro_bal_status_2"]) - (np.tanh((((data["inst_AMT_PAYMENT"]) * (((((((0.0) < (-3.0))*1.)) + (data["te_FLAG_CONT_MOBILE"]))/2.0)))))))) + (data["avg_buro_buro_bal_status_2"]))/2.0)))) +
                    0.086596*np.tanh(((data["AMT_ANNUITY"]) * ((((-1.0) > (((np.maximum(((-1.0)), ((2.0)))) + (((((data["AMT_CREDIT_x"]) + (data["AMT_APPLICATION"]))) * 2.0)))))*1.)))) +
                    0.097694*np.tanh(((((np.where(((data["AMT_ANNUITY_x"]) * (data["inst_NUM_INSTALMENT_VERSION"]))>0, data["NAME_CLIENT_TYPE_New"], np.tanh((data["inst_NUM_INSTALMENT_VERSION"])) )) / 2.0)) / 2.0)) +
                    0.098554*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (np.maximum(((data["NAME_PORTFOLIO_POS"])), ((((np.minimum(((((((0.0)) > (((data["PRODUCT_COMBINATION_Cash"]) * 2.0)))*1.))), ((0.0)))) - (data["te_FLAG_DOCUMENT_20"])))))))) * 2.0)) +
                    0.099492*np.tanh(((((data["avg_buro_buro_bal_status_2"]) * (data["te_FLAG_DOCUMENT_20"]))) * ((((data["AMT_CREDIT_y"]) < (((3.0) / 2.0)))*1.)))) +
                    0.075772*np.tanh((-1.0*((((np.minimum(((np.tanh(((-1.0*((data["avg_buro_buro_bal_status_1"]))))))), ((data["avg_buro_buro_bal_status_4"])))) * (((data["CNT_INSTALMENT_FUTURE"]) * 2.0))))))) +
                    0.087769*np.tanh(((np.minimum(((((data["cc_bal_SK_DPD"]) * (data["MONTHS_BALANCE"])))), ((data["cnt_NAME_INCOME_TYPE"])))) * (((data["MONTHS_BALANCE"]) / 2.0)))) +
                    0.100000*np.tanh(((((((((data["inst_DAYS_INSTALMENT"]) - ((((data["inst_DAYS_INSTALMENT"]) > (np.where(data["avg_buro_buro_bal_status_2"]>0, data["AMT_CREDIT_y"], (4.0) )))*1.)))) / 2.0)) / 2.0)) / 2.0)) +
                    0.099844*np.tanh(((((-3.0) * (data["avg_buro_buro_bal_status_4"]))) * (((-2.0) * ((-1.0*((((((-1.0*((data["avg_buro_buro_bal_status_2"])))) + (np.maximum(((data["PRODUCT_COMBINATION_Cash"])), ((np.tanh((0.0)))))))/2.0))))))))) +
                    0.099844*np.tanh((((np.where(data["avg_buro_buro_bal_status_4"]>0, (-1.0*((data["AMT_ANNUITY_x"]))), np.tanh((data["avg_buro_buro_bal_status_4"])) )) + (np.tanh((data["AMT_ANNUITY_x"]))))/2.0)) +
                    0.099766*np.tanh(((np.where(data["avg_buro_buro_bal_status_3"]>0, data["avg_buro_buro_bal_status_4"], (((data["nans"]) > (np.maximum(((0.0)), (((-1.0*((data["AMT_ANNUITY_x"]))))))))*1.) )) * (data["AMT_ANNUITY_x"]))) +
                    0.099766*np.tanh(np.minimum(((((data["AMT_ANNUITY_x"]) * (data["AMT_GOODS_PRICE_x"])))), ((((data["avg_buro_buro_bal_status_2"]) * (((((3.0) * (data["AMT_GOODS_PRICE_x"]))) - ((-1.0*((data["CNT_INSTALMENT"]))))))))))) +
                    0.099805*np.tanh(((np.minimum(((((((((data["AMT_CREDIT_x"]) + (-3.0))/2.0)) > (data["avg_buro_buro_bal_status_4"]))*1.))), ((((data["AMT_CREDIT_x"]) + (1.0)))))) * 2.0)) +
                    0.096835*np.tanh((-1.0*((((np.minimum(((((data["AMT_ANNUITY_x"]) - (((-1.0) + (((3.0) * ((((2.0) < (data["avg_buro_buro_bal_status_2"]))*1.))))))))), (((0.0))))) * 2.0))))) +
                    0.087182*np.tanh(np.minimum(((data["avg_buro_buro_bal_status_4"])), ((np.minimum((((((-1.0*((((np.where(((data["cnt_FLAG_DOCUMENT_3"]) * 2.0)>0, (3.0), -2.0 )) * 2.0))))) * (data["avg_buro_buro_bal_status_2"])))), (((-1.0*((data["avg_buro_buro_bal_status_4"])))))))))) +
                    0.099844*np.tanh(((-3.0) * ((((((data["AMT_CREDIT_x"]) * (-3.0))) > (np.maximum(((1.0)), ((((data["avg_buro_buro_bal_status_2"]) - ((((data["AMT_CREDIT_x"]) + (((-3.0) * 2.0)))/2.0))))))))*1.)))) +
                    0.099961*np.tanh((((data["te_FLAG_DOCUMENT_8"]) < (((np.where(data["avg_buro_buro_bal_status_2"]>0, -1.0, (-1.0*((data["te_FLAG_DOCUMENT_8"]))) )) + (-3.0))))*1.)) +
                    0.098828*np.tanh((((((data["avg_buro_buro_bal_status_2"]) > (np.where(-1.0>0, 0.0, np.minimum(((2.0)), ((data["avg_buro_buro_bal_status_2"]))) )))*1.)) * (((2.0) * (data["te_FLAG_DOCUMENT_20"]))))) +
                    0.061704*np.tanh(np.where(data["FLAG_DOCUMENT_8"]>0, data["te_FLAG_DOCUMENT_20"], ((np.where(data["avg_buro_buro_bal_status_4"]>0, data["FLAG_DOCUMENT_8"], 0.0 )) * (data["te_FLAG_DOCUMENT_20"])) )) +
                    0.069637*np.tanh(((((np.maximum(((data["te_FLAG_DOCUMENT_7"])), ((0.0)))) * (((data["te_FLAG_DOCUMENT_3"]) / 2.0)))) / 2.0)) +
                    0.083587*np.tanh(((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["FLAG_DOCUMENT_6"], np.minimum((((7.0))), ((((data["te_FLAG_MOBIL"]) * ((((0.0) < (data["FLAG_DOCUMENT_6"]))*1.)))))) )) / 2.0)) +
                    0.099805*np.tanh(((np.minimum(((data["HOUR_APPR_PROCESS_START_y"])), (((-1.0*(((((3.0) < (((((((data["AMT_ANNUITY_x"]) < (data["avg_buro_buro_bal_status_2"]))*1.)) + (data["avg_buro_buro_bal_status_2"]))/2.0)))*1.)))))))) * (data["avg_buro_buro_bal_status_2"]))) +
                    0.099766*np.tanh(np.where(np.minimum((((-1.0*((data["avg_buro_buro_bal_status_4"]))))), (((((np.tanh((data["NAME_GOODS_CATEGORY_Construction_Materials"]))) < (((data["NAME_GOODS_CATEGORY_Construction_Materials"]) * 2.0)))*1.))))>0, data["cnt_FLAG_DOCUMENT_10"], (((data["NAME_GOODS_CATEGORY_Construction_Materials"]) > (data["avg_buro_buro_bal_status_4"]))*1.) )) +
                    0.099961*np.tanh(np.where(((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0)>0, np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["cnt_NAME_TYPE_SUITE"], ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0) ), ((np.tanh((((data["NAME_SELLER_INDUSTRY_Construction"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))) * (data["NAME_SELLER_INDUSTRY_Construction"])) )) +
                    0.099336*np.tanh(np.minimum(((np.minimum(((np.where(data["NAME_GOODS_CATEGORY_Weapon"]>0, -3.0, data["avg_buro_buro_bal_status_4"] ))), ((data["cnt_FLAG_DOCUMENT_10"]))))), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((data["FLAG_DOCUMENT_4"]) * 2.0))))))) +
                    0.099922*np.tanh(np.minimum((((-1.0*((((data["FLAG_DOCUMENT_15"]) - (np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, ((data["FLAG_DOCUMENT_15"]) * 2.0), data["cnt_FLAG_DOCUMENT_10"] )))))))), ((((data["FLAG_DOCUMENT_15"]) * (data["AMT_ANNUITY_x"])))))) +
                    0.099375*np.tanh((-1.0*(((((((data["AMT_CREDIT_y"]) > (3.0))*1.)) * (np.where(data["avg_buro_buro_bal_status_4"]>0, np.minimum(((-3.0)), ((data["AMT_CREDIT_y"]))), 2.0 ))))))) +
                    0.059203*np.tanh(((data["te_FLAG_DOCUMENT_3"]) * ((((data["inst_AMT_PAYMENT"]) > ((((-3.0) < (np.tanh((np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))))*1.)))*1.)))) +
                    0.098124*np.tanh((((((data["te_FLAG_DOCUMENT_6"]) - ((((data["Amortized_debt"]) < (np.maximum(((((data["inst_AMT_PAYMENT"]) * (((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * 2.0))))), ((data["avg_buro_buro_bal_status_4"])))))*1.)))) > (data["inst_AMT_PAYMENT"]))*1.)) +
                    0.099101*np.tanh(np.tanh(((((((np.tanh((np.tanh((((data["cnt_FLAG_DOCUMENT_6"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))))) * 2.0)) < (np.minimum(((data["CHANNEL_TYPE_Stone"])), ((data["FLAG_DOCUMENT_6"])))))*1.)))) +
                    0.100000*np.tanh(((data["avg_buro_buro_bal_status_4"]) * (np.where(data["CNT_INSTALMENT"]>0, ((-2.0) - (data["LIVINGAPARTMENTS_MEDI"])), (((2.0) < (data["avg_buro_buro_bal_status_4"]))*1.) )))) +
                    0.089332*np.tanh(np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), (((((data["NAME_PAYMENT_TYPE_Cash_through_the_bank"]) + (np.maximum(((((data["inst_AMT_PAYMENT"]) * (((-3.0) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))), ((data["RATE_INTEREST_PRIVILEGED"])))))/2.0))))) +
                    0.099648*np.tanh(np.where(data["avg_buro_buro_bal_status_4"]>0, ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["NAME_TYPE_SUITE_Unaccompanied"])), np.tanh((((np.where(data["NAME_PORTFOLIO_XNA"]>0, data["NAME_TYPE_SUITE_Unaccompanied"], 0.0 )) * (data["cnt_NAME_INCOME_TYPE"])))) )) +
                    0.083079*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["te_FLAG_DOCUMENT_3"], ((((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((data["avg_buro_buro_bal_status_4"]) / 2.0)))) - (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]))) * 2.0)) * (((data["avg_buro_buro_bal_status_4"]) + (data["avg_buro_buro_bal_status_4"])))) )) +
                    0.076671*np.tanh((((((1.0) * 2.0)) < (((((np.where(((data["NAME_PAYMENT_TYPE_XNA"]) * ((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))) * 2.0)))>0, data["NAME_YIELD_GROUP_XNA"], data["cnt_FLAG_DOCUMENT_17"] )) * (3.0))) / 2.0)))*1.)) +
                    0.099297*np.tanh(((data["CHANNEL_TYPE_Credit_and_cash_offices"]) * ((((((((data["NAME_CONTRACT_STATUS_Approved"]) - ((-1.0*((np.tanh((np.maximum(((data["NAME_YIELD_GROUP_XNA"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))))))))) + (data["cc_bal_SK_ID_PREV"]))/2.0)) / 2.0)))) +
                    0.099961*np.tanh(((np.where(data["NAME_CASH_LOAN_PURPOSE_XNA"]>0, 2.0, data["CHANNEL_TYPE_Credit_and_cash_offices"] )) * (np.where((((np.tanh((data["NAME_CONTRACT_STATUS_Unused_offer"]))) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))/2.0)>0, data["NAME_PORTFOLIO_POS"], (-1.0*((0.0))) )))) +
                    0.051700*np.tanh(np.tanh(((((data["cnt_NAME_INCOME_TYPE"]) > (((((1.25729477405548096)) + (((((((data["NAME_SELLER_INDUSTRY_XNA"]) - (np.maximum(((data["CHANNEL_TYPE_Stone"])), ((0.0)))))) * 2.0)) + ((1.98952722549438477)))))/2.0)))*1.)))) +
                    0.100000*np.tanh((((((-1.0*((data["cnt_EMERGENCYSTATE_MODE"])))) * (np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["NAME_INCOME_TYPE"], (-1.0*((data["cnt_FLAG_DOCUMENT_17"]))) )))) * 2.0)) +
                    0.099726*np.tanh((((((np.where(data["avg_buro_buro_bal_status_4"]>0, data["AMT_REQ_CREDIT_BUREAU_QRT"], data["NAME_CASH_LOAN_PURPOSE_Business_development"] )) > ((((np.maximum(((-1.0)), ((0.0)))) > (-3.0))*1.)))*1.)) - (data["CHANNEL_TYPE_Car_dealer"]))) +
                    0.092927*np.tanh(np.minimum(((((np.where(data["te_WALLSMATERIAL_MODE"]>0, data["EXT_SOURCE_1"], ((data["EXT_SOURCE_1"]) * (data["NAME_INCOME_TYPE"])) )) * 2.0))), ((np.tanh((((data["EXT_SOURCE_1"]) * (data["NAME_INCOME_TYPE"])))))))) +
                    0.100000*np.tanh(np.tanh((np.where(np.tanh((((data["EXT_SOURCE_1"]) * (((data["EXT_SOURCE_1"]) / 2.0)))))>0, (((data["DAYS_BIRTH"]) < ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) > (3.0))*1.)))*1.), data["DAYS_BIRTH"] )))) +
                    0.099961*np.tanh((-1.0*(((((data["cnt_FLAG_DOCUMENT_6"]) > ((((data["DAYS_BIRTH"]) < ((((2.54294586181640625)) / 2.0)))*1.)))*1.))))) +
                    0.099961*np.tanh(((data["EXT_SOURCE_1"]) * ((-1.0*((np.tanh((((np.tanh(((((data["CNT_FAM_MEMBERS"]) > (np.tanh((data["EXT_SOURCE_1"]))))*1.)))) + (((data["cnt_DAYS_BIRTH"]) / 2.0))))))))))) +
                    0.099961*np.tanh(((np.tanh(((((data["cnt_DAYS_BIRTH"]) + (data["EXT_SOURCE_1"]))/2.0)))) * (((((-1.0*((data["DAYS_EMPLOYED"])))) + ((-1.0*((((2.0) / 2.0))))))/2.0)))) +
                    0.099805*np.tanh((((((((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (data["CNT_FAM_MEMBERS"]))/2.0)) * (data["cnt_CODE_GENDER"]))) - ((((data["CNT_CHILDREN"]) > ((-1.0*((-2.0)))))*1.)))) / 2.0)) / 2.0)) +
                    0.099883*np.tanh((((data["DAYS_ID_PUBLISH"]) > (((2.0) - (np.tanh((np.tanh(((((data["cnt_CODE_GENDER"]) + ((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))) * 2.0)))/2.0)))))))))*1.)) +
                    0.089175*np.tanh(((((((((data["avg_buro_buro_bal_status_4"]) + ((((((np.where(data["avg_buro_buro_bal_status_4"]>0, data["WALLSMATERIAL_MODE"], data["WALLSMATERIAL_MODE"] )) + (((data["HOUSETYPE_MODE"]) / 2.0)))/2.0)) * 2.0)))) * 2.0)) * (data["avg_buro_buro_bal_status_4"]))) * 2.0)) +
                    0.095975*np.tanh((((((2.0) < ((((((3.0) * (data["DAYS_REGISTRATION"]))) + (np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((data["CODE_REJECT_REASON_SCOFR"]) * (data["avg_buro_buro_bal_status_4"])))))))/2.0)))*1.)) * 2.0)) +
                    0.094724*np.tanh(np.minimum(((((((np.minimum(((0.0)), ((((np.minimum(((((-2.0) / 2.0))), ((data["NAME_GOODS_CATEGORY_Insurance"])))) / 2.0))))) / 2.0)) * (data["DAYS_REGISTRATION"])))), (((((data["DAYS_REGISTRATION"]) < (-2.0))*1.))))) +
                    0.084408*np.tanh(np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, (-1.0*((data["cnt_DAYS_BIRTH"]))), np.where((-1.0*((data["avg_buro_buro_bal_status_2"])))>0, data["cnt_DAYS_BIRTH"], (-1.0*(((((-1.0) > ((6.27440023422241211)))*1.)))) ) )) +
                    0.099609*np.tanh(((np.maximum(((np.maximum(((data["avg_buro_buro_bal_status_4"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))), ((((data["avg_buro_buro_bal_status_4"]) * (data["inst_DAYS_INSTALMENT"])))))) * (((data["inst_DAYS_INSTALMENT"]) - (1.0))))) +
                    0.096366*np.tanh(((((data["ELEVATORS_MODE"]) * (((((((data["NAME_GOODS_CATEGORY_Insurance"]) - ((-1.0*((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (data["NAME_INCOME_TYPE"])))))))) - (data["CNT_FAM_MEMBERS"]))) * (data["ELEVATORS_MODE"]))))) / 2.0)) +
                    0.087769*np.tanh(((np.where(3.0>0, data["FLAG_WORK_PHONE"], 3.0 )) * ((((data["FLAG_WORK_PHONE"]) < (((data["CNT_CHILDREN"]) - (((1.0) * 2.0)))))*1.)))) +
                    0.079719*np.tanh(((((((-1.0*((((data["cnt_NAME_INCOME_TYPE"]) / 2.0))))) < ((-1.0*((((((((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))) + (2.0))/2.0)) + (data["cnt_CNT_CHILDREN"])))))))*1.)) / 2.0)) +
                    0.099883*np.tanh(((np.where(np.where(data["cc_bal_SK_DPD"]>0, data["avg_buro_buro_bal_status_4"], data["AMT_DOWN_PAYMENT"] )>0, data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"], ((data["RATE_INTEREST_PRIVILEGED"]) * 2.0) )) * 2.0)) +
                    0.062329*np.tanh(np.where(data["AMT_DOWN_PAYMENT"]>0, np.minimum((((-1.0*(((-1.0*((data["DAYS_LAST_DUE_1ST_VERSION"])))))))), ((data["cc_bal_SK_DPD"]))), ((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]) * (((((data["cc_bal_SK_DPD"]) * 2.0)) * 2.0))) )) +
                    0.099883*np.tanh(((((data["RATE_DOWN_PAYMENT"]) * (np.where(data["cc_bal_SK_DPD_DEF"]>0, 1.0, data["NAME_CASH_LOAN_PURPOSE_Business_development"] )))) * (-3.0))) +
                    0.098867*np.tanh(np.tanh((((data["inst_NUM_INSTALMENT_VERSION"]) * ((((data["cc_bal_SK_DPD"]) > ((((((((((-1.0*((data["cc_bal_SK_DPD_DEF"])))) / 2.0)) > (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))*1.)) + (data["inst_NUM_INSTALMENT_VERSION"]))/2.0)))*1.)))))) +
                    0.068386*np.tanh(np.tanh(((-1.0*((np.where(data["DAYS_FIRST_DRAWING"]>0, data["cc_bal_MONTHS_BALANCE"], np.tanh(((((data["cc_bal_SK_DPD"]) + (-1.0))/2.0))) ))))))) +
                    0.099766*np.tanh((-1.0*((np.where(data["cc_bal_AMT_INST_MIN_REGULARITY"]>0, ((((data["avg_buro_buro_bal_status_4"]) * 2.0)) * (((((data["avg_buro_buro_bal_status_4"]) * 2.0)) * (data["SK_ID_PREV_y"])))), ((((data["avg_buro_buro_bal_status_4"]) * 2.0)) * (data["SK_ID_PREV_y"])) ))))) +
                    0.088159*np.tanh((-1.0*((((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) * 2.0)) * (np.where(data["AMT_DOWN_PAYMENT"]>0, data["AMT_DOWN_PAYMENT"], (((-1.0*((((data["AMT_DOWN_PAYMENT"]) * 2.0))))) * ((-1.0*((((data["CNT_INSTALMENT"]) * 2.0)))))) ))))))) +
                    0.097499*np.tanh(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * (np.where(data["avg_buro_buro_bal_status_4"]>0, data["CNT_INSTALMENT"], ((data["AMT_DOWN_PAYMENT"]) - ((-1.0*((np.tanh((0.0))))))) )))) +
                    0.100000*np.tanh((-1.0*((((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) * ((((data["inst_SK_ID_PREV"]) + (((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) * (((data["inst_SK_ID_PREV"]) - ((-1.0*((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"])))))))))/2.0))))))) +
                    0.083783*np.tanh(((((((data["inst_AMT_INSTALMENT"]) > (2.0))*1.)) < (((np.minimum(((data["inst_AMT_INSTALMENT"])), ((data["avg_buro_buro_bal_status_4"])))) + (((data["cc_bal_SK_DPD"]) * (data["AMT_DOWN_PAYMENT"]))))))*1.)) +
                    0.096913*np.tanh(np.where(np.minimum((((3.24788403511047363))), ((((np.minimum(((((-1.0) / 2.0))), ((data["cc_bal_SK_DPD"])))) + (np.tanh((data["SK_ID_PREV_y"])))))))>0, data["NAME_PRODUCT_TYPE_XNA"], data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"] )) +
                    0.098828*np.tanh(((data["cc_bal_SK_DPD"]) * (((data["AMT_CREDIT_y"]) * (((((((-1.0*(((3.0))))) + (((data["cc_bal_SK_DPD"]) * 2.0)))) + ((((-1.0*(((3.0))))) + (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]))))/2.0)))))) +
                    0.099609*np.tanh(((((data["avg_buro_buro_bal_status_2"]) * (((data["avg_buro_buro_bal_status_2"]) * 2.0)))) * (np.where(data["inst_AMT_PAYMENT"]>0, data["NAME_CASH_LOAN_PURPOSE_Business_development"], (((data["avg_buro_buro_bal_status_2"]) > (2.0))*1.) )))) +
                    0.099844*np.tanh(((data["avg_buro_buro_bal_status_2"]) * (((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]) + (((data["avg_buro_buro_bal_status_3"]) * ((-1.0*((data["cnt_CNT_CHILDREN"])))))))))) +
                    0.095819*np.tanh(((data["avg_buro_buro_bal_status_4"]) * (((np.where(data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]>0, ((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]) * 2.0), -2.0 )) + (((-3.0) * (data["cc_bal_SK_DPD"]))))))) +
                    0.099844*np.tanh(((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) * (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"])), ((np.minimum((((((((2.0) - (((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) * 2.0)))) + (data["FLAG_DOCUMENT_17"]))/2.0))), ((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]))))))))) * 2.0)) +
                    0.099961*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["inst_AMT_PAYMENT"], ((np.minimum(((data["inst_AMT_PAYMENT"])), ((np.maximum(((data["avg_buro_buro_bal_status_2"])), ((0.0))))))) * (((((data["avg_buro_buro_bal_status_5"]) / 2.0)) * 2.0))) )) +
                    0.099805*np.tanh(((data["NAME_CONTRACT_STATUS_Approved"]) * (((((data["avg_buro_buro_bal_status_5"]) - (np.maximum(((data["avg_buro_buro_bal_status_5"])), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) - (((data["avg_buro_buro_bal_status_5"]) - (data["cc_bal_SK_DPD"])))))))))) * (-1.0))))) +
                    0.098398*np.tanh((((((data["cnt_CNT_CHILDREN"]) * ((((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]) + (((data["cnt_CNT_CHILDREN"]) * (data["cc_bal_SK_DPD"]))))/2.0)))) + (((((((data["cnt_FLAG_DOCUMENT_17"]) / 2.0)) * 2.0)) / 2.0)))/2.0)) +
                    0.088081*np.tanh(np.where(((data["cc_bal_AMT_PAYMENT_CURRENT"]) / 2.0)>0, data["AMT_CREDIT_SUM_LIMIT"], (((-1.0*((data["AMT_CREDIT_SUM_LIMIT"])))) * (data["AMT_CREDIT_SUM_LIMIT"])) )) +
                    0.097421*np.tanh(((((data["NAME_PRODUCT_TYPE_x_sell"]) * (((data["NAME_PRODUCT_TYPE_x_sell"]) * (((np.minimum(((data["cnt_FLAG_DOCUMENT_10"])), ((data["avg_buro_buro_bal_status_5"])))) * (3.0))))))) - (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]))) +
                    0.099805*np.tanh(((((-1.0*((data["NAME_CONTRACT_TYPE_Consumer_loans"])))) < (np.where((((-2.0) > (data["cnt_CNT_CHILDREN"]))*1.)>0, ((np.where(data["NAME_PRODUCT_TYPE_XNA"]>0, data["avg_buro_buro_bal_status_5"], data["NAME_CONTRACT_TYPE_Consumer_loans"] )) / 2.0), data["NAME_GOODS_CATEGORY_XNA"] )))*1.)) +
                    0.098828*np.tanh(((np.maximum((((((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]) < (np.where(np.tanh((data["NAME_CONTRACT_TYPE_Consumer_loans"]))>0, data["NAME_PRODUCT_TYPE_x_sell"], data["PRODUCT_COMBINATION_Cash_X_Sell__middle"] )))*1.))), ((((-1.0) - (data["CNT_FAM_MEMBERS"])))))) / 2.0)) +
                    0.099922*np.tanh(np.where(((data["avg_buro_buro_bal_status_5"]) * 2.0)>0, data["NAME_CASH_LOAN_PURPOSE_XAP"], ((np.where(data["NAME_CASH_LOAN_PURPOSE_XAP"]>0, data["avg_buro_buro_bal_status_5"], (((np.tanh((0.0))) > (((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]) + (data["NAME_CONTRACT_TYPE_Cash_loans"]))))*1.) )) / 2.0) )) +
                    0.100000*np.tanh((((((((np.where(data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]>0, data["NAME_PORTFOLIO_Cash"], 1.0 )) * 2.0)) * 2.0)) < (((1.0) - (((data["NAME_PORTFOLIO_Cash"]) * 2.0)))))*1.)) +
                    0.099961*np.tanh(((((np.minimum(((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"])), ((np.tanh(((((((1.0) < (data["NAME_PORTFOLIO_Cash"]))*1.)) * (data["NAME_PRODUCT_TYPE_x_sell"])))))))) / 2.0)) / 2.0)) +
                    0.060531*np.tanh(np.where(np.minimum(((np.tanh((2.0)))), (((((np.tanh((data["cnt_CNT_CHILDREN"]))) + (data["avg_buro_buro_bal_status_C"]))/2.0))))>0, ((((np.tanh((data["SK_ID_PREV_y"]))) * (data["CNT_INSTALMENT"]))) / 2.0), data["NAME_CLIENT_TYPE_New"] )) +
                    0.099492*np.tanh(np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((((data["NAME_PRODUCT_TYPE_XNA"]) * (np.maximum(((data["NAME_PRODUCT_TYPE_XNA"])), ((data["avg_buro_buro_bal_status_C"]))))))))) +
                    0.084252*np.tanh(((((((((data["NAME_PORTFOLIO_POS"]) > (data["NAME_CONTRACT_TYPE_Cash_loans"]))*1.)) < ((-1.0*(((((((data["NAME_CONTRACT_TYPE_Cash_loans"]) + ((-1.0*((data["NAME_SELLER_INDUSTRY_XNA"])))))/2.0)) / 2.0))))))*1.)) / 2.0)) +
                    0.099609*np.tanh((((-1.0*((((((data["avg_buro_buro_bal_status_5"]) * (((data["DAYS_LAST_DUE_1ST_VERSION"]) * (data["NAME_CASH_LOAN_PURPOSE_XAP"]))))) * (((-3.0) * 2.0))))))) + ((((3.0) < (data["CNT_PAYMENT"]))*1.)))) +
                    0.099453*np.tanh((((((-1.0*((np.where(data["CNT_INSTALMENT"]>0, 2.0, 0.0 ))))) + ((((data["NAME_CASH_LOAN_PURPOSE_XAP"]) > (((0.0) * (1.0))))*1.)))) * (data["avg_buro_buro_bal_status_5"]))) +
                    0.080774*np.tanh((-1.0*((((((((((((1.0)) * (((data["CNT_INSTALMENT"]) / 2.0)))) + (np.maximum(((np.minimum(((3.0)), ((data["CNT_INSTALMENT"]))))), ((data["te_WALLSMATERIAL_MODE"])))))/2.0)) / 2.0)) / 2.0))))) +
                    0.099961*np.tanh((((((np.minimum(((data["NAME_PORTFOLIO_Cash"])), ((data["NAME_PRODUCT_TYPE_XNA"])))) > ((((0.0) > (((data["NAME_CONTRACT_TYPE_Cash_loans"]) + (np.tanh((data["CNT_INSTALMENT_FUTURE"]))))))*1.)))*1.)) * (data["CNT_INSTALMENT_FUTURE"]))) +
                    0.099844*np.tanh((((1.0) < (np.minimum((((((data["NAME_PRODUCT_TYPE_x_sell"]) + (data["NAME_CONTRACT_TYPE_Cash_loans"]))/2.0))), ((((np.maximum(((((data["NAME_PORTFOLIO_POS"]) * (data["NAME_PORTFOLIO_Cash"])))), (((((-1.0) < (data["NAME_PORTFOLIO_POS"]))*1.))))) * 2.0))))))*1.)) +
                    0.099648*np.tanh(np.where(data["NAME_CASH_LOAN_PURPOSE_XAP"]>0, data["NAME_CASH_LOAN_PURPOSE_Repairs"], ((np.where(data["CNT_INSTALMENT_FUTURE"]>0, data["NAME_CASH_LOAN_PURPOSE_Repairs"], (((((data["NAME_CASH_LOAN_PURPOSE_Repairs"]) < (data["PRODUCT_COMBINATION_Cash_Street__middle"]))*1.)) / 2.0) )) / 2.0) )) +
                    0.099883*np.tanh(((((((data["inst_AMT_PAYMENT"]) - (np.minimum(((3.0)), (((-1.0*((data["PRODUCT_COMBINATION_Cash_Street__low"]))))))))) * (np.minimum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((data["inst_AMT_PAYMENT"])))))) / 2.0)) +
                    0.099609*np.tanh(((np.where(data["CNT_INSTALMENT_FUTURE"]>0, data["NAME_TYPE_SUITE_Spouse__partner"], np.where(data["DAYS_LAST_DUE_1ST_VERSION"]>0, data["NAME_TYPE_SUITE_Spouse__partner"], data["NAME_CASH_LOAN_PURPOSE_Business_development"] ) )) / 2.0)) +
                    0.099648*np.tanh(((np.where(data["CNT_INSTALMENT_FUTURE"]>0, 0.0, np.tanh(((((((np.tanh((np.tanh((data["NAME_CASH_LOAN_PURPOSE_Other"]))))) > (((((data["te_FLAG_DOCUMENT_20"]) * 2.0)) * 2.0)))*1.)) * (data["NAME_CASH_LOAN_PURPOSE_Other"])))) )) * 2.0)) +
                    0.100000*np.tanh((-1.0*((((data["NAME_TYPE_SUITE_Spouse__partner"]) * (((((((data["inst_NUM_INSTALMENT_VERSION"]) + ((((0.0) + (np.maximum(((data["CNT_INSTALMENT_FUTURE"])), (((((-2.0) < (data["avg_buro_buro_bal_status_5"]))*1.))))))/2.0)))/2.0)) > (1.0))*1.))))))) +
                    0.099805*np.tanh(np.maximum(((np.tanh((((data["ty__Interbank_credit"]) + ((-1.0*((data["avg_buro_buro_bal_status_5"]))))))))), (((((((np.where(data["ty__Interbank_credit"]>0, data["CNT_INSTALMENT_FUTURE"], data["te_FLAG_EMAIL"] )) - (1.0))) > (0.0))*1.))))) +
                    0.099218*np.tanh(np.minimum(((np.minimum(((data["avg_buro_buro_bal_status_5"])), (((-1.0*(((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (np.tanh((data["te_FLAG_DOCUMENT_11"]))))/2.0)) / 2.0))))))))), ((np.where(data["CNT_INSTALMENT"]>0, data["te_FLAG_DOCUMENT_11"], data["NAME_CASH_LOAN_PURPOSE_Business_development"] ))))) +
                    0.071161*np.tanh(((((8.0)) < ((((-1.0*(((((np.where((10.64545440673828125)>0, data["avg_buro_buro_bal_status_5"], data["te_FLAG_CONT_MOBILE"] )) + (data["te_FLAG_DOCUMENT_11"]))/2.0))))) - (((data["te_FLAG_EMAIL"]) * 2.0)))))*1.)) +
                    0.092458*np.tanh(((np.where((-1.0*((np.where(((2.0) * (np.minimum(((data["cnt_FLAG_EMAIL"])), (((4.85187721252441406))))))>0, data["NAME_GOODS_CATEGORY_Insurance"], data["te_FLAG_DOCUMENT_20"] ))))>0, 0.0, data["avg_buro_buro_bal_status_5"] )) * ((13.72183322906494141)))) +
                    0.099492*np.tanh(np.where((((((((8.19193267822265625)) < (data["avg_buro_buro_bal_status_5"]))*1.)) < (data["te_FLAG_DOCUMENT_8"]))*1.)>0, ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]))) * 2.0), (((data["NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment"]) > (np.tanh((0.0))))*1.) )) +
                    0.047245*np.tanh((((-1.0*((np.minimum(((data["avg_buro_buro_bal_status_5"])), ((((2.0) * (np.minimum(((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) * (data["te_FLAG_DOCUMENT_20"]))) * (data["te_FLAG_DOCUMENT_20"])))), ((data["avg_buro_buro_bal_status_5"])))))))))))) * 2.0)) +
                    0.099687*np.tanh(((np.minimum(((data["avg_buro_buro_bal_status_5"])), ((np.where(np.minimum(((data["DAYS_LAST_DUE_1ST_VERSION"])), ((np.tanh((data["ty__Interbank_credit"])))))>0, (-1.0*(((((2.0) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))/2.0)))), data["ty__Interbank_credit"] ))))) * (data["DAYS_LAST_DUE_1ST_VERSION"]))) +
                    0.099687*np.tanh((((-3.0) > (((np.minimum(((data["te_FLAG_DOCUMENT_20"])), (((-1.0*((0.0))))))) - (data["NAME_PORTFOLIO_Cards"]))))*1.)) +
                    0.099414*np.tanh(np.minimum(((((data["CODE_REJECT_REASON_SYSTEM"]) * (data["NAME_YIELD_GROUP_high"])))), ((((data["CODE_REJECT_REASON_SYSTEM"]) + (np.maximum(((data["CODE_REJECT_REASON_SYSTEM"])), ((data["NAME_YIELD_GROUP_high"]))))))))) +
                    0.065416*np.tanh(((np.where(((data["NAME_YIELD_GROUP_high"]) / 2.0)>0, (((((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) + (2.0))/2.0)) + (0.0))/2.0)) + (data["DAYS_LAST_DUE"]))/2.0), data["NAME_CASH_LOAN_PURPOSE_Business_development"] )) / 2.0)) +
                    0.100000*np.tanh((((data["ty__Interbank_credit"]) > (np.minimum((((((((((((((3.0) / 2.0)) + (((((data["PRODUCT_COMBINATION_POS_mobile_with_interest"]) / 2.0)) / 2.0)))/2.0)) + (1.0))/2.0)) > (data["PRODUCT_COMBINATION_POS_mobile_with_interest"]))*1.))), ((data["ty__Interbank_credit"])))))*1.)) +
                    0.099726*np.tanh((((((data["NAME_GOODS_CATEGORY_Mobile"]) + (-1.0))) > (((data["NAME_SELLER_INDUSTRY_Connectivity"]) * 2.0)))*1.)) +
                    0.078976*np.tanh(((np.where(data["PRODUCT_COMBINATION_POS_mobile_with_interest"]>0, (9.37105369567871094), np.minimum(((data["NAME_GOODS_CATEGORY_Insurance"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) )) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) +
                    0.062524*np.tanh(((np.where((8.51268959045410156)>0, ((((data["CODE_REJECT_REASON_SCOFR"]) * (np.minimum(((data["NAME_YIELD_GROUP_high"])), ((((data["NAME_YIELD_GROUP_high"]) * (data["NAME_YIELD_GROUP_high"])))))))) / 2.0), -1.0 )) * 2.0)) +
                    0.099062*np.tanh(np.where(data["avg_buro_buro_bal_status_5"]>0, data["te_FLAG_DOCUMENT_20"], ((np.where(data["ty__Interbank_credit"]>0, (((np.tanh((0.0))) + (data["NAME_SELLER_INDUSTRY_Connectivity"]))/2.0), ((-2.0) * (data["NAME_GOODS_CATEGORY_Insurance"])) )) / 2.0) )) +
                    0.095506*np.tanh(((data["avg_buro_buro_bal_status_2"]) * (((((((((data["te_FLAG_DOCUMENT_20"]) * (data["avg_buro_buro_bal_status_2"]))) / 2.0)) - (data["avg_buro_buro_bal_status_5"]))) * (((np.tanh((data["avg_buro_buro_bal_status_2"]))) / 2.0)))))) +
                    0.099726*np.tanh(((data["avg_buro_buro_bal_status_2"]) * (((((((6.0)) > (data["avg_buro_buro_bal_status_2"]))*1.)) * (np.tanh((np.where(1.0>0, np.tanh((((((2.0)) < (data["avg_buro_buro_bal_status_2"]))*1.))), -1.0 )))))))) +
                    0.099023*np.tanh(((data["cc_bal_AMT_PAYMENT_CURRENT"]) * (((((data["avg_buro_buro_bal_status_2"]) * 2.0)) * (((3.0) - (data["cc_bal_AMT_PAYMENT_CURRENT"]))))))) +
                    0.099297*np.tanh(((((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, data["avg_buro_buro_bal_status_5"], np.where(data["ty__Real_estate_loan"]>0, data["NAME_GOODS_CATEGORY_Insurance"], (-1.0*((data["NAME_GOODS_CATEGORY_Insurance"]))) ) )) - (np.tanh((np.maximum(((data["ty__Real_estate_loan"])), ((data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))))) * 2.0)) +
                    0.084056*np.tanh(np.where(((((2.0) * 2.0)) - (data["avg_buro_buro_bal_status_2"]))>0, ((np.minimum(((data["avg_buro_buro_bal_status_2"])), ((data["avg_buro_buro_bal_status_2"])))) * ((((2.0) > (data["cc_bal_AMT_PAYMENT_CURRENT"]))*1.))), -1.0 )) +
                    0.070535*np.tanh(np.where(data["ty__Interbank_credit"]>0, ((data["cc_bal_AMT_PAYMENT_CURRENT"]) * 2.0), (-1.0*((np.where(-3.0>0, data["ty__Interbank_credit"], ((1.0) * (data["avg_buro_buro_bal_status_5"])) )))) )) +
                    0.099922*np.tanh(((((10.0)) < ((((((data["NAME_GOODS_CATEGORY_Direct_Sales"]) - (data["te_FLAG_DOCUMENT_20"]))) + (data["avg_buro_buro_bal_status_5"]))/2.0)))*1.)) +
                    0.088316*np.tanh(np.maximum(((((((np.maximum(((data["avg_buro_buro_bal_status_5"])), ((data["avg_buro_buro_bal_status_5"])))) + (((data["te_FLAG_DOCUMENT_20"]) / 2.0)))) * (((data["avg_buro_buro_bal_status_2"]) * 2.0))))), ((-3.0)))) +
                    0.099414*np.tanh((-1.0*(((((((data["FLAG_DOCUMENT_10"]) - (data["te_FLAG_DOCUMENT_20"]))) < ((((data["avg_buro_buro_bal_status_2"]) + (((-3.0) / 2.0)))/2.0)))*1.))))) +
                    0.099531*np.tanh(((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (((data["FLAG_DOCUMENT_19"]) - (data["avg_buro_buro_bal_status_5"]))))) +
                    0.099258*np.tanh(((((np.where(data["ty__Interbank_credit"]>0, data["CODE_REJECT_REASON_SYSTEM"], (((data["NAME_GOODS_CATEGORY_Insurance"]) + (((data["NAME_GOODS_CATEGORY_Fitness"]) - (data["CODE_REJECT_REASON_SYSTEM"]))))/2.0) )) * (data["CODE_REJECT_REASON_SYSTEM"]))) * 2.0)) +
                    0.081047*np.tanh(((data["cnt_FLAG_DOCUMENT_4"]) + ((((((((data["avg_buro_buro_bal_status_2"]) / 2.0)) + ((((data["cc_bal_cc_bal_status__Refused"]) + (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))/2.0)))) > (1.0))*1.)))) +
                    0.099883*np.tanh(((((np.where(((((data["ty__Interbank_credit"]) / 2.0)) / 2.0)>0, (-1.0*((data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"]))), data["NAME_CASH_LOAN_PURPOSE_Buying_a_new_car"] )) - (np.tanh((data["cnt_FLAG_DOCUMENT_10"]))))) - (np.maximum(((data["ty__Interbank_credit"])), ((data["FLAG_DOCUMENT_10"])))))) +
                    0.099805*np.tanh((((data["AMT_ANNUITY_y"]) > ((((((np.minimum((((-1.0*((((data["CODE_REJECT_REASON_VERIF"]) * 2.0)))))), ((((data["cc_bal_cc_bal_status__Refused"]) / 2.0))))) > (0.0))*1.)) / 2.0)))*1.)) +
                    0.093669*np.tanh((((((data["CODE_REJECT_REASON_VERIF"]) + ((((np.where(data["AMT_CREDIT_SUM"]>0, ((data["avg_buro_buro_bal_status_2"]) * 2.0), data["cc_bal_cc_bal_status__Refused"] )) < (((data["CODE_REJECT_REASON_VERIF"]) * (data["AMT_CREDIT_SUM"]))))*1.)))/2.0)) * (data["AMT_CREDIT_SUM"]))) +
                    0.098945*np.tanh(((((((np.where(data["ty__Car_loan"]>0, data["CODE_REJECT_REASON_VERIF"], (((((-1.0*((data["CODE_REJECT_REASON_VERIF"])))) / 2.0)) * (data["CNT_INSTALMENT"])) )) + (data["ty__Car_loan"]))) * (data["CODE_REJECT_REASON_VERIF"]))) * 2.0)) +
                    0.096874*np.tanh(((np.where(data["cc_bal_cc_bal_status__Refused"]>0, (-1.0*(((-1.0*((((np.where(np.where(data["avg_buro_buro_bal_status_5"]>0, data["avg_buro_buro_bal_status_2"], data["ty__Interbank_credit"] )>0, data["avg_buro_buro_bal_status_4"], data["ty__Interbank_credit"] )) * 2.0))))))), data["avg_buro_buro_bal_status_4"] )) * 2.0)) +
                    0.090504*np.tanh(np.where(3.0>0, ((data["AMT_REQ_CREDIT_BUREAU_MON"]) * ((((((((data["CHANNEL_TYPE_Car_dealer"]) + (((data["AMT_CREDIT_SUM"]) / 2.0)))) > (data["CODE_REJECT_REASON_VERIF"]))*1.)) * (data["AMT_REQ_CREDIT_BUREAU_MON"])))), np.tanh((data["NAME_CASH_LOAN_PURPOSE_Business_development"])) )) +
                    0.099961*np.tanh(((((data["AMT_ANNUITY_x"]) * (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, (((data["AMT_REQ_CREDIT_BUREAU_MON"]) < (0.0))*1.), data["AMT_REQ_CREDIT_BUREAU_MON"] )))) / 2.0)) +
                    0.099766*np.tanh((((((((-1.0) * ((((data["AMT_REQ_CREDIT_BUREAU_MON"]) + (data["avg_buro_buro_bal_status_2"]))/2.0)))) + (np.minimum((((((2.0)) - (data["AMT_CREDIT_SUM"])))), ((0.0)))))/2.0)) / 2.0)) +
                    0.099766*np.tanh(((np.where((((data["avg_buro_buro_bal_status_2"]) > (data["CODE_REJECT_REASON_VERIF"]))*1.)>0, data["NAME_CASH_LOAN_PURPOSE_Business_development"], data["AMT_GOODS_PRICE_x"] )) * (data["AMT_CREDIT_SUM"]))) +
                    0.099961*np.tanh(((data["AMT_CREDIT_SUM"]) * (np.minimum(((((3.0) * ((((((data["avg_buro_buro_bal_status_4"]) < (data["NAME_GOODS_CATEGORY_Insurance"]))*1.)) * (data["inst_AMT_PAYMENT"])))))), ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0))))))) +
                    0.100000*np.tanh(((((((np.maximum(((((((data["AMT_CREDIT_MAX_OVERDUE"]) * ((((7.54546737670898438)) * 2.0)))) * 2.0))), ((data["AMT_CREDIT_MAX_OVERDUE"])))) * (np.maximum((((10.0))), ((data["avg_buro_buro_bal_status_4"])))))) * 2.0)) * 2.0)) +
                    0.095506*np.tanh(((((-1.0*((np.where(data["SK_ID_BUREAU"]>0, data["avg_buro_buro_bal_status_1"], ((((-3.0) + (np.tanh((-3.0))))) * (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0))) ))))) + (((1.0) / 2.0)))/2.0)) +
                    0.099805*np.tanh(np.where(data["avg_buro_buro_bal_status_1"]>0, ((data["EXT_SOURCE_3"]) - (data["NAME_CONTRACT_STATUS_Refused"])), ((np.maximum(((data["avg_buro_buro_bal_status_2"])), ((((data["EXT_SOURCE_3"]) * 2.0))))) * (data["avg_buro_buro_bal_status_1"])) )) +
                    0.092224*np.tanh(((((data["cc_bal_cc_bal_status__Refused"]) + ((-1.0*((data["AMT_CREDIT_SUM"])))))) * (np.maximum((((((data["AMT_CREDIT_SUM"]) > ((1.58207809925079346)))*1.))), ((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])))))) +
                    0.094959*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["AMT_CREDIT_SUM_DEBT"]))) + (np.tanh(((((data["AMT_CREDIT_SUM_DEBT"]) + ((((data["avg_buro_buro_bal_status_2"]) < (np.minimum(((data["AMT_CREDIT_SUM_DEBT"])), (((((12.72768211364746094)) / 2.0))))))*1.)))/2.0)))))) +
                    0.099922*np.tanh(((np.tanh(((((-1.0*((data["AMT_CREDIT_SUM"])))) * (np.maximum((((((-1.0) > (((data["inst_AMT_PAYMENT"]) + (data["AMT_CREDIT_SUM"]))))*1.))), ((np.tanh((data["EXT_SOURCE_3"])))))))))) * 2.0)) +
                    0.078507*np.tanh(np.where(data["SK_ID_BUREAU"]>0, ((data["SK_ID_BUREAU"]) * ((((np.where(data["NAME_CASH_LOAN_PURPOSE_Business_development"]>0, 0.0, data["NAME_GOODS_CATEGORY_Insurance"] )) > (((np.minimum(((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"])), ((3.0)))) * 2.0)))*1.))), data["ty__Mortgage"] )) +
                    0.092614*np.tanh(np.where(data["DAYS_FIRST_DRAWING"]>0, ((data["inst_NUM_INSTALMENT_VERSION"]) * (data["inst_NUM_INSTALMENT_NUMBER"])), ((np.minimum(((0.0)), ((data["DAYS_FIRST_DRAWING"])))) * (data["inst_NUM_INSTALMENT_NUMBER"])) )) +
                    0.099883*np.tanh((((1.0) < (((data["inst_NUM_INSTALMENT_NUMBER"]) * (((((((-1.0*((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])))) > (np.minimum(((data["cc_bal_MONTHS_BALANCE"])), (((10.0))))))*1.)) * 2.0)))))*1.)) +
                    0.100000*np.tanh((((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"]) < ((-1.0*(((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) < ((((3.0) + (data["cc_bal_AMT_BALANCE"]))/2.0)))*1.))))))*1.)) +
                    0.099531*np.tanh((((data["cc_bal_AMT_RECIVABLE"]) > (np.maximum(((data["cc_bal_AMT_RECEIVABLE_PRINCIPAL"])), ((np.minimum((((((((1.0) < (data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]))*1.)) / 2.0))), ((((((((data["cc_bal_AMT_RECIVABLE"]) > (-2.0))*1.)) > ((0.0)))*1.)))))))))*1.)) +
                    0.100000*np.tanh((-1.0*((np.maximum(((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"])), (((-1.0*((np.minimum((((-1.0*((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]))))), (((((data["cc_bal_CNT_DRAWINGS_ATM_CURRENT"]) + (((((-1.0*((data["NAME_GOODS_CATEGORY_Insurance"])))) < (data["EXT_SOURCE_3"]))*1.)))/2.0)))))))))))))) +
                    0.099805*np.tanh((((((data["cc_bal_AMT_RECIVABLE"]) / 2.0)) > ((((((0.0) - ((((2.0) < (data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]))*1.)))) < ((((data["cc_bal_AMT_RECIVABLE"]) + ((-1.0*((data["cc_bal_AMT_PAYMENT_CURRENT"])))))/2.0)))*1.)))*1.)) +
                    0.072177*np.tanh(np.minimum(((((data["inst_NUM_INSTALMENT_VERSION"]) + ((1.0))))), ((np.where(np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Buying_a_garage"])), ((data["cc_bal_cc_bal_status__Refused"])))>0, data["inst_NUM_INSTALMENT_VERSION"], np.maximum(((data["cc_bal_cc_bal_status__Refused"])), ((data["cc_bal_cc_bal_status__Refused"]))) ))))) +
                    0.099883*np.tanh(((((((((((3.66407728195190430)) > ((0.14521482586860657)))*1.)) - ((((data["cc_bal_cc_bal_status__Refused"]) < ((((data["inst_NUM_INSTALMENT_NUMBER"]) < (np.maximum(((data["NAME_CASH_LOAN_PURPOSE_Business_development"])), ((data["CNT_INSTALMENT"])))))*1.)))*1.)))) * 2.0)) * 2.0)) +
                    0.099336*np.tanh(np.minimum((((((data["inst_AMT_PAYMENT"]) < (np.where((6.0)>0, ((data["NAME_GOODS_CATEGORY_Mobile"]) * 2.0), 0.0 )))*1.))), (((((np.tanh((data["CHANNEL_TYPE_Car_dealer"]))) + (((data["inst_AMT_PAYMENT"]) * (data["DAYS_FIRST_DRAWING"]))))/2.0))))) +
                    0.099726*np.tanh(((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) * (((-1.0) * (np.maximum(((data["cc_bal_AMT_PAYMENT_CURRENT"])), ((np.where(data["CHANNEL_TYPE_Car_dealer"]>0, (-1.0*((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_garage"]) * 2.0)))), ((((data["CHANNEL_TYPE_Car_dealer"]) * 2.0)) * 2.0) ))))))))) +
                    0.095584*np.tanh(np.minimum(((((data["CHANNEL_TYPE_Car_dealer"]) * (data["inst_AMT_PAYMENT"])))), ((np.tanh(((((-1.0*((np.minimum(((np.maximum(((-3.0)), ((np.minimum(((data["inst_AMT_PAYMENT"])), ((data["AMT_APPLICATION"])))))))), ((3.0))))))) / 2.0))))))) +
                    0.071473*np.tanh(np.minimum((((((np.tanh((np.tanh(((2.0)))))) < (data["inst_AMT_INSTALMENT"]))*1.))), ((((((data["inst_AMT_INSTALMENT"]) * 2.0)) - (-1.0)))))) +
                    0.099883*np.tanh((-1.0*((np.maximum((((((data["AMT_APPLICATION"]) < (-1.0))*1.))), ((np.tanh((((np.where((((data["inst_AMT_PAYMENT"]) + (-1.0))/2.0)>0, data["AMT_APPLICATION"], data["NAME_CASH_LOAN_PURPOSE_Buying_a_garage"] )) / 2.0)))))))))) +
                    0.096209*np.tanh(((((data["cnt_DAYS_BIRTH"]) * (data["CNT_INSTALMENT_FUTURE"]))) * ((((-1.0) > (data["CNT_INSTALMENT_FUTURE"]))*1.)))) +
                    0.079914*np.tanh(((data["avg_buro_buro_bal_status_4"]) * (((data["AMT_REQ_CREDIT_BUREAU_MON"]) * (np.where(((data["CNT_INSTALMENT_FUTURE"]) * 2.0)>0, data["CNT_INSTALMENT_FUTURE"], (10.0) )))))) +
                    0.099609*np.tanh(((np.where(data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]>0, ((data["DAYS_LAST_DUE"]) / 2.0), np.where(np.tanh((data["CODE_REJECT_REASON_VERIF"]))>0, data["DAYS_LAST_DUE"], data["ty__Interbank_credit"] ) )) / 2.0)) +
                    0.100000*np.tanh((((((((3.0) + (3.0))/2.0)) + ((3.0)))) * (np.minimum(((np.where(data["avg_buro_buro_bal_status_5"]>0, data["cc_bal_CNT_INSTALMENT_MATURE_CUM"], 0.0 ))), ((((data["avg_buro_buro_bal_status_5"]) * (data["CODE_REJECT_REASON_VERIF"])))))))) +
                    0.099648*np.tanh((((np.minimum((((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) > (data["ty__Interbank_credit"]))*1.))), (((((data["NAME_SELLER_INDUSTRY_Auto_technology"]) + (data["avg_buro_buro_bal_status_5"]))/2.0))))) > (np.tanh(((((data["CNT_INSTALMENT_FUTURE"]) > (np.minimum(((data["NAME_SELLER_INDUSTRY_Auto_technology"])), ((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"])))))*1.)))))*1.)) +
                    0.095389*np.tanh(((((data["FLAG_DOCUMENT_13"]) * ((((((((data["ty__Interbank_credit"]) / 2.0)) > (data["avg_buro_buro_bal_status_5"]))*1.)) + (data["NAME_GOODS_CATEGORY_Auto_Accessories"]))))) * 2.0)) +
                    0.099531*np.tanh(np.where(((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) < (((((-1.0*((data["avg_buro_buro_bal_status_5"])))) < (data["FLAG_DOCUMENT_13"]))*1.)))*1.)) < (data["avg_buro_buro_bal_status_5"]))*1.)>0, data["CNT_INSTALMENT_FUTURE"], (-1.0*((data["FLAG_DOCUMENT_13"]))) )) +
                    0.099375*np.tanh(((((np.minimum(((np.where(data["avg_buro_buro_bal_status_5"]>0, data["cc_bal_CNT_INSTALMENT_MATURE_CUM"], data["NAME_CASH_LOAN_PURPOSE_Buying_a_garage"] ))), ((((data["NAME_CASH_LOAN_PURPOSE_Buying_a_garage"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"])))))) - (data["NAME_CASH_LOAN_PURPOSE_Buying_a_garage"]))) * 2.0)) +
                    0.099766*np.tanh(np.where(-3.0>0, -3.0, ((((((((data["cc_bal_CNT_INSTALMENT_MATURE_CUM"]) - (np.minimum(((0.0)), ((data["CNT_INSTALMENT_FUTURE"])))))) * (data["avg_buro_buro_bal_status_5"]))) - (data["CNT_INSTALMENT_FUTURE"]))) * (data["avg_buro_buro_bal_status_3"])) )) +
                    0.098554*np.tanh((-1.0*((((((data["CNT_INSTALMENT"]) * (np.maximum(((data["CHANNEL_TYPE_Car_dealer"])), ((-2.0)))))) * ((((((data["CODE_REJECT_REASON_SCO"]) * (data["CODE_REJECT_REASON_SCO"]))) + (2.0))/2.0))))))) +
                    0.064127*np.tanh(((((np.tanh((((data["NAME_GOODS_CATEGORY_Weapon"]) * (data["AMT_DOWN_PAYMENT"]))))) * ((11.26369285583496094)))) * (((-2.0) * (np.tanh((-2.0))))))) +
                    0.099961*np.tanh(np.tanh((((data["AMT_CREDIT_x"]) * (np.where(((-3.0) + (np.where(data["CNT_INSTALMENT"]>0, data["SK_ID_PREV_y"], 0.0 )))>0, data["CNT_INSTALMENT"], 0.0 )))))) +
                    0.093200*np.tanh(np.minimum(((np.tanh(((((data["RATE_DOWN_PAYMENT"]) > ((((1.0) > (data["NAME_GOODS_CATEGORY_Weapon"]))*1.)))*1.))))), ((np.tanh((np.where(data["SK_ID_PREV_y"]>0, data["RATE_DOWN_PAYMENT"], np.tanh((data["avg_buro_buro_bal_status_5"])) ))))))) +
                    0.099531*np.tanh(((data["CNT_INSTALMENT"]) * (((np.where((((((np.maximum(((((data["AMT_DOWN_PAYMENT"]) / 2.0))), ((0.0)))) > (2.0))*1.)) / 2.0)>0, 3.0, data["NAME_SELLER_INDUSTRY_MLM_partners"] )) * 2.0)))) +
                    0.091403*np.tanh((((((((2.0)) > (((data["AMT_CREDIT_y"]) / 2.0)))*1.)) > ((-1.0*((((data["AMT_CREDIT_y"]) - (((data["AMT_CREDIT_y"]) + ((((((data["AMT_CREDIT_y"]) * 2.0)) < ((5.46582460403442383)))*1.))))))))))*1.)) +
                    0.089371*np.tanh(((((np.maximum((((12.36531925201416016))), ((data["ty__Interbank_credit"])))) * (((data["avg_buro_buro_bal_status_5"]) * ((-1.0*((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) - (data["NAME_GOODS_CATEGORY_Weapon"])))))))))) / 2.0)) +
                    0.085893*np.tanh(((((((((data["NAME_GOODS_CATEGORY_Insurance"]) + (2.0))/2.0)) < (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))*1.)) + ((-1.0*(((((((data["cc_bal_AMT_CREDIT_LIMIT_ACTUAL"]) + (((data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]) / 2.0)))/2.0)) / 2.0))))))) +
                    0.099726*np.tanh(np.minimum(((((data["NAME_GOODS_CATEGORY_Mobile"]) * ((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) + (data["cc_bal_AMT_DRAWINGS_CURRENT"]))/2.0))))), (((((((data["ty__Interbank_credit"]) + (((data["cc_bal_AMT_DRAWINGS_CURRENT"]) - (data["cc_bal_AMT_PAYMENT_TOTAL_CURRENT"]))))/2.0)) * 2.0))))) +
                    0.097733*np.tanh((((np.where((((data["CNT_INSTALMENT_FUTURE"]) > (data["inst_SK_ID_PREV"]))*1.)>0, (((data["inst_SK_ID_PREV"]) > (3.0))*1.), 3.0 )) < (data["inst_SK_ID_PREV"]))*1.)) +
                    0.071161*np.tanh((-1.0*((((((np.maximum(((np.tanh(((((data["AMT_GOODS_PRICE_y"]) > (2.0))*1.))))), (((((data["CNT_INSTALMENT"]) > ((((3.0)) + (((-1.0) / 2.0)))))*1.))))) * 2.0)) * 2.0))))) +
                    0.099726*np.tanh(np.where(((data["cc_bal_AMT_DRAWINGS_ATM_CURRENT"]) * (data["NAME_GOODS_CATEGORY_Insurance"]))>0, ((data["cc_bal_AMT_PAYMENT_CURRENT"]) * (data["AMT_DOWN_PAYMENT"])), (-1.0*((data["AMT_DOWN_PAYMENT"]))) )) +
                    0.081985*np.tanh(((np.where((((data["AMT_ANNUITY"]) + (data["AMT_CREDIT_y"]))/2.0)>0, data["avg_buro_buro_bal_status_5"], (((data["avg_buro_buro_bal_status_5"]) > (np.where(data["SK_ID_PREV_y"]>0, data["cc_bal_AMT_PAYMENT_CURRENT"], (((4.0)) + ((4.0))) )))*1.) )) * 2.0)) +
                    0.099726*np.tanh(((np.where((14.45024776458740234)>0, ((data["AMT_ANNUITY"]) * 2.0), ((-1.0) * 2.0) )) * ((((1.0) < (((data["AMT_DOWN_PAYMENT"]) - (((data["AMT_GOODS_PRICE_y"]) - (data["AMT_CREDIT_y"]))))))*1.)))) +
                    0.099375*np.tanh(np.where(((data["inst_SK_ID_PREV"]) * (((data["cc_bal_AMT_PAYMENT_CURRENT"]) * 2.0)))>0, data["SK_ID_PREV_y"], (-1.0*((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) / 2.0)))) )) +
                    0.099453*np.tanh(np.tanh((((np.tanh(((-1.0*((data["SK_ID_PREV_y"])))))) * ((((data["NAME_GOODS_CATEGORY_Insurance"]) > (np.maximum((((-1.0*(((8.21855163574218750)))))), ((((((data["PRODUCT_COMBINATION_Cash_X_Sell__middle"]) * (data["NAME_GOODS_CATEGORY_Insurance"]))) * 2.0))))))*1.)))))) +
                    0.099687*np.tanh(((np.minimum(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * (data["NAME_GOODS_CATEGORY_Mobile"])))), (((((((((data["NAME_SELLER_INDUSTRY_MLM_partners"]) < (((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) / 2.0)))*1.)) + (data["avg_buro_buro_bal_status_2"]))) * (data["cc_bal_AMT_PAYMENT_CURRENT"])))))) * (data["NAME_GOODS_CATEGORY_Mobile"]))) +
                    0.099531*np.tanh(((np.where(((((((((1.0) + (data["inst_SK_ID_PREV"]))/2.0)) / 2.0)) < (data["avg_buro_buro_bal_status_2"]))*1.)>0, 1.0, data["NAME_CASH_LOAN_PURPOSE_Business_development"] )) * (np.where(data["NAME_GOODS_CATEGORY_Insurance"]>0, data["CODE_REJECT_REASON_SCO"], data["CNT_INSTALMENT"] )))) +
                    0.099570*np.tanh(((((((((data["NAME_SELLER_INDUSTRY_MLM_partners"]) * (data["CODE_REJECT_REASON_SCO"]))) - ((((((data["NAME_GOODS_CATEGORY_Sport_and_Leisure"]) > (np.maximum(((data["CHANNEL_TYPE_Car_dealer"])), ((data["NAME_SELLER_INDUSTRY_MLM_partners"])))))*1.)) * 2.0)))) * 2.0)) - (data["CHANNEL_TYPE_Car_dealer"]))) +
                    0.099766*np.tanh(((((np.where(np.where(-2.0>0, -3.0, np.tanh((data["ty__Interbank_credit"])) )>0, data["CODE_REJECT_REASON_HC"], 0.0 )) + ((-1.0*((data["NAME_GOODS_CATEGORY_Weapon"])))))) / 2.0)) +
                    0.099805*np.tanh(((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) * 2.0)) * (((((data["inst_SK_ID_PREV"]) * (np.minimum(((data["NAME_SELLER_INDUSTRY_XNA"])), (((((((data["CODE_REJECT_REASON_HC"]) > (np.tanh((data["inst_SK_ID_PREV"]))))*1.)) * 2.0))))))) * (3.0))))) +
                    0.099687*np.tanh((((((data["CODE_REJECT_REASON_HC"]) * (np.where((3.33906126022338867)>0, data["NAME_CASH_LOAN_PURPOSE_Business_development"], data["NAME_CONTRACT_STATUS_Approved"] )))) + ((((data["cc_bal_AMT_PAYMENT_CURRENT"]) + ((-1.0*((((data["CHANNEL_TYPE_Credit_and_cash_offices"]) * (data["cc_bal_AMT_PAYMENT_CURRENT"])))))))/2.0)))/2.0)) +
                    0.084838*np.tanh((((((np.tanh(((-1.0*((np.minimum(((data["NAME_SELLER_INDUSTRY_XNA"])), ((data["CODE_REJECT_REASON_HC"]))))))))) / 2.0)) + (((data["CODE_REJECT_REASON_HC"]) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))))/2.0)) +
                    0.099766*np.tanh(np.tanh((((((np.where(((data["cc_bal_SK_DPD"]) * (np.minimum(((data["NAME_TYPE_SUITE_Spouse__partner"])), ((0.0)))))>0, data["LIVINGAPARTMENTS_MEDI"], data["NAME_GOODS_CATEGORY_Fitness"] )) / 2.0)) * (((3.0) * 2.0)))))) +
                    0.099687*np.tanh((-1.0*((((((3.0)) < (np.where(data["AMT_DOWN_PAYMENT"]>0, 3.0, np.where(data["ty__Interbank_credit"]>0, data["ty__Interbank_credit"], data["NAME_TYPE_SUITE_Spouse__partner"] ) )))*1.))))) +
                    0.088277*np.tanh(((((-1.0*((data["DAYS_CREDIT_ENDDATE"])))) > ((((np.maximum(((data["ENTRANCES_AVG"])), (((-1.0*((data["NAME_GOODS_CATEGORY_Fitness"]))))))) + (np.tanh(((9.0)))))/2.0)))*1.)) +
                    0.099726*np.tanh(np.where((((((((-1.0) - (3.0))) * (data["NAME_CASH_LOAN_PURPOSE_Business_development"]))) + (((((((data["NAME_CASH_LOAN_PURPOSE_Business_development"]) > (data["DAYS_CREDIT_ENDDATE"]))*1.)) + (data["DAYS_ENDDATE_FACT"]))/2.0)))/2.0)>0, data["NAME_CASH_LOAN_PURPOSE_Business_development"], data["DAYS_CREDIT"] )))

In [19]:
def GP(data):
    return (GPI(data)+GPII(data)+GPIII(data))/3.


In [20]:
from sklearn.metrics import roc_auc_score

In [21]:
roc_auc_score(train.TARGET,GPI(train))

In [22]:
roc_auc_score(train.TARGET,GPII(train))

In [23]:
roc_auc_score(train.TARGET,GPIII(train))

In [24]:
roc_auc_score(train.TARGET,GP(train))

In [25]:
Submission = pd.DataFrame({ 'SK_ID_CURR': ID,'TARGET': GP(test).values })
Submission.to_csv("sample_submission.csv", index=False)